In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import h5py
import numpy as np

DOMAIN_SIZE = 100.0
N_DOF = 200
DT = 0.05 

class KuramotoSivashinsky():
    def __init__(
        self,
        L,
        N,
        dt,
        a=-1.0,
        b=-1.0,
        c=-0.5,
    ):
        self.L = L
        self.N = N
        self.dt = dt
        self.a = a
        self.b = b
        self.c = c

        self.dx = L / N

        # Frequencies (cycles per unit length)
        freqs = jnp.fft.rfftfreq(N, d=self.dx)
        # Convert to angular wavenumbers k = 2pi * freq
        k = 2 * jnp.pi * freqs
        self.derivative_operator = 1j * k

        # Linear operator in Fourier space: a*d^2/dx^2 + b*d^4/dx^4
        linear_operator = self.a * (-k**2) + self.b * (k**4)

        self.exp_term = jnp.exp(dt * linear_operator)
        self.coef = jnp.where(
            linear_operator == 0.0,
            dt,
            (self.exp_term - 1.0) / linear_operator,
        )

        # 2/3 rule dealiasing mask for nonlinear term
        self.alias_mask = (freqs < 2/3 * jnp.max(freqs))

    def __call__(self, u):
        # Nonlinear term c * d/dx (u^2)
        u_nonlin = self.c * u**2
        u_hat = jnp.fft.rfft(u)
        u_nonlin_hat = jnp.fft.rfft(u_nonlin)
        u_nonlin_hat = self.alias_mask * u_nonlin_hat

        u_nonlin_der_hat = self.derivative_operator * u_nonlin_hat

        # Exponential time differencing step
        u_next_hat = self.exp_term * u_hat + self.coef * u_nonlin_der_hat
        u_next = jnp.fft.irfft(u_next_hat, n=self.N)

        return u_next

# Create spatial mesh
mesh = jnp.linspace(0.0, DOMAIN_SIZE, N_DOF, endpoint=False)

# Initial condition
u_0 = jnp.sin(16 * jnp.pi * mesh / DOMAIN_SIZE)

ks_stepper = KuramotoSivashinsky(
    L=DOMAIN_SIZE,
    N=N_DOF,
    dt=DT,
    a=-1.0,
    b=-1.0,
    c=-0.5,
)

ks_stepper = jax.jit(ks_stepper)

# METHOD 1: Spatial Translation (Pixels don't coincide between time steps)
def generate_spatially_shifted_data():
    """Apply slight translations between time steps"""
    u_current = u_0
    trj_shifted = [u_current]
    
    # Generate clean trajectory first
    for i in range(2000):
        u_current = ks_stepper(u_current)
        trj_shifted.append(u_current)
    
    trj_shifted = jnp.stack(trj_shifted)
    
    # Now apply random spatial shifts to break pixel alignment
    rng = np.random.default_rng(42)
    trj_unregistered = []
    
    for i in range(trj_shifted.shape[0]):
        # Random shift between -3 and +3 pixels
        shift = rng.integers(-3, 4)
        shifted_frame = jnp.roll(trj_shifted[i], shift)
        trj_unregistered.append(shifted_frame)
    
    return jnp.stack(trj_unregistered)

# METHOD 2: Varying Initial Conditions (Each time step from different IC)
def generate_varying_ic_data():
    """Each time step comes from slightly different initial conditions"""
    rng = np.random.default_rng(42)
    trj_varied = []
    
    # Base initial condition
    base_u0 = u_0
    
    for i in range(2001):  # Same number of time steps
        # Add small Gaussian noise to initial condition
        noise_level = 0.02  # 2% noise
        perturbed_u0 = base_u0 + noise_level * rng.normal(0, 1, base_u0.shape)
        
        # Simulate forward i steps from perturbed initial condition
        u_current = perturbed_u0
        for step in range(i):  # Simulate exactly i steps
            u_current = ks_stepper(u_current)
        
        trj_varied.append(u_current)
    
    return jnp.stack(trj_varied)

# Generate both types of unregistered data
print("Generating spatially shifted data (spatial misregistration noise)...")
trj_spatial_shift = generate_spatially_shifted_data()

print("Generating varying initial condition data...")  
trj_varied_ic = generate_varying_ic_data()

# Generate clean data for comparison
print("Generating clean data...")
u_current = u_0
trj_clean = [u_current]
for i in range(2000):
    u_current = ks_stepper(u_current)
    trj_clean.append(u_current)
trj_clean = jnp.stack(trj_clean)

# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Clean data
im0 = axes[0].imshow(trj_clean.T, cmap="RdBu", aspect="auto", origin="lower",
                    extent=(0, trj_clean.shape[0] * DT, 0, DOMAIN_SIZE))
axes[0].set_title("Clean Data\n(Pixels Aligned)")
axes[0].set_xlabel("Time")
axes[0].set_ylabel("Space")
plt.colorbar(im0, ax=axes[0])

# Spatially shifted
im1 = axes[1].imshow(trj_spatial_shift.T, cmap="RdBu", aspect="auto", origin="lower",
                    extent=(0, trj_spatial_shift.shape[0] * DT, 0, DOMAIN_SIZE))
axes[1].set_title("Spatially Shifted\n(Pixels Don't Coincide)")
axes[1].set_xlabel("Time")
axes[1].set_ylabel("Space")
plt.colorbar(im1, ax=axes[1])

# Varying IC
im2 = axes[2].imshow(trj_varied_ic.T, cmap="RdBu", aspect="auto", origin="lower",
                    extent=(0, trj_varied_ic.shape[0] * DT, 0, DOMAIN_SIZE))
axes[2].set_title("Varying Initial Conditions\n(Pixels Don't Coincide)")
axes[2].set_xlabel("Time")
axes[2].set_ylabel("Space")
plt.colorbar(im2, ax=axes[2])

plt.tight_layout()
plt.savefig("ks_unregistered_data_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

# Save all datasets
x = jnp.linspace(0.0, DOMAIN_SIZE, N_DOF, endpoint=False)
t = jnp.arange(trj_clean.shape[0]) * DT

# Save clean data
with h5py.File("test_solving_euler_ks_clean.h5", "w") as f:
    f.create_dataset("u", data=trj_clean)
    f.create_dataset("x", data=x)
    f.create_dataset("t", data=t)
    f.attrs["domain_size"] = DOMAIN_SIZE
    f.attrs["n_dof"] = N_DOF
    f.attrs["dt"] = DT
    f.attrs["n_steps"] = trj_clean.shape[0]
    f.attrs["registration"] = "aligned"

# Save spatially shifted data
with h5py.File("test_solving_euler_ks_spatial_shift.h5", "w") as f:
    f.create_dataset("u", data=trj_spatial_shift)
    f.create_dataset("x", data=x)
    f.create_dataset("t", data=t)
    f.attrs["domain_size"] = DOMAIN_SIZE
    f.attrs["n_dof"] = N_DOF
    f.attrs["dt"] = DT
    f.attrs["n_steps"] = trj_spatial_shift.shape[0]
    f.attrs["registration"] = "unregistered_spatial_shift"
    f.attrs["perturbation_type"] = "spatial_translation"

# Save varying IC data
with h5py.File("test_solving_euler_ks_varied_ic.h5", "w") as f:
    f.create_dataset("u", data=trj_varied_ic)
    f.create_dataset("x", data=x)
    f.create_dataset("t", data=t)
    f.attrs["domain_size"] = DOMAIN_SIZE
    f.attrs["n_dof"] = N_DOF
    f.attrs["dt"] = DT
    f.attrs["n_steps"] = trj_varied_ic.shape[0]
    f.attrs["registration"] = "unregistered_varied_ic"
    f.attrs["perturbation_type"] = "varying_initial_conditions"

print("All datasets saved successfully!")
print("Files created:")
print("- test_solving_euler_ks_clean.h5 (aligned pixels)")
print("- test_solving_euler_ks_spatial_shift.h5 (spatial shifts)")
print("- test_solving_euler_ks_varied_ic.h5 (varying initial conditions)")

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, ifft, fftfreq
from sklearn.linear_model import Lasso



# === Compute spatial derivatives using spectral method ===
def spectral_derivative(u_snapshot, k):
    """Compute first spatial derivative using FFT."""
    u_hat = fft(u_snapshot)
    return np.real(ifft(1j * k * u_hat))

def spectral_second_derivative(u_snapshot, k):
    """Compute second spatial derivative."""
    u_hat = fft(u_snapshot)
    return np.real(ifft(-k**2 * u_hat))

def spectral_fourth_derivative(u_snapshot, k):
    """Compute fourth spatial derivative."""
    u_hat = fft(u_snapshot)
    return np.real(ifft(k**4 * u_hat))

### FFT SINDy on data with spatial shift 

In [ ]:
with h5py.File("test_solving_euler_ks_spatial_shift.h5", "r") as f:
    u = np.array(f["u"])      # shape (Nt, Nx)
    x = np.array(f["x"])      # spatial grid
    t = np.array(f["t"])      # time grid
    
Nx = len(x)
Nt = len(t)
L = x[-1] - x[0] + (x[1]-x[0])
dx = x[1] - x[0]
dt = t[1] - t[0]
    
print(f"Loaded u(t,x) with shape {u.shape}, dx={dx:.3f}, dt={dt:.3f}")
    
    
# Define wavenumbers
k = 2 * np.pi * fftfreq(Nx, d=dx)
    
# === Compute temporal derivative u_t using finite difference ===
u_t = (u[2:] - u[:-2]) / (2 * dt)          # central difference in time
u_mid = u[1:-1]                            # align time dimension
    
# === Build feature library Θ(u) ===
# For each time snapshot, compute spatial derivatives
Theta = []
ut_flat = []
    
for snapshot, ut_snapshot in zip(u_mid, u_t):
    ux  = spectral_derivative(snapshot, k)
    uxx = spectral_second_derivative(snapshot, k)
    uxxxx = spectral_fourth_derivative(snapshot, k)
    
    # Feature terms
    Theta_snapshot = np.vstack([
            uxx,         # diffusion term
            uxxxx,       # hyperdiffusion
            snapshot * ux, # nonlinear advection term (u * ux)
        ]).T  # shape (Nx, 3)
    
    Theta.append(Theta_snapshot)
    ut_flat.append(ut_snapshot)
    
Theta = np.vstack(Theta)       # shape (Nt*Nx, 3)
ut_flat = np.hstack(ut_flat)   # flatten to 1D (Nt*Nx,)
    
print("Feature matrix Θ shape:", Theta.shape)
print("Target vector u_t shape:", ut_flat.shape)
    
# === Normalize features for numerical stability ===
Theta_mean = Theta.mean(axis=0)
Theta_std = Theta.std(axis=0)
Theta_norm = (Theta - Theta_mean) / Theta_std
    
# === Sparse regression (LASSO) ===
lasso = Lasso(alpha=1e-4, fit_intercept=False)
lasso.fit(Theta_norm, ut_flat)
coeffs = lasso.coef_ / Theta_std  # un-normalize coefficients
    
print("\nRecovered PDE coefficients (Spatial Translation):")
print(f"a (u_xx):     {coeffs[0]:.4f}")
print(f"b (u_xxxx):   {coeffs[1]:.4f}")
print(f"c (u*u_x):    {coeffs[2]:.4f}") # Updated label for the new term
    


## FFt basd SINDY on data with different initial conditions

In [ ]:
with h5py.File("test_solving_euler_ks_varied_ic.h5", "r") as f:
    u = np.array(f["u"])      # shape (Nt, Nx)
    x = np.array(f["x"])      # spatial grid
    t = np.array(f["t"])      # time grid
    
Nx = len(x)
Nt = len(t)
L = x[-1] - x[0] + (x[1]-x[0])
dx = x[1] - x[0]
dt = t[1] - t[0]
    
print(f"Loaded u(t,x) with shape {u.shape}, dx={dx:.3f}, dt={dt:.3f}")
    
    
# Define wavenumbers
k = 2 * np.pi * fftfreq(Nx, d=dx)
    
# === Compute temporal derivative u_t using finite difference ===
u_t = (u[2:] - u[:-2]) / (2 * dt)          # central difference in time
u_mid = u[1:-1]                            # align time dimension
    
# === Build feature library Θ(u) ===
# For each time snapshot, compute spatial derivatives
Theta = []
ut_flat = []
    
for snapshot, ut_snapshot in zip(u_mid, u_t):
    ux  = spectral_derivative(snapshot, k)
    uxx = spectral_second_derivative(snapshot, k)
    uxxxx = spectral_fourth_derivative(snapshot, k)
    
    # Feature terms
    Theta_snapshot = np.vstack([
            uxx,         # diffusion term
            uxxxx,       # hyperdiffusion
            snapshot * ux, # nonlinear advection term (u * ux)
        ]).T  # shape (Nx, 3)
    
    Theta.append(Theta_snapshot)
    ut_flat.append(ut_snapshot)
    
Theta = np.vstack(Theta)       # shape (Nt*Nx, 3)
ut_flat = np.hstack(ut_flat)   # flatten to 1D (Nt*Nx,)
    
print("Feature matrix Θ shape:", Theta.shape)
print("Target vector u_t shape:", ut_flat.shape)
    
# === Normalize features for numerical stability ===
Theta_mean = Theta.mean(axis=0)
Theta_std = Theta.std(axis=0)
Theta_norm = (Theta - Theta_mean) / Theta_std
    
# === Sparse regression (LASSO) ===
lasso = Lasso(alpha=1e-4, fit_intercept=False)
lasso.fit(Theta_norm, ut_flat)
coeffs = lasso.coef_ / Theta_std  # un-normalize coefficients
    
print("\nRecovered PDE coefficients (Varied Initial Conditions):")
print(f"a (u_xx):     {coeffs[0]:.4f}")
print(f"b (u_xxxx):   {coeffs[1]:.4f}")
print(f"c (u*u_x):    {coeffs[2]:.4f}") # Updated label for the new term
    


In [ ]:
### FFT based SINDy on data with clean ks data

In [ ]:
with h5py.File("test_solving_euler_ks_clean.h5", "r") as f:
    u = np.array(f["u"])      # shape (Nt, Nx)
    x = np.array(f["x"])      # spatial grid
    t = np.array(f["t"])      # time grid
    
Nx = len(x)
Nt = len(t)
L = x[-1] - x[0] + (x[1]-x[0])
dx = x[1] - x[0]
dt = t[1] - t[0]
    
print(f"Loaded u(t,x) with shape {u.shape}, dx={dx:.3f}, dt={dt:.3f}")
    
    
# Define wavenumbers
k = 2 * np.pi * fftfreq(Nx, d=dx)
    
# === Compute temporal derivative u_t using finite difference ===
u_t = (u[2:] - u[:-2]) / (2 * dt)          # central difference in time
u_mid = u[1:-1]                            # align time dimension
    
# === Build feature library Θ(u) ===
# For each time snapshot, compute spatial derivatives
Theta = []
ut_flat = []
    
for snapshot, ut_snapshot in zip(u_mid, u_t):
    ux  = spectral_derivative(snapshot, k)
    uxx = spectral_second_derivative(snapshot, k)
    uxxxx = spectral_fourth_derivative(snapshot, k)
    
    # Feature terms
    Theta_snapshot = np.vstack([
            uxx,         # diffusion term
            uxxxx,       # hyperdiffusion
            snapshot * ux, # nonlinear advection term (u * ux)
        ]).T  # shape (Nx, 3)
    
    Theta.append(Theta_snapshot)
    ut_flat.append(ut_snapshot)
    
Theta = np.vstack(Theta)       # shape (Nt*Nx, 3)
ut_flat = np.hstack(ut_flat)   # flatten to 1D (Nt*Nx,)
    
print("Feature matrix Θ shape:", Theta.shape)
print("Target vector u_t shape:", ut_flat.shape)
    
# === Normalize features for numerical stability ===
Theta_mean = Theta.mean(axis=0)
Theta_std = Theta.std(axis=0)
Theta_norm = (Theta - Theta_mean) / Theta_std
    
# === Sparse regression (LASSO) ===
lasso = Lasso(alpha=1e-4, fit_intercept=False)
lasso.fit(Theta_norm, ut_flat)
coeffs = lasso.coef_ / Theta_std  # un-normalize coefficients
    
print("\nRecovered PDE coefficients (Ks clean):")
print(f"a (u_xx):     {coeffs[0]:.4f}")
print(f"b (u_xxxx):   {coeffs[1]:.4f}")
print(f"c (u*u_x):    {coeffs[2]:.4f}") # Updated label for the new term
    


### FFT based SINDy on data 2 (20,000) steps clean ks data

In [ ]:
with h5py.File("ks_data_dtt100.h5", "r") as f:
    u = np.array(f["u"])      # shape (Nt, Nx)
    x = np.array(f["x"])      # spatial grid
    t = np.array(f["t"])      # time grid
    
Nx = len(x)
Nt = len(t)
L = x[-1] - x[0] + (x[1]-x[0])
dx = x[1] - x[0]
dt = t[1] - t[0]
    
print(f"Loaded u(t,x) with shape {u.shape}, dx={dx:.3f}, dt={dt:.3f}")
    
    
# Define wavenumbers
k = 2 * np.pi * fftfreq(Nx, d=dx)
    
# === Compute temporal derivative u_t using finite difference ===
u_t = (u[2:] - u[:-2]) / (2 * dt)          # central difference in time
u_mid = u[1:-1]                            # align time dimension
    
# === Build feature library Θ(u) ===
# For each time snapshot, compute spatial derivatives
Theta = []
ut_flat = []
    
for snapshot, ut_snapshot in zip(u_mid, u_t):
    ux  = spectral_derivative(snapshot, k)
    uxx = spectral_second_derivative(snapshot, k)
    uxxxx = spectral_fourth_derivative(snapshot, k)
    
    # Feature terms
    Theta_snapshot = np.vstack([
            uxx,         # diffusion term
            uxxxx,       # hyperdiffusion
            snapshot * ux, # nonlinear advection term (u * ux)
        ]).T  # shape (Nx, 3)
    
    Theta.append(Theta_snapshot)
    ut_flat.append(ut_snapshot)
    
Theta = np.vstack(Theta)       # shape (Nt*Nx, 3)
ut_flat = np.hstack(ut_flat)   # flatten to 1D (Nt*Nx,)
    
print("Feature matrix Θ shape:", Theta.shape)
print("Target vector u_t shape:", ut_flat.shape)
    
# === Normalize features for numerical stability ===
Theta_mean = Theta.mean(axis=0)
Theta_std = Theta.std(axis=0)
Theta_norm = (Theta - Theta_mean) / Theta_std
    
# === Sparse regression (LASSO) ===
lasso = Lasso(alpha=1e-4, fit_intercept=False)
lasso.fit(Theta_norm, ut_flat)
coeffs = lasso.coef_ / Theta_std  # un-normalize coefficients
    
print("\nRecovered PDE coefficients (Ks clean):")
print(f"a (u_xx):     {coeffs[0]:.4f}")
print(f"b (u_xxxx):   {coeffs[1]:.4f}")
print(f"c (u*u_x):    {coeffs[2]:.4f}") # Updated label for the new term
    


###### LASSO tries to minimize:
‖u_t − Θξ‖² + α‖ξ‖₁


But when Θ is unnormalized:

the columns are correlated

the magnitudes differ by orders of magnitude

LASSO distributes penalty unevenly

The optimizer ends up setting: a ≈ b ≈ c ≈ –0.96


###### We validated two SINDy pipelines. Code 1—using FFT derivatives and unnormalized regression—fails because spectral differentiation amplifies numerical noise and the feature matrix is ill-conditioned. This causes the three coefficients to collapse toward a common biased value around –0.96. Code 2 uses finite differences, normalization, subsampling, and a full SINDy library, which makes the regression well-conditioned. It recovers the correct KS coefficients with high R² and low RMSE. Therefore all noisy-data and denoising experiments use Code 2 as the reliable baseline."

### Finite Difference Derivatives sindy (periodic boundary conditions)

In [ ]:
import h5py, numpy as np
from sklearn.linear_model import Lasso

# ===== single baseline dataset =====
DATA_FILE = "ks_data_dtt100.h5"   # dt = 0.1 (your best dataset)

# ---- load ----
with h5py.File(DATA_FILE, "r") as f:
    u = np.array(f["u"], dtype=np.float32)
    x = np.array(f["x"], dtype=np.float32)
    t = np.array(f["t"], dtype=np.float32)

dx, dt = float(x[1] - x[0]), float(t[1] - t[0])
print(f"Loaded: u.shape={u.shape}, dx={dx:.4f}, dt={dt:.4f}")

# ---- finite difference operators ----
def d1(U): return (np.roll(U,-1,1)-np.roll(U,1,1)) / (2*dx)
def d2(U): return (np.roll(U,-1,1)-2*U+np.roll(U,1,1)) / (dx**2)
def d4(U):
    return (np.roll(U,-2,1)-4*np.roll(U,-1,1)+6*U-4*np.roll(U,1,1)+np.roll(U,2,1)) / (dx**4)

# ---- derivatives ----
U_t = (u[1:] - u[:-1]) / dt
U   = u[:-1]
U_x, U_xx, U_xxxx = d1(U), d2(U), d4(U)
U2_x = d1(U**2)  # nonlinear (u^2)_x term

# ---- flatten & sample ----
N_total = U_t.size
N_sample = 60000
idx = np.random.choice(N_total, N_sample, replace=False)
Theta = np.column_stack([
    np.ones_like(U).ravel(),
    U.ravel(),
    U_x.ravel(),
    U_xx.ravel(),
    U_xxxx.ravel(),
    U2_x.ravel()
])[idx]
y = U_t.ravel()[idx]
names = ["1", "u", "u_x", "u_xx", "u_xxxx", "(u^2)_x"]

# ---- normalize ----
Theta_mean, Theta_std = Theta.mean(0), Theta.std(0) + 1e-8
Theta_n = (Theta - Theta_mean) / Theta_std

# ---- Lasso regression ----
lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000, tol=1e-6)
lasso.fit(Theta_n, y)
coef = lasso.coef_ / Theta_std  # unscale

# ---- extract recovered coefficients ----
a = coef[names.index("u_xx")]
b = coef[names.index("u_xxxx")]
c_phys = coef[names.index("(u^2)_x")]

# ---- true coefficients ----
a_true, b_true, c_true = -1.0, -1.0, -0.5

# ---- metrics ----
Δa = a - a_true
Δb = b - b_true
Δc = c_phys - c_true

rel_a = abs(Δa / a_true)
rel_b = abs(Δb / b_true)
rel_c = abs(Δc / c_true)

L1_recov = abs(a) + abs(b) + abs(c_phys)
L1_true  = abs(a_true) + abs(b_true) + abs(c_true)
L1_err   = abs(L1_recov - L1_true)

# ---- report ----
print("\n==================== PDE DISCOVERY RESULTS ====================")
print("Discovered PDE (Euler–SINDy form):")
for n, c in zip(names, coef):
    print(f"{n:>8s} : {c:+.4f}")

print("\nRecovered KS coefficients:")
print(f"a (u_xx term)   = {a:+.4f}")
print(f"b (u_xxxx term) = {b:+.4f}")
print(f"c ((u^2)_x term)= {c_phys:+.4f}")

print("\n------------------ ERROR METRICS ------------------")
print(f"Δa (error) = {Δa:+.5f} | Relative = {rel_a*100:.3f}%")
print(f"Δb (error) = {Δb:+.5f} | Relative = {rel_b*100:.3f}%")
print(f"Δc (error) = {Δc:+.5f} | Relative = {rel_c*100:.3f}%")
print(f"L1 norm recovered = {L1_recov:.4f} (true = {L1_true:.4f}) | ΔL1 = {L1_err:.4f}")
print("===============================================================")


In [ ]:
import h5py
import numpy as np
from sklearn.linear_model import Lasso

# ===== Load data =====
DATA_FILE = "ks_data_dtt100.h5"

with h5py.File(DATA_FILE, "r") as f:
    u = np.array(f["u"], dtype=np.float32)
    x = np.array(f["x"], dtype=np.float32)
    t = np.array(f["t"], dtype=np.float32)

dx = float(x[1] - x[0])
dt = float(t[1] - t[0])
print(f"Loaded: u.shape={u.shape}, dx={dx:.4f}, dt={dt:.4f}")

# ===== Finite difference operators =====
def d1(U): 
    return (np.roll(U, -1, 1) - np.roll(U, 1, 1)) / (2*dx)

def d2(U): 
    return (np.roll(U, -1, 1) - 2*U + np.roll(U, 1, 1)) / (dx**2)

def d4(U):
    return (np.roll(U, -2, 1) - 4*np.roll(U, -1, 1) + 6*U - 
            4*np.roll(U, 1, 1) + np.roll(U, 2, 1)) / (dx**4)

# ===== Compute derivatives =====
U_t = (u[1:] - u[:-1]) / dt
U = u[:-1]
U_x = d1(U)
U_xx = d2(U)
U_xxxx = d4(U)
U2_x = d1(U**2)  # (u²)_x term

# ===== Build feature matrix (ALL data points) =====
Theta = np.column_stack([
    np.ones_like(U).ravel(),
    U.ravel(),
    U_x.ravel(),
    U_xx.ravel(),
    U_xxxx.ravel(),
    U2_x.ravel()
])
y = U_t.ravel()
names = ["1", "u", "u_x", "u_xx", "u_xxxx", "(u^2)_x"]

print(f"Using ALL data: {len(y):,} points")

# ===== Normalize features =====
Theta_mean = Theta.mean(0)
Theta_std = Theta.std(0) + 1e-8
Theta_n = (Theta - Theta_mean) / Theta_std

# ===== Lasso regression =====
lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000, tol=1e-6)
lasso.fit(Theta_n, y)
coef = lasso.coef_ / Theta_std  # un-normalize

# ===== Extract KS coefficients =====
a = coef[names.index("u_xx")]
b = coef[names.index("u_xxxx")]
c_phys = coef[names.index("(u^2)_x")]

# ===== True coefficients =====
a_true, b_true, c_true = -1.0, -1.0, -0.5

# ===== Compute errors =====
Δa = a - a_true
Δb = b - b_true
Δc = c_phys - c_true

rel_a = abs(Δa / a_true)
rel_b = abs(Δb / b_true)
rel_c = abs(Δc / c_true)

L1_recov = abs(a) + abs(b) + abs(c_phys)
L1_true = abs(a_true) + abs(b_true) + abs(c_true)
L1_err = abs(L1_recov - L1_true)

# ===== Print results =====
print("\n" + "="*60)
print("PDE DISCOVERY RESULTS")
print("="*60)
print("Discovered PDE (Euler-SINDy form):")
for n, c in zip(names, coef):
    print(f"{n:>8s} : {c:+.4f}")

print("\nRecovered KS coefficients:")
print(f"a (u_xx term)   = {a:+.4f}")
print(f"b (u_xxxx term) = {b:+.4f}")
print(f"c ((u^2)_x term)= {c_phys:+.4f}")

print("\n" + "-"*60)
print("ERROR METRICS")
print("-"*60)
print(f"Δa (error) = {Δa:+.5f} | Relative = {rel_a*100:.3f}%")
print(f"Δb (error) = {Δb:+.5f} | Relative = {rel_b*100:.3f}%")
print(f"Δc (error) = {Δc:+.5f} | Relative = {rel_c*100:.3f}%")
print(f"L1 norm: {L1_recov:.4f} (true: {L1_true:.4f}) | ΔL1 = {L1_err:.4f}")
print("="*60)

In [ ]:
test_solving_euler_ks_clean

Finite Difference Derivatives sindy (periodic boundary conditions) on 2000 step clean ks data

In [ ]:
import h5py
import numpy as np
from sklearn.linear_model import Lasso

# ===== Load data =====
DATA_FILE = "test_solving_euler_ks_clean.h5"

with h5py.File(DATA_FILE, "r") as f:
    u = np.array(f["u"], dtype=np.float32)
    x = np.array(f["x"], dtype=np.float32)
    t = np.array(f["t"], dtype=np.float32)

dx = float(x[1] - x[0])
dt = float(t[1] - t[0])
print(f"Loaded: u.shape={u.shape}, dx={dx:.4f}, dt={dt:.4f}")

# ===== Finite difference operators =====
def d1(U): 
    return (np.roll(U, -1, 1) - np.roll(U, 1, 1)) / (2*dx)

def d2(U): 
    return (np.roll(U, -1, 1) - 2*U + np.roll(U, 1, 1)) / (dx**2)

def d4(U):
    return (np.roll(U, -2, 1) - 4*np.roll(U, -1, 1) + 6*U - 
            4*np.roll(U, 1, 1) + np.roll(U, 2, 1)) / (dx**4)

# ===== Compute derivatives =====
U_t = (u[1:] - u[:-1]) / dt
U = u[:-1]
U_x = d1(U)
U_xx = d2(U)
U_xxxx = d4(U)
U2_x = d1(U**2)  # (u²)_x term

# ===== Build feature matrix (ALL data points) =====
Theta = np.column_stack([
    np.ones_like(U).ravel(),
    U.ravel(),
    U_x.ravel(),
    U_xx.ravel(),
    U_xxxx.ravel(),
    U2_x.ravel()
])
y = U_t.ravel()
names = ["1", "u", "u_x", "u_xx", "u_xxxx", "(u^2)_x"]

print(f"Using ALL data: {len(y):,} points")

# ===== Normalize features =====
Theta_mean = Theta.mean(0)
Theta_std = Theta.std(0) + 1e-8
Theta_n = (Theta - Theta_mean) / Theta_std

# ===== Lasso regression =====
lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000, tol=1e-6)
lasso.fit(Theta_n, y)
coef = lasso.coef_ / Theta_std  # un-normalize

# ===== Extract KS coefficients =====
a = coef[names.index("u_xx")]
b = coef[names.index("u_xxxx")]
c_phys = coef[names.index("(u^2)_x")]

# ===== True coefficients =====
a_true, b_true, c_true = -1.0, -1.0, -0.5

# ===== Compute errors =====
Δa = a - a_true
Δb = b - b_true
Δc = c_phys - c_true

rel_a = abs(Δa / a_true)
rel_b = abs(Δb / b_true)
rel_c = abs(Δc / c_true)

L1_recov = abs(a) + abs(b) + abs(c_phys)
L1_true = abs(a_true) + abs(b_true) + abs(c_true)
L1_err = abs(L1_recov - L1_true)

# ===== Print results =====
print("\n" + "="*60)
print("PDE DISCOVERY RESULTS")
print("="*60)
print("Discovered PDE (Euler-SINDy form):")
for n, c in zip(names, coef):
    print(f"{n:>8s} : {c:+.4f}")

print("\nRecovered KS coefficients:")
print(f"a (u_xx term)   = {a:+.4f}")
print(f"b (u_xxxx term) = {b:+.4f}")
print(f"c ((u^2)_x term)= {c_phys:+.4f}")

print("\n" + "-"*60)
print("ERROR METRICS")
print("-"*60)
print(f"Δa (error) = {Δa:+.5f} | Relative = {rel_a*100:.3f}%")
print(f"Δb (error) = {Δb:+.5f} | Relative = {rel_b*100:.3f}%")
print(f"Δc (error) = {Δc:+.5f} | Relative = {rel_c*100:.3f}%")
print(f"L1 norm: {L1_recov:.4f} (true: {L1_true:.4f}) | ΔL1 = {L1_err:.4f}")
print("="*60)

Finite Difference Derivatives sindy on 2000 step spatial shift data

In [ ]:
import h5py
import numpy as np
from sklearn.linear_model import Lasso

# ===== Load data =====
DATA_FILE = "test_solving_euler_ks_spatial_shift.h5"

with h5py.File(DATA_FILE, "r") as f:
    u = np.array(f["u"], dtype=np.float32)
    x = np.array(f["x"], dtype=np.float32)
    t = np.array(f["t"], dtype=np.float32)

dx = float(x[1] - x[0])
dt = float(t[1] - t[0])
print(f"Loaded: u.shape={u.shape}, dx={dx:.4f}, dt={dt:.4f}")

# ===== Finite difference operators =====
def d1(U): 
    return (np.roll(U, -1, 1) - np.roll(U, 1, 1)) / (2*dx)

def d2(U): 
    return (np.roll(U, -1, 1) - 2*U + np.roll(U, 1, 1)) / (dx**2)

def d4(U):
    return (np.roll(U, -2, 1) - 4*np.roll(U, -1, 1) + 6*U - 
            4*np.roll(U, 1, 1) + np.roll(U, 2, 1)) / (dx**4)

# ===== Compute derivatives =====
U_t = (u[1:] - u[:-1]) / dt
U = u[:-1]
U_x = d1(U)
U_xx = d2(U)
U_xxxx = d4(U)
U2_x = d1(U**2)  # (u²)_x term

# ===== Build feature matrix (ALL data points) =====
Theta = np.column_stack([
    np.ones_like(U).ravel(),
    U.ravel(),
    U_x.ravel(),
    U_xx.ravel(),
    U_xxxx.ravel(),
    U2_x.ravel()
])
y = U_t.ravel()
names = ["1", "u", "u_x", "u_xx", "u_xxxx", "(u^2)_x"]

print(f"Using ALL data: {len(y):,} points")

# ===== Normalize features =====
Theta_mean = Theta.mean(0)
Theta_std = Theta.std(0) + 1e-8
Theta_n = (Theta - Theta_mean) / Theta_std

# ===== Lasso regression =====
lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000, tol=1e-6)
lasso.fit(Theta_n, y)
coef = lasso.coef_ / Theta_std  # un-normalize

# ===== Extract KS coefficients =====
a = coef[names.index("u_xx")]
b = coef[names.index("u_xxxx")]
c_phys = coef[names.index("(u^2)_x")]

# ===== True coefficients =====
a_true, b_true, c_true = -1.0, -1.0, -0.5

# ===== Compute errors =====
Δa = a - a_true
Δb = b - b_true
Δc = c_phys - c_true

rel_a = abs(Δa / a_true)
rel_b = abs(Δb / b_true)
rel_c = abs(Δc / c_true)

L1_recov = abs(a) + abs(b) + abs(c_phys)
L1_true = abs(a_true) + abs(b_true) + abs(c_true)
L1_err = abs(L1_recov - L1_true)

# ===== Print results =====
print("\n" + "="*60)
print("PDE DISCOVERY RESULTS")
print("="*60)
print("Discovered PDE (Euler-SINDy form):")
for n, c in zip(names, coef):
    print(f"{n:>8s} : {c:+.4f}")

print("\nRecovered KS coefficients:")
print(f"a (u_xx term)   = {a:+.4f}")
print(f"b (u_xxxx term) = {b:+.4f}")
print(f"c ((u^2)_x term)= {c_phys:+.4f}")

print("\n" + "-"*60)
print("ERROR METRICS")
print("-"*60)
print(f"Δa (error) = {Δa:+.5f} | Relative = {rel_a*100:.3f}%")
print(f"Δb (error) = {Δb:+.5f} | Relative = {rel_b*100:.3f}%")
print(f"Δc (error) = {Δc:+.5f} | Relative = {rel_c*100:.3f}%")
print(f"L1 norm: {L1_recov:.4f} (true: {L1_true:.4f}) | ΔL1 = {L1_err:.4f}")
print("="*60)

### Finite Difference Derivatives SINDY on data with different initial conditions (2000 step)

In [ ]:
import h5py
import numpy as np
from sklearn.linear_model import Lasso

# ===== Load data =====
DATA_FILE = "test_solving_euler_ks_varied_ic.h5"

with h5py.File(DATA_FILE, "r") as f:
    u = np.array(f["u"], dtype=np.float32)
    x = np.array(f["x"], dtype=np.float32)
    t = np.array(f["t"], dtype=np.float32)

dx = float(x[1] - x[0])
dt = float(t[1] - t[0])
print(f"Loaded: u.shape={u.shape}, dx={dx:.4f}, dt={dt:.4f}")

# ===== Finite difference operators =====
def d1(U): 
    return (np.roll(U, -1, 1) - np.roll(U, 1, 1)) / (2*dx)

def d2(U): 
    return (np.roll(U, -1, 1) - 2*U + np.roll(U, 1, 1)) / (dx**2)

def d4(U):
    return (np.roll(U, -2, 1) - 4*np.roll(U, -1, 1) + 6*U - 
            4*np.roll(U, 1, 1) + np.roll(U, 2, 1)) / (dx**4)

# ===== Compute derivatives =====
U_t = (u[1:] - u[:-1]) / dt
U = u[:-1]
U_x = d1(U)
U_xx = d2(U)
U_xxxx = d4(U)
U2_x = d1(U**2)  # (u²)_x term

# ===== Build feature matrix (ALL data points) =====
Theta = np.column_stack([
    np.ones_like(U).ravel(),
    U.ravel(),
    U_x.ravel(),
    U_xx.ravel(),
    U_xxxx.ravel(),
    U2_x.ravel()
])
y = U_t.ravel()
names = ["1", "u", "u_x", "u_xx", "u_xxxx", "(u^2)_x"]

print(f"Using ALL data: {len(y):,} points")

# ===== Normalize features =====
Theta_mean = Theta.mean(0)
Theta_std = Theta.std(0) + 1e-8
Theta_n = (Theta - Theta_mean) / Theta_std

# ===== Lasso regression =====
lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000, tol=1e-6)
lasso.fit(Theta_n, y)
coef = lasso.coef_ / Theta_std  # un-normalize

# ===== Extract KS coefficients =====
a = coef[names.index("u_xx")]
b = coef[names.index("u_xxxx")]
c_phys = coef[names.index("(u^2)_x")]

# ===== True coefficients =====
a_true, b_true, c_true = -1.0, -1.0, -0.5

# ===== Compute errors =====
Δa = a - a_true
Δb = b - b_true
Δc = c_phys - c_true

rel_a = abs(Δa / a_true)
rel_b = abs(Δb / b_true)
rel_c = abs(Δc / c_true)

L1_recov = abs(a) + abs(b) + abs(c_phys)
L1_true = abs(a_true) + abs(b_true) + abs(c_true)
L1_err = abs(L1_recov - L1_true)

# ===== Print results =====
print("\n" + "="*60)
print("PDE DISCOVERY RESULTS")
print("="*60)
print("Discovered PDE (Euler-SINDy form):")
for n, c in zip(names, coef):
    print(f"{n:>8s} : {c:+.4f}")

print("\nRecovered KS coefficients:")
print(f"a (u_xx term)   = {a:+.4f}")
print(f"b (u_xxxx term) = {b:+.4f}")
print(f"c ((u^2)_x term)= {c_phys:+.4f}")

print("\n" + "-"*60)
print("ERROR METRICS")
print("-"*60)
print(f"Δa (error) = {Δa:+.5f} | Relative = {rel_a*100:.3f}%")
print(f"Δb (error) = {Δb:+.5f} | Relative = {rel_b*100:.3f}%")
print(f"Δc (error) = {Δc:+.5f} | Relative = {rel_c*100:.3f}%")
print(f"L1 norm: {L1_recov:.4f} (true: {L1_true:.4f}) | ΔL1 = {L1_err:.4f}")
print("="*60)

Generate Additive Gaussian Noise

In [ ]:
import h5py
import numpy as np


CLEAN_FILE = "ks_data_dtt100.h5"      # or "test_solving_euler_ks_clean.h5"
SIGMAS = [0.005, 0.01, 0.03, 0.05, 0.08, 0.10]   # noise levels you want
OUT_PREFIX = "ks20k_gaussian_sigma_"     # output file prefix


# =====================================================
# LOAD CLEAN REGISTERED KS DATA
# =====================================================

with h5py.File(CLEAN_FILE, "r") as f:
    u_clean = np.array(f["u"], dtype=np.float32)
    x = np.array(f["x"], dtype=np.float32)
    t = np.array(f["t"], dtype=np.float32)
    attrs = dict(f.attrs)

print(f"Loaded clean dataset: u.shape = {u_clean.shape}")


# =====================================================
# FUNCTION: Add Gaussian Noise
# =====================================================

def add_gaussian_noise(u, sigma):
    """
    Add Gaussian noise with zero mean and std = sigma.
    Noise is added independently to every pixel.
    """
    noise = sigma * np.random.randn(*u.shape)
    return u + noise


# =====================================================
# GENERATE & SAVE NOISY DATASETS
# =====================================================

for sigma in SIGMAS:
    print(f"Generating σ = {sigma} dataset...")

    u_noisy = add_gaussian_noise(u_clean, sigma)

    out_filename = f"{OUT_PREFIX}{sigma:.3f}.h5"
    
    with h5py.File(out_filename, "w") as f:
        f.create_dataset("u", data=u_noisy)
        f.create_dataset("x", data=x)
        f.create_dataset("t", data=t)

        # store metadata
        for k, v in attrs.items():
            f.attrs[k] = v
        f.attrs["noise_type"] = "gaussian"
        f.attrs["sigma"] = sigma
    
    print(f"Saved: {out_filename}")

print("\nAll noisy datasets generated successfully!")


In [ ]:
import h5py
import numpy as np


CLEAN_FILE = "test_solving_euler_ks_clean.h5"      # or "test_solving_euler_ks_clean.h5"
SIGMAS = [0.005, 0.01, 0.03, 0.05, 0.08, 0.10]   # noise levels you want
OUT_PREFIX = "ks_gaussian_sigma_"     # output file prefix


# =====================================================
# LOAD CLEAN REGISTERED KS DATA
# =====================================================

with h5py.File(CLEAN_FILE, "r") as f:
    u_clean = np.array(f["u"], dtype=np.float32)
    x = np.array(f["x"], dtype=np.float32)
    t = np.array(f["t"], dtype=np.float32)
    attrs = dict(f.attrs)

print(f"Loaded clean dataset: u.shape = {u_clean.shape}")


# =====================================================
# FUNCTION: Add Gaussian Noise
# =====================================================

def add_gaussian_noise(u, sigma):
    """
    Add Gaussian noise with zero mean and std = sigma.
    Noise is added independently to every pixel.
    """
    noise = sigma * np.random.randn(*u.shape)
    return u + noise


# =====================================================
# GENERATE & SAVE NOISY DATASETS
# =====================================================

for sigma in SIGMAS:
    print(f"Generating σ = {sigma} dataset...")

    u_noisy = add_gaussian_noise(u_clean, sigma)

    out_filename = f"{OUT_PREFIX}{sigma:.3f}.h5"
    
    with h5py.File(out_filename, "w") as f:
        f.create_dataset("u", data=u_noisy)
        f.create_dataset("x", data=x)
        f.create_dataset("t", data=t)

        # store metadata
        for k, v in attrs.items():
            f.attrs[k] = v
        f.attrs["noise_type"] = "gaussian"
        f.attrs["sigma"] = sigma
    
    print(f"Saved: {out_filename}")

print("\nAll noisy datasets generated successfully!")


Finite diff deriavative on additive noise 

In [ ]:
import h5py
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import pandas as pd

# ===== Evaluation Metrics Functions =====
def compute_rmse(y_true, y_pred):
    """Root Mean Squared Error"""
    return np.sqrt(np.mean((y_true - y_pred)**2))

def compute_r2(y_true, y_pred):
    """R-squared (coefficient of determination)"""
    return r2_score(y_true, y_pred)

def compute_l1_norm(coeffs):
    """L1 norm of coefficient vector"""
    return np.sum(np.abs(coeffs))

def compute_relative_error(true_val, pred_val):
    """Relative error in percentage"""
    if true_val == 0:
        return 0.0 if pred_val == 0 else 100.0
    return abs((pred_val - true_val) / true_val) * 100

def compute_f1_score(true_coeffs, pred_coeffs, threshold=1e-3):
    """
    F1 score for term selection stability
    Treats coefficients above threshold as 'selected'
    """
    true_selected = np.abs(true_coeffs) > threshold
    pred_selected = np.abs(pred_coeffs) > threshold
    
    # True Positives, False Positives, False Negatives
    TP = np.sum(true_selected & pred_selected)
    FP = np.sum(~true_selected & pred_selected)
    FN = np.sum(true_selected & ~pred_selected)
    
    # Precision and Recall
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    
    # F1 Score
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0
    
    return f1


# ===== List of noise datasets =====
noise_files = [
    "test_solving_euler_ks_clean.h5",              # Clean (no noise)
    "ks_gaussian_sigma_0.005.h5",
    "ks_gaussian_sigma_0.010.h5",
    "ks_gaussian_sigma_0.030.h5",
    "ks_gaussian_sigma_0.050.h5",
    "ks_gaussian_sigma_0.080.h5",
    "ks_gaussian_sigma_0.100.h5"
]

noise_levels = [0.0, 0.005, 0.01, 0.03, 0.05, 0.08, 0.1]

# ===== Store results =====
results = []

# ===== True coefficients =====
a_true, b_true, c_true = -1.0, -1.0, -0.5
true_coeffs = np.array([a_true, b_true, c_true])

# ===== Loop through each dataset =====
for i, (file, sigma) in enumerate(zip(noise_files, noise_levels)):
    
    print("\n" + "="*80)
    print(f"TESTING NOISE LEVEL σ = {sigma} (FINITE DIFFERENCE)")
    print("="*80)
    
    # ---- Load data ----
    with h5py.File(file, "r") as f:
        u = np.array(f["u"], dtype=np.float32)
        x = np.array(f["x"], dtype=np.float32)
        t = np.array(f["t"], dtype=np.float32)
    
    dx = float(x[1] - x[0])
    dt = float(t[1] - t[0])
    print(f"Loaded: {file}")
    print(f"Shape: u.shape={u.shape}, dx={dx:.4f}, dt={dt:.4f}")
    
    # ---- Finite difference operators ----
    def d1(U): 
        return (np.roll(U, -1, 1) - np.roll(U, 1, 1)) / (2*dx)
    
    def d2(U): 
        return (np.roll(U, -1, 1) - 2*U + np.roll(U, 1, 1)) / (dx**2)
    
    def d4(U):
        return (np.roll(U, -2, 1) - 4*np.roll(U, -1, 1) + 6*U - 
                4*np.roll(U, 1, 1) + np.roll(U, 2, 1)) / (dx**4)
    
    # ---- Compute derivatives ----
    U_t = (u[1:] - u[:-1]) / dt
    U = u[:-1]
    U_x = d1(U)
    U_xx = d2(U)
    U_xxxx = d4(U)
    U2_x = d1(U**2)
    
    # ---- Build feature matrix ----
    Theta = np.column_stack([
        np.ones_like(U).ravel(),
        U.ravel(),
        U_x.ravel(),
        U_xx.ravel(),
        U_xxxx.ravel(),
        U2_x.ravel()
    ])
    y = U_t.ravel()
    names = ["1", "u", "u_x", "u_xx", "u_xxxx", "(u^2)_x"]
    
    print(f"Feature matrix Θ shape: {Theta.shape}")
    print(f"Target vector u_t shape: {y.shape}")
    print(f"Using all data: {len(y):,} points")
    
    # ---- Normalize ----
    Theta_mean = Theta.mean(0)
    Theta_std = Theta.std(0) + 1e-8
    Theta_n = (Theta - Theta_mean) / Theta_std
    
    # ---- Lasso regression ----
    lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000, tol=1e-6)
    lasso.fit(Theta_n, y)
    coef = lasso.coef_ / Theta_std
    
    # ---- Extract coefficients ----
    a = coef[names.index("u_xx")]
    b = coef[names.index("u_xxxx")]
    c_phys = coef[names.index("(u^2)_x")]
    
    pred_coeffs = np.array([a, b, c_phys])
    
    # ---- Predict u_t using recovered coefficients ----
    y_pred = Theta @ coef
    
    # ==================== COMPUTE ALL METRICS ====================
    
    # 1. RMSE
    rmse = compute_rmse(y, y_pred)
    
    # 2. R² Score
    r2 = compute_r2(y, y_pred)
    
    # 3. L1 Norm
    l1_recovered = compute_l1_norm(pred_coeffs)
    l1_true = compute_l1_norm(true_coeffs)
    l1_error = abs(l1_recovered - l1_true)
    
    # 4. Relative Coefficient Errors
    rel_a = compute_relative_error(a_true, a)
    rel_b = compute_relative_error(b_true, b)
    rel_c = compute_relative_error(c_true, c_phys)
    mean_rel_error = (rel_a + rel_b + rel_c) / 3
    
    # 5. F1 Score (term selection stability)
    f1 = compute_f1_score(true_coeffs, pred_coeffs, threshold=1e-3)
    
    # 6. Individual coefficient errors (absolute)
    abs_error_a = abs(a - a_true)
    abs_error_b = abs(b - b_true)
    abs_error_c = abs(c_phys - c_true)
    
    # ---- Store results ----
    results.append({
        'sigma': sigma,
        'file': file,
        # Recovered coefficients
        'a': a,
        'b': b,
        'c': c_phys,
        # Relative errors per coefficient
        'rel_a': rel_a,
        'rel_b': rel_b,
        'rel_c': rel_c,
        # Absolute errors per coefficient
        'abs_error_a': abs_error_a,
        'abs_error_b': abs_error_b,
        'abs_error_c': abs_error_c,
        # Summary metrics
        'mean_rel_error': mean_rel_error,
        'rmse': rmse,
        'r2': r2,
        'l1_recovered': l1_recovered,
        'l1_true': l1_true,
        'l1_error': l1_error,
        'f1_score': f1
    })
    
    # ---- Print results ----
    print("\n" + "-"*80)
    print("RECOVERED PDE COEFFICIENTS:")
    print("-"*80)
    print(f"  a (u_xx):   {a:+.6f}  (true: {a_true:+.2f}) | Abs Error: {abs_error_a:.6f} | Rel Error: {rel_a:.2f}%")
    print(f"  b (u_xxxx): {b:+.6f}  (true: {b_true:+.2f}) | Abs Error: {abs_error_b:.6f} | Rel Error: {rel_b:.2f}%")
    print(f"  c ((u²)_x): {c_phys:+.6f}  (true: {c_true:+.2f}) | Abs Error: {abs_error_c:.6f} | Rel Error: {rel_c:.2f}%")
    
    print("\n" + "-"*80)
    print("EVALUATION METRICS:")
    print("-"*80)
    print(f"  RMSE:                {rmse:.6f}")
    print(f"  R² Score:            {r2:.6f}")
    print(f"  L1 Norm (recovered): {l1_recovered:.6f}")
    print(f"  L1 Norm (true):      {l1_true:.6f}")
    print(f"  L1 Error:            {l1_error:.6f}")
    print(f"  Mean Rel Error:      {mean_rel_error:.2f}%")
    print(f"  F1 Score:            {f1:.6f}")


# ===== SUMMARY TABLE 1: RECOVERED COEFFICIENTS =====
print("\n\n" + "="*100)
print("SUMMARY TABLE 1: RECOVERED PDE COEFFICIENTS (FINITE DIFFERENCE)")
print("="*100)
print(f"{'Sigma':>8} {'a (u_xx)':>12} {'b (u_xxxx)':>12} {'c ((u²)_x)':>12} {'Rel Err a':>12} {'Rel Err b':>12} {'Rel Err c':>12}")
print("-"*100)

for r in results:
    print(f"{r['sigma']:>8.3f} {r['a']:>12.6f} {r['b']:>12.6f} {r['c']:>12.6f} "
          f"{r['rel_a']:>11.2f}% {r['rel_b']:>11.2f}% {r['rel_c']:>11.2f}%")

print("="*100)
print(f"{'TRUE:':>8} {a_true:>12.6f} {b_true:>12.6f} {c_true:>12.6f}")
print("="*100)


# ===== SUMMARY TABLE 2: ALL EVALUATION METRICS =====
print("\n\n" + "="*120)
print("SUMMARY TABLE 2: COMPREHENSIVE EVALUATION METRICS (FINITE DIFFERENCE)")
print("="*120)
print(f"{'Sigma':>8} {'RMSE':>12} {'R²':>12} {'L1 Error':>12} {'Mean Rel Err':>15} {'F1 Score':>12}")
print("-"*120)

for r in results:
    print(f"{r['sigma']:>8.3f} {r['rmse']:>12.6f} {r['r2']:>12.6f} {r['l1_error']:>12.6f} "
          f"{r['mean_rel_error']:>14.2f}% {r['f1_score']:>12.6f}")

print("="*120)


# ===== SUMMARY TABLE 3: DETAILED COEFFICIENT ERRORS =====
print("\n\n" + "="*120)
print("SUMMARY TABLE 3: ABSOLUTE COEFFICIENT ERRORS (FINITE DIFFERENCE)")
print("="*120)
print(f"{'Sigma':>8} {'|Δa|':>12} {'|Δb|':>12} {'|Δc|':>12} {'Mean Abs Err':>15}")
print("-"*120)

for r in results:
    mean_abs_err = (r['abs_error_a'] + r['abs_error_b'] + r['abs_error_c']) / 3
    print(f"{r['sigma']:>8.3f} {r['abs_error_a']:>12.6f} {r['abs_error_b']:>12.6f} {r['abs_error_c']:>12.6f} "
          f"{mean_abs_err:>15.6f}")

print("="*120)


# ===== FIND BEST RESULTS =====
print("\n\n" + "="*80)
print("BEST RESULTS (FINITE DIFFERENCE SINDY)")
print("="*80)

best_rmse = min(results, key=lambda x: x['rmse'])
best_r2 = max(results, key=lambda x: x['r2'])
best_l1 = min(results, key=lambda x: x['l1_error'])
best_rel = min(results, key=lambda x: x['mean_rel_error'])
best_f1 = max(results, key=lambda x: x['f1_score'])

print(f"\n    BEST RMSE:            σ = {best_rmse['sigma']:.3f}  |  RMSE = {best_rmse['rmse']:.6f}")
print(f"    BEST R² Score:        σ = {best_r2['sigma']:.3f}  |  R² = {best_r2['r2']:.6f}")
print(f"    BEST L1 Error:        σ = {best_l1['sigma']:.3f}  |  L1 Error = {best_l1['l1_error']:.6f}")
print(f"    BEST Mean Rel Error:  σ = {best_rel['sigma']:.3f}  |  Mean Rel Error = {best_rel['mean_rel_error']:.2f}%")
print(f"    BEST F1 Score:        σ = {best_f1['sigma']:.3f}  |  F1 Score = {best_f1['f1_score']:.6f}")

print("\n" + "="*80)
print(f"OVERALL BEST: σ = {best_rel['sigma']:.3f} (based on Mean Relative Error)")
print(f"  Coefficients: a={best_rel['a']:.6f}, b={best_rel['b']:.6f}, c={best_rel['c']:.6f}")
print(f"  Mean Relative Error: {best_rel['mean_rel_error']:.2f}%")
print(f"  RMSE: {best_rel['rmse']:.6f}")
print(f"  R²: {best_rel['r2']:.6f}")
print(f"  F1 Score: {best_rel['f1_score']:.6f}")
print("="*80)


# ===== SAVE RESULTS TO CSV =====
df = pd.DataFrame(results)
df.to_csv('finite_diff_sindy_results.csv', index=False)
print("\n    Results saved to: finite_diff_sindy_results.csv")

FFT sindy with additive noise 

In [ ]:
import h5py
import numpy as np
from scipy.fft import fft, ifft, fftfreq
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import pandas as pd

# ===== Spectral derivative functions =====
def spectral_derivative(u_snapshot, k):
    """First derivative using FFT"""
    u_hat = fft(u_snapshot)
    du_hat = 1j * k * u_hat
    return np.real(ifft(du_hat))

def spectral_second_derivative(u_snapshot, k):
    """Second derivative using FFT"""
    u_hat = fft(u_snapshot)
    d2u_hat = -(k**2) * u_hat
    return np.real(ifft(d2u_hat))

def spectral_fourth_derivative(u_snapshot, k):
    """Fourth derivative using FFT"""
    u_hat = fft(u_snapshot)
    d4u_hat = (k**4) * u_hat
    return np.real(ifft(d4u_hat))


# ===== Evaluation Metrics Functions =====
def compute_rmse(y_true, y_pred):
    """Root Mean Squared Error"""
    return np.sqrt(np.mean((y_true - y_pred)**2))

def compute_r2(y_true, y_pred):
    """R-squared (coefficient of determination)"""
    return r2_score(y_true, y_pred)

def compute_l1_norm(coeffs):
    """L1 norm of coefficient vector"""
    return np.sum(np.abs(coeffs))

def compute_relative_error(true_val, pred_val):
    """Relative error in percentage"""
    if true_val == 0:
        return 0.0 if pred_val == 0 else 100.0
    return abs((pred_val - true_val) / true_val) * 100

def compute_f1_score(true_coeffs, pred_coeffs, threshold=1e-3):
    """
    F1 score for term selection stability
    Treats coefficients above threshold as 'selected'
    """
    true_selected = np.abs(true_coeffs) > threshold
    pred_selected = np.abs(pred_coeffs) > threshold
    
    # True Positives, False Positives, False Negatives
    TP = np.sum(true_selected & pred_selected)
    FP = np.sum(~true_selected & pred_selected)
    FN = np.sum(true_selected & ~pred_selected)
    
    # Precision and Recall
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    
    # F1 Score
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0
    
    return f1


# ===== List of noise datasets =====
noise_files = [
    "test_solving_euler_ks_clean.h5",              # Clean (no noise)
    "ks_gaussian_sigma_0.005.h5",
    "ks_gaussian_sigma_0.010.h5",
    "ks_gaussian_sigma_0.030.h5",
    "ks_gaussian_sigma_0.050.h5",
    "ks_gaussian_sigma_0.080.h5",
    "ks_gaussian_sigma_0.100.h5"
]

noise_levels = [0.0, 0.005, 0.01, 0.03, 0.05, 0.08, 0.1]

# ===== True coefficients (conservative form) =====
a_true, b_true, c_true = -1.0, -1.0, -1.0
true_coeffs = np.array([a_true, b_true, c_true])

# ===== Store results =====
results = []

# ===== Loop through each dataset =====
for file, sigma in zip(noise_files, noise_levels):
    
    print("\n" + "="*80)
    print(f"TESTING NOISE LEVEL σ = {sigma} (FFT/SPECTRAL)")
    print("="*80)
    
    # ---- Load data ----
    with h5py.File(file, "r") as f:
        u = np.array(f["u"])
        x = np.array(f["x"])
        t = np.array(f["t"])
    
    Nx = len(x)
    Nt = len(t)
    L = x[-1] - x[0] + (x[1] - x[0])
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    
    print(f"Loaded: {file}")
    print(f"Shape: u.shape={u.shape}, dx={dx:.3f}, dt={dt:.3f}")
    
    # ---- Define wavenumbers ----
    k = 2 * np.pi * fftfreq(Nx, d=dx)
    
    # ---- Compute temporal derivative (central difference) ----
    u_t = (u[2:] - u[:-2]) / (2 * dt)
    u_mid = u[1:-1]
    
    # ---- Build feature library using FFT ----
    Theta = []
    ut_flat = []
    
    for snapshot, ut_snapshot in zip(u_mid, u_t):
        ux = spectral_derivative(snapshot, k)
        uxx = spectral_second_derivative(snapshot, k)
        uxxxx = spectral_fourth_derivative(snapshot, k)
        
        # Feature terms: [u_xx, u_xxxx, u*u_x]
        Theta_snapshot = np.vstack([
            uxx,
            uxxxx,
            snapshot * ux
        ]).T  # shape (Nx, 3)
        
        Theta.append(Theta_snapshot)
        ut_flat.append(ut_snapshot)
    
    Theta = np.vstack(Theta)
    ut_flat = np.hstack(ut_flat)
    
    print(f"Feature matrix Θ shape: {Theta.shape}")
    print(f"Target vector u_t shape: {ut_flat.shape}")
    
    # ---- Normalize features ----
    Theta_mean = Theta.mean(axis=0)
    Theta_std = Theta.std(axis=0)
    Theta_norm = (Theta - Theta_mean) / Theta_std
    
    # ---- Lasso regression ----
    lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000)
    lasso.fit(Theta_norm, ut_flat)
    coeffs = lasso.coef_ / Theta_std
    
    # ---- Extract coefficients ----
    a = coeffs[0]  # u_xx
    b = coeffs[1]  # u_xxxx
    c = coeffs[2]  # u*u_x
    
    pred_coeffs = np.array([a, b, c])
    
    # ---- Predict u_t using recovered coefficients ----
    u_t_pred = Theta @ coeffs
    
    # ==================== COMPUTE ALL METRICS ====================
    
    # 1. RMSE
    rmse = compute_rmse(ut_flat, u_t_pred)
    
    # 2. R² Score
    r2 = compute_r2(ut_flat, u_t_pred)
    
    # 3. L1 Norm
    l1_recovered = compute_l1_norm(pred_coeffs)
    l1_true = compute_l1_norm(true_coeffs)
    l1_error = abs(l1_recovered - l1_true)
    
    # 4. Relative Coefficient Errors
    rel_a = compute_relative_error(a_true, a)
    rel_b = compute_relative_error(b_true, b)
    rel_c = compute_relative_error(c_true, c)
    mean_rel_error = (rel_a + rel_b + rel_c) / 3
    
    # 5. F1 Score (term selection stability)
    f1 = compute_f1_score(true_coeffs, pred_coeffs, threshold=1e-3)
    
    # 6. Individual coefficient errors (absolute)
    abs_error_a = abs(a - a_true)
    abs_error_b = abs(b - b_true)
    abs_error_c = abs(c - c_true)
    
    # ---- Store results ----
    results.append({
        'sigma': sigma,
        'file': file,
        # Recovered coefficients
        'a': a,
        'b': b,
        'c': c,
        # Relative errors per coefficient
        'rel_a': rel_a,
        'rel_b': rel_b,
        'rel_c': rel_c,
        # Absolute errors per coefficient
        'abs_error_a': abs_error_a,
        'abs_error_b': abs_error_b,
        'abs_error_c': abs_error_c,
        # Summary metrics
        'mean_rel_error': mean_rel_error,
        'rmse': rmse,
        'r2': r2,
        'l1_recovered': l1_recovered,
        'l1_true': l1_true,
        'l1_error': l1_error,
        'f1_score': f1
    })
    
    # ---- Print results ----
    print("\n" + "-"*80)
    print("RECOVERED PDE COEFFICIENTS:")
    print("-"*80)
    print(f"  a (u_xx):   {a:+.6f}  (true: {a_true:+.2f}) | Abs Error: {abs_error_a:.6f} | Rel Error: {rel_a:.2f}%")
    print(f"  b (u_xxxx): {b:+.6f}  (true: {b_true:+.2f}) | Abs Error: {abs_error_b:.6f} | Rel Error: {rel_b:.2f}%")
    print(f"  c (u*u_x):  {c:+.6f}  (true: {c_true:+.2f}) | Abs Error: {abs_error_c:.6f} | Rel Error: {rel_c:.2f}%")
    
    print("\n" + "-"*80)
    print("EVALUATION METRICS:")
    print("-"*80)
    print(f"  RMSE:                {rmse:.6f}")
    print(f"  R² Score:            {r2:.6f}")
    print(f"  L1 Norm (recovered): {l1_recovered:.6f}")
    print(f"  L1 Norm (true):      {l1_true:.6f}")
    print(f"  L1 Error:            {l1_error:.6f}")
    print(f"  Mean Rel Error:      {mean_rel_error:.2f}%")
    print(f"  F1 Score:            {f1:.6f}")


# ===== SUMMARY TABLE 1: RECOVERED COEFFICIENTS =====
print("\n\n" + "="*100)
print("SUMMARY TABLE 1: RECOVERED PDE COEFFICIENTS (FFT/SPECTRAL)")
print("="*100)
print(f"{'Sigma':>8} {'a (u_xx)':>12} {'b (u_xxxx)':>12} {'c (u*u_x)':>12} {'Rel Err a':>12} {'Rel Err b':>12} {'Rel Err c':>12}")
print("-"*100)

for r in results:
    print(f"{r['sigma']:>8.3f} {r['a']:>12.6f} {r['b']:>12.6f} {r['c']:>12.6f} "
          f"{r['rel_a']:>11.2f}% {r['rel_b']:>11.2f}% {r['rel_c']:>11.2f}%")

print("="*100)
print(f"{'TRUE:':>8} {a_true:>12.6f} {b_true:>12.6f} {c_true:>12.6f}")
print("="*100)


# ===== SUMMARY TABLE 2: ALL EVALUATION METRICS =====
print("\n\n" + "="*120)
print("SUMMARY TABLE 2: COMPREHENSIVE EVALUATION METRICS (FFT/SPECTRAL)")
print("="*120)
print(f"{'Sigma':>8} {'RMSE':>12} {'R²':>12} {'L1 Error':>12} {'Mean Rel Err':>15} {'F1 Score':>12}")
print("-"*120)

for r in results:
    print(f"{r['sigma']:>8.3f} {r['rmse']:>12.6f} {r['r2']:>12.6f} {r['l1_error']:>12.6f} "
          f"{r['mean_rel_error']:>14.2f}% {r['f1_score']:>12.6f}")

print("="*120)


# ===== SUMMARY TABLE 3: DETAILED COEFFICIENT ERRORS =====
print("\n\n" + "="*120)
print("SUMMARY TABLE 3: ABSOLUTE COEFFICIENT ERRORS (FFT/SPECTRAL)")
print("="*120)
print(f"{'Sigma':>8} {'|Δa|':>12} {'|Δb|':>12} {'|Δc|':>12} {'Mean Abs Err':>15}")
print("-"*120)

for r in results:
    mean_abs_err = (r['abs_error_a'] + r['abs_error_b'] + r['abs_error_c']) / 3
    print(f"{r['sigma']:>8.3f} {r['abs_error_a']:>12.6f} {r['abs_error_b']:>12.6f} {r['abs_error_c']:>12.6f} "
          f"{mean_abs_err:>15.6f}")

print("="*120)


# ===== FIND BEST RESULTS =====
print("\n\n" + "="*80)
print("BEST RESULTS (FFT/SPECTRAL SINDY)")
print("="*80)

best_rmse = min(results, key=lambda x: x['rmse'])
best_r2 = max(results, key=lambda x: x['r2'])
best_l1 = min(results, key=lambda x: x['l1_error'])
best_rel = min(results, key=lambda x: x['mean_rel_error'])
best_f1 = max(results, key=lambda x: x['f1_score'])

print(f"\n    BEST RMSE:            σ = {best_rmse['sigma']:.3f}  |  RMSE = {best_rmse['rmse']:.6f}")
print(f"    BEST R² Score:        σ = {best_r2['sigma']:.3f}  |  R² = {best_r2['r2']:.6f}")
print(f"    BEST L1 Error:        σ = {best_l1['sigma']:.3f}  |  L1 Error = {best_l1['l1_error']:.6f}")
print(f"    BEST Mean Rel Error:  σ = {best_rel['sigma']:.3f}  |  Mean Rel Error = {best_rel['mean_rel_error']:.2f}%")
print(f"    BEST F1 Score:        σ = {best_f1['sigma']:.3f}  |  F1 Score = {best_f1['f1_score']:.6f}")

print("\n" + "="*80)
print(f"OVERALL BEST: σ = {best_rel['sigma']:.3f} (based on Mean Relative Error)")
print(f"  Coefficients: a={best_rel['a']:.6f}, b={best_rel['b']:.6f}, c={best_rel['c']:.6f}")
print(f"  Mean Relative Error: {best_rel['mean_rel_error']:.2f}%")
print(f"  RMSE: {best_rel['rmse']:.6f}")
print(f"  R²: {best_rel['r2']:.6f}")
print(f"  F1 Score: {best_rel['f1_score']:.6f}")
print("="*80)


# ===== SAVE RESULTS TO CSV =====
df = pd.DataFrame(results)
df.to_csv('fft_sindy_results.csv', index=False)
print("\n    Results saved to: fft_sindy_results.csv")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 10

print("="*80)
print("SINDY METHODS COMPARISON: FFT vs FINITE DIFFERENCE")
print("="*80)

# ===== Load Results =====
print("\n📂 Loading results...")
try:
    fft_results = pd.read_csv('fft_sindy_results.csv')
    fd_results = pd.read_csv('finite_diff_sindy_results.csv')
    print("    Successfully loaded both result files")
except FileNotFoundError as e:
    print(f"    Error: {e}")
    print("Please run both enhanced scripts first to generate the CSV files.")
    exit()

# ===== Display Data Preview =====
print("\n" + "="*80)
print("DATA PREVIEW")
print("="*80)
print("\nFFT Results (first 3 rows):")
print(fft_results.head(3))
print("\nFinite Difference Results (first 3 rows):")
print(fd_results.head(3))

# ===== Compare Metrics =====
print("\n\n" + "="*80)
print("SIDE-BY-SIDE COMPARISON TABLE")
print("="*80)

comparison = pd.DataFrame({
    'Sigma': fft_results['sigma'],
    'FFT_a': fft_results['a'],
    'FD_a': fd_results['a'],
    'FFT_b': fft_results['b'],
    'FD_b': fd_results['b'],
    'FFT_c': fft_results['c'],
    'FD_c': fd_results['c'],
    'FFT_RMSE': fft_results['rmse'],
    'FD_RMSE': fd_results['rmse'],
    'FFT_R2': fft_results['r2'],
    'FD_R2': fd_results['r2'],
    'FFT_MeanRelErr': fft_results['mean_rel_error'],
    'FD_MeanRelErr': fd_results['mean_rel_error'],
    'FFT_F1': fft_results['f1_score'],
    'FD_F1': fd_results['f1_score']
})

print("\n1. RECOVERED COEFFICIENTS COMPARISON")
print("-"*80)
print(f"{'Sigma':>8} {'FFT a':>12} {'FD a':>12} {'FFT b':>12} {'FD b':>12} {'FFT c':>12} {'FD c':>12}")
print("-"*80)
for _, row in comparison.iterrows():
    print(f"{row['Sigma']:>8.3f} {row['FFT_a']:>12.6f} {row['FD_a']:>12.6f} "
          f"{row['FFT_b']:>12.6f} {row['FD_b']:>12.6f} {row['FFT_c']:>12.6f} {row['FD_c']:>12.6f}")

print("\n2. EVALUATION METRICS COMPARISON")
print("-"*80)
print(f"{'Sigma':>8} {'FFT RMSE':>12} {'FD RMSE':>12} {'FFT R²':>12} {'FD R²':>12} {'FFT F1':>12} {'FD F1':>12}")
print("-"*80)
for _, row in comparison.iterrows():
    print(f"{row['Sigma']:>8.3f} {row['FFT_RMSE']:>12.6f} {row['FD_RMSE']:>12.6f} "
          f"{row['FFT_R2']:>12.6f} {row['FD_R2']:>12.6f} {row['FFT_F1']:>12.6f} {row['FD_F1']:>12.6f}")

print("\n3. MEAN RELATIVE ERROR COMPARISON")
print("-"*80)
print(f"{'Sigma':>8} {'FFT MeanRelErr':>18} {'FD MeanRelErr':>18} {'Winner':>12}")
print("-"*80)
for _, row in comparison.iterrows():
    winner = "FFT" if row['FFT_MeanRelErr'] < row['FD_MeanRelErr'] else "FD"
    print(f"{row['Sigma']:>8.3f} {row['FFT_MeanRelErr']:>17.2f}% {row['FD_MeanRelErr']:>17.2f}% {winner:>12}")

# ===== Statistical Summary =====
print("\n\n" + "="*80)
print("STATISTICAL SUMMARY")
print("="*80)

metrics = ['rmse', 'r2', 'mean_rel_error', 'f1_score', 'l1_error']
metric_names = ['RMSE', 'R² Score', 'Mean Rel Error (%)', 'F1 Score', 'L1 Error']

print(f"\n{'Metric':<25} {'FFT Mean':>15} {'FD Mean':>15} {'Better Method':>15}")
print("-"*80)
for metric, name in zip(metrics, metric_names):
    fft_mean = fft_results[metric].mean()
    fd_mean = fd_results[metric].mean()
    
    # For RMSE, mean_rel_error, l1_error: lower is better
    # For R2, F1: higher is better
    if metric in ['rmse', 'mean_rel_error', 'l1_error']:
        better = "FFT" if fft_mean < fd_mean else "Finite Diff"
    else:
        better = "FFT" if fft_mean > fd_mean else "Finite Diff"
    
    print(f"{name:<25} {fft_mean:>15.6f} {fd_mean:>15.6f} {better:>15}")

# ===== Winner Count =====
print("\n\n" + "="*80)
print("WINNER COUNT BY NOISE LEVEL")
print("="*80)

fft_wins = 0
fd_wins = 0

for _, row in comparison.iterrows():
    if row['FFT_MeanRelErr'] < row['FD_MeanRelErr']:
        fft_wins += 1
    else:
        fd_wins += 1

print(f"\nBased on Mean Relative Error:")
print(f"  FFT wins: {fft_wins} times")
print(f"  Finite Difference wins: {fd_wins} times")

# ===== Best Overall =====
print("\n\n" + "="*80)
print("BEST OVERALL PERFORMANCE")
print("="*80)

fft_best = fft_results.loc[fft_results['mean_rel_error'].idxmin()]
fd_best = fd_results.loc[fd_results['mean_rel_error'].idxmin()]

print("\n🏆 FFT Best Performance:")
print(f"  Sigma: {fft_best['sigma']:.3f}")
print(f"  Mean Relative Error: {fft_best['mean_rel_error']:.2f}%")
print(f"  RMSE: {fft_best['rmse']:.6f}")
print(f"  R²: {fft_best['r2']:.6f}")
print(f"  F1: {fft_best['f1_score']:.6f}")
print(f"  Coefficients: a={fft_best['a']:.6f}, b={fft_best['b']:.6f}, c={fft_best['c']:.6f}")

print("\n🏆 Finite Difference Best Performance:")
print(f"  Sigma: {fd_best['sigma']:.3f}")
print(f"  Mean Relative Error: {fd_best['mean_rel_error']:.2f}%")
print(f"  RMSE: {fd_best['rmse']:.6f}")
print(f"  R²: {fd_best['r2']:.6f}")
print(f"  F1: {fd_best['f1_score']:.6f}")
print(f"  Coefficients: a={fd_best['a']:.6f}, b={fd_best['b']:.6f}, c={fd_best['c']:.6f}")

overall_winner = "FFT" if fft_best['mean_rel_error'] < fd_best['mean_rel_error'] else "Finite Difference"
print(f"\n🎯 Overall Winner: {overall_winner}")

# ===== VISUALIZATIONS =====
print("\n\n" + "="*80)
print("GENERATING COMPARISON PLOTS...")
print("="*80)

fig = plt.figure(figsize=(16, 12))

# Plot 1: Coefficient Recovery (a, b, c)
ax1 = plt.subplot(3, 3, 1)
plt.plot(fft_results['sigma'], fft_results['a'], 'o-', label='FFT', linewidth=2, markersize=6)
plt.plot(fd_results['sigma'], fd_results['a'], 's--', label='Finite Diff', linewidth=2, markersize=6)
plt.axhline(y=-1.0, color='r', linestyle=':', label='True value', linewidth=2)
plt.xlabel('Noise Level (σ)')
plt.ylabel('Coefficient a (u_xx)')
plt.title('Coefficient a Recovery')
plt.legend()
plt.grid(True, alpha=0.3)

ax2 = plt.subplot(3, 3, 2)
plt.plot(fft_results['sigma'], fft_results['b'], 'o-', label='FFT', linewidth=2, markersize=6)
plt.plot(fd_results['sigma'], fd_results['b'], 's--', label='Finite Diff', linewidth=2, markersize=6)
plt.axhline(y=-1.0, color='r', linestyle=':', label='True value', linewidth=2)
plt.xlabel('Noise Level (σ)')
plt.ylabel('Coefficient b (u_xxxx)')
plt.title('Coefficient b Recovery')
plt.legend()
plt.grid(True, alpha=0.3)

ax3 = plt.subplot(3, 3, 3)
plt.plot(fft_results['sigma'], fft_results['c'], 'o-', label='FFT', linewidth=2, markersize=6)
plt.plot(fd_results['sigma'], fd_results['c'], 's--', label='Finite Diff', linewidth=2, markersize=6)
plt.axhline(y=-1.0, color='r', linestyle=':', label='FFT True', linewidth=2)
plt.axhline(y=-0.5, color='orange', linestyle=':', label='FD True', linewidth=2)
plt.xlabel('Noise Level (σ)')
plt.ylabel('Coefficient c')
plt.title('Coefficient c Recovery')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: RMSE
ax4 = plt.subplot(3, 3, 4)
plt.plot(fft_results['sigma'], fft_results['rmse'], 'o-', label='FFT', linewidth=2, markersize=6)
plt.plot(fd_results['sigma'], fd_results['rmse'], 's--', label='Finite Diff', linewidth=2, markersize=6)
plt.xlabel('Noise Level (σ)')
plt.ylabel('RMSE')
plt.title('Root Mean Squared Error')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Plot 3: R² Score
ax5 = plt.subplot(3, 3, 5)
plt.plot(fft_results['sigma'], fft_results['r2'], 'o-', label='FFT', linewidth=2, markersize=6)
plt.plot(fd_results['sigma'], fd_results['r2'], 's--', label='Finite Diff', linewidth=2, markersize=6)
plt.xlabel('Noise Level (σ)')
plt.ylabel('R² Score')
plt.title('R² Coefficient of Determination')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 4: Mean Relative Error
ax6 = plt.subplot(3, 3, 6)
plt.plot(fft_results['sigma'], fft_results['mean_rel_error'], 'o-', label='FFT', linewidth=2, markersize=6)
plt.plot(fd_results['sigma'], fd_results['mean_rel_error'], 's--', label='Finite Diff', linewidth=2, markersize=6)
plt.xlabel('Noise Level (σ)')
plt.ylabel('Mean Relative Error (%)')
plt.title('Mean Relative Error')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 5: F1 Score
ax7 = plt.subplot(3, 3, 7)
plt.plot(fft_results['sigma'], fft_results['f1_score'], 'o-', label='FFT', linewidth=2, markersize=6)
plt.plot(fd_results['sigma'], fd_results['f1_score'], 's--', label='Finite Diff', linewidth=2, markersize=6)
plt.xlabel('Noise Level (σ)')
plt.ylabel('F1 Score')
plt.title('F1 Score (Term Selection)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 6: L1 Error
ax8 = plt.subplot(3, 3, 8)
plt.plot(fft_results['sigma'], fft_results['l1_error'], 'o-', label='FFT', linewidth=2, markersize=6)
plt.plot(fd_results['sigma'], fd_results['l1_error'], 's--', label='Finite Diff', linewidth=2, markersize=6)
plt.xlabel('Noise Level (σ)')
plt.ylabel('L1 Error')
plt.title('L1 Norm Error')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 7: Relative Errors per Coefficient
ax9 = plt.subplot(3, 3, 9)
width = 0.035
x = np.arange(len(fft_results))
plt.bar(x - 1.5*width, fft_results['rel_a'], width, label='FFT a', alpha=0.8)
plt.bar(x - 0.5*width, fft_results['rel_b'], width, label='FFT b', alpha=0.8)
plt.bar(x + 0.5*width, fft_results['rel_c'], width, label='FFT c', alpha=0.8)
plt.bar(x + 1.5*width, fd_results['mean_rel_error'], width, label='FD mean', alpha=0.8)
plt.xlabel('Noise Level Index')
plt.ylabel('Relative Error (%)')
plt.title('Relative Errors Comparison')
plt.xticks(x, [f"{s:.3f}" for s in fft_results['sigma']], rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('sindy_comparison_plots.png', dpi=300, bbox_inches='tight')
print("    Saved: sindy_comparison_plots.png")

# ===== Additional Visualization: Heatmap =====
fig2, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# FFT Heatmap
fft_data = fft_results[['rel_a', 'rel_b', 'rel_c']].T
sns.heatmap(fft_data, annot=True, fmt='.2f', cmap='RdYlGn_r', 
            xticklabels=[f"{s:.3f}" for s in fft_results['sigma']],
            yticklabels=['a (u_xx)', 'b (u_xxxx)', 'c'],
            ax=ax1, cbar_kws={'label': 'Relative Error (%)'})
ax1.set_title('FFT: Relative Errors Heatmap')
ax1.set_xlabel('Noise Level (σ)')

# Finite Difference Heatmap
fd_data = fd_results[['rel_a', 'rel_b', 'rel_c']].T
sns.heatmap(fd_data, annot=True, fmt='.2f', cmap='RdYlGn_r',
            xticklabels=[f"{s:.3f}" for s in fd_results['sigma']],
            yticklabels=['a (u_xx)', 'b (u_xxxx)', 'c ((u²)_x)'],
            ax=ax2, cbar_kws={'label': 'Relative Error (%)'})
ax2.set_title('Finite Difference: Relative Errors Heatmap')
ax2.set_xlabel('Noise Level (σ)')

plt.tight_layout()
plt.savefig('sindy_comparison_heatmap.png', dpi=300, bbox_inches='tight')
print("    Saved: sindy_comparison_heatmap.png")

# ===== Radar Chart for Best Results =====
from math import pi

fig3, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), subplot_kw=dict(projection='polar'))

categories = ['RMSE\n(lower better)', 'R²\n(higher better)', 'Mean Rel Err\n(lower better)', 
              'F1 Score\n(higher better)', 'L1 Error\n(lower better)']
N = len(categories)

# Normalize metrics to 0-1 scale for comparison
def normalize_metric(values, inverse=False):
    vmin, vmax = values.min(), values.max()
    if vmax == vmin:
        return np.ones_like(values) * 0.5
    normalized = (values - vmin) / (vmax - vmin)
    return 1 - normalized if inverse else normalized

# Get best result for each method
fft_best_idx = fft_results['mean_rel_error'].idxmin()
fd_best_idx = fd_results['mean_rel_error'].idxmin()

fft_metrics = [
    1 - normalize_metric(fft_results['rmse'].values, inverse=True)[fft_best_idx],
    normalize_metric(fft_results['r2'].values, inverse=False)[fft_best_idx],
    1 - normalize_metric(fft_results['mean_rel_error'].values, inverse=True)[fft_best_idx],
    normalize_metric(fft_results['f1_score'].values, inverse=False)[fft_best_idx],
    1 - normalize_metric(fft_results['l1_error'].values, inverse=True)[fft_best_idx]
]

fd_metrics = [
    1 - normalize_metric(fd_results['rmse'].values, inverse=True)[fd_best_idx],
    normalize_metric(fd_results['r2'].values, inverse=False)[fd_best_idx],
    1 - normalize_metric(fd_results['mean_rel_error'].values, inverse=True)[fd_best_idx],
    normalize_metric(fd_results['f1_score'].values, inverse=False)[fd_best_idx],
    1 - normalize_metric(fd_results['l1_error'].values, inverse=True)[fd_best_idx]
]

angles = [n / float(N) * 2 * pi for n in range(N)]
fft_metrics += fft_metrics[:1]
fd_metrics += fd_metrics[:1]
angles += angles[:1]

# FFT Radar
ax1.plot(angles, fft_metrics, 'o-', linewidth=2, color='blue', label='FFT')
ax1.fill(angles, fft_metrics, alpha=0.25, color='blue')
ax1.set_xticks(angles[:-1])
ax1.set_xticklabels(categories, size=9)
ax1.set_ylim(0, 1)
ax1.set_title(f'FFT Best Performance\n(σ={fft_results.loc[fft_best_idx, "sigma"]:.3f})', 
              size=12, pad=20)
ax1.grid(True)

# FD Radar
ax2.plot(angles, fd_metrics, 's-', linewidth=2, color='green', label='Finite Diff')
ax2.fill(angles, fd_metrics, alpha=0.25, color='green')
ax2.set_xticks(angles[:-1])
ax2.set_xticklabels(categories, size=9)
ax2.set_ylim(0, 1)
ax2.set_title(f'Finite Difference Best Performance\n(σ={fd_results.loc[fd_best_idx, "sigma"]:.3f})', 
              size=12, pad=20)
ax2.grid(True)

plt.tight_layout()
plt.savefig('sindy_comparison_radar.png', dpi=300, bbox_inches='tight')
print("      Saved: sindy_comparison_radar.png")

# ===== Save Comparison Table =====
comparison.to_csv('sindy_comparison_table.csv', index=False)
print("    Saved: sindy_comparison_table.csv")

print("\n" + "="*80)
print("   COMPARISON COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. sindy_comparison_plots.png - Main comparison plots")
print("  2. sindy_comparison_heatmap.png - Relative errors heatmap")
print("  3. sindy_comparison_radar.png - Radar charts for best results")
print("  4. sindy_comparison_table.csv - Side-by-side comparison data")
print("="*80)

Denoising methods 

Implement 3 Denoising Methods on KS Synthetic Noisy Data

We will implement:

1. Gaussian smoothing (spatial filtering)
2. Savitzky–Golay smoothing (polynomial filtering)
3. Spectral low-pass filtering (Fourier truncation)

We then evaluate:

recovered coefficients a, b, c
r2
RMSE

In [ ]:
import h5py
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter

# ============================================================
# 1. List your noisy datasets here
#    Replace the filenames with your actual .h5 file names.
# ============================================================
data_files = [
    (0.00, "test_solving_euler_ks_clean.h5"),  # clean (no added noise)
    (0.01, "ks_gaussian_sigma_0.005.h5"),
    (0.03, "ks_gaussian_sigma_0.010.h5"),
    (0.05, "ks_gaussian_sigma_0.030.h5"),
    (0.08, "ks_gaussian_sigma_0.080.h5"),
    (0.10, "ks_gaussian_sigma_0.100.h5"),
]

# ============================================================
# 2. Denoising operators (space-only, along x)
# ============================================================

def denoise_gaussian(U, sigma_x=1.0):
    """
    Gaussian smoothing along space (axis=1) with periodic boundary (wrap).
    U: array (T, X)
    """
    return gaussian_filter1d(U, sigma=sigma_x, axis=1, mode="wrap")

def denoise_savgol(U, window=11, poly=3):
    """
    Savitzky–Golay smoothing along space (axis=1) with periodic boundary (wrap).
    window must be odd and <= number of spatial points.
    """
    T, X = U.shape
    # ensure window is odd and <= X
    window = min(window, X - (1 - X % 2))
    if window % 2 == 0:
        window -= 1
    return savgol_filter(U, window_length=window, polyorder=poly,
                         axis=1, mode="wrap")

def denoise_spectral(U, cutoff_ratio=0.5, dx=0.5):
    """
    Spectral low-pass denoising:
    - FFT in space (axis=1)
    - zero out frequencies above cutoff_ratio * k_max
    """
    T, X = U.shape
    U_hat = np.fft.rfft(U, axis=1)
    freqs = np.fft.rfftfreq(X, d=dx)
    k_max = freqs.max()
    mask = (np.abs(freqs) <= cutoff_ratio * k_max).astype(U_hat.dtype)
    U_hat_filtered = U_hat * mask[None, :]
    U_lp = np.fft.irfft(U_hat_filtered, n=X, axis=1)
    return U_lp

# ============================================================
# 3. Main loop: load each σ–dataset, apply denoisers, save
# ============================================================

for sigma, fname in data_files:
    print(f"\n=== Processing σ={sigma:.3f}, file={fname} ===")

    # ---- load original noisy dataset ----
    with h5py.File(fname, "r") as f:
        u = np.array(f["u"], dtype=np.float32)   # (T, X)
        x = np.array(f["x"], dtype=np.float32)
        t = np.array(f["t"], dtype=np.float32)
        attrs = dict(f.attrs)  # copy attributes

    T, X = u.shape
    dx = float(x[1] - x[0])
    dt = float(t[1] - t[0])
    print(f"  u.shape={u.shape}, dx={dx:.4f}, dt={dt:.4f}")

    # ---- apply denoising methods ----
    u_gauss   = denoise_gaussian(u, sigma_x=1.0)
    u_savgol  = denoise_savgol(u, window=11, poly=3)
    u_spectral= denoise_spectral(u, cutoff_ratio=0.5, dx=dx)

    # ---- base name for outputs ----
    if fname.lower().endswith(".h5"):
        base = fname[:-3]
    else:
        base = fname

    # ---- helper to save a denoised version ----
    def save_denoised(u_denoised, suffix, method_name):
        out_name = f"{base}_{suffix}.h5"
        print(f"  Saving {method_name} denoised data → {out_name}")
        with h5py.File(out_name, "w") as f_out:
            f_out.create_dataset("u", data=u_denoised)
            f_out.create_dataset("x", data=x)
            f_out.create_dataset("t", data=t)
            # copy original attrs, then add denoising info
            for k, v in attrs.items():
                f_out.attrs[k] = v
            f_out.attrs["denoising_method"] = method_name
            f_out.attrs["sigma_source"] = sigma

    # ---- save all three denoised versions ----
    save_denoised(u_gauss,    "gauss",   "gaussian_spatial")
    save_denoised(u_savgol,   "savgol",  "savgol_spatial")
    save_denoised(u_spectral, "spectral","spectral_lowpass")

print("\nAll denoised datasets saved.")


## Phase 2: Denoising methods

### 3 denoising methods, FFT-based SINDy, and useful visualizations.

In [ ]:
import h5py
import numpy as np
from scipy.fft import fft, ifft, fftfreq
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 10)

print("="*80)
print("DENOISING METHODS + FFT-BASED SINDY EVALUATION")
print("="*80)

# ===== Spectral derivative functions =====
def spectral_derivative(u_snapshot, k):
    """First derivative using FFT"""
    u_hat = fft(u_snapshot)
    du_hat = 1j * k * u_hat
    return np.real(ifft(du_hat))

def spectral_second_derivative(u_snapshot, k):
    """Second derivative using FFT"""
    u_hat = fft(u_snapshot)
    d2u_hat = -(k**2) * u_hat
    return np.real(ifft(d2u_hat))

def spectral_fourth_derivative(u_snapshot, k):
    """Fourth derivative using FFT"""
    u_hat = fft(u_snapshot)
    d4u_hat = (k**4) * u_hat
    return np.real(ifft(d4u_hat))


# ===== Denoising Methods =====

def gaussian_denoising(u, sigma=2.0):
    """
    Gaussian smoothing - spatial filtering
    Apply Gaussian filter along spatial dimension
    """
    u_denoised = np.zeros_like(u)
    for i in range(u.shape[0]):  # For each time snapshot
        u_denoised[i] = gaussian_filter1d(u[i], sigma=sigma)
    return u_denoised


def savitzky_golay_denoising(u, window_length=11, polyorder=3):
    """
    Savitzky-Golay smoothing - polynomial filtering
    Preserves edges and peaks better than Gaussian
    Apply along spatial dimension
    """
    u_denoised = np.zeros_like(u)
    # Ensure window_length is odd and less than data length
    window_length = min(window_length, u.shape[1] - 1)
    if window_length % 2 == 0:
        window_length -= 1
    
    for i in range(u.shape[0]):  # For each time snapshot
        u_denoised[i] = savgol_filter(u[i], window_length=window_length, 
                                       polyorder=polyorder, mode='wrap')
    return u_denoised


def spectral_lowpass_denoising(u, cutoff_ratio=0.5):
    """
    Spectral low-pass filtering - Fourier truncation
    Remove high-frequency components (noise)
    cutoff_ratio: fraction of frequencies to keep (0.5 = keep lower 50%)
    """
    u_denoised = np.zeros_like(u)
    Nx = u.shape[1]
    cutoff_idx = int(Nx * cutoff_ratio / 2)
    
    for i in range(u.shape[0]):  # For each time snapshot
        u_hat = fft(u[i])
        # Zero out high frequencies
        u_hat_filtered = u_hat.copy()
        u_hat_filtered[cutoff_idx:-cutoff_idx] = 0
        u_denoised[i] = np.real(ifft(u_hat_filtered))
    
    return u_denoised


# ===== Evaluation Function =====
def evaluate_sindy_fft(u, x, t, method_name="Unknown", sigma_noise=0.0):
    """
    Apply FFT-based SINDy and compute evaluation metrics
    Returns: dict with coefficients, R², RMSE, errors
    """
    Nx = len(x)
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    
    # Define wavenumbers
    k = 2 * np.pi * fftfreq(Nx, d=dx)
    
    # Compute temporal derivative (central difference)
    u_t = (u[2:] - u[:-2]) / (2 * dt)
    u_mid = u[1:-1]
    
    # Build feature library using FFT
    Theta = []
    ut_flat = []
    
    for snapshot, ut_snapshot in zip(u_mid, u_t):
        ux = spectral_derivative(snapshot, k)
        uxx = spectral_second_derivative(snapshot, k)
        uxxxx = spectral_fourth_derivative(snapshot, k)
        
        # Feature terms: [u_xx, u_xxxx, u*u_x]
        Theta_snapshot = np.vstack([
            uxx,
            uxxxx,
            snapshot * ux
        ]).T  # shape (Nx, 3)
        
        Theta.append(Theta_snapshot)
        ut_flat.append(ut_snapshot)
    
    Theta = np.vstack(Theta)
    ut_flat = np.hstack(ut_flat)
    
    # Normalize features
    Theta_mean = Theta.mean(axis=0)
    Theta_std = Theta.std(axis=0)
    Theta_norm = (Theta - Theta_mean) / Theta_std
    
    # Lasso regression
    lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000)
    lasso.fit(Theta_norm, ut_flat)
    coeffs = lasso.coef_ / Theta_std
    
    # Extract coefficients
    a = coeffs[0]  # u_xx
    b = coeffs[1]  # u_xxxx
    c = coeffs[2]  # u*u_x
    
    # Predict u_t
    u_t_pred = Theta @ coeffs
    
    # Compute metrics
    rmse = np.sqrt(np.mean((ut_flat - u_t_pred)**2))
    r2 = r2_score(ut_flat, u_t_pred)
    
    # True coefficients
    a_true, b_true, c_true = -1.0, -1.0, -1.0
    
    # Coefficient errors
    error_a = a - a_true
    error_b = b - b_true
    error_c = c - c_true
    
    abs_error_a = abs(error_a)
    abs_error_b = abs(error_b)
    abs_error_c = abs(error_c)
    
    return {
        'method': method_name,
        'sigma': sigma_noise,
        'a': a,
        'b': b,
        'c': c,
        'error_a': error_a,
        'error_b': error_b,
        'error_c': error_c,
        'abs_error_a': abs_error_a,
        'abs_error_b': abs_error_b,
        'abs_error_c': abs_error_c,
        'rmse': rmse,
        'r2': r2
    }


# ===== Main Processing =====

# Noise datasets (excluding clean data)
noise_files = [
    "ks_gaussian_sigma_0.005.h5",
    "ks_gaussian_sigma_0.010.h5",
    "ks_gaussian_sigma_0.030.h5",
    "ks_gaussian_sigma_0.050.h5",
    "ks_gaussian_sigma_0.080.h5",
    "ks_gaussian_sigma_0.100.h5"
]

noise_levels = [0.005, 0.01, 0.03, 0.05, 0.08, 0.1]

# Store all results
all_results = []

# Process each noise level
for file, sigma in zip(noise_files, noise_levels):
    
    print("\n" + "="*80)
    print(f"PROCESSING NOISE LEVEL σ = {sigma}")
    print("="*80)
    
    # Load noisy data
    with h5py.File(file, "r") as f:
        u_noisy = np.array(f["u"])
        x = np.array(f["x"])
        t = np.array(f["t"])
    
    print(f"Loaded: {file}")
    print(f"Shape: {u_noisy.shape}")
    
    # ===== 1. NO DENOISING (Baseline) =====
    print("\n--- Method 1: No Denoising (Baseline) ---")
    result_baseline = evaluate_sindy_fft(u_noisy, x, t, 
                                          method_name="No Denoising", 
                                          sigma_noise=sigma)
    all_results.append(result_baseline)
    print(f"  Coefficients: a={result_baseline['a']:.6f}, b={result_baseline['b']:.6f}, c={result_baseline['c']:.6f}")
    print(f"  RMSE: {result_baseline['rmse']:.6f}, R²: {result_baseline['r2']:.6f}")
    
    
    # ===== 2. GAUSSIAN DENOISING =====
    print("\n--- Method 2: Gaussian Smoothing ---")
    u_gaussian = gaussian_denoising(u_noisy, sigma=2.0)
    result_gaussian = evaluate_sindy_fft(u_gaussian, x, t, 
                                          method_name="Gaussian", 
                                          sigma_noise=sigma)
    all_results.append(result_gaussian)
    print(f"  Coefficients: a={result_gaussian['a']:.6f}, b={result_gaussian['b']:.6f}, c={result_gaussian['c']:.6f}")
    print(f"  RMSE: {result_gaussian['rmse']:.6f}, R²: {result_gaussian['r2']:.6f}")
    
    
    # ===== 3. SAVITZKY-GOLAY DENOISING =====
    print("\n--- Method 3: Savitzky-Golay Smoothing ---")
    u_savgol = savitzky_golay_denoising(u_noisy, window_length=11, polyorder=3)
    result_savgol = evaluate_sindy_fft(u_savgol, x, t, 
                                        method_name="Savitzky-Golay", 
                                        sigma_noise=sigma)
    all_results.append(result_savgol)
    print(f"  Coefficients: a={result_savgol['a']:.6f}, b={result_savgol['b']:.6f}, c={result_savgol['c']:.6f}")
    print(f"  RMSE: {result_savgol['rmse']:.6f}, R²: {result_savgol['r2']:.6f}")
    
    
    # ===== 4. SPECTRAL LOW-PASS DENOISING =====
    print("\n--- Method 4: Spectral Low-Pass Filtering ---")
    u_spectral = spectral_lowpass_denoising(u_noisy, cutoff_ratio=0.5)
    result_spectral = evaluate_sindy_fft(u_spectral, x, t, 
                                          method_name="Spectral", 
                                          sigma_noise=sigma)
    all_results.append(result_spectral)
    print(f"  Coefficients: a={result_spectral['a']:.6f}, b={result_spectral['b']:.6f}, c={result_spectral['c']:.6f}")
    print(f"  RMSE: {result_spectral['rmse']:.6f}, R²: {result_spectral['r2']:.6f}")


# ===== Convert to DataFrame =====
df_results = pd.DataFrame(all_results)

# ===== SUMMARY TABLES =====
print("\n\n" + "="*100)
print("SUMMARY TABLE 1: RECOVERED COEFFICIENTS BY DENOISING METHOD")
print("="*100)
print(f"{'Sigma':>8} {'Method':>18} {'a (u_xx)':>12} {'b (u_xxxx)':>12} {'c (u*u_x)':>12} {'Err_a':>10} {'Err_b':>10} {'Err_c':>10}")
print("-"*100)

for _, row in df_results.iterrows():
    print(f"{row['sigma']:>8.3f} {row['method']:>18} {row['a']:>12.6f} {row['b']:>12.6f} {row['c']:>12.6f} "
          f"{row['error_a']:>10.6f} {row['error_b']:>10.6f} {row['error_c']:>10.6f}")

print("="*100)
print(f"{'TRUE:':>27} {-1.0:>12.6f} {-1.0:>12.6f} {-1.0:>12.6f}")
print("="*100)


print("\n\n" + "="*100)
print("SUMMARY TABLE 2: EVALUATION METRICS BY DENOISING METHOD")
print("="*100)
print(f"{'Sigma':>8} {'Method':>18} {'RMSE':>12} {'R²':>12} {'|Err_a|':>10} {'|Err_b|':>10} {'|Err_c|':>10}")
print("-"*100)

for _, row in df_results.iterrows():
    print(f"{row['sigma']:>8.3f} {row['method']:>18} {row['rmse']:>12.6f} {row['r2']:>12.6f} "
          f"{row['abs_error_a']:>10.6f} {row['abs_error_b']:>10.6f} {row['abs_error_c']:>10.6f}")

print("="*100)


# ===== COMPARISON BY NOISE LEVEL =====
print("\n\n" + "="*100)
print("SUMMARY TABLE 3: BEST METHOD PER NOISE LEVEL (by RMSE)")
print("="*100)
print(f"{'Sigma':>8} {'Best Method':>18} {'RMSE':>12} {'R²':>12} {'Coeff a':>12} {'Coeff b':>12} {'Coeff c':>12}")
print("-"*100)

for sigma in noise_levels:
    sigma_data = df_results[df_results['sigma'] == sigma]
    best_idx = sigma_data['rmse'].idxmin()
    best = df_results.loc[best_idx]
    print(f"{sigma:>8.3f} {best['method']:>18} {best['rmse']:>12.6f} {best['r2']:>12.6f} "
          f"{best['a']:>12.6f} {best['b']:>12.6f} {best['c']:>12.6f}")

print("="*100)


# ===== OVERALL STATISTICS =====
print("\n\n" + "="*80)
print("OVERALL STATISTICS BY METHOD (Averaged Across All Noise Levels)")
print("="*80)

method_stats = df_results.groupby('method').agg({
    'rmse': 'mean',
    'r2': 'mean',
    'abs_error_a': 'mean',
    'abs_error_b': 'mean',
    'abs_error_c': 'mean'
}).round(6)

print(method_stats)
print("="*80)

best_method = method_stats['rmse'].idxmin()
print(f"\n🏆 BEST OVERALL METHOD (by average RMSE): {best_method}")
print(f"   Average RMSE: {method_stats.loc[best_method, 'rmse']:.6f}")
print(f"   Average R²: {method_stats.loc[best_method, 'r2']:.6f}")


# ===== VISUALIZATION 1: Coefficient Recovery =====
print("\n\n" + "="*80)
print("GENERATING VISUALIZATIONS...")
print("="*80)

fig = plt.figure(figsize=(18, 12))

methods = df_results['method'].unique()
colors = {'No Denoising': 'red', 'Gaussian': 'blue', 
          'Savitzky-Golay': 'green', 'Spectral': 'purple'}
markers = {'No Denoising': 'x', 'Gaussian': 'o', 
           'Savitzky-Golay': 's', 'Spectral': '^'}

# Plot 1: Coefficient a recovery
ax1 = plt.subplot(3, 3, 1)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['a'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.axhline(y=-1.0, color='black', linestyle='--', linewidth=2, label='True value')
plt.xlabel('Noise Level (σ)')
plt.ylabel('Coefficient a (u_xx)')
plt.title('Coefficient a Recovery')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Coefficient b recovery
ax2 = plt.subplot(3, 3, 2)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['b'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.axhline(y=-1.0, color='black', linestyle='--', linewidth=2, label='True value')
plt.xlabel('Noise Level (σ)')
plt.ylabel('Coefficient b (u_xxxx)')
plt.title('Coefficient b Recovery')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Coefficient c recovery
ax3 = plt.subplot(3, 3, 3)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['c'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.axhline(y=-1.0, color='black', linestyle='--', linewidth=2, label='True value')
plt.xlabel('Noise Level (σ)')
plt.ylabel('Coefficient c (u*u_x)')
plt.title('Coefficient c Recovery')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 4: RMSE comparison
ax4 = plt.subplot(3, 3, 4)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['rmse'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.xlabel('Noise Level (σ)')
plt.ylabel('RMSE')
plt.title('Root Mean Squared Error')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Plot 5: R² comparison
ax5 = plt.subplot(3, 3, 5)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['r2'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.xlabel('Noise Level (σ)')
plt.ylabel('R² Score')
plt.title('R² Coefficient of Determination')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 6: Absolute error in a
ax6 = plt.subplot(3, 3, 6)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['abs_error_a'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.xlabel('Noise Level (σ)')
plt.ylabel('|Error in a|')
plt.title('Absolute Error in Coefficient a')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 7: Absolute error in b
ax7 = plt.subplot(3, 3, 7)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['abs_error_b'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.xlabel('Noise Level (σ)')
plt.ylabel('|Error in b|')
plt.title('Absolute Error in Coefficient b')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 8: Absolute error in c
ax8 = plt.subplot(3, 3, 8)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['abs_error_c'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.xlabel('Noise Level (σ)')
plt.ylabel('|Error in c|')
plt.title('Absolute Error in Coefficient c')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 9: Bar chart of average performance
ax9 = plt.subplot(3, 3, 9)
x = np.arange(len(methods))
width = 0.35
rmse_means = [method_stats.loc[m, 'rmse'] for m in methods]
r2_means = [method_stats.loc[m, 'r2'] for m in methods]
ax9_twin = ax9.twinx()
bars1 = ax9.bar(x - width/2, rmse_means, width, label='Avg RMSE', alpha=0.8)
bars2 = ax9_twin.bar(x + width/2, r2_means, width, label='Avg R²', alpha=0.8, color='orange')
ax9.set_xlabel('Denoising Method')
ax9.set_ylabel('Average RMSE', color='blue')
ax9_twin.set_ylabel('Average R²', color='orange')
ax9.set_xticks(x)
ax9.set_xticklabels(methods, rotation=45, ha='right')
ax9.set_title('Average Performance by Method')
ax9.legend(loc='upper left')
ax9_twin.legend(loc='upper right')
ax9.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('denoising_fft_comparison.png', dpi=300, bbox_inches='tight')
print("    Saved: denoising_fft_comparison.png")


# ===== VISUALIZATION 2: Heatmaps =====
fig2, axes = plt.subplots(2, 2, figsize=(16, 12))

# Heatmap 1: RMSE
pivot_rmse = df_results.pivot(index='method', columns='sigma', values='rmse')
sns.heatmap(pivot_rmse, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=axes[0, 0],
            cbar_kws={'label': 'RMSE'})
axes[0, 0].set_title('RMSE by Method and Noise Level', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Noise Level (σ)')
axes[0, 0].set_ylabel('Denoising Method')

# Heatmap 2: R²
pivot_r2 = df_results.pivot(index='method', columns='sigma', values='r2')
sns.heatmap(pivot_r2, annot=True, fmt='.4f', cmap='RdYlGn', ax=axes[0, 1],
            cbar_kws={'label': 'R² Score'})
axes[0, 1].set_title('R² Score by Method and Noise Level', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Noise Level (σ)')
axes[0, 1].set_ylabel('Denoising Method')

# Heatmap 3: Absolute Error in coefficient a
pivot_err_a = df_results.pivot(index='method', columns='sigma', values='abs_error_a')
sns.heatmap(pivot_err_a, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=axes[1, 0],
            cbar_kws={'label': '|Error in a|'})
axes[1, 0].set_title('Absolute Error in Coefficient a', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Noise Level (σ)')
axes[1, 0].set_ylabel('Denoising Method')

# Heatmap 4: Absolute Error in coefficient b
pivot_err_b = df_results.pivot(index='method', columns='sigma', values='abs_error_b')
sns.heatmap(pivot_err_b, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=axes[1, 1],
            cbar_kws={'label': '|Error in b|'})
axes[1, 1].set_title('Absolute Error in Coefficient b', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Noise Level (σ)')
axes[1, 1].set_ylabel('Denoising Method')

plt.tight_layout()
plt.savefig('denoising_fft_heatmaps.png', dpi=300, bbox_inches='tight')
print("    Saved: denoising_fft_heatmaps.png")


# ===== Save Results to CSV =====
df_results.to_csv('denoising_fft_results.csv', index=False)
print("    Saved: denoising_fft_results.csv")

print("\n" + "="*80)
print("   DENOISING + FFT SINDY EVALUATION COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. denoising_fft_comparison.png - Main comparison plots (9 subplots)")
print("  2. denoising_fft_heatmaps.png - Heatmaps of metrics")
print("  3. denoising_fft_results.csv - Complete results data")
print("="*80)

 3 denoising methods, Finite Difference sindy

In [ ]:
import h5py
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter
from scipy.fft import fft, ifft
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 10)

print("="*80)
print("DENOISING METHODS + FINITE DIFFERENCE SINDY EVALUATION")
print("="*80)

# ===== Denoising Methods =====

def gaussian_denoising(u, sigma=2.0):
    """
    Gaussian smoothing - spatial filtering
    Apply Gaussian filter along spatial dimension
    """
    u_denoised = np.zeros_like(u)
    for i in range(u.shape[0]):  # For each time snapshot
        u_denoised[i] = gaussian_filter1d(u[i], sigma=sigma)
    return u_denoised


def savitzky_golay_denoising(u, window_length=11, polyorder=3):
    """
    Savitzky-Golay smoothing - polynomial filtering
    Preserves edges and peaks better than Gaussian
    Apply along spatial dimension
    """
    u_denoised = np.zeros_like(u)
    # Ensure window_length is odd and less than data length
    window_length = min(window_length, u.shape[1] - 1)
    if window_length % 2 == 0:
        window_length -= 1
    
    for i in range(u.shape[0]):  # For each time snapshot
        u_denoised[i] = savgol_filter(u[i], window_length=window_length, 
                                       polyorder=polyorder, mode='wrap')
    return u_denoised


def spectral_lowpass_denoising(u, cutoff_ratio=0.5):
    """
    Spectral low-pass filtering - Fourier truncation
    Remove high-frequency components (noise)
    cutoff_ratio: fraction of frequencies to keep (0.5 = keep lower 50%)
    """
    u_denoised = np.zeros_like(u)
    Nx = u.shape[1]
    cutoff_idx = int(Nx * cutoff_ratio / 2)
    
    for i in range(u.shape[0]):  # For each time snapshot
        u_hat = fft(u[i])
        # Zero out high frequencies
        u_hat_filtered = u_hat.copy()
        u_hat_filtered[cutoff_idx:-cutoff_idx] = 0
        u_denoised[i] = np.real(ifft(u_hat_filtered))
    
    return u_denoised


# ===== Evaluation Function with Finite Difference =====
def evaluate_sindy_finite_diff(u, x, t, method_name="Unknown", sigma_noise=0.0):
    """
    Apply Finite Difference SINDy and compute evaluation metrics
    Returns: dict with coefficients, R², RMSE, errors
    """
    dx = float(x[1] - x[0])
    dt = float(t[1] - t[0])
    
    # Finite difference operators
    def d1(U): 
        return (np.roll(U, -1, 1) - np.roll(U, 1, 1)) / (2*dx)
    
    def d2(U): 
        return (np.roll(U, -1, 1) - 2*U + np.roll(U, 1, 1)) / (dx**2)
    
    def d4(U):
        return (np.roll(U, -2, 1) - 4*np.roll(U, -1, 1) + 6*U - 
                4*np.roll(U, 1, 1) + np.roll(U, 2, 1)) / (dx**4)
    
    # Compute derivatives
    U_t = (u[1:] - u[:-1]) / dt
    U = u[:-1]
    U_x = d1(U)
    U_xx = d2(U)
    U_xxxx = d4(U)
    U2_x = d1(U**2)
    
    # Build feature matrix
    Theta = np.column_stack([
        np.ones_like(U).ravel(),
        U.ravel(),
        U_x.ravel(),
        U_xx.ravel(),
        U_xxxx.ravel(),
        U2_x.ravel()
    ])
    y = U_t.ravel()
    names = ["1", "u", "u_x", "u_xx", "u_xxxx", "(u^2)_x"]
    
    # Normalize
    Theta_mean = Theta.mean(0)
    Theta_std = Theta.std(0) + 1e-8
    Theta_n = (Theta - Theta_mean) / Theta_std
    
    # Lasso regression
    lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000, tol=1e-6)
    lasso.fit(Theta_n, y)
    coef = lasso.coef_ / Theta_std
    
    # Extract coefficients
    a = coef[names.index("u_xx")]
    b = coef[names.index("u_xxxx")]
    c_phys = coef[names.index("(u^2)_x")]
    
    # Predict u_t
    y_pred = Theta @ coef
    
    # Compute metrics
    rmse = np.sqrt(np.mean((y - y_pred)**2))
    r2 = r2_score(y, y_pred)
    
    # True coefficients (physical form)
    a_true, b_true, c_true = -1.0, -1.0, -0.5
    
    # Coefficient errors
    error_a = a - a_true
    error_b = b - b_true
    error_c = c_phys - c_true
    
    abs_error_a = abs(error_a)
    abs_error_b = abs(error_b)
    abs_error_c = abs(error_c)
    
    return {
        'method': method_name,
        'sigma': sigma_noise,
        'a': a,
        'b': b,
        'c': c_phys,
        'error_a': error_a,
        'error_b': error_b,
        'error_c': error_c,
        'abs_error_a': abs_error_a,
        'abs_error_b': abs_error_b,
        'abs_error_c': abs_error_c,
        'rmse': rmse,
        'r2': r2
    }


# ===== Main Processing =====

# Noise datasets (excluding clean data)
noise_files = [
    "ks_gaussian_sigma_0.005.h5",
    "ks_gaussian_sigma_0.010.h5",
    "ks_gaussian_sigma_0.030.h5",
    "ks_gaussian_sigma_0.050.h5",
    "ks_gaussian_sigma_0.080.h5",
    "ks_gaussian_sigma_0.100.h5"
]

noise_levels = [0.005, 0.01, 0.03, 0.05, 0.08, 0.1]

# Store all results
all_results = []

# Process each noise level
for file, sigma in zip(noise_files, noise_levels):
    
    print("\n" + "="*80)
    print(f"PROCESSING NOISE LEVEL σ = {sigma}")
    print("="*80)
    
    # Load noisy data
    with h5py.File(file, "r") as f:
        u_noisy = np.array(f["u"], dtype=np.float32)
        x = np.array(f["x"], dtype=np.float32)
        t = np.array(f["t"], dtype=np.float32)
    
    print(f"Loaded: {file}")
    print(f"Shape: {u_noisy.shape}")
    
    # ===== 1. NO DENOISING (Baseline) =====
    print("\n--- Method 1: No Denoising (Baseline) ---")
    result_baseline = evaluate_sindy_finite_diff(u_noisy, x, t, 
                                                   method_name="No Denoising", 
                                                   sigma_noise=sigma)
    all_results.append(result_baseline)
    print(f"  Coefficients: a={result_baseline['a']:.6f}, b={result_baseline['b']:.6f}, c={result_baseline['c']:.6f}")
    print(f"  RMSE: {result_baseline['rmse']:.6f}, R²: {result_baseline['r2']:.6f}")
    
    
    # ===== 2. GAUSSIAN DENOISING =====
    print("\n--- Method 2: Gaussian Smoothing ---")
    u_gaussian = gaussian_denoising(u_noisy, sigma=2.0)
    result_gaussian = evaluate_sindy_finite_diff(u_gaussian, x, t, 
                                                   method_name="Gaussian", 
                                                   sigma_noise=sigma)
    all_results.append(result_gaussian)
    print(f"  Coefficients: a={result_gaussian['a']:.6f}, b={result_gaussian['b']:.6f}, c={result_gaussian['c']:.6f}")
    print(f"  RMSE: {result_gaussian['rmse']:.6f}, R²: {result_gaussian['r2']:.6f}")
    
    
    # ===== 3. SAVITZKY-GOLAY DENOISING =====
    print("\n--- Method 3: Savitzky-Golay Smoothing ---")
    u_savgol = savitzky_golay_denoising(u_noisy, window_length=11, polyorder=3)
    result_savgol = evaluate_sindy_finite_diff(u_savgol, x, t, 
                                                 method_name="Savitzky-Golay", 
                                                 sigma_noise=sigma)
    all_results.append(result_savgol)
    print(f"  Coefficients: a={result_savgol['a']:.6f}, b={result_savgol['b']:.6f}, c={result_savgol['c']:.6f}")
    print(f"  RMSE: {result_savgol['rmse']:.6f}, R²: {result_savgol['r2']:.6f}")
    
    
    # ===== 4. SPECTRAL LOW-PASS DENOISING =====
    print("\n--- Method 4: Spectral Low-Pass Filtering ---")
    u_spectral = spectral_lowpass_denoising(u_noisy, cutoff_ratio=0.5)
    result_spectral = evaluate_sindy_finite_diff(u_spectral, x, t, 
                                                   method_name="Spectral", 
                                                   sigma_noise=sigma)
    all_results.append(result_spectral)
    print(f"  Coefficients: a={result_spectral['a']:.6f}, b={result_spectral['b']:.6f}, c={result_spectral['c']:.6f}")
    print(f"  RMSE: {result_spectral['rmse']:.6f}, R²: {result_spectral['r2']:.6f}")


# ===== Convert to DataFrame =====
df_results = pd.DataFrame(all_results)

# ===== SUMMARY TABLES =====
print("\n\n" + "="*100)
print("SUMMARY TABLE 1: RECOVERED COEFFICIENTS BY DENOISING METHOD")
print("="*100)
print(f"{'Sigma':>8} {'Method':>18} {'a (u_xx)':>12} {'b (u_xxxx)':>12} {'c ((u²)_x)':>12} {'Err_a':>10} {'Err_b':>10} {'Err_c':>10}")
print("-"*100)

for _, row in df_results.iterrows():
    print(f"{row['sigma']:>8.3f} {row['method']:>18} {row['a']:>12.6f} {row['b']:>12.6f} {row['c']:>12.6f} "
          f"{row['error_a']:>10.6f} {row['error_b']:>10.6f} {row['error_c']:>10.6f}")

print("="*100)
print(f"{'TRUE:':>27} {-1.0:>12.6f} {-1.0:>12.6f} {-0.5:>12.6f}")
print("="*100)


print("\n\n" + "="*100)
print("SUMMARY TABLE 2: EVALUATION METRICS BY DENOISING METHOD")
print("="*100)
print(f"{'Sigma':>8} {'Method':>18} {'RMSE':>12} {'R²':>12} {'|Err_a|':>10} {'|Err_b|':>10} {'|Err_c|':>10}")
print("-"*100)

for _, row in df_results.iterrows():
    print(f"{row['sigma']:>8.3f} {row['method']:>18} {row['rmse']:>12.6f} {row['r2']:>12.6f} "
          f"{row['abs_error_a']:>10.6f} {row['abs_error_b']:>10.6f} {row['abs_error_c']:>10.6f}")

print("="*100)


# ===== COMPARISON BY NOISE LEVEL =====
print("\n\n" + "="*100)
print("SUMMARY TABLE 3: BEST METHOD PER NOISE LEVEL (by RMSE)")
print("="*100)
print(f"{'Sigma':>8} {'Best Method':>18} {'RMSE':>12} {'R²':>12} {'Coeff a':>12} {'Coeff b':>12} {'Coeff c':>12}")
print("-"*100)

for sigma in noise_levels:
    sigma_data = df_results[df_results['sigma'] == sigma]
    best_idx = sigma_data['rmse'].idxmin()
    best = df_results.loc[best_idx]
    print(f"{sigma:>8.3f} {best['method']:>18} {best['rmse']:>12.6f} {best['r2']:>12.6f} "
          f"{best['a']:>12.6f} {best['b']:>12.6f} {best['c']:>12.6f}")

print("="*100)


# ===== OVERALL STATISTICS =====
print("\n\n" + "="*80)
print("OVERALL STATISTICS BY METHOD (Averaged Across All Noise Levels)")
print("="*80)

method_stats = df_results.groupby('method').agg({
    'rmse': 'mean',
    'r2': 'mean',
    'abs_error_a': 'mean',
    'abs_error_b': 'mean',
    'abs_error_c': 'mean'
}).round(6)

print(method_stats)
print("="*80)

best_method = method_stats['rmse'].idxmin()
print(f"\n🏆 BEST OVERALL METHOD (by average RMSE): {best_method}")
print(f"   Average RMSE: {method_stats.loc[best_method, 'rmse']:.6f}")
print(f"   Average R²: {method_stats.loc[best_method, 'r2']:.6f}")


# ===== VISUALIZATION 1: Coefficient Recovery =====
print("\n\n" + "="*80)
print("GENERATING VISUALIZATIONS...")
print("="*80)

fig = plt.figure(figsize=(18, 12))

methods = df_results['method'].unique()
colors = {'No Denoising': 'red', 'Gaussian': 'blue', 
          'Savitzky-Golay': 'green', 'Spectral': 'purple'}
markers = {'No Denoising': 'x', 'Gaussian': 'o', 
           'Savitzky-Golay': 's', 'Spectral': '^'}

# Plot 1: Coefficient a recovery
ax1 = plt.subplot(3, 3, 1)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['a'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.axhline(y=-1.0, color='black', linestyle='--', linewidth=2, label='True value')
plt.xlabel('Noise Level (σ)')
plt.ylabel('Coefficient a (u_xx)')
plt.title('Coefficient a Recovery')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Coefficient b recovery
ax2 = plt.subplot(3, 3, 2)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['b'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.axhline(y=-1.0, color='black', linestyle='--', linewidth=2, label='True value')
plt.xlabel('Noise Level (σ)')
plt.ylabel('Coefficient b (u_xxxx)')
plt.title('Coefficient b Recovery')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Coefficient c recovery
ax3 = plt.subplot(3, 3, 3)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['c'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.axhline(y=-0.5, color='black', linestyle='--', linewidth=2, label='True value')
plt.xlabel('Noise Level (σ)')
plt.ylabel('Coefficient c ((u²)_x)')
plt.title('Coefficient c Recovery')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 4: RMSE comparison
ax4 = plt.subplot(3, 3, 4)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['rmse'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.xlabel('Noise Level (σ)')
plt.ylabel('RMSE')
plt.title('Root Mean Squared Error')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Plot 5: R² comparison
ax5 = plt.subplot(3, 3, 5)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['r2'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.xlabel('Noise Level (σ)')
plt.ylabel('R² Score')
plt.title('R² Coefficient of Determination')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 6: Absolute error in a
ax6 = plt.subplot(3, 3, 6)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['abs_error_a'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.xlabel('Noise Level (σ)')
plt.ylabel('|Error in a|')
plt.title('Absolute Error in Coefficient a')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 7: Absolute error in b
ax7 = plt.subplot(3, 3, 7)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['abs_error_b'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.xlabel('Noise Level (σ)')
plt.ylabel('|Error in b|')
plt.title('Absolute Error in Coefficient b')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 8: Absolute error in c
ax8 = plt.subplot(3, 3, 8)
for method in methods:
    data = df_results[df_results['method'] == method]
    plt.plot(data['sigma'], data['abs_error_c'], marker=markers[method], 
             label=method, linewidth=2, markersize=8, color=colors[method])
plt.xlabel('Noise Level (σ)')
plt.ylabel('|Error in c|')
plt.title('Absolute Error in Coefficient c')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 9: Bar chart of average performance
ax9 = plt.subplot(3, 3, 9)
x = np.arange(len(methods))
width = 0.35
rmse_means = [method_stats.loc[m, 'rmse'] for m in methods]
r2_means = [method_stats.loc[m, 'r2'] for m in methods]
ax9_twin = ax9.twinx()
bars1 = ax9.bar(x - width/2, rmse_means, width, label='Avg RMSE', alpha=0.8)
bars2 = ax9_twin.bar(x + width/2, r2_means, width, label='Avg R²', alpha=0.8, color='orange')
ax9.set_xlabel('Denoising Method')
ax9.set_ylabel('Average RMSE', color='blue')
ax9_twin.set_ylabel('Average R²', color='orange')
ax9.set_xticks(x)
ax9.set_xticklabels(methods, rotation=45, ha='right')
ax9.set_title('Average Performance by Method')
ax9.legend(loc='upper left')
ax9_twin.legend(loc='upper right')
ax9.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('denoising_finitediff_comparison.png', dpi=300, bbox_inches='tight')
print("    Saved: denoising_finitediff_comparison.png")


# ===== VISUALIZATION 2: Heatmaps =====
fig2, axes = plt.subplots(2, 2, figsize=(16, 12))

# Heatmap 1: RMSE
pivot_rmse = df_results.pivot(index='method', columns='sigma', values='rmse')
sns.heatmap(pivot_rmse, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=axes[0, 0],
            cbar_kws={'label': 'RMSE'})
axes[0, 0].set_title('RMSE by Method and Noise Level', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Noise Level (σ)')
axes[0, 0].set_ylabel('Denoising Method')

# Heatmap 2: R²
pivot_r2 = df_results.pivot(index='method', columns='sigma', values='r2')
sns.heatmap(pivot_r2, annot=True, fmt='.4f', cmap='RdYlGn', ax=axes[0, 1],
            cbar_kws={'label': 'R² Score'})
axes[0, 1].set_title('R² Score by Method and Noise Level', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Noise Level (σ)')
axes[0, 1].set_ylabel('Denoising Method')

# Heatmap 3: Absolute Error in coefficient a
pivot_err_a = df_results.pivot(index='method', columns='sigma', values='abs_error_a')
sns.heatmap(pivot_err_a, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=axes[1, 0],
            cbar_kws={'label': '|Error in a|'})
axes[1, 0].set_title('Absolute Error in Coefficient a', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Noise Level (σ)')
axes[1, 0].set_ylabel('Denoising Method')

# Heatmap 4: Absolute Error in coefficient b
pivot_err_b = df_results.pivot(index='method', columns='sigma', values='abs_error_b')
sns.heatmap(pivot_err_b, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=axes[1, 1],
            cbar_kws={'label': '|Error in b|'})
axes[1, 1].set_title('Absolute Error in Coefficient b', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Noise Level (σ)')
axes[1, 1].set_ylabel('Denoising Method')

plt.tight_layout()
plt.savefig('denoising_finitediff_heatmaps.png', dpi=300, bbox_inches='tight')
print("    Saved: denoising_finitediff_heatmaps.png")


# ===== Save Results to CSV =====
df_results.to_csv('denoising_finitediff_results.csv', index=False)
print("    Saved: denoising_finitediff_results.csv")

print("\n" + "="*80)
print("   DENOISING + FINITE DIFFERENCE SINDY EVALUATION COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. denoising_finitediff_comparison.png - Main comparison plots (9 subplots)")
print("  2. denoising_finitediff_heatmaps.png - Heatmaps of metrics")
print("  3. denoising_finitediff_results.csv - Complete results data")
print("="*80)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (18, 12)

print("="*80)
print("DENOISING COMPARISON: FFT vs FINITE DIFFERENCE")
print("="*80)

# ===== Load Results =====
print("\n📂 Loading results...")
try:
    fft_results = pd.read_csv('denoising_fft_results.csv')
    fd_results = pd.read_csv('denoising_finitediff_results.csv')
    print("    Successfully loaded both result files")
except FileNotFoundError as e:
    print(f"    Error: {e}")
    print("Please run both denoising scripts first to generate the CSV files.")
    exit()

print(f"\nFFT Results: {len(fft_results)} rows")
print(f"Finite Diff Results: {len(fd_results)} rows")

# ===== Overall Statistics =====
print("\n\n" + "="*80)
print("OVERALL STATISTICS: FFT vs FINITE DIFFERENCE")
print("="*80)

fft_stats = fft_results.groupby('method').agg({
    'rmse': 'mean',
    'r2': 'mean',
    'abs_error_a': 'mean',
    'abs_error_b': 'mean',
    'abs_error_c': 'mean'
}).round(6)

fd_stats = fd_results.groupby('method').agg({
    'rmse': 'mean',
    'r2': 'mean',
    'abs_error_a': 'mean',
    'abs_error_b': 'mean',
    'abs_error_c': 'mean'
}).round(6)

print("\n--- FFT-Based SINDy (Average Metrics) ---")
print(fft_stats)

print("\n--- Finite Difference SINDy (Average Metrics) ---")
print(fd_stats)

# ===== Best Method Per Derivative Type =====
print("\n\n" + "="*80)
print("BEST DENOISING METHOD BY DERIVATIVE TYPE")
print("="*80)

fft_best = fft_stats['rmse'].idxmin()
fd_best = fd_stats['rmse'].idxmin()

print(f"\n🏆 FFT Best Method: {fft_best}")
print(f"   Average RMSE: {fft_stats.loc[fft_best, 'rmse']:.6f}")
print(f"   Average R²: {fft_stats.loc[fft_best, 'r2']:.6f}")

print(f"\n🏆 Finite Difference Best Method: {fd_best}")
print(f"   Average RMSE: {fd_stats.loc[fd_best, 'rmse']:.6f}")
print(f"   Average R²: {fd_stats.loc[fd_best, 'r2']:.6f}")

# ===== Method-by-Method Comparison =====
print("\n\n" + "="*100)
print("METHOD-BY-METHOD COMPARISON (Average RMSE)")
print("="*100)
print(f"{'Method':>18} {'FFT RMSE':>15} {'FD RMSE':>15} {'Winner':>15} {'Improvement':>15}")
print("-"*100)

methods = fft_stats.index
for method in methods:
    fft_rmse = fft_stats.loc[method, 'rmse']
    fd_rmse = fd_stats.loc[method, 'rmse']
    winner = "FFT" if fft_rmse < fd_rmse else "Finite Diff"
    improvement = abs(fft_rmse - fd_rmse) / max(fft_rmse, fd_rmse) * 100
    print(f"{method:>18} {fft_rmse:>15.6f} {fd_rmse:>15.6f} {winner:>15} {improvement:>14.2f}%")

print("="*100)

# ===== Noise Level Analysis =====
print("\n\n" + "="*100)
print("BEST METHOD PER NOISE LEVEL")
print("="*100)
print(f"{'Sigma':>8} {'FFT Best':>18} {'FFT RMSE':>12} {'FD Best':>18} {'FD RMSE':>12} {'Overall Winner':>18}")
print("-"*100)

noise_levels = sorted(fft_results['sigma'].unique())
for sigma in noise_levels:
    fft_sigma = fft_results[fft_results['sigma'] == sigma]
    fd_sigma = fd_results[fd_results['sigma'] == sigma]
    
    fft_best_idx = fft_sigma['rmse'].idxmin()
    fd_best_idx = fd_sigma['rmse'].idxmin()
    
    fft_best_method = fft_results.loc[fft_best_idx, 'method']
    fft_best_rmse = fft_results.loc[fft_best_idx, 'rmse']
    
    fd_best_method = fd_results.loc[fd_best_idx, 'method']
    fd_best_rmse = fd_results.loc[fd_best_idx, 'rmse']
    
    overall_winner = "FFT" if fft_best_rmse < fd_best_rmse else "Finite Diff"
    
    print(f"{sigma:>8.3f} {fft_best_method:>18} {fft_best_rmse:>12.6f} {fd_best_method:>18} {fd_best_rmse:>12.6f} {overall_winner:>18}")

print("="*100)

# ===== VISUALIZATION 1: Side-by-Side Comparison =====
print("\n\n" + "="*80)
print("GENERATING COMPARISON VISUALIZATIONS...")
print("="*80)

fig, axes = plt.subplots(3, 4, figsize=(20, 14))

methods = ['No Denoising', 'Gaussian', 'Savitzky-Golay', 'Spectral']
colors_fft = 'blue'
colors_fd = 'green'

# Row 1: Coefficient a
for i, method in enumerate(methods):
    ax = axes[0, i]
    fft_data = fft_results[fft_results['method'] == method]
    fd_data = fd_results[fd_results['method'] == method]
    
    ax.plot(fft_data['sigma'], fft_data['a'], 'o-', label='FFT', 
            linewidth=2, markersize=8, color=colors_fft)
    ax.plot(fd_data['sigma'], fd_data['a'], 's--', label='Finite Diff', 
            linewidth=2, markersize=8, color=colors_fd)
    ax.axhline(y=-1.0, color='red', linestyle=':', linewidth=2, label='True')
    ax.set_title(f'{method}\nCoefficient a', fontweight='bold')
    ax.set_xlabel('Noise Level (σ)')
    ax.set_ylabel('Coefficient a')
    ax.legend()
    ax.grid(True, alpha=0.3)

# Row 2: RMSE
for i, method in enumerate(methods):
    ax = axes[1, i]
    fft_data = fft_results[fft_results['method'] == method]
    fd_data = fd_results[fd_results['method'] == method]
    
    ax.plot(fft_data['sigma'], fft_data['rmse'], 'o-', label='FFT', 
            linewidth=2, markersize=8, color=colors_fft)
    ax.plot(fd_data['sigma'], fd_data['rmse'], 's--', label='Finite Diff', 
            linewidth=2, markersize=8, color=colors_fd)
    ax.set_title(f'{method}\nRMSE', fontweight='bold')
    ax.set_xlabel('Noise Level (σ)')
    ax.set_ylabel('RMSE')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')

# Row 3: R²
for i, method in enumerate(methods):
    ax = axes[2, i]
    fft_data = fft_results[fft_results['method'] == method]
    fd_data = fd_results[fd_results['method'] == method]
    
    ax.plot(fft_data['sigma'], fft_data['r2'], 'o-', label='FFT', 
            linewidth=2, markersize=8, color=colors_fft)
    ax.plot(fd_data['sigma'], fd_data['r2'], 's--', label='Finite Diff', 
            linewidth=2, markersize=8, color=colors_fd)
    ax.set_title(f'{method}\nR² Score', fontweight='bold')
    ax.set_xlabel('Noise Level (σ)')
    ax.set_ylabel('R²')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('denoising_fft_vs_fd_comparison.png', dpi=300, bbox_inches='tight')
print("    Saved: denoising_fft_vs_fd_comparison.png")


# ===== VISUALIZATION 2: Average Performance Comparison =====
fig2, axes2 = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Average RMSE
ax1 = axes2[0, 0]
x = np.arange(len(methods))
width = 0.35
fft_rmse = [fft_stats.loc[m, 'rmse'] for m in methods]
fd_rmse = [fd_stats.loc[m, 'rmse'] for m in methods]
ax1.bar(x - width/2, fft_rmse, width, label='FFT', alpha=0.8, color='blue')
ax1.bar(x + width/2, fd_rmse, width, label='Finite Diff', alpha=0.8, color='green')
ax1.set_xlabel('Denoising Method')
ax1.set_ylabel('Average RMSE')
ax1.set_title('Average RMSE by Method', fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(methods, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Average R²
ax2 = axes2[0, 1]
fft_r2 = [fft_stats.loc[m, 'r2'] for m in methods]
fd_r2 = [fd_stats.loc[m, 'r2'] for m in methods]
ax2.bar(x - width/2, fft_r2, width, label='FFT', alpha=0.8, color='blue')
ax2.bar(x + width/2, fd_r2, width, label='Finite Diff', alpha=0.8, color='green')
ax2.set_xlabel('Denoising Method')
ax2.set_ylabel('Average R²')
ax2.set_title('Average R² Score by Method', fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(methods, rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# Plot 3: Average Absolute Error in coefficient a
ax3 = axes2[1, 0]
fft_err_a = [fft_stats.loc[m, 'abs_error_a'] for m in methods]
fd_err_a = [fd_stats.loc[m, 'abs_error_a'] for m in methods]
ax3.bar(x - width/2, fft_err_a, width, label='FFT', alpha=0.8, color='blue')
ax3.bar(x + width/2, fd_err_a, width, label='Finite Diff', alpha=0.8, color='green')
ax3.set_xlabel('Denoising Method')
ax3.set_ylabel('Average |Error in a|')
ax3.set_title('Average Absolute Error in Coefficient a', fontweight='bold')
ax3.set_xticks(x)
ax3.set_xticklabels(methods, rotation=45, ha='right')
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

# Plot 4: Average Absolute Error in coefficient b
ax4 = axes2[1, 1]
fft_err_b = [fft_stats.loc[m, 'abs_error_b'] for m in methods]
fd_err_b = [fd_stats.loc[m, 'abs_error_b'] for m in methods]
ax4.bar(x - width/2, fft_err_b, width, label='FFT', alpha=0.8, color='blue')
ax4.bar(x + width/2, fd_err_b, width, label='Finite Diff', alpha=0.8, color='green')
ax4.set_xlabel('Denoising Method')
ax4.set_ylabel('Average |Error in b|')
ax4.set_title('Average Absolute Error in Coefficient b', fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(methods, rotation=45, ha='right')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('denoising_average_performance.png', dpi=300, bbox_inches='tight')
print("    Saved: denoising_average_performance.png")


# ===== VISUALIZATION 3: Heatmap Comparison =====
fig3, axes3 = plt.subplots(2, 2, figsize=(16, 12))

# FFT RMSE Heatmap
pivot_fft_rmse = fft_results.pivot(index='method', columns='sigma', values='rmse')
sns.heatmap(pivot_fft_rmse, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=axes3[0, 0],
            cbar_kws={'label': 'RMSE'})
axes3[0, 0].set_title('FFT: RMSE by Method and Noise Level', fontsize=12, fontweight='bold')
axes3[0, 0].set_xlabel('Noise Level (σ)')
axes3[0, 0].set_ylabel('Denoising Method')

# Finite Diff RMSE Heatmap
pivot_fd_rmse = fd_results.pivot(index='method', columns='sigma', values='rmse')
sns.heatmap(pivot_fd_rmse, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=axes3[0, 1],
            cbar_kws={'label': 'RMSE'})
axes3[0, 1].set_title('Finite Diff: RMSE by Method and Noise Level', fontsize=12, fontweight='bold')
axes3[0, 1].set_xlabel('Noise Level (σ)')
axes3[0, 1].set_ylabel('Denoising Method')

# FFT R² Heatmap
pivot_fft_r2 = fft_results.pivot(index='method', columns='sigma', values='r2')
sns.heatmap(pivot_fft_r2, annot=True, fmt='.4f', cmap='RdYlGn', ax=axes3[1, 0],
            cbar_kws={'label': 'R² Score'})
axes3[1, 0].set_title('FFT: R² Score by Method and Noise Level', fontsize=12, fontweight='bold')
axes3[1, 0].set_xlabel('Noise Level (σ)')
axes3[1, 0].set_ylabel('Denoising Method')

# Finite Diff R² Heatmap
pivot_fd_r2 = fd_results.pivot(index='method', columns='sigma', values='r2')
sns.heatmap(pivot_fd_r2, annot=True, fmt='.4f', cmap='RdYlGn', ax=axes3[1, 1],
            cbar_kws={'label': 'R² Score'})
axes3[1, 1].set_title('Finite Diff: R² Score by Method and Noise Level', fontsize=12, fontweight='bold')
axes3[1, 1].set_xlabel('Noise Level (σ)')
axes3[1, 1].set_ylabel('Denoising Method')

plt.tight_layout()
plt.savefig('denoising_heatmap_comparison.png', dpi=300, bbox_inches='tight')
print("    Saved: denoising_heatmap_comparison.png")


# ===== Save Comprehensive Comparison =====
comparison_data = []
for sigma in noise_levels:
    for method in methods:
        fft_row = fft_results[(fft_results['sigma'] == sigma) & (fft_results['method'] == method)]
        fd_row = fd_results[(fd_results['sigma'] == sigma) & (fd_results['method'] == method)]
        
        if not fft_row.empty and not fd_row.empty:
            comparison_data.append({
                'sigma': sigma,
                'method': method,
                'fft_a': fft_row.iloc[0]['a'],
                'fd_a': fd_row.iloc[0]['a'],
                'fft_b': fft_row.iloc[0]['b'],
                'fd_b': fd_row.iloc[0]['b'],
                'fft_c': fft_row.iloc[0]['c'],
                'fd_c': fd_row.iloc[0]['c'],
                'fft_rmse': fft_row.iloc[0]['rmse'],
                'fd_rmse': fd_row.iloc[0]['rmse'],
                'fft_r2': fft_row.iloc[0]['r2'],
                'fd_r2': fd_row.iloc[0]['r2']
            })

df_comparison = pd.DataFrame(comparison_data)
df_comparison.to_csv('denoising_fft_vs_fd_comparison.csv', index=False)
print("    Saved: denoising_fft_vs_fd_comparison.csv")

print("\n" + "="*80)
print("   COMPREHENSIVE COMPARISON COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. denoising_fft_vs_fd_comparison.png - Side-by-side comparisons")
print("  2. denoising_average_performance.png - Average performance metrics")
print("  3. denoising_heatmap_comparison.png - Heatmap comparisons")
print("  4. denoising_fft_vs_fd_comparison.csv - Complete comparison data")
print("="*80)

Phase 3

Cross-correlation registration + FFT derivatives

In [ ]:
import h5py
import numpy as np
from scipy.fft import fft, ifft, fftfreq
from scipy.signal import correlate
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 10)

print("="*80)
print("PHASE 3A: SPATIAL TRANSLATION - CROSS-CORRELATION REGISTRATION + FFT SINDY")
print("="*80)

# ===== Spectral derivative functions =====
def spectral_derivative(u_snapshot, k):
    """First derivative using FFT"""
    u_hat = fft(u_snapshot)
    du_hat = 1j * k * u_hat
    return np.real(ifft(du_hat))

def spectral_second_derivative(u_snapshot, k):
    """Second derivative using FFT"""
    u_hat = fft(u_snapshot)
    d2u_hat = -(k**2) * u_hat
    return np.real(ifft(d2u_hat))

def spectral_fourth_derivative(u_snapshot, k):
    """Fourth derivative using FFT"""
    u_hat = fft(u_snapshot)
    d4u_hat = (k**4) * u_hat
    return np.real(ifft(d4u_hat))


# ===== Cross-Correlation Registration =====
def find_shift_crosscorr(frame, reference):
    """
    Find the shift between frame and reference using cross-correlation
    Returns: optimal shift in pixels
    """
    # Compute cross-correlation
    correlation = correlate(frame, reference, mode='same', method='fft')
    
    # Find peak location
    max_idx = np.argmax(correlation)
    center = len(frame) // 2
    shift = max_idx - center
    
    return shift


def align_data_crosscorr(u_data, reference_frame):
    """
    Align all frames to reference using cross-correlation
    Returns: aligned data, detected shifts
    """
    aligned_data = np.zeros_like(u_data)
    detected_shifts = []
    
    for i in range(u_data.shape[0]):
        shift = find_shift_crosscorr(u_data[i], reference_frame)
        aligned_data[i] = np.roll(u_data[i], -shift)  # Undo the shift
        detected_shifts.append(shift)
    
    return aligned_data, np.array(detected_shifts)


# ===== Registration Quality Metrics =====
def compute_registration_metrics(detected_shifts, aligned_data, reference_frame):
    """
    Compute metrics to evaluate registration quality
    """
    # Shift statistics
    mean_shift = np.mean(np.abs(detected_shifts))
    std_shift = np.std(detected_shifts)
    max_shift = np.max(np.abs(detected_shifts))
    
    # Alignment quality: correlation with reference
    correlations = []
    for i in range(aligned_data.shape[0]):
        corr = np.corrcoef(aligned_data[i], reference_frame)[0, 1]
        correlations.append(corr)
    
    mean_correlation = np.mean(correlations)
    
    # Residual variance (lower is better)
    residuals = aligned_data - reference_frame
    residual_variance = np.var(residuals)
    
    return {
        'mean_absolute_shift': mean_shift,
        'std_shift': std_shift,
        'max_shift': max_shift,
        'mean_correlation': mean_correlation,
        'residual_variance': residual_variance,
        'detected_shifts': detected_shifts
    }


# ===== Evaluation Function =====
def evaluate_sindy_fft(u, x, t, method_name="Unknown"):
    """
    Apply FFT-based SINDy and compute evaluation metrics
    Returns: dict with coefficients, R², RMSE, errors
    """
    Nx = len(x)
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    
    # Define wavenumbers
    k = 2 * np.pi * fftfreq(Nx, d=dx)
    
    # Compute temporal derivative (central difference)
    u_t = (u[2:] - u[:-2]) / (2 * dt)
    u_mid = u[1:-1]
    
    # Build feature library using FFT
    Theta = []
    ut_flat = []
    
    for snapshot, ut_snapshot in zip(u_mid, u_t):
        ux = spectral_derivative(snapshot, k)
        uxx = spectral_second_derivative(snapshot, k)
        uxxxx = spectral_fourth_derivative(snapshot, k)
        
        # Feature terms: [u_xx, u_xxxx, u*u_x]
        Theta_snapshot = np.vstack([
            uxx,
            uxxxx,
            snapshot * ux
        ]).T  # shape (Nx, 3)
        
        Theta.append(Theta_snapshot)
        ut_flat.append(ut_snapshot)
    
    Theta = np.vstack(Theta)
    ut_flat = np.hstack(ut_flat)
    
    # Normalize features
    Theta_mean = Theta.mean(axis=0)
    Theta_std = Theta.std(axis=0)
    Theta_norm = (Theta - Theta_mean) / Theta_std
    
    # Lasso regression
    lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000)
    lasso.fit(Theta_norm, ut_flat)
    coeffs = lasso.coef_ / Theta_std
    
    # Extract coefficients
    a = coeffs[0]  # u_xx
    b = coeffs[1]  # u_xxxx
    c = coeffs[2]  # u*u_x
    
    # Predict u_t
    u_t_pred = Theta @ coeffs
    
    # Compute metrics
    rmse = np.sqrt(np.mean((ut_flat - u_t_pred)**2))
    r2 = r2_score(ut_flat, u_t_pred)
    
    # True coefficients
    a_true, b_true, c_true = -1.0, -1.0, -1.0
    
    # Coefficient errors
    error_a = a - a_true
    error_b = b - b_true
    error_c = c - c_true
    
    abs_error_a = abs(error_a)
    abs_error_b = abs(error_b)
    abs_error_c = abs(error_c)
    
    return {
        'method': method_name,
        'a': a,
        'b': b,
        'c': c,
        'error_a': error_a,
        'error_b': error_b,
        'error_c': error_c,
        'abs_error_a': abs_error_a,
        'abs_error_b': abs_error_b,
        'abs_error_c': abs_error_c,
        'rmse': rmse,
        'r2': r2
    }


# ===== Main Processing =====

print("\n📂 Loading spatial translation data...")
with h5py.File("test_solving_euler_ks_spatial_shift.h5", "r") as f:
    u_shifted = np.array(f["u"])
    x = np.array(f["x"])
    t = np.array(f["t"])

print(f"Loaded: test_solving_euler_ks_spatial_shift.h5")
print(f"Shape: {u_shifted.shape}")

# ===== Compute Mean Frame as Reference =====
print("\n🎯 Computing mean frame as reference...")
reference_frame = np.mean(u_shifted, axis=0)
print(f"Reference frame shape: {reference_frame.shape}")

# ===== Method 1: No Registration (Baseline) =====
print("\n" + "="*80)
print("METHOD 1: NO REGISTRATION (BASELINE)")
print("="*80)

result_baseline = evaluate_sindy_fft(u_shifted, x, t, method_name="No Registration")

print(f"\nRecovered Coefficients:")
print(f"  a (u_xx):   {result_baseline['a']:.6f}  (true: -1.0)")
print(f"  b (u_xxxx): {result_baseline['b']:.6f}  (true: -1.0)")
print(f"  c (u*u_x):  {result_baseline['c']:.6f}  (true: -1.0)")
print(f"\nEvaluation Metrics:")
print(f"  RMSE: {result_baseline['rmse']:.6f}")
print(f"  R²:   {result_baseline['r2']:.6f}")

# ===== Method 2: Cross-Correlation Registration =====
print("\n" + "="*80)
print("METHOD 2: CROSS-CORRELATION REGISTRATION")
print("="*80)

print("Aligning frames using cross-correlation...")
u_aligned, detected_shifts = align_data_crosscorr(u_shifted, reference_frame)

print(f"    Alignment complete!")
print(f"  Detected shifts: min={detected_shifts.min()}, max={detected_shifts.max()}")
print(f"  Mean absolute shift: {np.mean(np.abs(detected_shifts)):.2f} pixels")

# Compute registration quality metrics
reg_metrics = compute_registration_metrics(detected_shifts, u_aligned, reference_frame)

print(f"\nRegistration Quality Metrics:")
print(f"  Mean Absolute Shift: {reg_metrics['mean_absolute_shift']:.4f} pixels")
print(f"  Std Shift:           {reg_metrics['std_shift']:.4f} pixels")
print(f"  Max Shift:           {reg_metrics['max_shift']:.1f} pixels")
print(f"  Mean Correlation:    {reg_metrics['mean_correlation']:.6f}")
print(f"  Residual Variance:   {reg_metrics['residual_variance']:.6f}")

# Apply SINDy to aligned data
result_registered = evaluate_sindy_fft(u_aligned, x, t, method_name="Cross-Correlation")

print(f"\nRecovered Coefficients (After Registration):")
print(f"  a (u_xx):   {result_registered['a']:.6f}  (true: -1.0)")
print(f"  b (u_xxxx): {result_registered['b']:.6f}  (true: -1.0)")
print(f"  c (u*u_x):  {result_registered['c']:.6f}  (true: -1.0)")
print(f"\nEvaluation Metrics:")
print(f"  RMSE: {result_registered['rmse']:.6f}")
print(f"  R²:   {result_registered['r2']:.6f}")

# ===== Load Clean Data for Comparison =====
print("\n" + "="*80)
print("LOADING CLEAN DATA FOR COMPARISON")
print("="*80)

with h5py.File("test_solving_euler_ks_clean.h5", "r") as f:
    u_clean = np.array(f["u"])

result_clean = evaluate_sindy_fft(u_clean, x, t, method_name="Clean Data")

print(f"\nRecovered Coefficients (Clean Data):")
print(f"  a (u_xx):   {result_clean['a']:.6f}  (true: -1.0)")
print(f"  b (u_xxxx): {result_clean['b']:.6f}  (true: -1.0)")
print(f"  c (u*u_x):  {result_clean['c']:.6f}  (true: -1.0)")
print(f"\nEvaluation Metrics:")
print(f"  RMSE: {result_clean['rmse']:.6f}")
print(f"  R²:   {result_clean['r2']:.6f}")

# ===== Summary Comparison =====
print("\n\n" + "="*80)
print("SUMMARY COMPARISON")
print("="*80)

results = [result_clean, result_baseline, result_registered]
df_results = pd.DataFrame(results)

print(f"\n{'Method':<25} {'a (u_xx)':>12} {'b (u_xxxx)':>12} {'c (u*u_x)':>12} {'RMSE':>12} {'R²':>12}")
print("-"*80)
for _, row in df_results.iterrows():
    print(f"{row['method']:<25} {row['a']:>12.6f} {row['b']:>12.6f} {row['c']:>12.6f} "
          f"{row['rmse']:>12.6f} {row['r2']:>12.6f}")
print("="*80)
print(f"{'TRUE VALUES:':<25} {-1.0:>12.6f} {-1.0:>12.6f} {-1.0:>12.6f}")
print("="*80)

# ===== Improvement Analysis =====
print("\n" + "="*80)
print("IMPROVEMENT FROM REGISTRATION")
print("="*80)

improvement_rmse = (result_baseline['rmse'] - result_registered['rmse']) / result_baseline['rmse'] * 100
improvement_r2 = (result_registered['r2'] - result_baseline['r2']) / max(abs(result_baseline['r2']), 1e-10) * 100

print(f"\nRMSE Improvement:  {improvement_rmse:+.2f}%")
print(f"R² Improvement:    {improvement_r2:+.2f}%")
print(f"\nCoefficient Error Reduction:")
print(f"  a: {result_baseline['abs_error_a']:.6f} → {result_registered['abs_error_a']:.6f} "
      f"({(result_baseline['abs_error_a'] - result_registered['abs_error_a'])/result_baseline['abs_error_a']*100:+.1f}%)")
print(f"  b: {result_baseline['abs_error_b']:.6f} → {result_registered['abs_error_b']:.6f} "
      f"({(result_baseline['abs_error_b'] - result_registered['abs_error_b'])/result_baseline['abs_error_b']*100:+.1f}%)")
print(f"  c: {result_baseline['abs_error_c']:.6f} → {result_registered['abs_error_c']:.6f} "
      f"({(result_baseline['abs_error_c'] - result_registered['abs_error_c'])/result_baseline['abs_error_c']*100:+.1f}%)")

# ===== VISUALIZATIONS =====
print("\n\n" + "="*80)
print("GENERATING VISUALIZATIONS...")
print("="*80)

fig = plt.figure(figsize=(18, 12))

# Plot 1: Detected Shifts Histogram
ax1 = plt.subplot(3, 3, 1)
plt.hist(detected_shifts, bins=range(int(detected_shifts.min())-1, int(detected_shifts.max())+2), 
         edgecolor='black', alpha=0.7)
plt.xlabel('Detected Shift (pixels)')
plt.ylabel('Frequency')
plt.title('Distribution of Detected Shifts\n(Cross-Correlation)')
plt.grid(True, alpha=0.3)
plt.axvline(0, color='red', linestyle='--', linewidth=2, label='Zero shift')
plt.legend()

# Plot 2: Shifts Over Time
ax2 = plt.subplot(3, 3, 2)
plt.plot(detected_shifts, 'o-', markersize=2, linewidth=0.5)
plt.xlabel('Frame Index')
plt.ylabel('Detected Shift (pixels)')
plt.title('Detected Shifts Over Time')
plt.grid(True, alpha=0.3)
plt.axhline(0, color='red', linestyle='--', linewidth=2)

# Plot 3: Sample Frame Comparison
ax3 = plt.subplot(3, 3, 3)
sample_idx = len(u_shifted) // 2
plt.plot(x, u_shifted[sample_idx], 'r-', label='Before Registration', alpha=0.7, linewidth=2)
plt.plot(x, u_aligned[sample_idx], 'b-', label='After Registration', alpha=0.7, linewidth=2)
plt.plot(x, reference_frame, 'k--', label='Reference (Mean)', alpha=0.5, linewidth=2)
plt.xlabel('x')
plt.ylabel('u(x)')
plt.title(f'Sample Frame Alignment\n(Frame {sample_idx})')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 4: Coefficient Comparison
ax4 = plt.subplot(3, 3, 4)
methods = ['Clean', 'No Reg', 'Cross-Corr']
a_vals = [result_clean['a'], result_baseline['a'], result_registered['a']]
b_vals = [result_clean['b'], result_baseline['b'], result_registered['b']]
c_vals = [result_clean['c'], result_baseline['c'], result_registered['c']]
x_pos = np.arange(len(methods))
width = 0.25
plt.bar(x_pos - width, a_vals, width, label='a (u_xx)', alpha=0.8)
plt.bar(x_pos, b_vals, width, label='b (u_xxxx)', alpha=0.8)
plt.bar(x_pos + width, c_vals, width, label='c (u*u_x)', alpha=0.8)
plt.axhline(y=-1.0, color='red', linestyle='--', linewidth=2, label='True')
plt.ylabel('Coefficient Value')
plt.title('Recovered Coefficients Comparison')
plt.xticks(x_pos, methods)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

# Plot 5: RMSE Comparison
ax5 = plt.subplot(3, 3, 5)
rmse_vals = [result_clean['rmse'], result_baseline['rmse'], result_registered['rmse']]
colors = ['green', 'red', 'blue']
plt.bar(methods, rmse_vals, color=colors, alpha=0.7, edgecolor='black')
plt.ylabel('RMSE')
plt.title('RMSE Comparison')
plt.grid(True, alpha=0.3, axis='y')
plt.yscale('log')

# Plot 6: R² Comparison
ax6 = plt.subplot(3, 3, 6)
r2_vals = [result_clean['r2'], result_baseline['r2'], result_registered['r2']]
plt.bar(methods, r2_vals, color=colors, alpha=0.7, edgecolor='black')
plt.ylabel('R² Score')
plt.title('R² Comparison')
plt.grid(True, alpha=0.3, axis='y')
plt.axhline(y=1.0, color='black', linestyle='--', linewidth=1)

# Plot 7: Absolute Errors Comparison
ax7 = plt.subplot(3, 3, 7)
err_a = [result_clean['abs_error_a'], result_baseline['abs_error_a'], result_registered['abs_error_a']]
err_b = [result_clean['abs_error_b'], result_baseline['abs_error_b'], result_registered['abs_error_b']]
err_c = [result_clean['abs_error_c'], result_baseline['abs_error_c'], result_registered['abs_error_c']]
x_pos = np.arange(len(methods))
width = 0.25
plt.bar(x_pos - width, err_a, width, label='|Err a|', alpha=0.8)
plt.bar(x_pos, err_b, width, label='|Err b|', alpha=0.8)
plt.bar(x_pos + width, err_c, width, label='|Err c|', alpha=0.8)
plt.ylabel('Absolute Error')
plt.title('Absolute Coefficient Errors')
plt.xticks(x_pos, methods)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.yscale('log')

# Plot 8: Spatiotemporal plot - Before Registration
ax8 = plt.subplot(3, 3, 8)
im1 = plt.imshow(u_shifted[:500].T, cmap='RdBu', aspect='auto', origin='lower',
                 extent=(0, 500*t[1], 0, x[-1]))
plt.xlabel('Time')
plt.ylabel('Space (x)')
plt.title('Before Registration\n(First 500 frames)')
plt.colorbar(im1, ax=ax8)

# Plot 9: Spatiotemporal plot - After Registration
ax9 = plt.subplot(3, 3, 9)
im2 = plt.imshow(u_aligned[:500].T, cmap='RdBu', aspect='auto', origin='lower',
                 extent=(0, 500*t[1], 0, x[-1]))
plt.xlabel('Time')
plt.ylabel('Space (x)')
plt.title('After Registration\n(First 500 frames)')
plt.colorbar(im2, ax=ax9)

plt.tight_layout()
plt.savefig('registration_crosscorr_fft_results.png', dpi=300, bbox_inches='tight')
print("    Saved: registration_crosscorr_fft_results.png")

# ===== Save Results =====
results_dict = {
    'method': ['Clean Data', 'No Registration', 'Cross-Correlation'],
    'a': [result_clean['a'], result_baseline['a'], result_registered['a']],
    'b': [result_clean['b'], result_baseline['b'], result_registered['b']],
    'c': [result_clean['c'], result_baseline['c'], result_registered['c']],
    'rmse': [result_clean['rmse'], result_baseline['rmse'], result_registered['rmse']],
    'r2': [result_clean['r2'], result_baseline['r2'], result_registered['r2']],
    'abs_error_a': [result_clean['abs_error_a'], result_baseline['abs_error_a'], result_registered['abs_error_a']],
    'abs_error_b': [result_clean['abs_error_b'], result_baseline['abs_error_b'], result_registered['abs_error_b']],
    'abs_error_c': [result_clean['abs_error_c'], result_baseline['abs_error_c'], result_registered['abs_error_c']]
}

# Add registration metrics for cross-correlation method
results_dict['mean_shift'] = [0.0, np.nan, reg_metrics['mean_absolute_shift']]
results_dict['std_shift'] = [0.0, np.nan, reg_metrics['std_shift']]
results_dict['max_shift'] = [0.0, np.nan, reg_metrics['max_shift']]
results_dict['mean_correlation'] = [1.0, np.nan, reg_metrics['mean_correlation']]

df_results_full = pd.DataFrame(results_dict)
df_results_full.to_csv('registration_crosscorr_fft_results.csv', index=False)
print("    Saved: registration_crosscorr_fft_results.csv")

print("\n" + "="*80)
print("   CROSS-CORRELATION REGISTRATION + FFT SINDY COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. registration_crosscorr_fft_results.png - Visualization")
print("  2. registration_crosscorr_fft_results.csv - Results data")
print("="*80)

Cross-correlation registration + Finite Difference derivatives

In [ ]:
import h5py
import numpy as np
from scipy.signal import correlate
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 10)

print("="*80)
print("PHASE 3A: SPATIAL TRANSLATION - CROSS-CORRELATION REGISTRATION + FINITE DIFFERENCE SINDY")
print("="*80)

# ===== Cross-Correlation Registration =====
def find_shift_crosscorr(frame, reference):
    """Find the shift between frame and reference using cross-correlation"""
    correlation = correlate(frame, reference, mode='same', method='fft')
    max_idx = np.argmax(correlation)
    center = len(frame) // 2
    shift = max_idx - center
    return shift

def align_data_crosscorr(u_data, reference_frame):
    """Align all frames to reference using cross-correlation"""
    aligned_data = np.zeros_like(u_data)
    detected_shifts = []
    
    for i in range(u_data.shape[0]):
        shift = find_shift_crosscorr(u_data[i], reference_frame)
        aligned_data[i] = np.roll(u_data[i], -shift)
        detected_shifts.append(shift)
    
    return aligned_data, np.array(detected_shifts)

# ===== Registration Quality Metrics =====
def compute_registration_metrics(detected_shifts, aligned_data, reference_frame):
    """Compute metrics to evaluate registration quality"""
    mean_shift = np.mean(np.abs(detected_shifts))
    std_shift = np.std(detected_shifts)
    max_shift = np.max(np.abs(detected_shifts))
    
    correlations = []
    for i in range(aligned_data.shape[0]):
        corr = np.corrcoef(aligned_data[i], reference_frame)[0, 1]
        correlations.append(corr)
    
    mean_correlation = np.mean(correlations)
    residuals = aligned_data - reference_frame
    residual_variance = np.var(residuals)
    
    return {
        'mean_absolute_shift': mean_shift,
        'std_shift': std_shift,
        'max_shift': max_shift,
        'mean_correlation': mean_correlation,
        'residual_variance': residual_variance,
        'detected_shifts': detected_shifts
    }

# ===== Evaluation Function with Finite Difference =====
def evaluate_sindy_finite_diff(u, x, t, method_name="Unknown"):
    """Apply Finite Difference SINDy and compute evaluation metrics"""
    dx = float(x[1] - x[0])
    dt = float(t[1] - t[0])
    
    # Finite difference operators
    def d1(U): 
        return (np.roll(U, -1, 1) - np.roll(U, 1, 1)) / (2*dx)
    
    def d2(U): 
        return (np.roll(U, -1, 1) - 2*U + np.roll(U, 1, 1)) / (dx**2)
    
    def d4(U):
        return (np.roll(U, -2, 1) - 4*np.roll(U, -1, 1) + 6*U - 
                4*np.roll(U, 1, 1) + np.roll(U, 2, 1)) / (dx**4)
    
    # Compute derivatives
    U_t = (u[1:] - u[:-1]) / dt
    U = u[:-1]
    U_x = d1(U)
    U_xx = d2(U)
    U_xxxx = d4(U)
    U2_x = d1(U**2)
    
    # Build feature matrix
    Theta = np.column_stack([
        np.ones_like(U).ravel(),
        U.ravel(),
        U_x.ravel(),
        U_xx.ravel(),
        U_xxxx.ravel(),
        U2_x.ravel()
    ])
    y = U_t.ravel()
    names = ["1", "u", "u_x", "u_xx", "u_xxxx", "(u^2)_x"]
    
    # Normalize
    Theta_mean = Theta.mean(0)
    Theta_std = Theta.std(0) + 1e-8
    Theta_n = (Theta - Theta_mean) / Theta_std
    
    # Lasso regression
    lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000, tol=1e-6)
    lasso.fit(Theta_n, y)
    coef = lasso.coef_ / Theta_std
    
    # Extract coefficients
    a = coef[names.index("u_xx")]
    b = coef[names.index("u_xxxx")]
    c_phys = coef[names.index("(u^2)_x")]
    
    # Predict u_t
    y_pred = Theta @ coef
    
    # Compute metrics
    rmse = np.sqrt(np.mean((y - y_pred)**2))
    r2 = r2_score(y, y_pred)
    
    # True coefficients (physical form)
    a_true, b_true, c_true = -1.0, -1.0, -0.5
    
    # Coefficient errors
    error_a = a - a_true
    error_b = b - b_true
    error_c = c_phys - c_true
    
    abs_error_a = abs(error_a)
    abs_error_b = abs(error_b)
    abs_error_c = abs(error_c)
    
    return {
        'method': method_name,
        'a': a,
        'b': b,
        'c': c_phys,
        'error_a': error_a,
        'error_b': error_b,
        'error_c': error_c,
        'abs_error_a': abs_error_a,
        'abs_error_b': abs_error_b,
        'abs_error_c': abs_error_c,
        'rmse': rmse,
        'r2': r2
    }

# ===== Main Processing =====
print("\n📂 Loading spatial translation data...")
with h5py.File("test_solving_euler_ks_spatial_shift.h5", "r") as f:
    u_shifted = np.array(f["u"], dtype=np.float32)
    x = np.array(f["x"], dtype=np.float32)
    t = np.array(f["t"], dtype=np.float32)

print(f"Loaded: test_solving_euler_ks_spatial_shift.h5")
print(f"Shape: {u_shifted.shape}")

# Compute Mean Frame as Reference
print("\n🎯 Computing mean frame as reference...")
reference_frame = np.mean(u_shifted, axis=0)

# Method 1: No Registration (Baseline)
print("\n" + "="*80)
print("METHOD 1: NO REGISTRATION (BASELINE)")
print("="*80)

result_baseline = evaluate_sindy_finite_diff(u_shifted, x, t, method_name="No Registration")

print(f"\nRecovered Coefficients:")
print(f"  a (u_xx):    {result_baseline['a']:.6f}  (true: -1.0)")
print(f"  b (u_xxxx):  {result_baseline['b']:.6f}  (true: -1.0)")
print(f"  c ((u²)_x):  {result_baseline['c']:.6f}  (true: -0.5)")
print(f"\nEvaluation Metrics:")
print(f"  RMSE: {result_baseline['rmse']:.6f}")
print(f"  R²:   {result_baseline['r2']:.6f}")

# Method 2: Cross-Correlation Registration
print("\n" + "="*80)
print("METHOD 2: CROSS-CORRELATION REGISTRATION")
print("="*80)

print("Aligning frames using cross-correlation...")
u_aligned, detected_shifts = align_data_crosscorr(u_shifted, reference_frame)

print(f"    Alignment complete!")
print(f"  Detected shifts: min={detected_shifts.min()}, max={detected_shifts.max()}")
print(f"  Mean absolute shift: {np.mean(np.abs(detected_shifts)):.2f} pixels")

# Compute registration quality metrics
reg_metrics = compute_registration_metrics(detected_shifts, u_aligned, reference_frame)

print(f"\nRegistration Quality Metrics:")
print(f"  Mean Absolute Shift: {reg_metrics['mean_absolute_shift']:.4f} pixels")
print(f"  Std Shift:           {reg_metrics['std_shift']:.4f} pixels")
print(f"  Max Shift:           {reg_metrics['max_shift']:.1f} pixels")
print(f"  Mean Correlation:    {reg_metrics['mean_correlation']:.6f}")
print(f"  Residual Variance:   {reg_metrics['residual_variance']:.6f}")

# Apply SINDy to aligned data
result_registered = evaluate_sindy_finite_diff(u_aligned, x, t, method_name="Cross-Correlation")

print(f"\nRecovered Coefficients (After Registration):")
print(f"  a (u_xx):    {result_registered['a']:.6f}  (true: -1.0)")
print(f"  b (u_xxxx):  {result_registered['b']:.6f}  (true: -1.0)")
print(f"  c ((u²)_x):  {result_registered['c']:.6f}  (true: -0.5)")
print(f"\nEvaluation Metrics:")
print(f"  RMSE: {result_registered['rmse']:.6f}")
print(f"  R²:   {result_registered['r2']:.6f}")

# Load Clean Data for Comparison
print("\n" + "="*80)
print("LOADING CLEAN DATA FOR COMPARISON")
print("="*80)

with h5py.File("test_solving_euler_ks_clean.h5", "r") as f:
    u_clean = np.array(f["u"], dtype=np.float32)

result_clean = evaluate_sindy_finite_diff(u_clean, x, t, method_name="Clean Data")

print(f"\nRecovered Coefficients (Clean Data):")
print(f"  a (u_xx):    {result_clean['a']:.6f}  (true: -1.0)")
print(f"  b (u_xxxx):  {result_clean['b']:.6f}  (true: -1.0)")
print(f"  c ((u²)_x):  {result_clean['c']:.6f}  (true: -0.5)")
print(f"\nEvaluation Metrics:")
print(f"  RMSE: {result_clean['rmse']:.6f}")
print(f"  R²:   {result_clean['r2']:.6f}")

# Summary Comparison
print("\n\n" + "="*80)
print("SUMMARY COMPARISON")
print("="*80)

print(f"\n{'Method':<25} {'a (u_xx)':>12} {'b (u_xxxx)':>12} {'c ((u²)_x)':>12} {'RMSE':>12} {'R²':>12}")
print("-"*80)
results = [result_clean, result_baseline, result_registered]
for r in results:
    print(f"{r['method']:<25} {r['a']:>12.6f} {r['b']:>12.6f} {r['c']:>12.6f} "
          f"{r['rmse']:>12.6f} {r['r2']:>12.6f}")
print("="*80)
print(f"{'TRUE VALUES:':<25} {-1.0:>12.6f} {-1.0:>12.6f} {-0.5:>12.6f}")
print("="*80)

# Improvement Analysis
print("\n" + "="*80)
print("IMPROVEMENT FROM REGISTRATION")
print("="*80)

improvement_rmse = (result_baseline['rmse'] - result_registered['rmse']) / result_baseline['rmse'] * 100
improvement_r2 = (result_registered['r2'] - result_baseline['r2']) / max(abs(result_baseline['r2']), 1e-10) * 100

print(f"\nRMSE Improvement:  {improvement_rmse:+.2f}%")
print(f"R² Improvement:    {improvement_r2:+.2f}%")
print(f"\nCoefficient Error Reduction:")
print(f"  a: {result_baseline['abs_error_a']:.6f} → {result_registered['abs_error_a']:.6f}")
print(f"  b: {result_baseline['abs_error_b']:.6f} → {result_registered['abs_error_b']:.6f}")
print(f"  c: {result_baseline['abs_error_c']:.6f} → {result_registered['abs_error_c']:.6f}")

# Save Results
results_dict = {
    'method': ['Clean Data', 'No Registration', 'Cross-Correlation'],
    'a': [result_clean['a'], result_baseline['a'], result_registered['a']],
    'b': [result_clean['b'], result_baseline['b'], result_registered['b']],
    'c': [result_clean['c'], result_baseline['c'], result_registered['c']],
    'rmse': [result_clean['rmse'], result_baseline['rmse'], result_registered['rmse']],
    'r2': [result_clean['r2'], result_baseline['r2'], result_registered['r2']],
    'abs_error_a': [result_clean['abs_error_a'], result_baseline['abs_error_a'], result_registered['abs_error_a']],
    'abs_error_b': [result_clean['abs_error_b'], result_baseline['abs_error_b'], result_registered['abs_error_b']],
    'abs_error_c': [result_clean['abs_error_c'], result_baseline['abs_error_c'], result_registered['abs_error_c']],
    'mean_shift': [0.0, np.nan, reg_metrics['mean_absolute_shift']],
    'std_shift': [0.0, np.nan, reg_metrics['std_shift']],
    'max_shift': [0.0, np.nan, reg_metrics['max_shift']],
    'mean_correlation': [1.0, np.nan, reg_metrics['mean_correlation']]
}

df_results = pd.DataFrame(results_dict)
df_results.to_csv('registration_crosscorr_fd_results.csv', index=False)
print("\n    Saved: registration_crosscorr_fd_results.csv")

print("\n" + "="*80)
print("  CROSS-CORRELATION REGISTRATION + FINITE DIFFERENCE SINDY COMPLETE!")
print("="*80)

Phase correlation registration + FFT derivatives

In [ ]:
import h5py
import numpy as np
from scipy.fft import fft, ifft, fftfreq, fft2, ifft2
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import pandas as pd

print("="*80)
print("PHASE 3A: SPATIAL TRANSLATION - PHASE CORRELATION REGISTRATION + FFT SINDY")
print("="*80)

# ===== Spectral derivative functions =====
def spectral_derivative(u_snapshot, k):
    """First derivative using FFT"""
    u_hat = fft(u_snapshot)
    du_hat = 1j * k * u_hat
    return np.real(ifft(du_hat))

def spectral_second_derivative(u_snapshot, k):
    """Second derivative using FFT"""
    u_hat = fft(u_snapshot)
    d2u_hat = -(k**2) * u_hat
    return np.real(ifft(d2u_hat))

def spectral_fourth_derivative(u_snapshot, k):
    """Fourth derivative using FFT"""
    u_hat = fft(u_snapshot)
    d4u_hat = (k**4) * u_hat
    return np.real(ifft(d4u_hat))

# ===== Phase Correlation Registration =====
def find_shift_phasecorr(frame, reference):
    """
    Find the shift between frame and reference using phase correlation
    More robust to noise than standard cross-correlation
    """
    # FFT of both signals
    F1 = fft(frame)
    F2 = fft(reference)
    
    # Cross-power spectrum
    cross_power = F1 * np.conj(F2)
    # Normalize to get phase correlation
    cross_power = cross_power / (np.abs(cross_power) + 1e-10)
    
    # Inverse FFT to get correlation
    correlation = np.real(ifft(cross_power))
    
    # Find peak
    max_idx = np.argmax(correlation)
    
    # Convert to shift (handle wrap-around)
    N = len(frame)
    if max_idx > N // 2:
        shift = max_idx - N
    else:
        shift = max_idx
    
    return shift

def align_data_phasecorr(u_data, reference_frame):
    """Align all frames to reference using phase correlation"""
    aligned_data = np.zeros_like(u_data)
    detected_shifts = []
    
    for i in range(u_data.shape[0]):
        shift = find_shift_phasecorr(u_data[i], reference_frame)
        aligned_data[i] = np.roll(u_data[i], -shift)
        detected_shifts.append(shift)
    
    return aligned_data, np.array(detected_shifts)

# ===== Registration Quality Metrics =====
def compute_registration_metrics(detected_shifts, aligned_data, reference_frame):
    """Compute metrics to evaluate registration quality"""
    mean_shift = np.mean(np.abs(detected_shifts))
    std_shift = np.std(detected_shifts)
    max_shift = np.max(np.abs(detected_shifts))
    
    correlations = []
    for i in range(aligned_data.shape[0]):
        corr = np.corrcoef(aligned_data[i], reference_frame)[0, 1]
        correlations.append(corr)
    
    mean_correlation = np.mean(correlations)
    residuals = aligned_data - reference_frame
    residual_variance = np.var(residuals)
    
    return {
        'mean_absolute_shift': mean_shift,
        'std_shift': std_shift,
        'max_shift': max_shift,
        'mean_correlation': mean_correlation,
        'residual_variance': residual_variance,
        'detected_shifts': detected_shifts
    }

# ===== Evaluation Function =====
def evaluate_sindy_fft(u, x, t, method_name="Unknown"):
    """Apply FFT-based SINDy and compute evaluation metrics"""
    Nx = len(x)
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    
    # Define wavenumbers
    k = 2 * np.pi * fftfreq(Nx, d=dx)
    
    # Compute temporal derivative (central difference)
    u_t = (u[2:] - u[:-2]) / (2 * dt)
    u_mid = u[1:-1]
    
    # Build feature library using FFT
    Theta = []
    ut_flat = []
    
    for snapshot, ut_snapshot in zip(u_mid, u_t):
        ux = spectral_derivative(snapshot, k)
        uxx = spectral_second_derivative(snapshot, k)
        uxxxx = spectral_fourth_derivative(snapshot, k)
        
        # Feature terms: [u_xx, u_xxxx, u*u_x]
        Theta_snapshot = np.vstack([
            uxx,
            uxxxx,
            snapshot * ux
        ]).T
        
        Theta.append(Theta_snapshot)
        ut_flat.append(ut_snapshot)
    
    Theta = np.vstack(Theta)
    ut_flat = np.hstack(ut_flat)
    
    # Normalize features
    Theta_mean = Theta.mean(axis=0)
    Theta_std = Theta.std(axis=0)
    Theta_norm = (Theta - Theta_mean) / Theta_std
    
    # Lasso regression
    lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000)
    lasso.fit(Theta_norm, ut_flat)
    coeffs = lasso.coef_ / Theta_std
    
    # Extract coefficients
    a = coeffs[0]
    b = coeffs[1]
    c = coeffs[2]
    
    # Predict u_t
    u_t_pred = Theta @ coeffs
    
    # Compute metrics
    rmse = np.sqrt(np.mean((ut_flat - u_t_pred)**2))
    r2 = r2_score(ut_flat, u_t_pred)
    
    # True coefficients
    a_true, b_true, c_true = -1.0, -1.0, -1.0
    
    # Coefficient errors
    error_a = a - a_true
    error_b = b - b_true
    error_c = c - c_true
    
    abs_error_a = abs(error_a)
    abs_error_b = abs(error_b)
    abs_error_c = abs(error_c)
    
    return {
        'method': method_name,
        'a': a,
        'b': b,
        'c': c,
        'error_a': error_a,
        'error_b': error_b,
        'error_c': error_c,
        'abs_error_a': abs_error_a,
        'abs_error_b': abs_error_b,
        'abs_error_c': abs_error_c,
        'rmse': rmse,
        'r2': r2
    }

# ===== Main Processing =====
print("\n📂 Loading spatial translation data...")
with h5py.File("test_solving_euler_ks_spatial_shift.h5", "r") as f:
    u_shifted = np.array(f["u"])
    x = np.array(f["x"])
    t = np.array(f["t"])

print(f"Loaded: test_solving_euler_ks_spatial_shift.h5")
print(f"Shape: {u_shifted.shape}")

# Compute Mean Frame as Reference
print("\n🎯 Computing mean frame as reference...")
reference_frame = np.mean(u_shifted, axis=0)

# Method 1: No Registration (Baseline)
print("\n" + "="*80)
print("METHOD 1: NO REGISTRATION (BASELINE)")
print("="*80)

result_baseline = evaluate_sindy_fft(u_shifted, x, t, method_name="No Registration")

print(f"\nRecovered Coefficients:")
print(f"  a (u_xx):   {result_baseline['a']:.6f}  (true: -1.0)")
print(f"  b (u_xxxx): {result_baseline['b']:.6f}  (true: -1.0)")
print(f"  c (u*u_x):  {result_baseline['c']:.6f}  (true: -1.0)")
print(f"\nEvaluation Metrics:")
print(f"  RMSE: {result_baseline['rmse']:.6f}")
print(f"  R²:   {result_baseline['r2']:.6f}")

# Method 2: Phase Correlation Registration
print("\n" + "="*80)
print("METHOD 2: PHASE CORRELATION REGISTRATION")
print("="*80)

print("Aligning frames using phase correlation...")
u_aligned, detected_shifts = align_data_phasecorr(u_shifted, reference_frame)

print(f"    Alignment complete!")
print(f"  Detected shifts: min={detected_shifts.min()}, max={detected_shifts.max()}")
print(f"  Mean absolute shift: {np.mean(np.abs(detected_shifts)):.2f} pixels")

# Compute registration quality metrics
reg_metrics = compute_registration_metrics(detected_shifts, u_aligned, reference_frame)

print(f"\nRegistration Quality Metrics:")
print(f"  Mean Absolute Shift: {reg_metrics['mean_absolute_shift']:.4f} pixels")
print(f"  Std Shift:           {reg_metrics['std_shift']:.4f} pixels")
print(f"  Max Shift:           {reg_metrics['max_shift']:.1f} pixels")
print(f"  Mean Correlation:    {reg_metrics['mean_correlation']:.6f}")
print(f"  Residual Variance:   {reg_metrics['residual_variance']:.6f}")

# Apply SINDy to aligned data
result_registered = evaluate_sindy_fft(u_aligned, x, t, method_name="Phase Correlation")

print(f"\nRecovered Coefficients (After Registration):")
print(f"  a (u_xx):   {result_registered['a']:.6f}  (true: -1.0)")
print(f"  b (u_xxxx): {result_registered['b']:.6f}  (true: -1.0)")
print(f"  c (u*u_x):  {result_registered['c']:.6f}  (true: -1.0)")
print(f"\nEvaluation Metrics:")
print(f"  RMSE: {result_registered['rmse']:.6f}")
print(f"  R²:   {result_registered['r2']:.6f}")

# Load Clean Data for Comparison
print("\n" + "="*80)
print("LOADING CLEAN DATA FOR COMPARISON")
print("="*80)

with h5py.File("test_solving_euler_ks_clean.h5", "r") as f:
    u_clean = np.array(f["u"])

result_clean = evaluate_sindy_fft(u_clean, x, t, method_name="Clean Data")

print(f"\nRecovered Coefficients (Clean Data):")
print(f"  a (u_xx):   {result_clean['a']:.6f}  (true: -1.0)")
print(f"  b (u_xxxx): {result_clean['b']:.6f}  (true: -1.0)")
print(f"  c (u*u_x):  {result_clean['c']:.6f}  (true: -1.0)")
print(f"\nEvaluation Metrics:")
print(f"  RMSE: {result_clean['rmse']:.6f}")
print(f"  R²:   {result_clean['r2']:.6f}")

# Summary Comparison
print("\n\n" + "="*80)
print("SUMMARY COMPARISON")
print("="*80)

print(f"\n{'Method':<25} {'a (u_xx)':>12} {'b (u_xxxx)':>12} {'c (u*u_x)':>12} {'RMSE':>12} {'R²':>12}")
print("-"*80)
results = [result_clean, result_baseline, result_registered]
for r in results:
    print(f"{r['method']:<25} {r['a']:>12.6f} {r['b']:>12.6f} {r['c']:>12.6f} "
          f"{r['rmse']:>12.6f} {r['r2']:>12.6f}")
print("="*80)
print(f"{'TRUE VALUES:':<25} {-1.0:>12.6f} {-1.0:>12.6f} {-1.0:>12.6f}")
print("="*80)

# Improvement Analysis
print("\n" + "="*80)
print("IMPROVEMENT FROM REGISTRATION")
print("="*80)

improvement_rmse = (result_baseline['rmse'] - result_registered['rmse']) / result_baseline['rmse'] * 100
improvement_r2 = (result_registered['r2'] - result_baseline['r2']) / max(abs(result_baseline['r2']), 1e-10) * 100

print(f"\nRMSE Improvement:  {improvement_rmse:+.2f}%")
print(f"R² Improvement:    {improvement_r2:+.2f}%")

# Save Results
results_dict = {
    'method': ['Clean Data', 'No Registration', 'Phase Correlation'],
    'a': [result_clean['a'], result_baseline['a'], result_registered['a']],
    'b': [result_clean['b'], result_baseline['b'], result_registered['b']],
    'c': [result_clean['c'], result_baseline['c'], result_registered['c']],
    'rmse': [result_clean['rmse'], result_baseline['rmse'], result_registered['rmse']],
    'r2': [result_clean['r2'], result_baseline['r2'], result_registered['r2']],
    'abs_error_a': [result_clean['abs_error_a'], result_baseline['abs_error_a'], result_registered['abs_error_a']],
    'abs_error_b': [result_clean['abs_error_b'], result_baseline['abs_error_b'], result_registered['abs_error_b']],
    'abs_error_c': [result_clean['abs_error_c'], result_baseline['abs_error_c'], result_registered['abs_error_c']],
    'mean_shift': [0.0, np.nan, reg_metrics['mean_absolute_shift']],
    'std_shift': [0.0, np.nan, reg_metrics['std_shift']],
    'max_shift': [0.0, np.nan, reg_metrics['max_shift']],
    'mean_correlation': [1.0, np.nan, reg_metrics['mean_correlation']]
}

df_results = pd.DataFrame(results_dict)
df_results.to_csv('registration_phasecorr_fft_results.csv', index=False)
print("\n    Saved: registration_phasecorr_fft_results.csv")

print("\n" + "="*80)
print("   PHASE CORRELATION REGISTRATION + FFT SINDY COMPLETE!")
print("="*80)

Phase correlation registration + Finite Difference derivatives

In [ ]:
import h5py
import numpy as np
from scipy.fft import fft, ifft
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import pandas as pd

print("="*80)
print("PHASE 3A: SPATIAL TRANSLATION - PHASE CORRELATION REGISTRATION + FINITE DIFFERENCE SINDY")
print("="*80)

# ===== Phase Correlation Registration =====
def find_shift_phasecorr(frame, reference):
    """Find the shift between frame and reference using phase correlation"""
    F1 = fft(frame)
    F2 = fft(reference)
    cross_power = F1 * np.conj(F2)
    cross_power = cross_power / (np.abs(cross_power) + 1e-10)
    correlation = np.real(ifft(cross_power))
    max_idx = np.argmax(correlation)
    N = len(frame)
    if max_idx > N // 2:
        shift = max_idx - N
    else:
        shift = max_idx
    return shift

def align_data_phasecorr(u_data, reference_frame):
    """Align all frames to reference using phase correlation"""
    aligned_data = np.zeros_like(u_data)
    detected_shifts = []
    for i in range(u_data.shape[0]):
        shift = find_shift_phasecorr(u_data[i], reference_frame)
        aligned_data[i] = np.roll(u_data[i], -shift)
        detected_shifts.append(shift)
    return aligned_data, np.array(detected_shifts)

# ===== Registration Quality Metrics =====
def compute_registration_metrics(detected_shifts, aligned_data, reference_frame):
    """Compute metrics to evaluate registration quality"""
    mean_shift = np.mean(np.abs(detected_shifts))
    std_shift = np.std(detected_shifts)
    max_shift = np.max(np.abs(detected_shifts))
    correlations = []
    for i in range(aligned_data.shape[0]):
        corr = np.corrcoef(aligned_data[i], reference_frame)[0, 1]
        correlations.append(corr)
    mean_correlation = np.mean(correlations)
    residuals = aligned_data - reference_frame
    residual_variance = np.var(residuals)
    return {
        'mean_absolute_shift': mean_shift,
        'std_shift': std_shift,
        'max_shift': max_shift,
        'mean_correlation': mean_correlation,
        'residual_variance': residual_variance,
        'detected_shifts': detected_shifts
    }

# ===== Evaluation Function with Finite Difference =====
def evaluate_sindy_finite_diff(u, x, t, method_name="Unknown"):
    """Apply Finite Difference SINDy and compute evaluation metrics"""
    dx = float(x[1] - x[0])
    dt = float(t[1] - t[0])
    
    def d1(U): 
        return (np.roll(U, -1, 1) - np.roll(U, 1, 1)) / (2*dx)
    def d2(U): 
        return (np.roll(U, -1, 1) - 2*U + np.roll(U, 1, 1)) / (dx**2)
    def d4(U):
        return (np.roll(U, -2, 1) - 4*np.roll(U, -1, 1) + 6*U - 
                4*np.roll(U, 1, 1) + np.roll(U, 2, 1)) / (dx**4)
    
    U_t = (u[1:] - u[:-1]) / dt
    U = u[:-1]
    U_x = d1(U)
    U_xx = d2(U)
    U_xxxx = d4(U)
    U2_x = d1(U**2)
    
    Theta = np.column_stack([
        np.ones_like(U).ravel(),
        U.ravel(),
        U_x.ravel(),
        U_xx.ravel(),
        U_xxxx.ravel(),
        U2_x.ravel()
    ])
    y = U_t.ravel()
    names = ["1", "u", "u_x", "u_xx", "u_xxxx", "(u^2)_x"]
    
    Theta_mean = Theta.mean(0)
    Theta_std = Theta.std(0) + 1e-8
    Theta_n = (Theta - Theta_mean) / Theta_std
    
    lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000, tol=1e-6)
    lasso.fit(Theta_n, y)
    coef = lasso.coef_ / Theta_std
    
    a = coef[names.index("u_xx")]
    b = coef[names.index("u_xxxx")]
    c_phys = coef[names.index("(u^2)_x")]
    
    y_pred = Theta @ coef
    rmse = np.sqrt(np.mean((y - y_pred)**2))
    r2 = r2_score(y, y_pred)
    
    a_true, b_true, c_true = -1.0, -1.0, -0.5
    
    return {
        'method': method_name,
        'a': a,
        'b': b,
        'c': c_phys,
        'error_a': a - a_true,
        'error_b': b - b_true,
        'error_c': c_phys - c_true,
        'abs_error_a': abs(a - a_true),
        'abs_error_b': abs(b - b_true),
        'abs_error_c': abs(c_phys - c_true),
        'rmse': rmse,
        'r2': r2
    }

# ===== Main Processing =====
print("\n📂 Loading spatial translation data...")
with h5py.File("test_solving_euler_ks_spatial_shift.h5", "r") as f:
    u_shifted = np.array(f["u"], dtype=np.float32)
    x = np.array(f["x"], dtype=np.float32)
    t = np.array(f["t"], dtype=np.float32)

print(f"Loaded: test_solving_euler_ks_spatial_shift.h5")
print(f"Shape: {u_shifted.shape}")

print("\n🎯 Computing mean frame as reference...")
reference_frame = np.mean(u_shifted, axis=0)

# Method 1: No Registration
print("\n" + "="*80)
print("METHOD 1: NO REGISTRATION (BASELINE)")
print("="*80)
result_baseline = evaluate_sindy_finite_diff(u_shifted, x, t, method_name="No Registration")
print(f"\nRecovered: a={result_baseline['a']:.6f}, b={result_baseline['b']:.6f}, c={result_baseline['c']:.6f}")
print(f"RMSE={result_baseline['rmse']:.6f}, R²={result_baseline['r2']:.6f}")

# Method 2: Phase Correlation
print("\n" + "="*80)
print("METHOD 2: PHASE CORRELATION REGISTRATION")
print("="*80)
u_aligned, detected_shifts = align_data_phasecorr(u_shifted, reference_frame)
print(f"    Alignment complete! Mean shift: {np.mean(np.abs(detected_shifts)):.2f} pixels")

reg_metrics = compute_registration_metrics(detected_shifts, u_aligned, reference_frame)
print(f"Registration metrics: mean_shift={reg_metrics['mean_absolute_shift']:.4f}, "
      f"correlation={reg_metrics['mean_correlation']:.6f}")

result_registered = evaluate_sindy_finite_diff(u_aligned, x, t, method_name="Phase Correlation")
print(f"\nRecovered: a={result_registered['a']:.6f}, b={result_registered['b']:.6f}, c={result_registered['c']:.6f}")
print(f"RMSE={result_registered['rmse']:.6f}, R²={result_registered['r2']:.6f}")

# Clean Data
print("\n" + "="*80)
print("CLEAN DATA COMPARISON")
print("="*80)
with h5py.File("test_solving_euler_ks_clean.h5", "r") as f:
    u_clean = np.array(f["u"], dtype=np.float32)
result_clean = evaluate_sindy_finite_diff(u_clean, x, t, method_name="Clean Data")
print(f"Recovered: a={result_clean['a']:.6f}, b={result_clean['b']:.6f}, c={result_clean['c']:.6f}")
print(f"RMSE={result_clean['rmse']:.6f}, R²={result_clean['r2']:.6f}")

# Summary
print("\n\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"{'Method':<25} {'a':>12} {'b':>12} {'c':>12} {'RMSE':>12} {'R²':>12}")
print("-"*80)
for r in [result_clean, result_baseline, result_registered]:
    print(f"{r['method']:<25} {r['a']:>12.6f} {r['b']:>12.6f} {r['c']:>12.6f} {r['rmse']:>12.6f} {r['r2']:>12.6f}")
print("="*80)

# Save
results_dict = {
    'method': ['Clean Data', 'No Registration', 'Phase Correlation'],
    'a': [result_clean['a'], result_baseline['a'], result_registered['a']],
    'b': [result_clean['b'], result_baseline['b'], result_registered['b']],
    'c': [result_clean['c'], result_baseline['c'], result_registered['c']],
    'rmse': [result_clean['rmse'], result_baseline['rmse'], result_registered['rmse']],
    'r2': [result_clean['r2'], result_baseline['r2'], result_registered['r2']],
    'abs_error_a': [result_clean['abs_error_a'], result_baseline['abs_error_a'], result_registered['abs_error_a']],
    'abs_error_b': [result_clean['abs_error_b'], result_baseline['abs_error_b'], result_registered['abs_error_b']],
    'abs_error_c': [result_clean['abs_error_c'], result_baseline['abs_error_c'], result_registered['abs_error_c']],
    'mean_shift': [0.0, np.nan, reg_metrics['mean_absolute_shift']],
    'std_shift': [0.0, np.nan, reg_metrics['std_shift']],
    'max_shift': [0.0, np.nan, reg_metrics['max_shift']],
    'mean_correlation': [1.0, np.nan, reg_metrics['mean_correlation']]
}
pd.DataFrame(results_dict).to_csv('registration_phasecorr_fd_results.csv', index=False)
print("\n    Saved: registration_phasecorr_fd_results.csv")
print("\n   PHASE CORRELATION REGISTRATION + FINITE DIFFERENCE SINDY COMPLETE!")

Registration Comparison

In [ ]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import correlate

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (18, 14)

print("="*80)
print("PHASE 3A: REGISTRATION METHODS COMPARISON")
print("="*80)

# ===== Load All Results =====
print("\n📂 Loading all registration results...")

try:
    # Cross-Correlation Results
    crosscorr_fft = pd.read_csv('registration_crosscorr_fft_results.csv')
    crosscorr_fd = pd.read_csv('registration_crosscorr_fd_results.csv')
    
    # Phase Correlation Results
    phasecorr_fft = pd.read_csv('registration_phasecorr_fft_results.csv')
    phasecorr_fd = pd.read_csv('registration_phasecorr_fd_results.csv')
    
    print("    Successfully loaded all 4 result files")
    
except FileNotFoundError as e:
    print(f"    Error: {e}")
    print("Please run all 4 registration scripts first.")
    exit()

# ===== Load Data for Visual Comparison =====
print("\n📂 Loading spatial data for visual comparison...")

with h5py.File("test_solving_euler_ks_clean.h5", "r") as f:
    u_clean = np.array(f["u"])
    x = np.array(f["x"])
    t = np.array(f["t"])

with h5py.File("test_solving_euler_ks_spatial_shift.h5", "r") as f:
    u_shifted = np.array(f["u"])

# Reconstruct aligned data (using cross-correlation)
def find_shift_crosscorr(frame, reference):
    """Find shift using cross-correlation"""
    correlation = correlate(frame, reference, mode='same', method='fft')
    max_idx = np.argmax(correlation)
    center = len(frame) // 2
    shift = max_idx - center
    return shift

print("Reconstructing aligned data...")
reference_frame = np.mean(u_shifted, axis=0)
u_aligned = np.zeros_like(u_shifted)
for i in range(u_shifted.shape[0]):
    shift = find_shift_crosscorr(u_shifted[i], reference_frame)
    u_aligned[i] = np.roll(u_shifted[i], -shift)

print("    Data loaded successfully")

# ===== COMPARISON TABLE 1: All Methods Overview =====
print("\n\n" + "="*100)
print("COMPARISON TABLE 1: ALL REGISTRATION METHODS")
print("="*100)

comparison_data = []

# FFT Methods
for idx, row in crosscorr_fft.iterrows():
    if row['method'] != 'Clean Data':
        comparison_data.append({
            'Derivative': 'FFT',
            'Registration': row['method'],
            'a': row['a'],
            'b': row['b'],
            'c': row['c'],
            'rmse': row['rmse'],
            'r2': row['r2'],
            'abs_error_a': row['abs_error_a'],
            'abs_error_b': row['abs_error_b'],
            'abs_error_c': row['abs_error_c'],
            'mean_shift': row['mean_shift']
        })

for idx, row in phasecorr_fft.iterrows():
    if row['method'] not in ['Clean Data', 'No Registration']:
        comparison_data.append({
            'Derivative': 'FFT',
            'Registration': row['method'],
            'a': row['a'],
            'b': row['b'],
            'c': row['c'],
            'rmse': row['rmse'],
            'r2': row['r2'],
            'abs_error_a': row['abs_error_a'],
            'abs_error_b': row['abs_error_b'],
            'abs_error_c': row['abs_error_c'],
            'mean_shift': row['mean_shift']
        })

# Finite Difference Methods
for idx, row in crosscorr_fd.iterrows():
    if row['method'] != 'Clean Data':
        comparison_data.append({
            'Derivative': 'Finite Diff',
            'Registration': row['method'],
            'a': row['a'],
            'b': row['b'],
            'c': row['c'],
            'rmse': row['rmse'],
            'r2': row['r2'],
            'abs_error_a': row['abs_error_a'],
            'abs_error_b': row['abs_error_b'],
            'abs_error_c': row['abs_error_c'],
            'mean_shift': row['mean_shift']
        })

for idx, row in phasecorr_fd.iterrows():
    if row['method'] not in ['Clean Data', 'No Registration']:
        comparison_data.append({
            'Derivative': 'Finite Diff',
            'Registration': row['method'],
            'a': row['a'],
            'b': row['b'],
            'c': row['c'],
            'rmse': row['rmse'],
            'r2': row['r2'],
            'abs_error_a': row['abs_error_a'],
            'abs_error_b': row['abs_error_b'],
            'abs_error_c': row['abs_error_c'],
            'mean_shift': row['mean_shift']
        })

df_comparison = pd.DataFrame(comparison_data)

print(f"\n{'Derivative':>12} {'Registration':>20} {'a':>12} {'b':>12} {'c':>12} {'RMSE':>12} {'R²':>10}")
print("-"*100)
for _, row in df_comparison.iterrows():
    print(f"{row['Derivative']:>12} {row['Registration']:>20} {row['a']:>12.6f} {row['b']:>12.6f} "
          f"{row['c']:>12.6f} {row['rmse']:>12.6f} {row['r2']:>10.6f}")
print("="*100)

# ===== COMPARISON TABLE 2: Clean Data Baseline =====
print("\n\n" + "="*100)
print("COMPARISON TABLE 2: CLEAN DATA BASELINE")
print("="*100)

clean_fft = crosscorr_fft[crosscorr_fft['method'] == 'Clean Data'].iloc[0]
clean_fd = crosscorr_fd[crosscorr_fd['method'] == 'Clean Data'].iloc[0]

print(f"\n{'Derivative':>12} {'a':>12} {'b':>12} {'c':>12} {'RMSE':>12} {'R²':>10}")
print("-"*100)
print(f"{'FFT':>12} {clean_fft['a']:>12.6f} {clean_fft['b']:>12.6f} {clean_fft['c']:>12.6f} "
      f"{clean_fft['rmse']:>12.6f} {clean_fft['r2']:>10.6f}")
print(f"{'Finite Diff':>12} {clean_fd['a']:>12.6f} {clean_fd['b']:>12.6f} {clean_fd['c']:>12.6f} "
      f"{clean_fd['rmse']:>12.6f} {clean_fd['r2']:>10.6f}")
print("="*100)

# ===== BEST METHOD ANALYSIS =====
print("\n\n" + "="*80)
print("BEST METHODS BY METRIC")
print("="*80)

# Best by RMSE
best_rmse = df_comparison.loc[df_comparison['rmse'].idxmin()]
print(f"\n🏆 BEST RMSE: {best_rmse['Derivative']} + {best_rmse['Registration']}")
print(f"   RMSE = {best_rmse['rmse']:.6f}")
print(f"   R² = {best_rmse['r2']:.6f}")

# Best by R²
best_r2 = df_comparison.loc[df_comparison['r2'].idxmax()]
print(f"\n🏆 BEST R²: {best_r2['Derivative']} + {best_r2['Registration']}")
print(f"   RMSE = {best_r2['rmse']:.6f}")
print(f"   R² = {best_r2['r2']:.6f}")

# Best by coefficient accuracy (mean absolute error)
df_comparison['mean_coef_error'] = (df_comparison['abs_error_a'] + 
                                     df_comparison['abs_error_b'] + 
                                     df_comparison['abs_error_c']) / 3
best_coef = df_comparison.loc[df_comparison['mean_coef_error'].idxmin()]
print(f"\n🏆 BEST COEFFICIENT ACCURACY: {best_coef['Derivative']} + {best_coef['Registration']}")
print(f"   Mean Coef Error = {best_coef['mean_coef_error']:.6f}")
print(f"   RMSE = {best_coef['rmse']:.6f}")

# ===== IMPROVEMENT ANALYSIS =====
print("\n\n" + "="*80)
print("IMPROVEMENT FROM REGISTRATION (vs No Registration Baseline)")
print("="*80)

baseline_fft = crosscorr_fft[crosscorr_fft['method'] == 'No Registration'].iloc[0]
baseline_fd = crosscorr_fd[crosscorr_fd['method'] == 'No Registration'].iloc[0]

print("\n--- FFT Methods ---")
for idx, row in df_comparison[df_comparison['Derivative'] == 'FFT'].iterrows():
    if row['Registration'] != 'No Registration':
        rmse_improv = (baseline_fft['rmse'] - row['rmse']) / baseline_fft['rmse'] * 100
        r2_improv = (row['r2'] - baseline_fft['r2']) / max(abs(baseline_fft['r2']), 1e-10) * 100
        print(f"{row['Registration']:>20}: RMSE {rmse_improv:+.2f}%, R² {r2_improv:+.2f}%")

print("\n--- Finite Difference Methods ---")
for idx, row in df_comparison[df_comparison['Derivative'] == 'Finite Diff'].iterrows():
    if row['Registration'] != 'No Registration':
        rmse_improv = (baseline_fd['rmse'] - row['rmse']) / baseline_fd['rmse'] * 100
        r2_improv = (row['r2'] - baseline_fd['r2']) / max(abs(baseline_fd['r2']), 1e-10) * 100
        print(f"{row['Registration']:>20}: RMSE {rmse_improv:+.2f}%, R² {r2_improv:+.2f}%")

# ===== VISUALIZATION 1: VISUAL DATA COMPARISON =====
print("\n\n" + "="*80)
print("GENERATING VISUALIZATIONS...")
print("="*80)

fig1 = plt.figure(figsize=(20, 12))

# Sample frame index
sample_idx = len(u_clean) // 2

# Row 1: Spatiotemporal plots (first 500 frames)
n_frames = 500

ax1 = plt.subplot(3, 3, 1)
im1 = plt.imshow(u_clean[:n_frames].T, cmap='RdBu', aspect='auto', origin='lower',
                 extent=(0, n_frames*t[1], x[0], x[-1]), vmin=-2, vmax=2)
plt.xlabel('Time', fontsize=10)
plt.ylabel('Space (x)', fontsize=10)
plt.title('Clean Data (Aligned)\n(First 500 frames)', fontweight='bold', fontsize=11)
plt.colorbar(im1, ax=ax1, label='u(x,t)')

ax2 = plt.subplot(3, 3, 2)
im2 = plt.imshow(u_shifted[:n_frames].T, cmap='RdBu', aspect='auto', origin='lower',
                 extent=(0, n_frames*t[1], x[0], x[-1]), vmin=-2, vmax=2)
plt.xlabel('Time', fontsize=10)
plt.ylabel('Space (x)', fontsize=10)
plt.title('No Registration (Unaligned)\n(First 500 frames)', fontweight='bold', fontsize=11)
plt.colorbar(im2, ax=ax2, label='u(x,t)')

ax3 = plt.subplot(3, 3, 3)
im3 = plt.imshow(u_aligned[:n_frames].T, cmap='RdBu', aspect='auto', origin='lower',
                 extent=(0, n_frames*t[1], x[0], x[-1]), vmin=-2, vmax=2)
plt.xlabel('Time', fontsize=10)
plt.ylabel('Space (x)', fontsize=10)
plt.title('With Registration (Aligned)\n(First 500 frames)', fontweight='bold', fontsize=11)
plt.colorbar(im3, ax=ax3, label='u(x,t)')

# Row 2: Sample frame comparison
ax4 = plt.subplot(3, 3, 4)
plt.plot(x, u_clean[sample_idx], 'g-', label='Clean', linewidth=2.5, alpha=0.8)
plt.plot(x, u_shifted[sample_idx], 'r--', label='Unaligned', linewidth=2, alpha=0.7)
plt.plot(x, u_aligned[sample_idx], 'b:', label='Aligned', linewidth=2.5, alpha=0.8)
plt.xlabel('x', fontsize=10)
plt.ylabel('u(x)', fontsize=10)
plt.title(f'Sample Frame Comparison\n(Frame {sample_idx})', fontweight='bold', fontsize=11)
plt.legend(fontsize=9)
plt.grid(True, alpha=0.3)

# Row 2: Zoomed region
zoom_start, zoom_end = len(x)//3, len(x)//3 + 50
ax5 = plt.subplot(3, 3, 5)
plt.plot(x[zoom_start:zoom_end], u_clean[sample_idx, zoom_start:zoom_end], 
         'g-', label='Clean', linewidth=2.5, marker='o', markersize=4, alpha=0.8)
plt.plot(x[zoom_start:zoom_end], u_shifted[sample_idx, zoom_start:zoom_end], 
         'r--', label='Unaligned', linewidth=2, marker='s', markersize=4, alpha=0.7)
plt.plot(x[zoom_start:zoom_end], u_aligned[sample_idx, zoom_start:zoom_end], 
         'b:', label='Aligned', linewidth=2.5, marker='^', markersize=4, alpha=0.8)
plt.xlabel('x', fontsize=10)
plt.ylabel('u(x)', fontsize=10)
plt.title('Zoomed Region\n(Shows Spatial Shifts)', fontweight='bold', fontsize=11)
plt.legend(fontsize=9)
plt.grid(True, alpha=0.3)

# Difference plots
ax6 = plt.subplot(3, 3, 6)
diff_unaligned = np.abs(u_shifted[sample_idx] - u_clean[sample_idx])
diff_aligned = np.abs(u_aligned[sample_idx] - u_clean[sample_idx])
plt.plot(x, diff_unaligned, 'r-', label='|Unaligned - Clean|', linewidth=2, alpha=0.7)
plt.plot(x, diff_aligned, 'b-', label='|Aligned - Clean|', linewidth=2, alpha=0.7)
plt.xlabel('x', fontsize=10)
plt.ylabel('Absolute Difference', fontsize=10)
plt.title('Spatial Error Comparison\n(Lower is Better)', fontweight='bold', fontsize=11)
plt.legend(fontsize=9)
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Row 3: Coefficient comparison
ax7 = plt.subplot(3, 3, 7)
methods = ['No Reg\nFFT', 'CrossCorr\nFFT', 'PhaseCorr\nFFT', 
           'No Reg\nFD', 'CrossCorr\nFD', 'PhaseCorr\nFD']
a_vals = [baseline_fft['a'], 
          crosscorr_fft[crosscorr_fft['method']=='Cross-Correlation'].iloc[0]['a'],
          phasecorr_fft[phasecorr_fft['method']=='Phase Correlation'].iloc[0]['a'],
          baseline_fd['a'],
          crosscorr_fd[crosscorr_fd['method']=='Cross-Correlation'].iloc[0]['a'],
          phasecorr_fd[phasecorr_fd['method']=='Phase Correlation'].iloc[0]['a']]

b_vals = [baseline_fft['b'], 
          crosscorr_fft[crosscorr_fft['method']=='Cross-Correlation'].iloc[0]['b'],
          phasecorr_fft[phasecorr_fft['method']=='Phase Correlation'].iloc[0]['b'],
          baseline_fd['b'],
          crosscorr_fd[crosscorr_fd['method']=='Cross-Correlation'].iloc[0]['b'],
          phasecorr_fd[phasecorr_fd['method']=='Phase Correlation'].iloc[0]['b']]

x_pos = np.arange(len(methods))
width = 0.35
plt.bar(x_pos - width/2, a_vals, width, label='a (u_xx)', alpha=0.8)
plt.bar(x_pos + width/2, b_vals, width, label='b (u_xxxx)', alpha=0.8)
plt.axhline(y=-1.0, color='red', linestyle='--', linewidth=2, label='True')
plt.ylabel('Coefficient Value', fontsize=10)
plt.title('Recovered Coefficients\nAll Methods', fontweight='bold', fontsize=11)
plt.xticks(x_pos, methods, rotation=45, ha='right', fontsize=8)
plt.legend(fontsize=9)
plt.grid(True, alpha=0.3, axis='y')

# RMSE comparison
ax8 = plt.subplot(3, 3, 8)
rmse_vals = [baseline_fft['rmse'], 
             crosscorr_fft[crosscorr_fft['method']=='Cross-Correlation'].iloc[0]['rmse'],
             phasecorr_fft[phasecorr_fft['method']=='Phase Correlation'].iloc[0]['rmse'],
             baseline_fd['rmse'],
             crosscorr_fd[crosscorr_fd['method']=='Cross-Correlation'].iloc[0]['rmse'],
             phasecorr_fd[phasecorr_fd['method']=='Phase Correlation'].iloc[0]['rmse']]

colors = ['red', 'blue', 'purple', 'red', 'blue', 'purple']
plt.bar(methods, rmse_vals, color=colors, alpha=0.7, edgecolor='black')
plt.ylabel('RMSE', fontsize=10)
plt.title('RMSE Comparison\n(Lower is Better)', fontweight='bold', fontsize=11)
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.grid(True, alpha=0.3, axis='y')
plt.yscale('log')

# R² comparison
ax9 = plt.subplot(3, 3, 9)
r2_vals = [baseline_fft['r2'], 
           crosscorr_fft[crosscorr_fft['method']=='Cross-Correlation'].iloc[0]['r2'],
           phasecorr_fft[phasecorr_fft['method']=='Phase Correlation'].iloc[0]['r2'],
           baseline_fd['r2'],
           crosscorr_fd[crosscorr_fd['method']=='Cross-Correlation'].iloc[0]['r2'],
           phasecorr_fd[phasecorr_fd['method']=='Phase Correlation'].iloc[0]['r2']]

plt.bar(methods, r2_vals, color=colors, alpha=0.7, edgecolor='black')
plt.ylabel('R² Score', fontsize=10)
plt.title('R² Comparison\n(Higher is Better)', fontweight='bold', fontsize=11)
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.grid(True, alpha=0.3, axis='y')
plt.axhline(y=1.0, color='green', linestyle='--', linewidth=1)

plt.tight_layout()
plt.savefig('registration_visual_comparison.png', dpi=300, bbox_inches='tight')
print("    Saved: registration_visual_comparison.png")

# ===== VISUALIZATION 2: Detailed Performance Heatmaps =====
fig2, axes = plt.subplots(2, 2, figsize=(14, 10))

# Prepare data for heatmaps
methods_clean = ['No Reg', 'CrossCorr', 'PhaseCorr']
derivatives = ['FFT', 'Finite Diff']

# RMSE Heatmap
rmse_matrix = np.array([
    [baseline_fft['rmse'], 
     crosscorr_fft[crosscorr_fft['method']=='Cross-Correlation'].iloc[0]['rmse'],
     phasecorr_fft[phasecorr_fft['method']=='Phase Correlation'].iloc[0]['rmse']],
    [baseline_fd['rmse'],
     crosscorr_fd[crosscorr_fd['method']=='Cross-Correlation'].iloc[0]['rmse'],
     phasecorr_fd[phasecorr_fd['method']=='Phase Correlation'].iloc[0]['rmse']]
])

sns.heatmap(rmse_matrix, annot=True, fmt='.4f', cmap='RdYlGn_r', 
            xticklabels=methods_clean, yticklabels=derivatives, ax=axes[0,0],
            cbar_kws={'label': 'RMSE'})
axes[0,0].set_title('RMSE by Method', fontweight='bold', fontsize=12)
axes[0,0].set_xlabel('Registration Method')
axes[0,0].set_ylabel('Derivative Type')

# R² Heatmap
r2_matrix = np.array([
    [baseline_fft['r2'], 
     crosscorr_fft[crosscorr_fft['method']=='Cross-Correlation'].iloc[0]['r2'],
     phasecorr_fft[phasecorr_fft['method']=='Phase Correlation'].iloc[0]['r2']],
    [baseline_fd['r2'],
     crosscorr_fd[crosscorr_fd['method']=='Cross-Correlation'].iloc[0]['r2'],
     phasecorr_fd[phasecorr_fd['method']=='Phase Correlation'].iloc[0]['r2']]
])

sns.heatmap(r2_matrix, annot=True, fmt='.4f', cmap='RdYlGn', 
            xticklabels=methods_clean, yticklabels=derivatives, ax=axes[0,1],
            cbar_kws={'label': 'R² Score'})
axes[0,1].set_title('R² Score by Method', fontweight='bold', fontsize=12)
axes[0,1].set_xlabel('Registration Method')
axes[0,1].set_ylabel('Derivative Type')

# Coefficient Error a
error_a_matrix = np.array([
    [baseline_fft['abs_error_a'], 
     crosscorr_fft[crosscorr_fft['method']=='Cross-Correlation'].iloc[0]['abs_error_a'],
     phasecorr_fft[phasecorr_fft['method']=='Phase Correlation'].iloc[0]['abs_error_a']],
    [baseline_fd['abs_error_a'],
     crosscorr_fd[crosscorr_fd['method']=='Cross-Correlation'].iloc[0]['abs_error_a'],
     phasecorr_fd[phasecorr_fd['method']=='Phase Correlation'].iloc[0]['abs_error_a']]
])

sns.heatmap(error_a_matrix, annot=True, fmt='.4f', cmap='RdYlGn_r', 
            xticklabels=methods_clean, yticklabels=derivatives, ax=axes[1,0],
            cbar_kws={'label': '|Error in a|'})
axes[1,0].set_title('Absolute Error in Coefficient a', fontweight='bold', fontsize=12)
axes[1,0].set_xlabel('Registration Method')
axes[1,0].set_ylabel('Derivative Type')

# Coefficient Error b
error_b_matrix = np.array([
    [baseline_fft['abs_error_b'], 
     crosscorr_fft[crosscorr_fft['method']=='Cross-Correlation'].iloc[0]['abs_error_b'],
     phasecorr_fft[phasecorr_fft['method']=='Phase Correlation'].iloc[0]['abs_error_b']],
    [baseline_fd['abs_error_b'],
     crosscorr_fd[crosscorr_fd['method']=='Cross-Correlation'].iloc[0]['abs_error_b'],
     phasecorr_fd[phasecorr_fd['method']=='Phase Correlation'].iloc[0]['abs_error_b']]
])

sns.heatmap(error_b_matrix, annot=True, fmt='.4f', cmap='RdYlGn_r', 
            xticklabels=methods_clean, yticklabels=derivatives, ax=axes[1,1],
            cbar_kws={'label': '|Error in b|'})
axes[1,1].set_title('Absolute Error in Coefficient b', fontweight='bold', fontsize=12)
axes[1,1].set_xlabel('Registration Method')
axes[1,1].set_ylabel('Derivative Type')

plt.tight_layout()
plt.savefig('registration_heatmap_comparison.png', dpi=300, bbox_inches='tight')
print("    Saved: registration_heatmap_comparison.png")

# ===== Save Comprehensive Comparison =====
df_comparison.to_csv('registration_all_methods_comparison.csv', index=False)
print("    Saved: registration_all_methods_comparison.csv")

print("\n" + "="*80)
print("   REGISTRATION COMPARISON COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. registration_visual_comparison.png - Visual data & performance comparison")
print("  2. registration_heatmap_comparison.png - Performance heatmaps")
print("  3. registration_all_methods_comparison.csv - Complete comparison data")
print("="*80)

###  phase 3b Varying IC Problem

Each frame comes from different initial condition
Frames represent an ensemble of trajectories
Trajectories diverge due to chaotic dynamics
Cannot "undo" this - it's fundamental trajectory uncertainty

Baseline (no treatment) + FFT derivatives

In [ ]:
import h5py
import numpy as np
from scipy.fft import fft, ifft, fftfreq
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import pandas as pd

print("="*80)
print("PHASE 3B: VARYING INITIAL CONDITIONS - BASELINE (NO TREATMENT) + FFT SINDY")
print("="*80)

# ===== Spectral derivative functions =====
def spectral_derivative(u_snapshot, k):
    """First derivative using FFT"""
    u_hat = fft(u_snapshot)
    du_hat = 1j * k * u_hat
    return np.real(ifft(du_hat))

def spectral_second_derivative(u_snapshot, k):
    """Second derivative using FFT"""
    u_hat = fft(u_snapshot)
    d2u_hat = -(k**2) * u_hat
    return np.real(ifft(d2u_hat))

def spectral_fourth_derivative(u_snapshot, k):
    """Fourth derivative using FFT"""
    u_hat = fft(u_snapshot)
    d4u_hat = (k**4) * u_hat
    return np.real(ifft(d4u_hat))

# ===== Evaluation Function =====
def evaluate_sindy_fft(u, x, t, method_name="Unknown"):
    """Apply FFT-based SINDy and compute evaluation metrics"""
    Nx = len(x)
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    
    # Define wavenumbers
    k = 2 * np.pi * fftfreq(Nx, d=dx)
    
    # Compute temporal derivative (central difference)
    u_t = (u[2:] - u[:-2]) / (2 * dt)
    u_mid = u[1:-1]
    
    # Build feature library using FFT
    Theta = []
    ut_flat = []
    
    for snapshot, ut_snapshot in zip(u_mid, u_t):
        ux = spectral_derivative(snapshot, k)
        uxx = spectral_second_derivative(snapshot, k)
        uxxxx = spectral_fourth_derivative(snapshot, k)
        
        # Feature terms: [u_xx, u_xxxx, u*u_x]
        Theta_snapshot = np.vstack([
            uxx,
            uxxxx,
            snapshot * ux
        ]).T
        
        Theta.append(Theta_snapshot)
        ut_flat.append(ut_snapshot)
    
    Theta = np.vstack(Theta)
    ut_flat = np.hstack(ut_flat)
    
    # Normalize features
    Theta_mean = Theta.mean(axis=0)
    Theta_std = Theta.std(axis=0)
    Theta_norm = (Theta - Theta_mean) / Theta_std
    
    # Lasso regression
    lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000)
    lasso.fit(Theta_norm, ut_flat)
    coeffs = lasso.coef_ / Theta_std
    
    # Extract coefficients
    a = coeffs[0]
    b = coeffs[1]
    c = coeffs[2]
    
    # Predict u_t
    u_t_pred = Theta @ coeffs
    
    # Compute metrics
    rmse = np.sqrt(np.mean((ut_flat - u_t_pred)**2))
    r2 = r2_score(ut_flat, u_t_pred)
    
    # True coefficients
    a_true, b_true, c_true = -1.0, -1.0, -1.0
    
    # Coefficient errors
    error_a = a - a_true
    error_b = b - b_true
    error_c = c - c_true
    
    abs_error_a = abs(error_a)
    abs_error_b = abs(error_b)
    abs_error_c = abs(error_c)
    
    return {
        'method': method_name,
        'a': a,
        'b': b,
        'c': c,
        'error_a': error_a,
        'error_b': error_b,
        'error_c': error_c,
        'abs_error_a': abs_error_a,
        'abs_error_b': abs_error_b,
        'abs_error_c': abs_error_c,
        'rmse': rmse,
        'r2': r2
    }

# ===== Trajectory Ensemble Statistics =====
def compute_ensemble_statistics(u_data):
    """
    Compute statistics about trajectory ensemble
    Measures trajectory divergence and uncertainty
    """
    # Frame-wise statistics
    mean_traj = np.mean(u_data, axis=0)
    std_traj = np.std(u_data, axis=0)
    
    # Temporal variance (how much frames differ)
    frame_variance = np.var(u_data, axis=1)
    mean_frame_var = np.mean(frame_variance)
    
    # Trajectory divergence (distance from mean)
    divergence = np.sqrt(np.mean((u_data - mean_traj)**2, axis=1))
    mean_divergence = np.mean(divergence)
    max_divergence = np.max(divergence)
    
    # Spatial coherence (correlation between frames)
    n_frames = u_data.shape[0]
    sample_size = min(100, n_frames)  # Sample for efficiency
    indices = np.random.choice(n_frames, sample_size, replace=False)
    
    correlations = []
    for i in range(len(indices)-1):
        corr = np.corrcoef(u_data[indices[i]], u_data[indices[i+1]])[0, 1]
        correlations.append(corr)
    
    mean_correlation = np.mean(correlations)
    
    return {
        'mean_frame_variance': mean_frame_var,
        'mean_divergence': mean_divergence,
        'max_divergence': max_divergence,
        'mean_correlation': mean_correlation,
        'std_spatial_mean': np.mean(std_traj)
    }

# ===== Main Processing =====
print("\n📂 Loading varying initial condition data...")
with h5py.File("test_solving_euler_ks_varied_ic.h5", "r") as f:
    u_varied = np.array(f["u"])
    x = np.array(f["x"])
    t = np.array(f["t"])

print(f"Loaded: test_solving_euler_ks_varied_ic.h5")
print(f"Shape: {u_varied.shape}")

# Compute ensemble statistics
print("\n📊 Computing trajectory ensemble statistics...")
ensemble_stats = compute_ensemble_statistics(u_varied)

print("\nEnsemble Statistics:")
print(f"  Mean Frame Variance:     {ensemble_stats['mean_frame_variance']:.6f}")
print(f"  Mean Trajectory Divergence: {ensemble_stats['mean_divergence']:.6f}")
print(f"  Max Trajectory Divergence:  {ensemble_stats['max_divergence']:.6f}")
print(f"  Mean Frame Correlation:     {ensemble_stats['mean_correlation']:.6f}")
print(f"  Spatial Std (mean):         {ensemble_stats['std_spatial_mean']:.6f}")

# Method 1: Baseline - Use data as-is (no treatment)
print("\n" + "="*80)
print("METHOD 1: NO TREATMENT (BASELINE)")
print("="*80)
print("Applying SINDy to varying IC data without any filtering or treatment...")

result_baseline = evaluate_sindy_fft(u_varied, x, t, method_name="No Treatment")

print(f"\nRecovered Coefficients:")
print(f"  a (u_xx):   {result_baseline['a']:.6f}  (true: -1.0)")
print(f"  b (u_xxxx): {result_baseline['b']:.6f}  (true: -1.0)")
print(f"  c (u*u_x):  {result_baseline['c']:.6f}  (true: -1.0)")
print(f"\nEvaluation Metrics:")
print(f"  RMSE: {result_baseline['rmse']:.6f}")
print(f"  R²:   {result_baseline['r2']:.6f}")

# Load Clean Data for Comparison
print("\n" + "="*80)
print("LOADING CLEAN DATA FOR COMPARISON")
print("="*80)

with h5py.File("test_solving_euler_ks_clean.h5", "r") as f:
    u_clean = np.array(f["u"])

result_clean = evaluate_sindy_fft(u_clean, x, t, method_name="Clean Data")

print(f"\nRecovered Coefficients (Clean Data):")
print(f"  a (u_xx):   {result_clean['a']:.6f}  (true: -1.0)")
print(f"  b (u_xxxx): {result_clean['b']:.6f}  (true: -1.0)")
print(f"  c (u*u_x):  {result_clean['c']:.6f}  (true: -1.0)")
print(f"\nEvaluation Metrics:")
print(f"  RMSE: {result_clean['rmse']:.6f}")
print(f"  R²:   {result_clean['r2']:.6f}")

# Summary Comparison
print("\n\n" + "="*80)
print("SUMMARY COMPARISON")
print("="*80)

print(f"\n{'Method':<25} {'a (u_xx)':>12} {'b (u_xxxx)':>12} {'c (u*u_x)':>12} {'RMSE':>12} {'R²':>12}")
print("-"*80)
print(f"{result_clean['method']:<25} {result_clean['a']:>12.6f} {result_clean['b']:>12.6f} "
      f"{result_clean['c']:>12.6f} {result_clean['rmse']:>12.6f} {result_clean['r2']:>12.6f}")
print(f"{result_baseline['method']:<25} {result_baseline['a']:>12.6f} {result_baseline['b']:>12.6f} "
      f"{result_baseline['c']:>12.6f} {result_baseline['rmse']:>12.6f} {result_baseline['r2']:>12.6f}")
print("="*80)
print(f"{'TRUE VALUES:':<25} {-1.0:>12.6f} {-1.0:>12.6f} {-1.0:>12.6f}")
print("="*80)

# Degradation Analysis
print("\n" + "="*80)
print("DEGRADATION FROM VARYING INITIAL CONDITIONS")
print("="*80)

degradation_rmse = (result_baseline['rmse'] - result_clean['rmse']) / result_clean['rmse'] * 100
degradation_r2 = (result_clean['r2'] - result_baseline['r2']) / result_clean['r2'] * 100

print(f"\nRMSE Degradation:  {degradation_rmse:+.2f}%")
print(f"R² Degradation:    {degradation_r2:+.2f}%")
print(f"\nCoefficient Error Increase:")
print(f"  a: {result_clean['abs_error_a']:.6f} → {result_baseline['abs_error_a']:.6f} "
      f"({(result_baseline['abs_error_a'] - result_clean['abs_error_a'])/result_clean['abs_error_a']*100:+.1f}%)")
print(f"  b: {result_clean['abs_error_b']:.6f} → {result_baseline['abs_error_b']:.6f} "
      f"({(result_baseline['abs_error_b'] - result_clean['abs_error_b'])/result_clean['abs_error_b']*100:+.1f}%)")
print(f"  c: {result_clean['abs_error_c']:.6f} → {result_baseline['abs_error_c']:.6f} "
      f"({(result_baseline['abs_error_c'] - result_clean['abs_error_c'])/result_clean['abs_error_c']*100:+.1f}%)")

# Visualizations
print("\n\n" + "="*80)
print("GENERATING VISUALIZATIONS...")
print("="*80)

fig = plt.figure(figsize=(16, 10))

# Plot 1: Spatiotemporal - Clean
ax1 = plt.subplot(2, 3, 1)
im1 = plt.imshow(u_clean[:500].T, cmap='RdBu', aspect='auto', origin='lower',
                 extent=(0, 500*t[1], x[0], x[-1]), vmin=-2, vmax=2)
plt.xlabel('Time')
plt.ylabel('Space (x)')
plt.title('Clean Data\n(Consistent Trajectory)', fontweight='bold')
plt.colorbar(im1, ax=ax1)

# Plot 2: Spatiotemporal - Varying IC
ax2 = plt.subplot(2, 3, 2)
im2 = plt.imshow(u_varied[:500].T, cmap='RdBu', aspect='auto', origin='lower',
                 extent=(0, 500*t[1], x[0], x[-1]), vmin=-2, vmax=2)
plt.xlabel('Time')
plt.ylabel('Space (x)')
plt.title('Varying Initial Conditions\n(Ensemble of Trajectories)', fontweight='bold')
plt.colorbar(im2, ax=ax2)

# Plot 3: Sample frames comparison
ax3 = plt.subplot(2, 3, 3)
sample_idx = len(u_clean) // 2
plt.plot(x, u_clean[sample_idx], 'g-', label='Clean', linewidth=2.5, alpha=0.8)
plt.plot(x, u_varied[sample_idx], 'r--', label='Varying IC', linewidth=2, alpha=0.7)
plt.xlabel('x')
plt.ylabel('u(x)')
plt.title(f'Sample Frame Comparison\n(Frame {sample_idx})', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 4: Trajectory divergence over time
ax4 = plt.subplot(2, 3, 4)
mean_traj = np.mean(u_varied, axis=0)
divergence = np.sqrt(np.mean((u_varied - mean_traj)**2, axis=1))
plt.plot(t, divergence, 'b-', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Trajectory Divergence')
plt.title('Trajectory Divergence from Mean\n(Shows IC Uncertainty)', fontweight='bold')
plt.grid(True, alpha=0.3)

# Plot 5: Coefficient comparison
ax5 = plt.subplot(2, 3, 5)
methods = ['Clean', 'Varying IC']
a_vals = [result_clean['a'], result_baseline['a']]
b_vals = [result_clean['b'], result_baseline['b']]
c_vals = [result_clean['c'], result_baseline['c']]
x_pos = np.arange(len(methods))
width = 0.25
plt.bar(x_pos - width, a_vals, width, label='a (u_xx)', alpha=0.8)
plt.bar(x_pos, b_vals, width, label='b (u_xxxx)', alpha=0.8)
plt.bar(x_pos + width, c_vals, width, label='c (u*u_x)', alpha=0.8)
plt.axhline(y=-1.0, color='red', linestyle='--', linewidth=2, label='True')
plt.ylabel('Coefficient Value')
plt.title('Recovered Coefficients', fontweight='bold')
plt.xticks(x_pos, methods)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

# Plot 6: RMSE and R² comparison
ax6 = plt.subplot(2, 3, 6)
metrics = ['RMSE', 'R²']
clean_vals = [result_clean['rmse'], result_clean['r2']]
varied_vals = [result_baseline['rmse'], result_baseline['r2']]
x_pos = np.arange(len(metrics))
width = 0.35
bars1 = plt.bar(x_pos - width/2, clean_vals, width, label='Clean', alpha=0.8, color='green')
bars2 = plt.bar(x_pos + width/2, varied_vals, width, label='Varying IC', alpha=0.8, color='red')
plt.ylabel('Metric Value')
plt.title('Metrics Comparison', fontweight='bold')
plt.xticks(x_pos, metrics)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('varying_ic_baseline_fft_results.png', dpi=300, bbox_inches='tight')
print("    Saved: varying_ic_baseline_fft_results.png")

# Save Results
results_dict = {
    'method': ['Clean Data', 'Varying IC (No Treatment)'],
    'a': [result_clean['a'], result_baseline['a']],
    'b': [result_clean['b'], result_baseline['b']],
    'c': [result_clean['c'], result_baseline['c']],
    'rmse': [result_clean['rmse'], result_baseline['rmse']],
    'r2': [result_clean['r2'], result_baseline['r2']],
    'abs_error_a': [result_clean['abs_error_a'], result_baseline['abs_error_a']],
    'abs_error_b': [result_clean['abs_error_b'], result_baseline['abs_error_b']],
    'abs_error_c': [result_clean['abs_error_c'], result_baseline['abs_error_c']],
    'mean_frame_variance': [0.0, ensemble_stats['mean_frame_variance']],
    'mean_divergence': [0.0, ensemble_stats['mean_divergence']],
    'max_divergence': [0.0, ensemble_stats['max_divergence']]
}

df_results = pd.DataFrame(results_dict)
df_results.to_csv('varying_ic_baseline_fft_results.csv', index=False)
print("    Saved: varying_ic_baseline_fft_results.csv")

print("\n" + "="*80)
print("   VARYING IC BASELINE + FFT SINDY COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. varying_ic_baseline_fft_results.png - Visualization")
print("  2. varying_ic_baseline_fft_results.csv - Results data")
print("="*80)

Baseline (no treatment) + Finite Difference derivatives

In [ ]:
import h5py
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import pandas as pd

print("="*80)
print("PHASE 3B: VARYING INITIAL CONDITIONS - BASELINE (NO TREATMENT) + FINITE DIFFERENCE SINDY")
print("="*80)

# ===== Evaluation Function with Finite Difference =====
def evaluate_sindy_finite_diff(u, x, t, method_name="Unknown"):
    """Apply Finite Difference SINDy and compute evaluation metrics"""
    dx = float(x[1] - x[0])
    dt = float(t[1] - t[0])
    
    def d1(U): 
        return (np.roll(U, -1, 1) - np.roll(U, 1, 1)) / (2*dx)
    def d2(U): 
        return (np.roll(U, -1, 1) - 2*U + np.roll(U, 1, 1)) / (dx**2)
    def d4(U):
        return (np.roll(U, -2, 1) - 4*np.roll(U, -1, 1) + 6*U - 
                4*np.roll(U, 1, 1) + np.roll(U, 2, 1)) / (dx**4)
    
    U_t = (u[1:] - u[:-1]) / dt
    U = u[:-1]
    U_x = d1(U)
    U_xx = d2(U)
    U_xxxx = d4(U)
    U2_x = d1(U**2)
    
    Theta = np.column_stack([
        np.ones_like(U).ravel(),
        U.ravel(),
        U_x.ravel(),
        U_xx.ravel(),
        U_xxxx.ravel(),
        U2_x.ravel()
    ])
    y = U_t.ravel()
    names = ["1", "u", "u_x", "u_xx", "u_xxxx", "(u^2)_x"]
    
    Theta_mean = Theta.mean(0)
    Theta_std = Theta.std(0) + 1e-8
    Theta_n = (Theta - Theta_mean) / Theta_std
    
    lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000, tol=1e-6)
    lasso.fit(Theta_n, y)
    coef = lasso.coef_ / Theta_std
    
    a = coef[names.index("u_xx")]
    b = coef[names.index("u_xxxx")]
    c_phys = coef[names.index("(u^2)_x")]
    
    y_pred = Theta @ coef
    rmse = np.sqrt(np.mean((y - y_pred)**2))
    r2 = r2_score(y, y_pred)
    
    a_true, b_true, c_true = -1.0, -1.0, -0.5
    
    return {
        'method': method_name,
        'a': a,
        'b': b,
        'c': c_phys,
        'error_a': a - a_true,
        'error_b': b - b_true,
        'error_c': c_phys - c_true,
        'abs_error_a': abs(a - a_true),
        'abs_error_b': abs(b - b_true),
        'abs_error_c': abs(c_phys - c_true),
        'rmse': rmse,
        'r2': r2
    }

# ===== Trajectory Ensemble Statistics =====
def compute_ensemble_statistics(u_data):
    """Compute statistics about trajectory ensemble"""
    mean_traj = np.mean(u_data, axis=0)
    std_traj = np.std(u_data, axis=0)
    frame_variance = np.var(u_data, axis=1)
    mean_frame_var = np.mean(frame_variance)
    divergence = np.sqrt(np.mean((u_data - mean_traj)**2, axis=1))
    mean_divergence = np.mean(divergence)
    max_divergence = np.max(divergence)
    
    n_frames = u_data.shape[0]
    sample_size = min(100, n_frames)
    indices = np.random.choice(n_frames, sample_size, replace=False)
    correlations = []
    for i in range(len(indices)-1):
        corr = np.corrcoef(u_data[indices[i]], u_data[indices[i+1]])[0, 1]
        correlations.append(corr)
    mean_correlation = np.mean(correlations)
    
    return {
        'mean_frame_variance': mean_frame_var,
        'mean_divergence': mean_divergence,
        'max_divergence': max_divergence,
        'mean_correlation': mean_correlation,
        'std_spatial_mean': np.mean(std_traj)
    }

# ===== Main Processing =====
print("\n📂 Loading varying initial condition data...")
with h5py.File("test_solving_euler_ks_varied_ic.h5", "r") as f:
    u_varied = np.array(f["u"], dtype=np.float32)
    x = np.array(f["x"], dtype=np.float32)
    t = np.array(f["t"], dtype=np.float32)

print(f"Loaded: test_solving_euler_ks_varied_ic.h5")
print(f"Shape: {u_varied.shape}")

# Compute ensemble statistics
print("\n📊 Computing trajectory ensemble statistics...")
ensemble_stats = compute_ensemble_statistics(u_varied)
print(f"\nEnsemble Statistics:")
print(f"  Mean Frame Variance:     {ensemble_stats['mean_frame_variance']:.6f}")
print(f"  Mean Trajectory Divergence: {ensemble_stats['mean_divergence']:.6f}")
print(f"  Max Trajectory Divergence:  {ensemble_stats['max_divergence']:.6f}")
print(f"  Mean Frame Correlation:     {ensemble_stats['mean_correlation']:.6f}")

# Method 1: Baseline - No treatment
print("\n" + "="*80)
print("METHOD 1: NO TREATMENT (BASELINE)")
print("="*80)

result_baseline = evaluate_sindy_finite_diff(u_varied, x, t, method_name="No Treatment")
print(f"\nRecovered: a={result_baseline['a']:.6f}, b={result_baseline['b']:.6f}, c={result_baseline['c']:.6f}")
print(f"RMSE={result_baseline['rmse']:.6f}, R²={result_baseline['r2']:.6f}")

# Load Clean Data
print("\n" + "="*80)
print("CLEAN DATA COMPARISON")
print("="*80)

with h5py.File("test_solving_euler_ks_clean.h5", "r") as f:
    u_clean = np.array(f["u"], dtype=np.float32)

result_clean = evaluate_sindy_finite_diff(u_clean, x, t, method_name="Clean Data")
print(f"Recovered: a={result_clean['a']:.6f}, b={result_clean['b']:.6f}, c={result_clean['c']:.6f}")
print(f"RMSE={result_clean['rmse']:.6f}, R²={result_clean['r2']:.6f}")

# Summary
print("\n\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"{'Method':<25} {'a':>12} {'b':>12} {'c':>12} {'RMSE':>12} {'R²':>12}")
print("-"*80)
print(f"{result_clean['method']:<25} {result_clean['a']:>12.6f} {result_clean['b']:>12.6f} "
      f"{result_clean['c']:>12.6f} {result_clean['rmse']:>12.6f} {result_clean['r2']:>12.6f}")
print(f"{result_baseline['method']:<25} {result_baseline['a']:>12.6f} {result_baseline['b']:>12.6f} "
      f"{result_baseline['c']:>12.6f} {result_baseline['rmse']:>12.6f} {result_baseline['r2']:>12.6f}")
print("="*80)

# Save Results
results_dict = {
    'method': ['Clean Data', 'Varying IC (No Treatment)'],
    'a': [result_clean['a'], result_baseline['a']],
    'b': [result_clean['b'], result_baseline['b']],
    'c': [result_clean['c'], result_baseline['c']],
    'rmse': [result_clean['rmse'], result_baseline['rmse']],
    'r2': [result_clean['r2'], result_baseline['r2']],
    'abs_error_a': [result_clean['abs_error_a'], result_baseline['abs_error_a']],
    'abs_error_b': [result_clean['abs_error_b'], result_baseline['abs_error_b']],
    'abs_error_c': [result_clean['abs_error_c'], result_baseline['abs_error_c']],
    'mean_frame_variance': [0.0, ensemble_stats['mean_frame_variance']],
    'mean_divergence': [0.0, ensemble_stats['mean_divergence']],
    'max_divergence': [0.0, ensemble_stats['max_divergence']]
}

pd.DataFrame(results_dict).to_csv('varying_ic_baseline_fd_results.csv', index=False)
print("\n    Saved: varying_ic_baseline_fd_results.csv")
print("\n   VARYING IC BASELINE + FINITE DIFFERENCE SINDY COMPLETE!")

Robust regression (Huber loss) + FFT derivatives
Tests 2 epsilon values (1.35, 1.0)

In [ ]:
import h5py
import numpy as np
from scipy.fft import fft, ifft, fftfreq
from sklearn.linear_model import Lasso, HuberRegressor
from sklearn.metrics import r2_score
import pandas as pd

print("="*80)
print("PHASE 3B: VARYING INITIAL CONDITIONS - ROBUST REGRESSION + FFT SINDY")
print("="*80)

# ===== Spectral derivative functions =====
def spectral_derivative(u_snapshot, k):
    """First derivative using FFT"""
    u_hat = fft(u_snapshot)
    du_hat = 1j * k * u_hat
    return np.real(ifft(du_hat))

def spectral_second_derivative(u_snapshot, k):
    """Second derivative using FFT"""
    u_hat = fft(u_snapshot)
    d2u_hat = -(k**2) * u_hat
    return np.real(ifft(d2u_hat))

def spectral_fourth_derivative(u_snapshot, k):
    """Fourth derivative using FFT"""
    u_hat = fft(u_snapshot)
    d4u_hat = (k**4) * u_hat
    return np.real(ifft(d4u_hat))

# ===== Evaluation Functions =====
def evaluate_sindy_fft_standard(u, x, t, method_name="Unknown"):
    """Standard Lasso regression (L1 penalty)"""
    Nx = len(x)
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    k = 2 * np.pi * fftfreq(Nx, d=dx)
    
    u_t = (u[2:] - u[:-2]) / (2 * dt)
    u_mid = u[1:-1]
    
    Theta = []
    ut_flat = []
    
    for snapshot, ut_snapshot in zip(u_mid, u_t):
        ux = spectral_derivative(snapshot, k)
        uxx = spectral_second_derivative(snapshot, k)
        uxxxx = spectral_fourth_derivative(snapshot, k)
        Theta_snapshot = np.vstack([uxx, uxxxx, snapshot * ux]).T
        Theta.append(Theta_snapshot)
        ut_flat.append(ut_snapshot)
    
    Theta = np.vstack(Theta)
    ut_flat = np.hstack(ut_flat)
    
    Theta_mean = Theta.mean(axis=0)
    Theta_std = Theta.std(axis=0)
    Theta_norm = (Theta - Theta_mean) / Theta_std
    
    lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000)
    lasso.fit(Theta_norm, ut_flat)
    coeffs = lasso.coef_ / Theta_std
    
    a, b, c = coeffs[0], coeffs[1], coeffs[2]
    u_t_pred = Theta @ coeffs
    rmse = np.sqrt(np.mean((ut_flat - u_t_pred)**2))
    r2 = r2_score(ut_flat, u_t_pred)
    
    a_true, b_true, c_true = -1.0, -1.0, -1.0
    
    return {
        'method': method_name,
        'a': a, 'b': b, 'c': c,
        'error_a': a - a_true,
        'error_b': b - b_true,
        'error_c': c - c_true,
        'abs_error_a': abs(a - a_true),
        'abs_error_b': abs(b - b_true),
        'abs_error_c': abs(c - c_true),
        'rmse': rmse,
        'r2': r2
    }

def evaluate_sindy_fft_robust(u, x, t, method_name="Unknown", epsilon=1.35):
    """
    Robust regression using Huber loss
    Epsilon parameter controls outlier threshold (default 1.35 is standard)
    Smaller epsilon = more aggressive outlier downweighting
    """
    Nx = len(x)
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    k = 2 * np.pi * fftfreq(Nx, d=dx)
    
    u_t = (u[2:] - u[:-2]) / (2 * dt)
    u_mid = u[1:-1]
    
    Theta = []
    ut_flat = []
    
    for snapshot, ut_snapshot in zip(u_mid, u_t):
        ux = spectral_derivative(snapshot, k)
        uxx = spectral_second_derivative(snapshot, k)
        uxxxx = spectral_fourth_derivative(snapshot, k)
        Theta_snapshot = np.vstack([uxx, uxxxx, snapshot * ux]).T
        Theta.append(Theta_snapshot)
        ut_flat.append(ut_snapshot)
    
    Theta = np.vstack(Theta)
    ut_flat = np.hstack(ut_flat)
    
    Theta_mean = Theta.mean(axis=0)
    Theta_std = Theta.std(axis=0)
    Theta_norm = (Theta - Theta_mean) / Theta_std
    
    # Huber Regressor (robust to outliers)
    huber = HuberRegressor(epsilon=epsilon, fit_intercept=False, max_iter=1000, alpha=1e-4)
    huber.fit(Theta_norm, ut_flat)
    coeffs = huber.coef_ / Theta_std
    
    # Count outliers detected
    residuals = ut_flat - huber.predict(Theta_norm)
    threshold = epsilon * np.median(np.abs(residuals))
    n_outliers = np.sum(np.abs(residuals) > threshold)
    outlier_fraction = n_outliers / len(residuals)
    
    a, b, c = coeffs[0], coeffs[1], coeffs[2]
    u_t_pred = Theta @ coeffs
    rmse = np.sqrt(np.mean((ut_flat - u_t_pred)**2))
    r2 = r2_score(ut_flat, u_t_pred)
    
    a_true, b_true, c_true = -1.0, -1.0, -1.0
    
    return {
        'method': method_name,
        'a': a, 'b': b, 'c': c,
        'error_a': a - a_true,
        'error_b': b - b_true,
        'error_c': c - c_true,
        'abs_error_a': abs(a - a_true),
        'abs_error_b': abs(b - b_true),
        'abs_error_c': abs(c - c_true),
        'rmse': rmse,
        'r2': r2,
        'n_outliers': n_outliers,
        'outlier_fraction': outlier_fraction
    }

# ===== Main Processing =====
print("\n📂 Loading varying initial condition data...")
with h5py.File("test_solving_euler_ks_varied_ic.h5", "r") as f:
    u_varied = np.array(f["u"])
    x = np.array(f["x"])
    t = np.array(f["t"])

print(f"Loaded: test_solving_euler_ks_varied_ic.h5")
print(f"Shape: {u_varied.shape}")

# Method 1: Standard Lasso (baseline)
print("\n" + "="*80)
print("METHOD 1: STANDARD LASSO (BASELINE)")
print("="*80)

result_standard = evaluate_sindy_fft_standard(u_varied, x, t, method_name="Standard Lasso")
print(f"\nRecovered: a={result_standard['a']:.6f}, b={result_standard['b']:.6f}, c={result_standard['c']:.6f}")
print(f"RMSE={result_standard['rmse']:.6f}, R²={result_standard['r2']:.6f}")

# Method 2: Robust Regression (epsilon=1.35, standard)
print("\n" + "="*80)
print("METHOD 2: ROBUST REGRESSION (Huber Loss, ε=1.35)")
print("="*80)

result_robust = evaluate_sindy_fft_robust(u_varied, x, t, method_name="Robust (ε=1.35)", epsilon=1.35)
print(f"\nRecovered: a={result_robust['a']:.6f}, b={result_robust['b']:.6f}, c={result_robust['c']:.6f}")
print(f"RMSE={result_robust['rmse']:.6f}, R²={result_robust['r2']:.6f}")
print(f"Outliers detected: {result_robust['n_outliers']} ({result_robust['outlier_fraction']*100:.2f}%)")

# Method 3: More aggressive outlier rejection (epsilon=1.0)
print("\n" + "="*80)
print("METHOD 3: AGGRESSIVE ROBUST REGRESSION (Huber Loss, ε=1.0)")
print("="*80)

result_robust_aggressive = evaluate_sindy_fft_robust(u_varied, x, t, method_name="Robust (ε=1.0)", epsilon=1.0)
print(f"\nRecovered: a={result_robust_aggressive['a']:.6f}, b={result_robust_aggressive['b']:.6f}, c={result_robust_aggressive['c']:.6f}")
print(f"RMSE={result_robust_aggressive['rmse']:.6f}, R²={result_robust_aggressive['r2']:.6f}")
print(f"Outliers detected: {result_robust_aggressive['n_outliers']} ({result_robust_aggressive['outlier_fraction']*100:.2f}%)")

# Load Clean Data
print("\n" + "="*80)
print("CLEAN DATA COMPARISON")
print("="*80)

with h5py.File("test_solving_euler_ks_clean.h5", "r") as f:
    u_clean = np.array(f["u"])

result_clean = evaluate_sindy_fft_standard(u_clean, x, t, method_name="Clean Data")
print(f"Recovered: a={result_clean['a']:.6f}, b={result_clean['b']:.6f}, c={result_clean['c']:.6f}")
print(f"RMSE={result_clean['rmse']:.6f}, R²={result_clean['r2']:.6f}")

# Summary
print("\n\n" + "="*80)
print("SUMMARY COMPARISON")
print("="*80)
print(f"{'Method':<25} {'a':>12} {'b':>12} {'c':>12} {'RMSE':>12} {'R²':>10}")
print("-"*80)
for r in [result_clean, result_standard, result_robust, result_robust_aggressive]:
    print(f"{r['method']:<25} {r['a']:>12.6f} {r['b']:>12.6f} {r['c']:>12.6f} "
          f"{r['rmse']:>12.6f} {r['r2']:>10.6f}")
print("="*80)

# Improvement Analysis
print("\n" + "="*80)
print("IMPROVEMENT FROM ROBUST REGRESSION")
print("="*80)

improv_robust = (result_standard['rmse'] - result_robust['rmse']) / result_standard['rmse'] * 100
improv_aggressive = (result_standard['rmse'] - result_robust_aggressive['rmse']) / result_standard['rmse'] * 100

print(f"\nRobust (ε=1.35):  RMSE {improv_robust:+.2f}%, R² change {(result_robust['r2']-result_standard['r2'])*100:+.2f}%")
print(f"Robust (ε=1.0):   RMSE {improv_aggressive:+.2f}%, R² change {(result_robust_aggressive['r2']-result_standard['r2'])*100:+.2f}%")

# Save Results
results_dict = {
    'method': ['Clean Data', 'Standard Lasso', 'Robust (ε=1.35)', 'Robust (ε=1.0)'],
    'a': [result_clean['a'], result_standard['a'], result_robust['a'], result_robust_aggressive['a']],
    'b': [result_clean['b'], result_standard['b'], result_robust['b'], result_robust_aggressive['b']],
    'c': [result_clean['c'], result_standard['c'], result_robust['c'], result_robust_aggressive['c']],
    'rmse': [result_clean['rmse'], result_standard['rmse'], result_robust['rmse'], result_robust_aggressive['rmse']],
    'r2': [result_clean['r2'], result_standard['r2'], result_robust['r2'], result_robust_aggressive['r2']],
    'abs_error_a': [result_clean['abs_error_a'], result_standard['abs_error_a'], result_robust['abs_error_a'], result_robust_aggressive['abs_error_a']],
    'abs_error_b': [result_clean['abs_error_b'], result_standard['abs_error_b'], result_robust['abs_error_b'], result_robust_aggressive['abs_error_b']],
    'abs_error_c': [result_clean['abs_error_c'], result_standard['abs_error_c'], result_robust['abs_error_c'], result_robust_aggressive['abs_error_c']],
    'outliers_detected': [0, 0, result_robust['n_outliers'], result_robust_aggressive['n_outliers']]
}

pd.DataFrame(results_dict).to_csv('varying_ic_robust_fft_results.csv', index=False)
print("\n    Saved: varying_ic_robust_fft_results.csv")
print("\n   VARYING IC ROBUST REGRESSION + FFT SINDY COMPLETE!")

Robust regression (Huber loss) + Finite Difference derivatives
Tests 2 epsilon values (1.35, 1.0)

In [ ]:
import h5py
import numpy as np
from sklearn.linear_model import Lasso, HuberRegressor
from sklearn.metrics import r2_score
import pandas as pd

print("="*80)
print("PHASE 3B: VARYING INITIAL CONDITIONS - ROBUST REGRESSION + FINITE DIFFERENCE SINDY")
print("="*80)

# ===== Evaluation Functions =====
def evaluate_sindy_fd_standard(u, x, t, method_name="Unknown"):
    """Standard Lasso regression"""
    dx = float(x[1] - x[0])
    dt = float(t[1] - t[0])
    
    def d1(U): return (np.roll(U, -1, 1) - np.roll(U, 1, 1)) / (2*dx)
    def d2(U): return (np.roll(U, -1, 1) - 2*U + np.roll(U, 1, 1)) / (dx**2)
    def d4(U): return (np.roll(U, -2, 1) - 4*np.roll(U, -1, 1) + 6*U - 4*np.roll(U, 1, 1) + np.roll(U, 2, 1)) / (dx**4)
    
    U_t = (u[1:] - u[:-1]) / dt
    U = u[:-1]
    Theta = np.column_stack([np.ones_like(U).ravel(), U.ravel(), d1(U).ravel(), d2(U).ravel(), d4(U).ravel(), d1(U**2).ravel()])
    y = U_t.ravel()
    names = ["1", "u", "u_x", "u_xx", "u_xxxx", "(u^2)_x"]
    
    Theta_mean = Theta.mean(0)
    Theta_std = Theta.std(0) + 1e-8
    Theta_n = (Theta - Theta_mean) / Theta_std
    
    lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=50000, tol=1e-6)
    lasso.fit(Theta_n, y)
    coef = lasso.coef_ / Theta_std
    
    a = coef[names.index("u_xx")]
    b = coef[names.index("u_xxxx")]
    c = coef[names.index("(u^2)_x")]
    
    y_pred = Theta @ coef
    rmse = np.sqrt(np.mean((y - y_pred)**2))
    r2 = r2_score(y, y_pred)
    
    a_true, b_true, c_true = -1.0, -1.0, -0.5
    
    return {
        'method': method_name, 'a': a, 'b': b, 'c': c,
        'error_a': a - a_true, 'error_b': b - b_true, 'error_c': c - c_true,
        'abs_error_a': abs(a - a_true), 'abs_error_b': abs(b - b_true), 'abs_error_c': abs(c - c_true),
        'rmse': rmse, 'r2': r2
    }

def evaluate_sindy_fd_robust(u, x, t, method_name="Unknown", epsilon=1.35):
    """Robust regression using Huber loss"""
    dx = float(x[1] - x[0])
    dt = float(t[1] - t[0])
    
    def d1(U): return (np.roll(U, -1, 1) - np.roll(U, 1, 1)) / (2*dx)
    def d2(U): return (np.roll(U, -1, 1) - 2*U + np.roll(U, 1, 1)) / (dx**2)
    def d4(U): return (np.roll(U, -2, 1) - 4*np.roll(U, -1, 1) + 6*U - 4*np.roll(U, 1, 1) + np.roll(U, 2, 1)) / (dx**4)
    
    U_t = (u[1:] - u[:-1]) / dt
    U = u[:-1]
    Theta = np.column_stack([np.ones_like(U).ravel(), U.ravel(), d1(U).ravel(), d2(U).ravel(), d4(U).ravel(), d1(U**2).ravel()])
    y = U_t.ravel()
    names = ["1", "u", "u_x", "u_xx", "u_xxxx", "(u^2)_x"]
    
    Theta_mean = Theta.mean(0)
    Theta_std = Theta.std(0) + 1e-8
    Theta_n = (Theta - Theta_mean) / Theta_std
    
    huber = HuberRegressor(epsilon=epsilon, fit_intercept=False, max_iter=1000, alpha=1e-4)
    huber.fit(Theta_n, y)
    coef = huber.coef_ / Theta_std
    
    residuals = y - huber.predict(Theta_n)
    threshold = epsilon * np.median(np.abs(residuals))
    n_outliers = np.sum(np.abs(residuals) > threshold)
    outlier_fraction = n_outliers / len(residuals)
    
    a = coef[names.index("u_xx")]
    b = coef[names.index("u_xxxx")]
    c = coef[names.index("(u^2)_x")]
    
    y_pred = Theta @ coef
    rmse = np.sqrt(np.mean((y - y_pred)**2))
    r2 = r2_score(y, y_pred)
    
    a_true, b_true, c_true = -1.0, -1.0, -0.5
    
    return {
        'method': method_name, 'a': a, 'b': b, 'c': c,
        'error_a': a - a_true, 'error_b': b - b_true, 'error_c': c - c_true,
        'abs_error_a': abs(a - a_true), 'abs_error_b': abs(b - b_true), 'abs_error_c': abs(c - c_true),
        'rmse': rmse, 'r2': r2, 'n_outliers': n_outliers, 'outlier_fraction': outlier_fraction
    }

# ===== Main Processing =====
print("\n📂 Loading varying initial condition data...")
with h5py.File("test_solving_euler_ks_varied_ic.h5", "r") as f:
    u_varied = np.array(f["u"], dtype=np.float32)
    x = np.array(f["x"], dtype=np.float32)
    t = np.array(f["t"], dtype=np.float32)

print(f"Loaded: {u_varied.shape}")

# Method 1: Standard Lasso
print("\n" + "="*80)
print("METHOD 1: STANDARD LASSO")
print("="*80)
result_standard = evaluate_sindy_fd_standard(u_varied, x, t, "Standard Lasso")
print(f"Recovered: a={result_standard['a']:.6f}, b={result_standard['b']:.6f}, c={result_standard['c']:.6f}")
print(f"RMSE={result_standard['rmse']:.6f}, R²={result_standard['r2']:.6f}")

# Method 2: Robust (ε=1.35)
print("\n" + "="*80)
print("METHOD 2: ROBUST REGRESSION (ε=1.35)")
print("="*80)
result_robust = evaluate_sindy_fd_robust(u_varied, x, t, "Robust (ε=1.35)", 1.35)
print(f"Recovered: a={result_robust['a']:.6f}, b={result_robust['b']:.6f}, c={result_robust['c']:.6f}")
print(f"RMSE={result_robust['rmse']:.6f}, R²={result_robust['r2']:.6f}")
print(f"Outliers: {result_robust['n_outliers']} ({result_robust['outlier_fraction']*100:.2f}%)")

# Method 3: Aggressive (ε=1.0)
print("\n" + "="*80)
print("METHOD 3: AGGRESSIVE ROBUST (ε=1.0)")
print("="*80)
result_robust_agg = evaluate_sindy_fd_robust(u_varied, x, t, "Robust (ε=1.0)", 1.0)
print(f"Recovered: a={result_robust_agg['a']:.6f}, b={result_robust_agg['b']:.6f}, c={result_robust_agg['c']:.6f}")
print(f"RMSE={result_robust_agg['rmse']:.6f}, R²={result_robust_agg['r2']:.6f}")
print(f"Outliers: {result_robust_agg['n_outliers']} ({result_robust_agg['outlier_fraction']*100:.2f}%)")

# Clean Data
with h5py.File("test_solving_euler_ks_clean.h5", "r") as f:
    u_clean = np.array(f["u"], dtype=np.float32)
result_clean = evaluate_sindy_fd_standard(u_clean, x, t, "Clean Data")

# Summary
print("\n\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"{'Method':<25} {'a':>12} {'b':>12} {'c':>12} {'RMSE':>12} {'R²':>10}")
print("-"*80)
for r in [result_clean, result_standard, result_robust, result_robust_agg]:
    print(f"{r['method']:<25} {r['a']:>12.6f} {r['b']:>12.6f} {r['c']:>12.6f} {r['rmse']:>12.6f} {r['r2']:>10.6f}")
print("="*80)

# Save
pd.DataFrame({
    'method': ['Clean Data', 'Standard Lasso', 'Robust (ε=1.35)', 'Robust (ε=1.0)'],
    'a': [result_clean['a'], result_standard['a'], result_robust['a'], result_robust_agg['a']],
    'b': [result_clean['b'], result_standard['b'], result_robust['b'], result_robust_agg['b']],
    'c': [result_clean['c'], result_standard['c'], result_robust['c'], result_robust_agg['c']],
    'rmse': [result_clean['rmse'], result_standard['rmse'], result_robust['rmse'], result_robust_agg['rmse']],
    'r2': [result_clean['r2'], result_standard['r2'], result_robust['r2'], result_robust_agg['r2']],
    'abs_error_a': [result_clean['abs_error_a'], result_standard['abs_error_a'], result_robust['abs_error_a'], result_robust_agg['abs_error_a']],
    'abs_error_b': [result_clean['abs_error_b'], result_standard['abs_error_b'], result_robust['abs_error_b'], result_robust_agg['abs_error_b']],
    'abs_error_c': [result_clean['abs_error_c'], result_standard['abs_error_c'], result_robust['abs_error_c'], result_robust_agg['abs_error_c']],
    'outliers_detected': [0, 0, result_robust['n_outliers'], result_robust_agg['n_outliers']]
}).to_csv('varying_ic_robust_fd_results.csv', index=False)

print("\n    Saved: varying_ic_robust_fd_results.csv")
print("\n   VARYING IC ROBUST REGRESSION + FINITE DIFFERENCE SINDY COMPLETE!")

In [ ]:
# In Jupyter, after running the script:
import pandas as pd
df = pd.read_csv('varying_ic_baseline_fft_results.csv')
print(df[['method', 'a', 'b', 'c']])

Comparison

In [ ]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (20, 14)

print("="*80)
print("PHASE 3B: VARYING INITIAL CONDITIONS - COMPREHENSIVE COMPARISON")
print("="*80)

# ===== Load All Results =====
print("\n📂 Loading all varying IC results...")

try:
    baseline_fft = pd.read_csv('varying_ic_baseline_fft_results.csv')
    baseline_fd = pd.read_csv('varying_ic_baseline_fd_results.csv')
    robust_fft = pd.read_csv('varying_ic_robust_fft_results.csv')
    robust_fd = pd.read_csv('varying_ic_robust_fd_results.csv')
    print("    Successfully loaded all 4 result files")
except FileNotFoundError as e:
    print(f"    Error: {e}")
    print("Please run all 4 varying IC scripts first.")
    exit()

# ===== Load Data for Visual Comparison =====
print("\n📂 Loading spatial data for visual comparison...")

with h5py.File("test_solving_euler_ks_clean.h5", "r") as f:
    u_clean = np.array(f["u"])
    x = np.array(f["x"])
    t = np.array(f["t"])

with h5py.File("test_solving_euler_ks_varied_ic.h5", "r") as f:
    u_varied = np.array(f["u"])

print("    Data loaded successfully")

# ===== Build Comparison DataFrame =====
comparison_data = []

# FFT Methods
for idx, row in baseline_fft.iterrows():
    if row['method'] != 'Clean Data':
        comparison_data.append({
            'Derivative': 'FFT', 'Treatment': 'No Treatment',
            'a': row['a'], 'b': row['b'], 'c': row['c'],
            'rmse': row['rmse'], 'r2': row['r2'],
            'abs_error_a': row['abs_error_a'], 'abs_error_b': row['abs_error_b'], 'abs_error_c': row['abs_error_c']
        })

for idx, row in robust_fft.iterrows():
    if row['method'] not in ['Clean Data', 'Standard Lasso']:
        comparison_data.append({
            'Derivative': 'FFT', 'Treatment': row['method'],
            'a': row['a'], 'b': row['b'], 'c': row['c'],
            'rmse': row['rmse'], 'r2': row['r2'],
            'abs_error_a': row['abs_error_a'], 'abs_error_b': row['abs_error_b'], 'abs_error_c': row['abs_error_c']
        })

# Finite Difference Methods
for idx, row in baseline_fd.iterrows():
    if row['method'] != 'Clean Data':
        comparison_data.append({
            'Derivative': 'Finite Diff', 'Treatment': 'No Treatment',
            'a': row['a'], 'b': row['b'], 'c': row['c'],
            'rmse': row['rmse'], 'r2': row['r2'],
            'abs_error_a': row['abs_error_a'], 'abs_error_b': row['abs_error_b'], 'abs_error_c': row['abs_error_c']
        })

for idx, row in robust_fd.iterrows():
    if row['method'] not in ['Clean Data', 'Standard Lasso']:
        comparison_data.append({
            'Derivative': 'Finite Diff', 'Treatment': row['method'],
            'a': row['a'], 'b': row['b'], 'c': row['c'],
            'rmse': row['rmse'], 'r2': row['r2'],
            'abs_error_a': row['abs_error_a'], 'abs_error_b': row['abs_error_b'], 'abs_error_c': row['abs_error_c']
        })

df_comparison = pd.DataFrame(comparison_data)

# ===== Extract Clean Data Baselines =====
clean_fft = baseline_fft[baseline_fft['method'] == 'Clean Data'].iloc[0]
clean_fd = baseline_fd[baseline_fd['method'] == 'Clean Data'].iloc[0]

baseline_fft_notreat = baseline_fft[baseline_fft['method'] == 'Varying IC (No Treatment)'].iloc[0]
baseline_fd_notreat = baseline_fd[baseline_fd['method'] == 'Varying IC (No Treatment)'].iloc[0]

# ===== SUMMARY TABLES =====
print("\n\n" + "="*100)
print("COMPARISON TABLE 1: ALL METHODS")
print("="*100)
print(f"\n{'Derivative':>12} {'Treatment':>20} {'a':>12} {'b':>12} {'c':>12} {'RMSE':>12} {'R²':>10}")
print("-"*100)
for _, row in df_comparison.iterrows():
    print(f"{row['Derivative']:>12} {row['Treatment']:>20} {row['a']:>12.6f} {row['b']:>12.6f} "
          f"{row['c']:>12.6f} {row['rmse']:>12.6f} {row['r2']:>10.6f}")
print("="*100)

print("\n\n" + "="*100)
print("COMPARISON TABLE 2: CLEAN DATA BASELINE")
print("="*100)
print(f"\n{'Derivative':>12} {'a':>12} {'b':>12} {'c':>12} {'RMSE':>12} {'R²':>10}")
print("-"*100)
print(f"{'FFT':>12} {clean_fft['a']:>12.6f} {clean_fft['b']:>12.6f} {clean_fft['c']:>12.6f} "
      f"{clean_fft['rmse']:>12.6f} {clean_fft['r2']:>10.6f}")
print(f"{'Finite Diff':>12} {clean_fd['a']:>12.6f} {clean_fd['b']:>12.6f} {clean_fd['c']:>12.6f} "
      f"{clean_fd['rmse']:>12.6f} {clean_fd['r2']:>10.6f}")
print("="*100)

# ===== BEST METHOD ANALYSIS =====
df_comparison['mean_coef_error'] = (df_comparison['abs_error_a'] + 
                                     df_comparison['abs_error_b'] + 
                                     df_comparison['abs_error_c']) / 3

best_rmse = df_comparison.loc[df_comparison['rmse'].idxmin()]
best_r2 = df_comparison.loc[df_comparison['r2'].idxmax()]
best_coef = df_comparison.loc[df_comparison['mean_coef_error'].idxmin()]

print("\n\n" + "="*80)
print("BEST METHODS")
print("="*80)
print(f"\n🏆 BEST RMSE: {best_rmse['Derivative']} + {best_rmse['Treatment']}")
print(f"   RMSE={best_rmse['rmse']:.6f}, R²={best_rmse['r2']:.6f}")
print(f"\n🏆 BEST R²: {best_r2['Derivative']} + {best_r2['Treatment']}")
print(f"   RMSE={best_r2['rmse']:.6f}, R²={best_r2['r2']:.6f}")
print(f"\n🏆 BEST COEFFICIENT ACCURACY: {best_coef['Derivative']} + {best_coef['Treatment']}")
print(f"   Mean Error={best_coef['mean_coef_error']:.6f}")

# ===== VISUALIZATION 1: Main Comparison =====
print("\n\n" + "="*80)
print("GENERATING VISUALIZATIONS...")
print("="*80)

fig1 = plt.figure(figsize=(20, 14))

n_frames = 500
sample_idx = len(u_clean) // 2

# Row 1: Spatiotemporal Comparison
ax1 = plt.subplot(3, 4, 1)
im1 = plt.imshow(u_clean[:n_frames].T, cmap='RdBu', aspect='auto', origin='lower',
                 extent=(0, n_frames*t[1], x[0], x[-1]), vmin=-2, vmax=2)
plt.xlabel('Time', fontsize=10)
plt.ylabel('Space (x)', fontsize=10)
plt.title('Clean Data\n(Single Trajectory)', fontweight='bold', fontsize=11)
plt.colorbar(im1, ax=ax1, label='u(x,t)')

ax2 = plt.subplot(3, 4, 2)
im2 = plt.imshow(u_varied[:n_frames].T, cmap='RdBu', aspect='auto', origin='lower',
                 extent=(0, n_frames*t[1], x[0], x[-1]), vmin=-2, vmax=2)
plt.xlabel('Time', fontsize=10)
plt.ylabel('Space (x)', fontsize=10)
plt.title('Varying IC (No Treatment)\n(Ensemble of Trajectories)', fontweight='bold', fontsize=11)
plt.colorbar(im2, ax=ax2, label='u(x,t)')

# Row 1: Sample Frame Comparison
ax3 = plt.subplot(3, 4, 3)
plt.plot(x, u_clean[sample_idx], 'g-', label='Clean', linewidth=2.5, alpha=0.8)
plt.plot(x, u_varied[sample_idx], 'r--', label='Varying IC', linewidth=2, alpha=0.7)
plt.xlabel('x', fontsize=10)
plt.ylabel('u(x)', fontsize=10)
plt.title(f'Sample Frame Comparison\n(Frame {sample_idx})', fontweight='bold', fontsize=11)
plt.legend(fontsize=9)
plt.grid(True, alpha=0.3)

# Row 1: Trajectory Divergence
ax4 = plt.subplot(3, 4, 4)
mean_traj = np.mean(u_varied, axis=0)
divergence = np.sqrt(np.mean((u_varied - mean_traj)**2, axis=1))
plt.plot(t, divergence, 'b-', linewidth=2)
plt.xlabel('Time', fontsize=10)
plt.ylabel('Trajectory Divergence', fontsize=10)
plt.title('Trajectory Divergence from Mean\n(Shows IC Uncertainty)', fontweight='bold', fontsize=11)
plt.grid(True, alpha=0.3)

# Row 2: Coefficient Recovery - FFT
ax5 = plt.subplot(3, 4, 5)
methods_fft = ['No Treat', 'Robust\n(ε=1.35)', 'Robust\n(ε=1.0)']
a_vals_fft = [
    baseline_fft_notreat['a'],
    robust_fft[robust_fft['method']=='Robust (ε=1.35)'].iloc[0]['a'],
    robust_fft[robust_fft['method']=='Robust (ε=1.0)'].iloc[0]['a']
]
b_vals_fft = [
    baseline_fft_notreat['b'],
    robust_fft[robust_fft['method']=='Robust (ε=1.35)'].iloc[0]['b'],
    robust_fft[robust_fft['method']=='Robust (ε=1.0)'].iloc[0]['b']
]
c_vals_fft = [
    baseline_fft_notreat['c'],
    robust_fft[robust_fft['method']=='Robust (ε=1.35)'].iloc[0]['c'],
    robust_fft[robust_fft['method']=='Robust (ε=1.0)'].iloc[0]['c']
]
x_pos = np.arange(len(methods_fft))
width = 0.25
plt.bar(x_pos - width, a_vals_fft, width, label='a (u_xx)', alpha=0.8)
plt.bar(x_pos, b_vals_fft, width, label='b (u_xxxx)', alpha=0.8)
plt.bar(x_pos + width, c_vals_fft, width, label='c (u*u_x)', alpha=0.8)
plt.axhline(y=-1.0, color='red', linestyle='--', linewidth=2, label='True')
plt.ylabel('Coefficient Value', fontsize=10)
plt.title('FFT: Coefficient Recovery', fontweight='bold', fontsize=11)
plt.xticks(x_pos, methods_fft, fontsize=8)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3, axis='y')

# Row 2: Coefficient Recovery - FD
ax6 = plt.subplot(3, 4, 6)
a_vals_fd = [
    baseline_fd_notreat['a'],
    robust_fd[robust_fd['method']=='Robust (ε=1.35)'].iloc[0]['a'],
    robust_fd[robust_fd['method']=='Robust (ε=1.0)'].iloc[0]['a']
]
b_vals_fd = [
    baseline_fd_notreat['b'],
    robust_fd[robust_fd['method']=='Robust (ε=1.35)'].iloc[0]['b'],
    robust_fd[robust_fd['method']=='Robust (ε=1.0)'].iloc[0]['b']
]
c_vals_fd = [
    baseline_fd_notreat['c'],
    robust_fd[robust_fd['method']=='Robust (ε=1.35)'].iloc[0]['c'],
    robust_fd[robust_fd['method']=='Robust (ε=1.0)'].iloc[0]['c']
]
plt.bar(x_pos - width, a_vals_fd, width, label='a (u_xx)', alpha=0.8)
plt.bar(x_pos, b_vals_fd, width, label='b (u_xxxx)', alpha=0.8)
plt.bar(x_pos + width, c_vals_fd, width, label='c ((u²)_x)', alpha=0.8)
plt.axhline(y=-1.0, color='red', linestyle='--', linewidth=2)
plt.axhline(y=-0.5, color='orange', linestyle='--', linewidth=2, label='True (c)')
plt.ylabel('Coefficient Value', fontsize=10)
plt.title('Finite Diff: Coefficient Recovery', fontweight='bold', fontsize=11)
plt.xticks(x_pos, methods_fft, fontsize=8)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3, axis='y')

# Row 2: RMSE Comparison
ax7 = plt.subplot(3, 4, 7)
rmse_fft = [
    baseline_fft_notreat['rmse'],
    robust_fft[robust_fft['method']=='Robust (ε=1.35)'].iloc[0]['rmse'],
    robust_fft[robust_fft['method']=='Robust (ε=1.0)'].iloc[0]['rmse']
]
rmse_fd = [
    baseline_fd_notreat['rmse'],
    robust_fd[robust_fd['method']=='Robust (ε=1.35)'].iloc[0]['rmse'],
    robust_fd[robust_fd['method']=='Robust (ε=1.0)'].iloc[0]['rmse']
]
width2 = 0.35
plt.bar(x_pos - width2/2, rmse_fft, width2, label='FFT', alpha=0.8, color='blue')
plt.bar(x_pos + width2/2, rmse_fd, width2, label='Finite Diff', alpha=0.8, color='green')
plt.axhline(y=clean_fft['rmse'], color='blue', linestyle='--', linewidth=1, alpha=0.5)
plt.axhline(y=clean_fd['rmse'], color='green', linestyle='--', linewidth=1, alpha=0.5)
plt.ylabel('RMSE', fontsize=10)
plt.title('RMSE Comparison\n(Dashed = Clean)', fontweight='bold', fontsize=11)
plt.xticks(x_pos, methods_fft, fontsize=8)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3, axis='y')
plt.yscale('log')

# Row 2: R² Comparison
ax8 = plt.subplot(3, 4, 8)
r2_fft = [
    baseline_fft_notreat['r2'],
    robust_fft[robust_fft['method']=='Robust (ε=1.35)'].iloc[0]['r2'],
    robust_fft[robust_fft['method']=='Robust (ε=1.0)'].iloc[0]['r2']
]
r2_fd = [
    baseline_fd_notreat['r2'],
    robust_fd[robust_fd['method']=='Robust (ε=1.35)'].iloc[0]['r2'],
    robust_fd[robust_fd['method']=='Robust (ε=1.0)'].iloc[0]['r2']
]
plt.bar(x_pos - width2/2, r2_fft, width2, label='FFT', alpha=0.8, color='blue')
plt.bar(x_pos + width2/2, r2_fd, width2, label='Finite Diff', alpha=0.8, color='green')
plt.axhline(y=clean_fft['r2'], color='blue', linestyle='--', linewidth=1, alpha=0.5)
plt.axhline(y=clean_fd['r2'], color='green', linestyle='--', linewidth=1, alpha=0.5)
plt.axhline(y=1.0, color='black', linestyle=':', linewidth=1)
plt.ylabel('R² Score', fontsize=10)
plt.title('R² Comparison\n(Dashed = Clean)', fontweight='bold', fontsize=11)
plt.xticks(x_pos, methods_fft, fontsize=8)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3, axis='y')

# Row 3: Absolute Errors - FFT
ax9 = plt.subplot(3, 4, 9)
err_a_fft = [baseline_fft_notreat['abs_error_a'],
             robust_fft[robust_fft['method']=='Robust (ε=1.35)'].iloc[0]['abs_error_a'],
             robust_fft[robust_fft['method']=='Robust (ε=1.0)'].iloc[0]['abs_error_a']]
err_b_fft = [baseline_fft_notreat['abs_error_b'],
             robust_fft[robust_fft['method']=='Robust (ε=1.35)'].iloc[0]['abs_error_b'],
             robust_fft[robust_fft['method']=='Robust (ε=1.0)'].iloc[0]['abs_error_b']]
err_c_fft = [baseline_fft_notreat['abs_error_c'],
             robust_fft[robust_fft['method']=='Robust (ε=1.35)'].iloc[0]['abs_error_c'],
             robust_fft[robust_fft['method']=='Robust (ε=1.0)'].iloc[0]['abs_error_c']]
plt.bar(x_pos - width, err_a_fft, width, label='|Err a|', alpha=0.8)
plt.bar(x_pos, err_b_fft, width, label='|Err b|', alpha=0.8)
plt.bar(x_pos + width, err_c_fft, width, label='|Err c|', alpha=0.8)
plt.ylabel('Absolute Error', fontsize=10)
plt.title('FFT: Absolute Coefficient Errors', fontweight='bold', fontsize=11)
plt.xticks(x_pos, methods_fft, fontsize=8)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3, axis='y')
plt.yscale('log')

# Row 3: Absolute Errors - FD
ax10 = plt.subplot(3, 4, 10)
err_a_fd = [baseline_fd_notreat['abs_error_a'],
            robust_fd[robust_fd['method']=='Robust (ε=1.35)'].iloc[0]['abs_error_a'],
            robust_fd[robust_fd['method']=='Robust (ε=1.0)'].iloc[0]['abs_error_a']]
err_b_fd = [baseline_fd_notreat['abs_error_b'],
            robust_fd[robust_fd['method']=='Robust (ε=1.35)'].iloc[0]['abs_error_b'],
            robust_fd[robust_fd['method']=='Robust (ε=1.0)'].iloc[0]['abs_error_b']]
err_c_fd = [baseline_fd_notreat['abs_error_c'],
            robust_fd[robust_fd['method']=='Robust (ε=1.35)'].iloc[0]['abs_error_c'],
            robust_fd[robust_fd['method']=='Robust (ε=1.0)'].iloc[0]['abs_error_c']]
plt.bar(x_pos - width, err_a_fd, width, label='|Err a|', alpha=0.8)
plt.bar(x_pos, err_b_fd, width, label='|Err b|', alpha=0.8)
plt.bar(x_pos + width, err_c_fd, width, label='|Err c|', alpha=0.8)
plt.ylabel('Absolute Error', fontsize=10)
plt.title('Finite Diff: Absolute Coefficient Errors', fontweight='bold', fontsize=11)
plt.xticks(x_pos, methods_fft, fontsize=8)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3, axis='y')
plt.yscale('log')

# Row 3: Outliers Detected - FFT
ax11 = plt.subplot(3, 4, 11)
if 'outliers_detected' in robust_fft.columns:
    outliers_fft = [0,
                    robust_fft[robust_fft['method']=='Robust (ε=1.35)'].iloc[0]['outliers_detected'],
                    robust_fft[robust_fft['method']=='Robust (ε=1.0)'].iloc[0]['outliers_detected']]
    colors = ['red', 'blue', 'green']
    plt.bar(x_pos, outliers_fft, color=colors, alpha=0.7, edgecolor='black')
    plt.ylabel('Number of Outliers', fontsize=10)
    plt.title('FFT: Outliers Detected', fontweight='bold', fontsize=11)
    plt.xticks(x_pos, methods_fft, fontsize=8)
    plt.grid(True, alpha=0.3, axis='y')

# Row 3: Outliers Detected - FD
ax12 = plt.subplot(3, 4, 12)
if 'outliers_detected' in robust_fd.columns:
    outliers_fd = [0,
                   robust_fd[robust_fd['method']=='Robust (ε=1.35)'].iloc[0]['outliers_detected'],
                   robust_fd[robust_fd['method']=='Robust (ε=1.0)'].iloc[0]['outliers_detected']]
    plt.bar(x_pos, outliers_fd, color=colors, alpha=0.7, edgecolor='black')
    plt.ylabel('Number of Outliers', fontsize=10)
    plt.title('Finite Diff: Outliers Detected', fontweight='bold', fontsize=11)
    plt.xticks(x_pos, methods_fft, fontsize=8)
    plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('varying_ic_visual_comparison.png', dpi=300, bbox_inches='tight')
print("    Saved: varying_ic_visual_comparison.png")

# ===== VISUALIZATION 2: Heatmaps =====
fig2, axes = plt.subplots(2, 2, figsize=(14, 10))

methods_clean = ['No Treat', 'Robust (ε=1.35)', 'Robust (ε=1.0)']
derivatives = ['FFT', 'Finite Diff']

# RMSE Heatmap
rmse_matrix = np.array([
    rmse_fft,
    rmse_fd
])
sns.heatmap(rmse_matrix, annot=True, fmt='.4f', cmap='RdYlGn_r',
            xticklabels=methods_clean, yticklabels=derivatives, ax=axes[0,0],
            cbar_kws={'label': 'RMSE'})
axes[0,0].set_title('RMSE by Method', fontweight='bold', fontsize=12)
axes[0,0].set_xlabel('Treatment Method')
axes[0,0].set_ylabel('Derivative Type')

# R² Heatmap
r2_matrix = np.array([
    r2_fft,
    r2_fd
])
sns.heatmap(r2_matrix, annot=True, fmt='.4f', cmap='RdYlGn',
            xticklabels=methods_clean, yticklabels=derivatives, ax=axes[0,1],
            cbar_kws={'label': 'R² Score'})
axes[0,1].set_title('R² Score by Method', fontweight='bold', fontsize=12)
axes[0,1].set_xlabel('Treatment Method')
axes[0,1].set_ylabel('Derivative Type')

# Coefficient Error a Heatmap
err_a_matrix = np.array([
    err_a_fft,
    err_a_fd
])
sns.heatmap(err_a_matrix, annot=True, fmt='.4f', cmap='RdYlGn_r',
            xticklabels=methods_clean, yticklabels=derivatives, ax=axes[1,0],
            cbar_kws={'label': '|Error in a|'})
axes[1,0].set_title('Absolute Error in Coefficient a', fontweight='bold', fontsize=12)
axes[1,0].set_xlabel('Treatment Method')
axes[1,0].set_ylabel('Derivative Type')

# Coefficient Error b Heatmap
err_b_matrix = np.array([
    err_b_fft,
    err_b_fd
])
sns.heatmap(err_b_matrix, annot=True, fmt='.4f', cmap='RdYlGn_r',
            xticklabels=methods_clean, yticklabels=derivatives, ax=axes[1,1],
            cbar_kws={'label': '|Error in b|'})
axes[1,1].set_title('Absolute Error in Coefficient b', fontweight='bold', fontsize=12)
axes[1,1].set_xlabel('Treatment Method')
axes[1,1].set_ylabel('Derivative Type')

plt.tight_layout()
plt.savefig('varying_ic_heatmap_comparison.png', dpi=300, bbox_inches='tight')
print("    Saved: varying_ic_heatmap_comparison.png")

# ===== Save Comparison Data =====
df_comparison.to_csv('varying_ic_all_methods_comparison.csv', index=False)
print("    Saved: varying_ic_all_methods_comparison.csv")

print("\n" + "="*80)
print("   VARYING IC COMPREHENSIVE COMPARISON COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. varying_ic_visual_comparison.png - Visual data & performance comparison (12 subplots)")
print("  2. varying_ic_heatmap_comparison.png - Performance heatmaps (4 heatmaps)")
print("  3. varying_ic_all_methods_comparison.csv - Complete comparison data")
print("="*80)

In [ ]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.fft import fft, ifft, fftfreq
from sklearn.linear_model import HuberRegressor

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (20, 14)

print("="*80)
print("PHASE 3B: VARYING INITIAL CONDITIONS - ENHANCED COMPARISON")
print("="*80)

# ===== Load All Results =====
print("\n📂 Loading all varying IC results...")

try:
    baseline_fft = pd.read_csv('varying_ic_baseline_fft_results.csv')
    baseline_fd = pd.read_csv('varying_ic_baseline_fd_results.csv')
    robust_fft = pd.read_csv('varying_ic_robust_fft_results.csv')
    robust_fd = pd.read_csv('varying_ic_robust_fd_results.csv')
    print("    Successfully loaded all 4 result files")
except FileNotFoundError as e:
    print(f" Error: {e}")
    print("Please run all 4 varying IC scripts first.")
    exit()

# ===== Load Data for Visual Comparison =====
print("\n📂 Loading spatial data for outlier detection...")

with h5py.File("test_solving_euler_ks_clean.h5", "r") as f:
    u_clean = np.array(f["u"])
    x = np.array(f["x"])
    t = np.array(f["t"])

with h5py.File("test_solving_euler_ks_varied_ic.h5", "r") as f:
    u_varied = np.array(f["u"])

print("    Data loaded successfully")

# Print data information
dt = t[1] - t[0]
n_frames_viz = 500
time_duration = n_frames_viz * dt
sample_idx = len(u_clean) // 2
sample_time = t[sample_idx]

print(f"\n📊 Data Information:")
print(f"  Total time steps: {len(t)}")
print(f"  Time step size (dt): {dt:.6f}")
print(f"  Spatiotemporal plots show: {n_frames_viz} frames (Time: 0 to {time_duration:.3f} units)")
print(f"  Sample frame index: {sample_idx} (Time: {sample_time:.3f} units)")
print(f"  Spatial domain: x ∈ [{x[0]:.2f}, {x[-1]:.2f}]")

# ===== Detect Outliers using Robust Regression =====
print("\n🔍 Computing outlier detection maps...")

def detect_outliers_fft(u, x, t, epsilon=1.35):
    """Detect outliers using Huber regression with FFT derivatives"""
    Nx = len(x)
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    k = 2 * np.pi * fftfreq(Nx, d=dx)
    
    def spectral_derivative(u_snapshot, k):
        u_hat = fft(u_snapshot)
        du_hat = 1j * k * u_hat
        return np.real(ifft(du_hat))
    
    def spectral_second_derivative(u_snapshot, k):
        u_hat = fft(u_snapshot)
        d2u_hat = -(k**2) * u_hat
        return np.real(ifft(d2u_hat))
    
    def spectral_fourth_derivative(u_snapshot, k):
        u_hat = fft(u_snapshot)
        d4u_hat = (k**4) * u_hat
        return np.real(ifft(d4u_hat))
    
    u_t = (u[2:] - u[:-2]) / (2 * dt)
    u_mid = u[1:-1]
    
    Theta = []
    ut_flat = []
    
    for snapshot, ut_snapshot in zip(u_mid, u_t):
        ux = spectral_derivative(snapshot, k)
        uxx = spectral_second_derivative(snapshot, k)
        uxxxx = spectral_fourth_derivative(snapshot, k)
        Theta_snapshot = np.vstack([uxx, uxxxx, snapshot * ux]).T
        Theta.append(Theta_snapshot)
        ut_flat.append(ut_snapshot)
    
    Theta = np.vstack(Theta)
    ut_flat = np.hstack(ut_flat)
    
    Theta_mean = Theta.mean(axis=0)
    Theta_std = Theta.std(axis=0)
    Theta_norm = (Theta - Theta_mean) / Theta_std
    
    huber = HuberRegressor(epsilon=epsilon, fit_intercept=False, max_iter=1000, alpha=1e-4)
    huber.fit(Theta_norm, ut_flat)
    
    residuals = ut_flat - huber.predict(Theta_norm)
    threshold = epsilon * np.median(np.abs(residuals))
    
    # Reshape residuals back to (time, space)
    n_time = len(u_mid)
    n_space = Nx
    residuals_2d = residuals.reshape(n_time, n_space)
    
    # Mark outliers (per frame)
    outlier_map = np.abs(residuals_2d) > threshold
    
    return outlier_map, residuals_2d

print("  Computing outliers for ε=1.35...")
outlier_map_135, residuals_135 = detect_outliers_fft(u_varied, x, t, epsilon=1.35)

print("  Computing outliers for ε=1.0...")
outlier_map_10, residuals_10 = detect_outliers_fft(u_varied, x, t, epsilon=1.0)

print("    Outlier detection complete")

# ===== Build Comparison DataFrame =====
comparison_data = []

# FFT Methods
for idx, row in baseline_fft.iterrows():
    comparison_data.append({
        'Derivative': 'FFT', 'Treatment': 'Clean Data' if row['method'] == 'Clean Data' else row['method'],
        'a': row['a'], 'b': row['b'], 'c': row['c'],
        'rmse': row['rmse'], 'r2': row['r2'],
        'abs_error_a': row['abs_error_a'], 'abs_error_b': row['abs_error_b'], 'abs_error_c': row['abs_error_c']
    })

for idx, row in robust_fft.iterrows():
    if row['method'] not in ['Standard Lasso']:
        comparison_data.append({
            'Derivative': 'FFT', 'Treatment': 'Clean Data' if row['method'] == 'Clean Data' else row['method'],
            'a': row['a'], 'b': row['b'], 'c': row['c'],
            'rmse': row['rmse'], 'r2': row['r2'],
            'abs_error_a': row['abs_error_a'], 'abs_error_b': row['abs_error_b'], 'abs_error_c': row['abs_error_c']
        })

# Finite Difference Methods
for idx, row in baseline_fd.iterrows():
    comparison_data.append({
        'Derivative': 'Finite Diff', 'Treatment': 'Clean Data' if row['method'] == 'Clean Data' else row['method'],
        'a': row['a'], 'b': row['b'], 'c': row['c'],
        'rmse': row['rmse'], 'r2': row['r2'],
        'abs_error_a': row['abs_error_a'], 'abs_error_b': row['abs_error_b'], 'abs_error_c': row['abs_error_c']
    })

for idx, row in robust_fd.iterrows():
    if row['method'] not in ['Standard Lasso']:
        comparison_data.append({
            'Derivative': 'Finite Diff', 'Treatment': 'Clean Data' if row['method'] == 'Clean Data' else row['method'],
            'a': row['a'], 'b': row['b'], 'c': row['c'],
            'rmse': row['rmse'], 'r2': row['r2'],
            'abs_error_a': row['abs_error_a'], 'abs_error_b': row['abs_error_b'], 'abs_error_c': row['abs_error_c']
        })

df_comparison = pd.DataFrame(comparison_data)

# ===== Extract Data for Plotting =====
clean_fft = baseline_fft[baseline_fft['method'] == 'Clean Data'].iloc[0]
clean_fd = baseline_fd[baseline_fd['method'] == 'Clean Data'].iloc[0]
baseline_fft_notreat = baseline_fft[baseline_fft['method'] == 'Varying IC (No Treatment)'].iloc[0]
baseline_fd_notreat = baseline_fd[baseline_fd['method'] == 'Varying IC (No Treatment)'].iloc[0]
robust_fft_135 = robust_fft[robust_fft['method'] == 'Robust (ε=1.35)'].iloc[0]
robust_fft_10 = robust_fft[robust_fft['method'] == 'Robust (ε=1.0)'].iloc[0]
robust_fd_135 = robust_fd[robust_fd['method'] == 'Robust (ε=1.35)'].iloc[0]
robust_fd_10 = robust_fd[robust_fd['method'] == 'Robust (ε=1.0)'].iloc[0]

# ===== SUMMARY TABLES =====
print("\n\n" + "="*100)
print("COMPARISON TABLE 1: ALL METHODS (INCLUDING CLEAN DATA)")
print("="*100)
print(f"\n{'Derivative':>12} {'Treatment':>25} {'a':>12} {'b':>12} {'c':>12} {'RMSE':>12} {'R²':>10}")
print("-"*100)

# Sort to show clean first
df_sorted = df_comparison.copy()
df_sorted['sort_key'] = df_sorted['Treatment'].map({
    'Clean Data': 0,
    'Varying IC (No Treatment)': 1,
    'Robust (ε=1.35)': 2,
    'Robust (ε=1.0)': 3
})
df_sorted = df_sorted.sort_values(['Derivative', 'sort_key'])

for _, row in df_sorted.iterrows():
    print(f"{row['Derivative']:>12} {row['Treatment']:>25} {row['a']:>12.6f} {row['b']:>12.6f} "
          f"{row['c']:>12.6f} {row['rmse']:>12.6f} {row['r2']:>10.6f}")
print("="*100)

# ===== VISUALIZATION 1: Main Comparison with Outlier Maps =====
print("\n\n" + "="*80)
print("GENERATING VISUALIZATIONS...")
print("="*80)

fig1 = plt.figure(figsize=(22, 14))

# Row 1: Spatiotemporal Comparison (4 plots)
ax1 = plt.subplot(3, 4, 1)
im1 = plt.imshow(u_clean[:n_frames_viz].T, cmap='RdBu', aspect='auto', origin='lower',
                 extent=(0, time_duration, x[0], x[-1]), vmin=-2, vmax=2)
plt.xlabel('Time (units)', fontsize=10)
plt.ylabel('Space (x)', fontsize=10)
plt.title(f'Clean Data (Single Trajectory)\nTime: 0-{time_duration:.1f} units ({n_frames_viz} frames)', 
          fontweight='bold', fontsize=10)
plt.colorbar(im1, ax=ax1, label='u(x,t)')

ax2 = plt.subplot(3, 4, 2)
im2 = plt.imshow(u_varied[:n_frames_viz].T, cmap='RdBu', aspect='auto', origin='lower',
                 extent=(0, time_duration, x[0], x[-1]), vmin=-2, vmax=2)
plt.xlabel('Time (units)', fontsize=10)
plt.ylabel('Space (x)', fontsize=10)
plt.title(f'Varying IC - No Treatment\nEnsemble of Trajectories', 
          fontweight='bold', fontsize=10)
plt.colorbar(im2, ax=ax2, label='u(x,t)')

# Adjust outlier maps to match visualization frames (1:-1 due to time derivative)
outlier_viz_135 = outlier_map_135[:min(n_frames_viz-2, outlier_map_135.shape[0])]
outlier_viz_10 = outlier_map_10[:min(n_frames_viz-2, outlier_map_10.shape[0])]

ax3 = plt.subplot(3, 4, 3)
im3 = plt.imshow(outlier_viz_135.T, cmap='RdYlGn_r', aspect='auto', origin='lower',
                 extent=(0, outlier_viz_135.shape[0]*dt, x[0], x[-1]), vmin=0, vmax=1)
plt.xlabel('Time (units)', fontsize=10)
plt.ylabel('Space (x)', fontsize=10)
outlier_pct_135 = np.sum(outlier_viz_135) / outlier_viz_135.size * 100
plt.title(f'Outlier Map (ε=1.35)\n{outlier_pct_135:.1f}% outliers detected', 
          fontweight='bold', fontsize=10)
cbar3 = plt.colorbar(im3, ax=ax3)
cbar3.set_ticks([0, 1])
cbar3.set_ticklabels(['Normal', 'Outlier'])

ax4 = plt.subplot(3, 4, 4)
im4 = plt.imshow(outlier_viz_10.T, cmap='RdYlGn_r', aspect='auto', origin='lower',
                 extent=(0, outlier_viz_10.shape[0]*dt, x[0], x[-1]), vmin=0, vmax=1)
plt.xlabel('Time (units)', fontsize=10)
plt.ylabel('Space (x)', fontsize=10)
outlier_pct_10 = np.sum(outlier_viz_10) / outlier_viz_10.size * 100
plt.title(f'Outlier Map (ε=1.0)\n{outlier_pct_10:.1f}% outliers detected (More Aggressive)', 
          fontweight='bold', fontsize=10)
cbar4 = plt.colorbar(im4, ax=ax4)
cbar4.set_ticks([0, 1])
cbar4.set_ticklabels(['Normal', 'Outlier'])

# Row 2: Sample Frame and Trajectory Divergence
ax5 = plt.subplot(3, 4, 5)
plt.plot(x, u_clean[sample_idx], 'g-', label='Clean', linewidth=2.5, alpha=0.8)
plt.plot(x, u_varied[sample_idx], 'r--', label='Varying IC', linewidth=2, alpha=0.7)
plt.xlabel('x', fontsize=10)
plt.ylabel('u(x)', fontsize=10)
plt.title(f'Sample Frame at t={sample_time:.1f}\n(Frame {sample_idx})', fontweight='bold', fontsize=10)
plt.legend(fontsize=9)
plt.grid(True, alpha=0.3)

ax6 = plt.subplot(3, 4, 6)
mean_traj = np.mean(u_varied, axis=0)
divergence = np.sqrt(np.mean((u_varied - mean_traj)**2, axis=1))
plt.plot(t, divergence, 'b-', linewidth=2)
plt.xlabel('Time (units)', fontsize=10)
plt.ylabel('Divergence', fontsize=10)
plt.title('Trajectory Divergence from Mean\n(Quantifies IC Uncertainty)', fontweight='bold', fontsize=10)
plt.grid(True, alpha=0.3)

# Row 2: Coefficient Recovery - FFT
ax7 = plt.subplot(3, 4, 7)
methods_fft = ['Clean', 'No Treat', 'ε=1.35', 'ε=1.0']
a_vals_fft = [clean_fft['a'], baseline_fft_notreat['a'], robust_fft_135['a'], robust_fft_10['a']]
b_vals_fft = [clean_fft['b'], baseline_fft_notreat['b'], robust_fft_135['b'], robust_fft_10['b']]
c_vals_fft = [clean_fft['c'], baseline_fft_notreat['c'], robust_fft_135['c'], robust_fft_10['c']]
x_pos = np.arange(len(methods_fft))
width = 0.25
plt.bar(x_pos - width, a_vals_fft, width, label='a (u_xx)', alpha=0.8)
plt.bar(x_pos, b_vals_fft, width, label='b (u_xxxx)', alpha=0.8)
plt.bar(x_pos + width, c_vals_fft, width, label='c (u*u_x)', alpha=0.8)
plt.axhline(y=-1.0, color='red', linestyle='--', linewidth=2, label='True')
plt.ylabel('Coefficient Value', fontsize=10)
plt.title('FFT: Coefficient Recovery', fontweight='bold', fontsize=10)
plt.xticks(x_pos, methods_fft, fontsize=8)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3, axis='y')

# Row 2: Coefficient Recovery - FD
ax8 = plt.subplot(3, 4, 8)
a_vals_fd = [clean_fd['a'], baseline_fd_notreat['a'], robust_fd_135['a'], robust_fd_10['a']]
b_vals_fd = [clean_fd['b'], baseline_fd_notreat['b'], robust_fd_135['b'], robust_fd_10['b']]
c_vals_fd = [clean_fd['c'], baseline_fd_notreat['c'], robust_fd_135['c'], robust_fd_10['c']]
plt.bar(x_pos - width, a_vals_fd, width, label='a (u_xx)', alpha=0.8)
plt.bar(x_pos, b_vals_fd, width, label='b (u_xxxx)', alpha=0.8)
plt.bar(x_pos + width, c_vals_fd, width, label='c ((u²)_x)', alpha=0.8)
plt.axhline(y=-1.0, color='red', linestyle='--', linewidth=2)
plt.axhline(y=-0.5, color='orange', linestyle='--', linewidth=2, label='True (c)')
plt.ylabel('Coefficient Value', fontsize=10)
plt.title('Finite Diff: Coefficient Recovery', fontweight='bold', fontsize=10)
plt.xticks(x_pos, methods_fft, fontsize=8)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3, axis='y')

# Row 3: RMSE and R² Comparisons
ax9 = plt.subplot(3, 4, 9)
rmse_fft = [clean_fft['rmse'], baseline_fft_notreat['rmse'], robust_fft_135['rmse'], robust_fft_10['rmse']]
rmse_fd = [clean_fd['rmse'], baseline_fd_notreat['rmse'], robust_fd_135['rmse'], robust_fd_10['rmse']]
width2 = 0.35
plt.bar(x_pos - width2/2, rmse_fft, width2, label='FFT', alpha=0.8, color='blue')
plt.bar(x_pos + width2/2, rmse_fd, width2, label='Finite Diff', alpha=0.8, color='green')
plt.ylabel('RMSE', fontsize=10)
plt.title('RMSE Comparison (Lower is Better)', fontweight='bold', fontsize=10)
plt.xticks(x_pos, methods_fft, fontsize=8)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3, axis='y')
plt.yscale('log')

ax10 = plt.subplot(3, 4, 10)
r2_fft = [clean_fft['r2'], baseline_fft_notreat['r2'], robust_fft_135['r2'], robust_fft_10['r2']]
r2_fd = [clean_fd['r2'], baseline_fd_notreat['r2'], robust_fd_135['r2'], robust_fd_10['r2']]
plt.bar(x_pos - width2/2, r2_fft, width2, label='FFT', alpha=0.8, color='blue')
plt.bar(x_pos + width2/2, r2_fd, width2, label='Finite Diff', alpha=0.8, color='green')
plt.axhline(y=1.0, color='black', linestyle=':', linewidth=1)
plt.ylabel('R² Score', fontsize=10)
plt.title('R² Comparison (Higher is Better)', fontweight='bold', fontsize=10)
plt.xticks(x_pos, methods_fft, fontsize=8)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3, axis='y')

# Row 3: Absolute Errors
ax11 = plt.subplot(3, 4, 11)
err_a_fft = [clean_fft['abs_error_a'], baseline_fft_notreat['abs_error_a'], 
             robust_fft_135['abs_error_a'], robust_fft_10['abs_error_a']]
err_b_fft = [clean_fft['abs_error_b'], baseline_fft_notreat['abs_error_b'],
             robust_fft_135['abs_error_b'], robust_fft_10['abs_error_b']]
err_c_fft = [clean_fft['abs_error_c'], baseline_fft_notreat['abs_error_c'],
             robust_fft_135['abs_error_c'], robust_fft_10['abs_error_c']]
plt.bar(x_pos - width, err_a_fft, width, label='|Err a|', alpha=0.8)
plt.bar(x_pos, err_b_fft, width, label='|Err b|', alpha=0.8)
plt.bar(x_pos + width, err_c_fft, width, label='|Err c|', alpha=0.8)
plt.ylabel('Absolute Error', fontsize=10)
plt.title('FFT: Absolute Coefficient Errors', fontweight='bold', fontsize=10)
plt.xticks(x_pos, methods_fft, fontsize=8)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3, axis='y')
plt.yscale('log')

ax12 = plt.subplot(3, 4, 12)
err_a_fd = [clean_fd['abs_error_a'], baseline_fd_notreat['abs_error_a'],
            robust_fd_135['abs_error_a'], robust_fd_10['abs_error_a']]
err_b_fd = [clean_fd['abs_error_b'], baseline_fd_notreat['abs_error_b'],
            robust_fd_135['abs_error_b'], robust_fd_10['abs_error_b']]
err_c_fd = [clean_fd['abs_error_c'], baseline_fd_notreat['abs_error_c'],
            robust_fd_135['abs_error_c'], robust_fd_10['abs_error_c']]
plt.bar(x_pos - width, err_a_fd, width, label='|Err a|', alpha=0.8)
plt.bar(x_pos, err_b_fd, width, label='|Err b|', alpha=0.8)
plt.bar(x_pos + width, err_c_fd, width, label='|Err c|', alpha=0.8)
plt.ylabel('Absolute Error', fontsize=10)
plt.title('Finite Diff: Absolute Coefficient Errors', fontweight='bold', fontsize=10)
plt.xticks(x_pos, methods_fft, fontsize=8)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3, axis='y')
plt.yscale('log')

plt.tight_layout()
plt.savefig('varying_ic_visual_comparison.png', dpi=300, bbox_inches='tight')
print("    Saved: varying_ic_visual_comparison.png")

# ===== VISUALIZATION 2: Enhanced Heatmaps with Clean Data =====
fig2, axes = plt.subplots(2, 2, figsize=(16, 10))

methods_all = ['Clean', 'No Treat', 'Robust\n(ε=1.35)', 'Robust\n(ε=1.0)']
derivatives = ['FFT', 'Finite Diff']

# RMSE Heatmap
rmse_matrix = np.array([rmse_fft, rmse_fd])
sns.heatmap(rmse_matrix, annot=True, fmt='.4f', cmap='RdYlGn_r',
            xticklabels=methods_all, yticklabels=derivatives, ax=axes[0,0],
            cbar_kws={'label': 'RMSE'})
axes[0,0].set_title('RMSE by Method (Including Clean Baseline)', fontweight='bold', fontsize=12)
axes[0,0].set_xlabel('Treatment Method')
axes[0,0].set_ylabel('Derivative Type')

# R² Heatmap
r2_matrix = np.array([r2_fft, r2_fd])
sns.heatmap(r2_matrix, annot=True, fmt='.4f', cmap='RdYlGn',
            xticklabels=methods_all, yticklabels=derivatives, ax=axes[0,1],
            cbar_kws={'label': 'R² Score'})
axes[0,1].set_title('R² Score by Method (Including Clean Baseline)', fontweight='bold', fontsize=12)
axes[0,1].set_xlabel('Treatment Method')
axes[0,1].set_ylabel('Derivative Type')

# Error a Heatmap
err_a_matrix = np.array([err_a_fft, err_a_fd])
sns.heatmap(err_a_matrix, annot=True, fmt='.4f', cmap='RdYlGn_r',
            xticklabels=methods_all, yticklabels=derivatives, ax=axes[1,0],
            cbar_kws={'label': '|Error in a|'})
axes[1,0].set_title('Absolute Error in Coefficient a', fontweight='bold', fontsize=12)
axes[1,0].set_xlabel('Treatment Method')
axes[1,0].set_ylabel('Derivative Type')

# Error b Heatmap
err_b_matrix = np.array([err_b_fft, err_b_fd])
sns.heatmap(err_b_matrix, annot=True, fmt='.4f', cmap='RdYlGn_r',
            xticklabels=methods_all, yticklabels=derivatives, ax=axes[1,1],
            cbar_kws={'label': '|Error in b|'})
axes[1,1].set_title('Absolute Error in Coefficient b', fontweight='bold', fontsize=12)
axes[1,1].set_xlabel('Treatment Method')
axes[1,1].set_ylabel('Derivative Type')

plt.tight_layout()
plt.savefig('varying_ic_heatmap_comparison.png', dpi=300, bbox_inches='tight')
print("    Saved: varying_ic_heatmap_comparison.png")

# ===== Save Comparison Data =====
df_comparison.to_csv('varying_ic_all_methods_comparison.csv', index=False)
print("    Saved: varying_ic_all_methods_comparison.csv")

print("\n" + "="*80)
print("   VARYING IC ENHANCED COMPARISON COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. varying_ic_visual_comparison.png - Complete visualization (12 subplots)")
print("     - Row 1: Spatiotemporal (Clean, Varying IC, Outlier Maps)")
print("     - Row 2: Sample frames, divergence, coefficient recovery")
print("     - Row 3: RMSE, R², absolute errors")
print("  2. varying_ic_heatmap_comparison.png - Performance heatmaps with clean data")
print("  3. varying_ic_all_methods_comparison.csv - Complete data")
print("="*80)

print("\n" + "="*80)
print("KEY INSIGHTS")
print("="*80)
print(f"\n1. Outlier Detection:")
print(f"   - ε=1.35 detected {outlier_pct_135:.1f}% outliers")
print(f"   - ε=1.0 detected {outlier_pct_10:.1f}% outliers (more aggressive)")
print(f"   - Green regions = normal frames, Red regions = outlier frames")

print(f"\n2. Spatiotemporal Display:")
print(f"   - Showing first {n_frames_viz} frames (0 to {time_duration:.1f} time units)")
print(f"   - Clean vs Varying IC shows visual difference in patterns")
print(f"   - Outlier maps show which regions are downweighted")

print(f"\n3. Performance:")
print(f"   - Clean FFT: RMSE={clean_fft['rmse']:.6f}, R²={clean_fft['r2']:.6f}")
print(f"   - Best treated: Check heatmaps for greenest cells")
print("="*80)

# REAL IMAGE

In [ ]:
"""
PHASE 4.1: LOAD AND PREPROCESS REAL EXPERIMENTAL IMAGES

This script loads the 50-frame TIF image sequence from laser-matter interaction
experiments, converts to grayscale, extracts intensity arrays, and performs
initial quality checks.

Author: SINDy Project
Phase: 4.1 - Data Loading
"""

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import glob
from pathlib import Path

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (18, 10)

print("=" * 80)
print("PHASE 4.1: LOADING REAL EXPERIMENTAL IMAGES")
print("=" * 80)

# ===== CONFIGURATION =====
IMAGE_DIRECTORY = "Real-Images"  # Directory containing the 50 TIF files


# Output settings
SAVE_NPZ = True  # Save processed data to NPZ file
SAVE_PLOTS = True  # Save visualization plots

print(f"\nConfiguration:")
print(f"  Image directory: {IMAGE_DIRECTORY}")
print(f"  Looking for .tif files...")

# ===== STEP 1: FIND AND LOAD IMAGE FILES =====
print("\n" + "=" * 80)
print("STEP 1: LOCATING IMAGE FILES")
print("=" * 80)

# Find all TIF files
tif_files = sorted(glob.glob(os.path.join(IMAGE_DIRECTORY, "*.tif")))

if len(tif_files) == 0:
    print(f"  WARNING: No .tif files found in {IMAGE_DIRECTORY}")
    print(f"   Please ensure images are in the correct directory.")
    print(f"   Current working directory: {os.getcwd()}")
    exit(1)

print(f"\n Found {len(tif_files)} TIF files")
print(f"   First file: {os.path.basename(tif_files[0])}")
print(f"   Last file: {os.path.basename(tif_files[-1])}")

# Extract frame numbers from filenames
frame_numbers = []
for fname in tif_files:
    basename = os.path.basename(fname)
    # Try to extract number from filename (e.g., "0.tif" -> 0, "frame_10.tif" -> 10)
    try:
        num = int(''.join(filter(str.isdigit, basename.split('.')[0])))
        frame_numbers.append(num)
    except:
        frame_numbers.append(len(frame_numbers))  # Fallback: sequential numbering

print(f"   Frame range: {min(frame_numbers)} to {max(frame_numbers)}")

# ===== STEP 2: LOAD FIRST IMAGE TO GET DIMENSIONS =====
print("\n" + "=" * 80)
print("STEP 2: ANALYZING IMAGE PROPERTIES")
print("=" * 80)

first_img = Image.open(tif_files[0])
first_array = np.array(first_img)

print(f"\nFirst image properties:")
print(f"  File: {os.path.basename(tif_files[0])}")
print(f"  Shape: {first_array.shape}")
print(f"  Data type: {first_array.dtype}")
print(f"  Mode: {first_img.mode}")
print(f"  Value range: [{first_array.min()}, {first_array.max()}]")

# Determine if grayscale conversion needed
needs_conversion = len(first_array.shape) == 3
if needs_conversion:
    print(f"  → Image has {first_array.shape[2]} channels (needs grayscale conversion)")
else:
    print(f"  → Image is already grayscale")

# ===== STEP 3: LOAD ALL IMAGES =====
print("\n" + "=" * 80)
print("STEP 3: LOADING ALL IMAGES")
print("=" * 80)

images_raw = []
images_gray = []

print(f"\nLoading {len(tif_files)} frames...")
for idx, fname in enumerate(tif_files):
    img = Image.open(fname)
    img_array = np.array(img)
    images_raw.append(img_array)
    
    # Convert to grayscale if needed
    if len(img_array.shape) == 3:
        if img_array.shape[2] == 4:  # RGBA
            # Average RGB channels (ignore alpha)
            gray = np.mean(img_array[:, :, :3], axis=2)
        elif img_array.shape[2] == 3:  # RGB
            # Average all channels
            gray = np.mean(img_array, axis=2)
        else:
            gray = img_array[:, :, 0]  # Take first channel
    else:
        gray = img_array
    
    images_gray.append(gray)
    
    if (idx + 1) % 10 == 0:
        print(f"  Loaded {idx + 1}/{len(tif_files)} frames...")

print(f"\n Successfully loaded all {len(tif_files)} frames")

# Convert to numpy array
u_images = np.array(images_gray)  # Shape: (Nt, Ny, Nx)

print(f"\nProcessed data shape: {u_images.shape}")
print(f"  Nt (time): {u_images.shape[0]} frames")
print(f"  Ny (height): {u_images.shape[1]} pixels")
print(f"  Nx (width): {u_images.shape[2]} pixels")
print(f"  Data type: {u_images.dtype}")
print(f"  Memory: {u_images.nbytes / 1e6:.2f} MB")

# ===== STEP 4: DATA QUALITY CHECKS =====
print("\n" + "=" * 80)
print("STEP 4: DATA QUALITY ANALYSIS")
print("=" * 80)

# Compute statistics
print(f"\nIntensity statistics across all frames:")
print(f"  Global min: {u_images.min():.2f}")
print(f"  Global max: {u_images.max():.2f}")
print(f"  Global mean: {u_images.mean():.2f}")
print(f"  Global std: {u_images.std():.2f}")

# Per-frame statistics
frame_means = np.mean(u_images, axis=(1, 2))
frame_stds = np.std(u_images, axis=(1, 2))
frame_mins = np.min(u_images, axis=(1, 2))
frame_maxs = np.max(u_images, axis=(1, 2))

print(f"\nPer-frame statistics:")
print(f"  Mean intensity range: [{frame_means.min():.2f}, {frame_means.max():.2f}]")
print(f"  Std intensity range: [{frame_stds.min():.2f}, {frame_stds.max():.2f}]")

# Temporal consistency
print(f"\nTemporal consistency:")
frame_to_frame_diff = np.mean(np.abs(np.diff(u_images, axis=0)))
print(f"  Average frame-to-frame difference: {frame_to_frame_diff:.2f}")

# Frame-to-frame correlation
correlations = []
for i in range(len(u_images) - 1):
    corr = np.corrcoef(u_images[i].flatten(), u_images[i+1].flatten())[0, 1]
    correlations.append(corr)
mean_correlation = np.mean(correlations)
print(f"  Average frame-to-frame correlation: {mean_correlation:.4f}")

if mean_correlation > 0.8:
    print(f"  → HIGH correlation: Smooth temporal evolution ")
elif mean_correlation > 0.5:
    print(f"  → MODERATE correlation: Some temporal variation")
else:
    print(f"  → LOW correlation: High temporal variation or noise ")

# ===== STEP 5: NORMALIZATION (OPTIONAL) =====
print("\n" + "=" * 80)
print("STEP 5: INTENSITY NORMALIZATION")
print("=" * 80)

# Option 1: Keep as-is (0-255 range)
u_images_normalized = u_images.copy()

# Option 2: Normalize to [0, 1]
# u_images_normalized = (u_images - u_images.min()) / (u_images.max() - u_images.min())

# Option 3: Z-score normalization
# u_images_normalized = (u_images - u_images.mean()) / u_images.std()

print(f"Normalization: Keeping original intensity range [0, 255]")
print(f"  (Can be changed in script if needed)")

# ===== STEP 6: SAVE PROCESSED DATA =====
if SAVE_NPZ:
    print("\n" + "=" * 80)
    print("STEP 6: SAVING PROCESSED DATA")
    print("=" * 80)
    
    output_file = "phase4_01_loaded_images.npz"
    np.savez_compressed(output_file,
                       u_images=u_images_normalized,
                       frame_numbers=np.array(frame_numbers),
                       frame_means=frame_means,
                       frame_stds=frame_stds,
                       shape=u_images.shape)
    
    print(f"\nSaved processed data to: {output_file}")
    print(f"   Contains:")
    print(f"     - u_images: {u_images.shape} array")
    print(f"     - frame_numbers: Frame identifiers")
    print(f"     - frame_means, frame_stds: Per-frame statistics")

# ===== STEP 7: VISUALIZATION =====
if SAVE_PLOTS:
    print("\n" + "=" * 80)
    print("STEP 7: CREATING VISUALIZATIONS")
    print("=" * 80)
    
    # === Figure 1: Sample frames overview ===
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    fig.suptitle('Sample Frames from Real Experimental Data', fontsize=16, fontweight='bold')
    
    # Show 10 evenly spaced frames
    frame_indices = np.linspace(0, len(u_images) - 1, 10, dtype=int)
    
    for idx, (ax, frame_idx) in enumerate(zip(axes.flat, frame_indices)):
        ax.imshow(u_images[frame_idx], cmap='gray', aspect='auto')
        ax.set_title(f'Frame {frame_numbers[frame_idx]}')
        ax.axis('off')
    
    plt.tight_layout()
    plt.savefig('phase4_01_sample_frames.png', dpi=150, bbox_inches='tight')
    print(f"   Saved: phase4_01_sample_frames.png")
    
    # === Figure 2: Temporal evolution analysis ===
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle('Temporal Evolution Analysis', fontsize=16, fontweight='bold')
    
    # Mean intensity over time
    axes[0, 0].plot(frame_numbers, frame_means, 'o-', linewidth=2, markersize=4)
    axes[0, 0].set_xlabel('Frame Number')
    axes[0, 0].set_ylabel('Mean Intensity')
    axes[0, 0].set_title('Mean Intensity Evolution')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Std intensity over time
    axes[0, 1].plot(frame_numbers, frame_stds, 'o-', linewidth=2, markersize=4, color='orange')
    axes[0, 1].set_xlabel('Frame Number')
    axes[0, 1].set_ylabel('Std Intensity')
    axes[0, 1].set_title('Intensity Variability Evolution')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Frame-to-frame correlation
    axes[1, 0].plot(frame_numbers[:-1], correlations, 'o-', linewidth=2, markersize=4, color='green')
    axes[1, 0].axhline(mean_correlation, ls='--', color='red', label=f'Mean: {mean_correlation:.3f}')
    axes[1, 0].set_xlabel('Frame Number')
    axes[1, 0].set_ylabel('Correlation with Next Frame')
    axes[1, 0].set_title('Temporal Correlation')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Intensity distribution (all frames combined)
    axes[1, 1].hist(u_images.flatten(), bins=100, color='steelblue', alpha=0.7, edgecolor='black')
    axes[1, 1].set_xlabel('Intensity Value')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Overall Intensity Distribution')
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('phase4_01_temporal_analysis.png', dpi=150, bbox_inches='tight')
    print(f"  Saved: phase4_01_temporal_analysis.png")
    
    # === Figure 3: Spatiotemporal visualization ===
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle('Spatiotemporal Structure', fontsize=16, fontweight='bold')
    
    # Middle row evolution (1D slice over time)
    mid_row = u_images.shape[1] // 2
    middle_row_data = u_images[:, mid_row, :]
    
    axes[0].imshow(middle_row_data, aspect='auto', cmap='gray', 
                   extent=[0, u_images.shape[2], frame_numbers[-1], frame_numbers[0]])
    axes[0].set_xlabel('X Position (pixels)')
    axes[0].set_ylabel('Frame Number')
    axes[0].set_title('Middle Row Evolution (X-T Diagram)')
    
    # Middle column evolution
    mid_col = u_images.shape[2] // 2
    middle_col_data = u_images[:, :, mid_col]
    
    axes[1].imshow(middle_col_data, aspect='auto', cmap='gray',
                   extent=[0, u_images.shape[1], frame_numbers[-1], frame_numbers[0]])
    axes[1].set_xlabel('Y Position (pixels)')
    axes[1].set_ylabel('Frame Number')
    axes[1].set_title('Middle Column Evolution (Y-T Diagram)')
    
    # Sample spatial profiles
    axes[2].set_title('Spatial Profiles (Middle Row)')
    sample_frames = np.linspace(0, len(u_images) - 1, 5, dtype=int)
    for frame_idx in sample_frames:
        axes[2].plot(u_images[frame_idx, mid_row, :], 
                    label=f'Frame {frame_numbers[frame_idx]}', 
                    alpha=0.7, linewidth=1.5)
    axes[2].set_xlabel('X Position (pixels)')
    axes[2].set_ylabel('Intensity')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('phase4_01_spatiotemporal.png', dpi=150, bbox_inches='tight')
    print(f"  Saved: phase4_01_spatiotemporal.png")

# ===== FINAL SUMMARY =====
print("\n" + "=" * 80)
print("PHASE 4.1 COMPLETE - SUMMARY")
print("=" * 80)

print(f"\n Loaded Data:")
print(f"  • {len(tif_files)} frames loaded successfully")
print(f"  • Shape: {u_images.shape} (Nt, Ny, Nx)")
print(f"  • Intensity range: [{u_images.min():.1f}, {u_images.max():.1f}]")

print(f"\nQuality Assessment:")
print(f"  • Mean frame-to-frame correlation: {mean_correlation:.3f}")
if mean_correlation > 0.7:
    quality = "EXCELLENT - Good temporal continuity"
elif mean_correlation > 0.5:
    quality = "GOOD - Moderate temporal continuity"
else:
    quality = "FAIR - Significant temporal variation"
print(f"  • Data quality: {quality}")

print(f"\n Outputs Generated:")
if SAVE_NPZ:
    print(f"   phase4_01_loaded_images.npz")
if SAVE_PLOTS:
    print(f"   phase4_01_sample_frames.png")
    print(f"   phase4_01_temporal_analysis.png")
    print(f"   phase4_01_spatiotemporal.png")

print(f"\n Next Steps:")
print(f"  → Run Phase 4.2: Extract 1D slices for SINDy analysis")
print(f"  → Choose extraction strategy (single row, averaged rows, multiple slices)")
print(f"  → Continue with baseline SINDy application (Phase 4.3)")

print("\n" + "=" * 80)
print("READY FOR PHASE 4.2! ")
print("=" * 80)

In [ ]:
"""
PHASE 4.1: LOAD AND PREPROCESS REAL EXPERIMENTAL IMAGES

This script loads the 50-frame TIF image sequence from laser-matter interaction
experiments, converts to grayscale, extracts intensity arrays, and performs
initial quality checks.

Author: SINDy Project
Phase: 4.1 - Data Loading
Version: 2.0 (Improved)
"""

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import glob
import re
from pathlib import Path
from scipy import ndimage

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (18, 10)

print("=" * 80)
print("PHASE 4.1: LOADING REAL EXPERIMENTAL IMAGES")
print("=" * 80)

# ===== CONFIGURATION =====
IMAGE_DIRECTORY = "Real-Images"  # Directory containing the 50 TIF files
NORMALIZATION_METHOD = "keep"  # Options: "keep", "minmax", "zscore"
GRAYSCALE_METHOD = "luminosity"  # Options: "simple", "luminosity"
PIXEL_SIZE_UM = None  # Set to actual pixel size in micrometers if known (e.g., 0.5)
TIME_PER_FRAME_S = None  # Set to actual time between frames in seconds if known

# Output settings
SAVE_NPZ = True  # Save processed data to NPZ file
SAVE_PLOTS = True  # Save visualization plots
OUTPUT_DIR = "phase4_real_images"  # Directory for outputs

print(f"\nConfiguration:")
print(f"  Image directory: {IMAGE_DIRECTORY}")
print(f"  Normalization method: {NORMALIZATION_METHOD}")
print(f"  Grayscale conversion: {GRAYSCALE_METHOD}")
print(f"  Pixel size: {PIXEL_SIZE_UM} µm" if PIXEL_SIZE_UM else "  Pixel size: Not specified")
print(f"  Time per frame: {TIME_PER_FRAME_S} s" if TIME_PER_FRAME_S else "  Time per frame: Not specified")
print(f"  Looking for .tif files...")

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ===== STEP 1: FIND AND LOAD IMAGE FILES =====
print("\n" + "=" * 80)
print("STEP 1: LOCATING IMAGE FILES")
print("=" * 80)

# Find all TIF files
tif_files = sorted(glob.glob(os.path.join(IMAGE_DIRECTORY, "*.tif")))

if len(tif_files) == 0:
    print(f"\n      WARNING: No .tif files found in {IMAGE_DIRECTORY}")
    print(f"     Please ensure images are in the correct directory.")
    print(f"     Current working directory: {os.getcwd()}")
    exit(1)

print(f"\n   Found {len(tif_files)} TIF files")
print(f"   First file: {os.path.basename(tif_files[0])}")
print(f"   Last file: {os.path.basename(tif_files[-1])}")

# Extract frame numbers from filenames (IMPROVED)
frame_numbers = []
for fname in tif_files:
    basename = os.path.basename(fname)
    # Try to find the first integer in the filename
    match = re.search(r'\d+', basename)
    if match:
        frame_numbers.append(int(match.group()))
    else:
        # Fallback: use index
        frame_numbers.append(len(frame_numbers))
        print(f"      WARNING: Could not extract frame number from '{basename}', using index {len(frame_numbers)-1}")

print(f"   Frame range: {min(frame_numbers)} to {max(frame_numbers)}")

# ===== STEP 2: LOAD FIRST IMAGE TO GET DIMENSIONS =====
print("\n" + "=" * 80)
print("STEP 2: ANALYZING IMAGE PROPERTIES")
print("=" * 80)

first_img = Image.open(tif_files[0])
first_array = np.array(first_img)

print(f"\nFirst image properties:")
print(f"  File: {os.path.basename(tif_files[0])}")
print(f"  Shape: {first_array.shape}")
print(f"  Data type: {first_array.dtype}")
print(f"  Mode: {first_img.mode}")
print(f"  Value range: [{first_array.min()}, {first_array.max()}]")

# Determine if grayscale conversion needed
needs_conversion = len(first_array.shape) == 3
if needs_conversion:
    print(f"  → Image has {first_array.shape[2]} channels (needs grayscale conversion)")
    print(f"  → Using {GRAYSCALE_METHOD} method")
else:
    print(f"  → Image is already grayscale")

# Memory estimate (IMPROVED)
Nt = len(tif_files)
if len(first_array.shape) == 3:
    Ny, Nx = first_array.shape[0], first_array.shape[1]
else:
    Ny, Nx = first_array.shape

estimated_memory_gb = (Nt * Ny * Nx * 8) / 1e9  # 8 bytes per float64

print(f"\nMemory estimate:")
print(f"  Array size: ({Nt}, {Ny}, {Nx})")
print(f"  Estimated memory: {estimated_memory_gb:.2f} GB")

if estimated_memory_gb > 4.0:
    print(f"      WARNING: Large dataset detected!")
    print(f"     Consider processing in chunks or using memory-mapped arrays")

# ===== STEP 3: LOAD ALL IMAGES =====
print("\n" + "=" * 80)
print("STEP 3: LOADING ALL IMAGES")
print("=" * 80)

images_raw = []
images_gray = []

print(f"\nLoading {len(tif_files)} frames...")
for idx, fname in enumerate(tif_files):
    img = Image.open(fname)
    img_array = np.array(img)
    images_raw.append(img_array)
    
    # Convert to grayscale if needed (IMPROVED)
    if len(img_array.shape) == 3:
        if GRAYSCALE_METHOD == "luminosity":
            # Luminosity method (perceptually accurate)
            if img_array.shape[2] == 4:  # RGBA
                gray = (0.299 * img_array[:, :, 0] + 
                        0.587 * img_array[:, :, 1] + 
                        0.114 * img_array[:, :, 2])
            elif img_array.shape[2] == 3:  # RGB
                gray = (0.299 * img_array[:, :, 0] + 
                        0.587 * img_array[:, :, 1] + 
                        0.114 * img_array[:, :, 2])
            else:
                gray = img_array[:, :, 0]  # Take first channel
        else:  # "simple" method
            # Simple averaging
            if img_array.shape[2] == 4:  # RGBA
                gray = np.mean(img_array[:, :, :3], axis=2)
            elif img_array.shape[2] == 3:  # RGB
                gray = np.mean(img_array, axis=2)
            else:
                gray = img_array[:, :, 0]
    else:
        gray = img_array
    
    # Convert to float64 immediately (IMPROVED)
    images_gray.append(gray.astype(np.float64))
    
    if (idx + 1) % 10 == 0:
        print(f"      Loaded {idx + 1}/{len(tif_files)} frames...")

print(f"\n   Successfully loaded all {len(tif_files)} frames")

# Convert to numpy array
u_images = np.array(images_gray)  # Shape: (Nt, Ny, Nx)

print(f"\nProcessed data shape: {u_images.shape}")
print(f"  Nt (time): {u_images.shape[0]} frames")
print(f"  Ny (height): {u_images.shape[1]} pixels")
print(f"  Nx (width): {u_images.shape[2]} pixels")
print(f"  Data type: {u_images.dtype}")
print(f"  Memory: {u_images.nbytes / 1e6:.2f} MB")

# Physical dimensions (IMPROVED)
if PIXEL_SIZE_UM is not None:
    width_um = u_images.shape[2] * PIXEL_SIZE_UM
    height_um = u_images.shape[1] * PIXEL_SIZE_UM
    print(f"\nPhysical dimensions:")
    print(f"  Width: {width_um:.2f} µm ({u_images.shape[2]} pixels)")
    print(f"  Height: {height_um:.2f} µm ({u_images.shape[1]} pixels)")

if TIME_PER_FRAME_S is not None:
    total_time = (u_images.shape[0] - 1) * TIME_PER_FRAME_S
    print(f"\nTemporal dimensions:")
    print(f"  Total duration: {total_time:.4f} s")
    print(f"  Frame rate: {1/TIME_PER_FRAME_S:.2f} fps")

# ===== STEP 4: DATA QUALITY CHECKS =====
print("\n" + "=" * 80)
print("STEP 4: DATA QUALITY ANALYSIS")
print("=" * 80)

# Compute statistics
print(f"\nIntensity statistics across all frames:")
print(f"  Global min: {u_images.min():.2f}")
print(f"  Global max: {u_images.max():.2f}")
print(f"  Global mean: {u_images.mean():.2f}")
print(f"  Global std: {u_images.std():.2f}")

# Per-frame statistics
frame_means = np.mean(u_images, axis=(1, 2))
frame_stds = np.std(u_images, axis=(1, 2))
frame_mins = np.min(u_images, axis=(1, 2))
frame_maxs = np.max(u_images, axis=(1, 2))

print(f"\nPer-frame statistics:")
print(f"  Mean intensity range: [{frame_means.min():.2f}, {frame_means.max():.2f}]")
print(f"  Std intensity range: [{frame_stds.min():.2f}, {frame_stds.max():.2f}]")
print(f"  Intensity drift: {frame_means.max() - frame_means.min():.2f} (max - min mean)")

# Temporal consistency
print(f"\nTemporal consistency:")
frame_to_frame_diff = np.mean(np.abs(np.diff(u_images, axis=0)))
print(f"  Average frame-to-frame difference: {frame_to_frame_diff:.2f}")

# Frame-to-frame correlation
correlations = []
for i in range(len(u_images) - 1):
    corr = np.corrcoef(u_images[i].flatten(), u_images[i+1].flatten())[0, 1]
    correlations.append(corr)
mean_correlation = np.mean(correlations)
print(f"  Average frame-to-frame correlation: {mean_correlation:.4f}")

if mean_correlation > 0.8:
    print(f"  →    HIGH correlation: Smooth temporal evolution")
elif mean_correlation > 0.5:
    print(f"  →     MODERATE correlation: Some temporal variation")
else:
    print(f"  →     LOW correlation: High temporal variation or noise")

# ===== ADVANCED QUALITY METRICS (NEW) =====
print(f"\nAdvanced quality checks:")

# 1. Noise estimation (using Donoho-Johnstone estimator)
def estimate_noise(image):
    """Estimate noise using median absolute deviation of Laplacian"""
    H = np.array([[1, -2, 1], [-2, 4, -2], [1, -2, 1]])  # Laplacian kernel
    filtered = ndimage.convolve(image, H)
    sigma = np.median(np.abs(filtered)) / 0.6745
    return sigma

noise_estimates = [estimate_noise(frame) for frame in u_images]
mean_noise = np.mean(noise_estimates)
std_noise = np.std(noise_estimates)
print(f"  Estimated noise level (σ): {mean_noise:.2f} ± {std_noise:.2f}")

# 2. Signal-to-Noise Ratio
signal_power = np.var(u_images)  # Signal variance
noise_power = mean_noise**2
snr_db = 10 * np.log10(signal_power / noise_power)
print(f"  Signal-to-Noise Ratio: {snr_db:.2f} dB")

if snr_db > 20:
    print(f"    →    Excellent SNR (> 20 dB)")
elif snr_db > 10:
    print(f"    →   Moderate SNR (10-20 dB) - Denoising recommended")
else:
    print(f"    →  Low SNR (< 10 dB) - Aggressive denoising needed")

# 3. Check for dead/saturated pixels
dead_pixels = np.sum(u_images == 0)
saturated_pixels = np.sum(u_images == 255)
total_pixels = u_images.size
print(f"  Dead pixels (0): {dead_pixels} ({100*dead_pixels/total_pixels:.3f}%)")
print(f"  Saturated pixels (255): {saturated_pixels} ({100*saturated_pixels/total_pixels:.3f}%)")

if saturated_pixels > 0.01 * total_pixels:
    print(f"    →      WARNING: > 0.01% saturated pixels detected!")

# 4. Spatial gradient analysis
print(f"\nSpatial structure:")
sample_frame = u_images[len(u_images)//2]  # Middle frame
grad_x = np.gradient(sample_frame, axis=1)
grad_y = np.gradient(sample_frame, axis=0)
grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)
mean_gradient = np.mean(grad_magnitude)
print(f"  Mean gradient magnitude (middle frame): {mean_gradient:.2f}")

if mean_gradient > 10:
    print(f"    →    Strong spatial gradients detected (good for 1D slice extraction)")
elif mean_gradient > 5:
    print(f"    →      Moderate spatial gradients")
else:
    print(f"    →     Weak spatial gradients (may affect derivative accuracy)")

# ===== STEP 5: NORMALIZATION =====
print("\n" + "=" * 80)
print("STEP 5: INTENSITY NORMALIZATION")
print("=" * 80)

if NORMALIZATION_METHOD == "keep":
    u_images_normalized = u_images.copy()
    print(f"   Normalization: Keeping original intensity range [{u_images.min():.2f}, {u_images.max():.2f}]")
    
elif NORMALIZATION_METHOD == "minmax":
    u_min, u_max = u_images.min(), u_images.max()
    u_images_normalized = (u_images - u_min) / (u_max - u_min)
    print(f"   Normalization: Min-Max scaling to [0, 1]")
    print(f"   Original range: [{u_min:.2f}, {u_max:.2f}]")
    print(f"   New range: [{u_images_normalized.min():.4f}, {u_images_normalized.max():.4f}]")
    
elif NORMALIZATION_METHOD == "zscore":
    u_mean, u_std = u_images.mean(), u_images.std()
    u_images_normalized = (u_images - u_mean) / u_std
    print(f"   Normalization: Z-score (zero mean, unit variance)")
    print(f"   Original mean: {u_mean:.2f}, std: {u_std:.2f}")
    print(f"   New mean: {u_images_normalized.mean():.6f}, std: {u_images_normalized.std():.4f}")
    
else:
    print(f"     WARNING: Unknown normalization method '{NORMALIZATION_METHOD}', keeping original")
    u_images_normalized = u_images.copy()

print(f"\n Recommendation for SINDy:")
print(f"   • FFT derivatives: Use 'minmax' or 'zscore' (helps with numerical stability)")
print(f"   • Finite Difference: 'keep' or 'minmax' work well")
print(f"   • Current choice: '{NORMALIZATION_METHOD}'")

# ===== STEP 6: SAVE PROCESSED DATA =====
if SAVE_NPZ:
    print("\n" + "=" * 80)
    print("STEP 6: SAVING PROCESSED DATA")
    print("=" * 80)
    
    output_file = os.path.join(OUTPUT_DIR, "phase4_01_loaded_images.npz")
    
    # Prepare metadata
    metadata = {
        'normalization_method': NORMALIZATION_METHOD,
        'grayscale_method': GRAYSCALE_METHOD,
        'pixel_size_um': PIXEL_SIZE_UM if PIXEL_SIZE_UM else -1,
        'time_per_frame_s': TIME_PER_FRAME_S if TIME_PER_FRAME_S else -1,
        'mean_noise': mean_noise,
        'snr_db': snr_db,
        'mean_correlation': mean_correlation
    }
    
    np.savez_compressed(output_file,
                       u_images=u_images_normalized,
                       u_images_raw=u_images,  # Save both normalized and raw
                       frame_numbers=np.array(frame_numbers),
                       frame_means=frame_means,
                       frame_stds=frame_stds,
                       noise_estimates=np.array(noise_estimates),
                       correlations=np.array(correlations),
                       shape=u_images.shape,
                       **metadata)
    
    print(f"\n   Saved processed data to: {output_file}")
    print(f"   Contains:")
    print(f"     - u_images: {u_images.shape} array (normalized)")
    print(f"     - u_images_raw: {u_images.shape} array (original)")
    print(f"     - frame_numbers: Frame identifiers")
    print(f"     - frame_means, frame_stds: Per-frame statistics")
    print(f"     - noise_estimates: Per-frame noise levels")
    print(f"     - correlations: Frame-to-frame correlations")
    print(f"     - metadata: Processing parameters")

# ===== STEP 7: VISUALIZATION =====
if SAVE_PLOTS:
    print("\n" + "=" * 80)
    print("STEP 7: CREATING VISUALIZATIONS")
    print("=" * 80)
    
    # === Figure 1: Sample frames overview ===
    print("\n  Creating Figure 1: Sample frames...")
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    fig.suptitle('Sample Frames from Real Experimental Data', fontsize=16, fontweight='bold')
    
    # Show 10 evenly spaced frames
    frame_indices = np.linspace(0, len(u_images) - 1, 10, dtype=int)
    
    for idx, (ax, frame_idx) in enumerate(zip(axes.flat, frame_indices)):
        im = ax.imshow(u_images_normalized[frame_idx], cmap='gray', aspect='auto')
        ax.set_title(f'Frame {frame_numbers[frame_idx]}\nMean: {frame_means[frame_idx]:.1f}', 
                     fontsize=10)
        ax.axis('off')
        # Add colorbar to last plot
        if idx == 9:
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
    plt.tight_layout()
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_01_sample_frames.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"      Saved: {plot_file}")
    plt.close()
    
    # === Figure 2: Temporal evolution analysis ===
    print("  Creating Figure 2: Temporal evolution...")
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle('Temporal Evolution Analysis', fontsize=16, fontweight='bold')
    
    # Mean intensity over time
    axes[0, 0].plot(frame_numbers, frame_means, 'o-', linewidth=2, markersize=4)
    axes[0, 0].set_xlabel('Frame Number')
    axes[0, 0].set_ylabel('Mean Intensity')
    axes[0, 0].set_title('Mean Intensity Evolution')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Std intensity over time
    axes[0, 1].plot(frame_numbers, frame_stds, 'o-', linewidth=2, markersize=4, color='orange')
    axes[0, 1].set_xlabel('Frame Number')
    axes[0, 1].set_ylabel('Std Intensity')
    axes[0, 1].set_title('Intensity Variability Evolution')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Noise estimates over time (NEW)
    axes[0, 2].plot(frame_numbers, noise_estimates, 'o-', linewidth=2, markersize=4, color='red')
    axes[0, 2].axhline(mean_noise, ls='--', color='black', label=f'Mean: {mean_noise:.2f}')
    axes[0, 2].set_xlabel('Frame Number')
    axes[0, 2].set_ylabel('Estimated Noise (σ)')
    axes[0, 2].set_title('Noise Level Evolution')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # Frame-to-frame correlation
    axes[1, 0].plot(frame_numbers[:-1], correlations, 'o-', linewidth=2, markersize=4, color='green')
    axes[1, 0].axhline(mean_correlation, ls='--', color='red', label=f'Mean: {mean_correlation:.3f}')
    axes[1, 0].set_xlabel('Frame Number')
    axes[1, 0].set_ylabel('Correlation with Next Frame')
    axes[1, 0].set_title('Temporal Correlation')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Intensity distribution (all frames combined)
    axes[1, 1].hist(u_images_normalized.flatten(), bins=100, color='steelblue', 
                    alpha=0.7, edgecolor='black')
    axes[1, 1].set_xlabel('Intensity Value')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Overall Intensity Distribution')
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    # Intensity range evolution (NEW)
    axes[1, 2].fill_between(frame_numbers, frame_mins, frame_maxs, alpha=0.3, label='Min-Max Range')
    axes[1, 2].plot(frame_numbers, frame_means, 'k-', linewidth=2, label='Mean')
    axes[1, 2].set_xlabel('Frame Number')
    axes[1, 2].set_ylabel('Intensity')
    axes[1, 2].set_title('Intensity Range Evolution')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_01_temporal_analysis.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"      Saved: {plot_file}")
    plt.close()
    
    # === Figure 3: Spatiotemporal visualization ===
    print("  Creating Figure 3: Spatiotemporal structure...")
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Spatiotemporal Structure', fontsize=16, fontweight='bold')
    
    # Middle row evolution (1D slice over time)
    mid_row = u_images.shape[1] // 2
    middle_row_data = u_images_normalized[:, mid_row, :]
    
    im0 = axes[0, 0].imshow(middle_row_data, aspect='auto', cmap='gray', 
                   extent=[0, u_images.shape[2], frame_numbers[-1], frame_numbers[0]])
    axes[0, 0].set_xlabel('X Position (pixels)')
    axes[0, 0].set_ylabel('Frame Number')
    axes[0, 0].set_title('Middle Row Evolution (X-T Diagram)')
    plt.colorbar(im0, ax=axes[0, 0], label='Intensity')
    
    # Middle column evolution
    mid_col = u_images.shape[2] // 2
    middle_col_data = u_images_normalized[:, :, mid_col]
    
    im1 = axes[0, 1].imshow(middle_col_data, aspect='auto', cmap='gray',
                   extent=[0, u_images.shape[1], frame_numbers[-1], frame_numbers[0]])
    axes[0, 1].set_xlabel('Y Position (pixels)')
    axes[0, 1].set_ylabel('Frame Number')
    axes[0, 1].set_title('Middle Column Evolution (Y-T Diagram)')
    plt.colorbar(im1, ax=axes[0, 1], label='Intensity')
    
    # Sample spatial profiles (middle row)
    axes[1, 0].set_title('Spatial Profiles (Middle Row)')
    sample_frames = np.linspace(0, len(u_images) - 1, 5, dtype=int)
    for frame_idx in sample_frames:
        axes[1, 0].plot(u_images_normalized[frame_idx, mid_row, :], 
                    label=f'Frame {frame_numbers[frame_idx]}', 
                    alpha=0.7, linewidth=1.5)
    axes[1, 0].set_xlabel('X Position (pixels)')
    axes[1, 0].set_ylabel('Intensity')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Gradient magnitude of middle frame (NEW)
    mid_frame_idx = len(u_images) // 2
    grad_x = np.gradient(u_images_normalized[mid_frame_idx], axis=1)
    grad_y = np.gradient(u_images_normalized[mid_frame_idx], axis=0)
    grad_mag = np.sqrt(grad_x**2 + grad_y**2)
    
    im2 = axes[1, 1].imshow(grad_mag, aspect='auto', cmap='hot')
    axes[1, 1].set_title(f'Gradient Magnitude (Frame {frame_numbers[mid_frame_idx]})')
    axes[1, 1].set_xlabel('X Position (pixels)')
    axes[1, 1].set_ylabel('Y Position (pixels)')
    plt.colorbar(im2, ax=axes[1, 1], label='|∇u|')
    
    plt.tight_layout()
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_01_spatiotemporal.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"  ✓ Saved: {plot_file}")
    plt.close()
    
    # === Figure 4: Quality metrics summary (NEW) ===
    print("  Creating Figure 4: Quality metrics...")
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Data Quality Metrics Summary', fontsize=16, fontweight='bold')
    
    # SNR visualization
    axes[0, 0].bar(['Signal\nPower', 'Noise\nPower', 'SNR (dB)'], 
                   [signal_power, noise_power, snr_db],
                   color=['green', 'red', 'blue'], alpha=0.7)
    axes[0, 0].set_ylabel('Value')
    axes[0, 0].set_title(f'Signal-to-Noise Analysis\nSNR = {snr_db:.2f} dB')
    axes[0, 0].grid(True, alpha=0.3, axis='y')
    
    # Correlation histogram
    axes[0, 1].hist(correlations, bins=30, color='green', alpha=0.7, edgecolor='black')
    axes[0, 1].axvline(mean_correlation, color='red', linestyle='--', 
                       linewidth=2, label=f'Mean: {mean_correlation:.3f}')
    axes[0, 1].set_xlabel('Correlation Coefficient')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Distribution of Frame-to-Frame Correlations')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # Noise histogram
    axes[1, 0].hist(noise_estimates, bins=30, color='orange', alpha=0.7, edgecolor='black')
    axes[1, 0].axvline(mean_noise, color='red', linestyle='--', 
                       linewidth=2, label=f'Mean: {mean_noise:.2f}')
    axes[1, 0].set_xlabel('Estimated Noise (σ)')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Distribution of Noise Estimates')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3, axis='y')
    
    # Quality summary text
    axes[1, 1].axis('off')
    quality_text = f"""
    DATA QUALITY SUMMARY
    {'='*40}
    
    Temporal Quality:
      • Mean correlation: {mean_correlation:.4f}
      • Status: {' EXCELLENT' if mean_correlation > 0.8 else 'MODERATE' if mean_correlation > 0.5 else ' POOR'}
    
    Noise Characteristics:
      • Noise level (σ): {mean_noise:.2f} ± {std_noise:.2f}
      • SNR: {snr_db:.2f} dB
      • Status: {'EXCELLENT' if snr_db > 20 else ' MODERATE' if snr_db > 10 else 'LOW'}
    
    Spatial Structure:
      • Mean gradient: {mean_gradient:.2f}
      • Status: {' STRONG' if mean_gradient > 10 else ' MODERATE' if mean_gradient > 5 else 'WEAK'}
    
    Data Integrity:
      • Dead pixels: {100*dead_pixels/total_pixels:.3f}%
      • Saturated pixels: {100*saturated_pixels/total_pixels:.3f}%
      • Status: {' GOOD' if saturated_pixels < 0.01*total_pixels else ' CHECK SATURATION'}
    
    Recommendation:
      {' Data suitable for SINDy analysis' if mean_correlation > 0.7 and snr_db > 10 else '    Consider preprocessing before SINDy'}
    """
    axes[1, 1].text(0.1, 0.5, quality_text, fontsize=10, family='monospace',
                    verticalalignment='center', transform=axes[1, 1].transAxes)
    
    plt.tight_layout()
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_01_quality_metrics.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"      Saved: {plot_file}")
    plt.close()

# ===== FINAL SUMMARY =====
print("\n" + "=" * 80)
print("PHASE 4.1 COMPLETE - SUMMARY")
print("=" * 80)

print(f"\n Loaded Data:")
print(f"   • {len(tif_files)} frames loaded successfully")
print(f"   • Shape: {u_images.shape} (Nt, Ny, Nx)")
print(f"   • Intensity range: [{u_images.min():.1f}, {u_images.max():.1f}]")
print(f"   • Normalized range: [{u_images_normalized.min():.4f}, {u_images_normalized.max():.4f}]")

print(f"\n Quality Assessment:")
print(f"   • Mean frame-to-frame correlation: {mean_correlation:.3f}")
print(f"   • Signal-to-Noise Ratio: {snr_db:.2f} dB")
print(f"   • Mean noise level: {mean_noise:.2f}")
print(f"   • Mean spatial gradient: {mean_gradient:.2f}")

# Overall quality rating
quality_score = 0
if mean_correlation > 0.7:
    quality_score += 1
if snr_db > 10:
    quality_score += 1
if mean_gradient > 5:
    quality_score += 1
if saturated_pixels < 0.01 * total_pixels:
    quality_score += 1

quality_ratings = {
    4: " EXCELLENT - Data ready for SINDy",
    3: " GOOD - Minor preprocessing may help",
    2: "  FAIR - Preprocessing recommended",
    1: "  POOR - Significant preprocessing needed",
    0: " CRITICAL - Data quality issues detected"
}

print(f"   • Overall quality: {quality_ratings[quality_score]}")

print(f"\nOutputs Generated:")
if SAVE_NPZ:
    print(f"    {os.path.join(OUTPUT_DIR, 'phase4_01_loaded_images.npz')}")
if SAVE_PLOTS:
    print(f"    {os.path.join(OUTPUT_DIR, 'phase4_01_sample_frames.png')}")
    print(f"    {os.path.join(OUTPUT_DIR, 'phase4_01_temporal_analysis.png')}")
    print(f"    {os.path.join(OUTPUT_DIR, 'phase4_01_spatiotemporal.png')}")
    print(f"    {os.path.join(OUTPUT_DIR, 'phase4_01_quality_metrics.png')}")

print(f"\n Next Steps:")
print(f"   → Run Phase 4.2: Extract 1D slices for SINDy analysis")
print(f"   → Recommended extraction strategy: {'Averaged rows' if snr_db < 15 else 'Single middle row'}")
if snr_db < 10:
    print(f"   →   Consider aggressive denoising (Phase 4.5) before SINDy")
if mean_correlation < 0.5:
    print(f"   →   Check for registration issues (Phase 4.6)")

print("\n" + "=" * 80)
print(" READY FOR PHASE 4.2!    ")
print("=" * 80)

In [ ]:
data = np.load('phase4_real_images/phase4_01_loaded_images.npz')
u_images = data['u_images']  # (51, 2052, 2560)

In [ ]:
# Use frames 10-45 (skip weak early frames and potential outliers)
frame_start = 10
frame_end = 45
u_selected = u_images[frame_start:frame_end+1, :, :]  # (36, 2052, 2560)

In [ ]:
# Interior region (avoid boundaries)
y_start, y_end = 800, 1200  # Central 400 rows
x_start, x_end = 250, 2300  # Avoid edge artifacts

u_interior = u_selected[:, y_start:y_end, x_start:x_end]

In [ ]:
# Average rows to reduce noise
u_1d = np.mean(u_interior, axis=1)  # (36, 2050) - time × space

In [ ]:
from scipy.signal import detrend
u_1d_detrended = detrend(u_1d, axis=0, type='linear')  # Remove temporal drift

In [ ]:
Nt, Nx = u_1d_detrended.shape
t = np.arange(Nt)  # Time indices (will scale if TIME_PER_FRAME known)
x = np.arange(Nx)  # Spatial indices

# Create u(x,t) array
u_real = u_1d_detrended  # This is your SINDy input!

In [ ]:
# Spatiotemporal diagram
plt.imshow(u_real, aspect='auto', cmap='gray')
plt.xlabel('X Position (pixels)')
plt.ylabel('Frame Number')
plt.title('Extracted 1D Spatiotemporal Data (Averaged Rows)')
plt.colorbar(label='Intensity')
plt.savefig('phase4_02_extracted_1d_slices.png')

In [ ]:
np.savez_compressed('phase4_real_images/phase4_02_extracted_1d_slices.npz',
                   u_real=u_real,
                   t=t,
                   x=x,
                   Nt=Nt,
                   Nx=Nx,
                   frame_start=frame_start,
                   frame_end=frame_end,
                   y_region=(y_start, y_end),
                   x_region=(x_start, x_end))

In [ ]:
"""
PHASE 4.2B: COMPREHENSIVE VISUALIZATION & QUALITY METRICS
FOR EXTRACTED 1D SPATIOTEMPORAL DATA

This script performs detailed analysis of the extracted 1D slices to assess
data quality, identify potential issues, and validate readiness for SINDy.

Author: SINDy Project
Phase: 4.2B - Data Quality Assessment
"""

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal, stats
from scipy.fft import fft, fftfreq
import os

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (18, 12)

print("=" * 80)
print("PHASE 4.2B: QUALITY METRICS FOR EXTRACTED 1D DATA")
print("=" * 80)

# ===== CONFIGURATION =====
INPUT_FILE = "phase4_real_images/phase4_02_extracted_1d_slices.npz"
OUTPUT_DIR = "phase4_real_images"
SAVE_PLOTS = True

# ===== STEP 1: LOAD EXTRACTED DATA =====
print("\n" + "=" * 80)
print("STEP 1: LOADING EXTRACTED DATA")
print("=" * 80)

data = np.load(INPUT_FILE)
u_real = data['u_real']  # (Nt, Nx)
t = data['t']
x = data['x']
Nt, Nx = u_real.shape

print(f"\n✅ Loaded extracted 1D spatiotemporal data:")
print(f"   Shape: {u_real.shape} (Nt={Nt}, Nx={Nx})")
print(f"   Time range: {t[0]} to {t[-1]} ({len(t)} frames)")
print(f"   Spatial range: {x[0]} to {x[-1]} ({len(x)} pixels)")
print(f"   Intensity range: [{u_real.min():.2f}, {u_real.max():.2f}]")
print(f"   Mean: {u_real.mean():.4f}, Std: {u_real.std():.4f}")

# ===== STEP 2: BASIC STATISTICS =====
print("\n" + "=" * 80)
print("STEP 2: STATISTICAL PROPERTIES")
print("=" * 80)

# Overall statistics
print(f"\n📊 Overall Statistics:")
print(f"   Min: {u_real.min():.4f}")
print(f"   Max: {u_real.max():.4f}")
print(f"   Mean: {u_real.mean():.4f}")
print(f"   Median: {np.median(u_real):.4f}")
print(f"   Std: {u_real.std():.4f}")
print(f"   Range: {u_real.max() - u_real.min():.4f}")

# Temporal statistics
temporal_means = np.mean(u_real, axis=1)
temporal_stds = np.std(u_real, axis=1)
temporal_mins = np.min(u_real, axis=1)
temporal_maxs = np.max(u_real, axis=1)

print(f"\n⏱️  Temporal Statistics:")
print(f"   Mean over time: {temporal_means.mean():.4f} ± {temporal_means.std():.4f}")
print(f"   Std over time: {temporal_stds.mean():.4f} ± {temporal_stds.std():.4f}")
print(f"   Temporal drift: {temporal_means.max() - temporal_means.min():.4f}")

# Spatial statistics
spatial_means = np.mean(u_real, axis=0)
spatial_stds = np.std(u_real, axis=0)
spatial_mins = np.min(u_real, axis=0)
spatial_maxs = np.max(u_real, axis=0)

print(f"\n📏 Spatial Statistics:")
print(f"   Mean over space: {spatial_means.mean():.4f} ± {spatial_means.std():.4f}")
print(f"   Std over space: {spatial_stds.mean():.4f} ± {spatial_stds.std():.4f}")
print(f"   Spatial variation: {spatial_means.max() - spatial_means.min():.4f}")

# ===== STEP 3: NOISE ESTIMATION =====
print("\n" + "=" * 80)
print("STEP 3: NOISE CHARACTERIZATION")
print("=" * 80)

def estimate_noise_mad(data):
    """Estimate noise using Median Absolute Deviation"""
    # Apply Laplacian for high-frequency noise extraction
    laplacian = np.array([[1, -2, 1]])
    noise_estimate = []
    for row in data:
        filtered = np.convolve(row, laplacian[0], mode='same')
        sigma = np.median(np.abs(filtered)) / 0.6745
        noise_estimate.append(sigma)
    return np.array(noise_estimate)

noise_per_frame = estimate_noise_mad(u_real)
mean_noise = np.mean(noise_per_frame)
std_noise = np.std(noise_per_frame)

print(f"\n🔊 Noise Estimation (MAD method):")
print(f"   Mean noise level: {mean_noise:.4f} ± {std_noise:.4f}")
print(f"   Min noise: {noise_per_frame.min():.4f} (Frame {noise_per_frame.argmin()})")
print(f"   Max noise: {noise_per_frame.max():.4f} (Frame {noise_per_frame.argmax()})")

# Signal-to-Noise Ratio
signal_power = np.var(u_real)
noise_power = mean_noise**2
snr_db = 10 * np.log10(signal_power / noise_power)

print(f"\n📡 Signal-to-Noise Ratio:")
print(f"   Signal variance: {signal_power:.4f}")
print(f"   Noise variance: {noise_power:.4f}")
print(f"   SNR: {snr_db:.2f} dB")

if snr_db > 20:
    print(f"   → ✅ Excellent SNR (> 20 dB) - Ready for SINDy!")
elif snr_db > 10:
    print(f"   → ⚠️  Moderate SNR (10-20 dB) - Denoising recommended")
else:
    print(f"   → ❌ Low SNR (< 10 dB) - Denoising REQUIRED")

# ===== STEP 4: TEMPORAL CONSISTENCY =====
print("\n" + "=" * 80)
print("STEP 4: TEMPORAL CONSISTENCY ANALYSIS")
print("=" * 80)

# Frame-to-frame differences
frame_diffs = np.diff(u_real, axis=0)
mean_frame_diff = np.mean(np.abs(frame_diffs))
max_frame_diff = np.max(np.abs(frame_diffs))

print(f"\n📊 Frame-to-Frame Changes:")
print(f"   Mean absolute difference: {mean_frame_diff:.4f}")
print(f"   Max absolute difference: {max_frame_diff:.4f}")
print(f"   Relative change: {mean_frame_diff/u_real.std():.4f} (as fraction of std)")

# Temporal correlation
correlations = []
for i in range(Nt - 1):
    corr = np.corrcoef(u_real[i, :], u_real[i+1, :])[0, 1]
    correlations.append(corr)
correlations = np.array(correlations)
mean_correlation = np.mean(correlations)

print(f"\n🔗 Temporal Correlation:")
print(f"   Mean correlation: {mean_correlation:.4f}")
print(f"   Min correlation: {correlations.min():.4f} (Frame {correlations.argmin()} → {correlations.argmin()+1})")
print(f"   Max correlation: {correlations.max():.4f} (Frame {correlations.argmax()} → {correlations.argmax()+1})")

if mean_correlation > 0.8:
    print(f"   → ✅ High correlation - Smooth evolution")
elif mean_correlation > 0.6:
    print(f"   → ⚠️  Moderate correlation - Some variation")
else:
    print(f"   → ❌ Low correlation - Check for discontinuities")

# Detect sudden jumps
correlation_drops = np.where(correlations < 0.5)[0]
if len(correlation_drops) > 0:
    print(f"\n   ⚠️  WARNING: Low correlation detected at frames:")
    for idx in correlation_drops:
        print(f"      Frame {idx} → {idx+1}: correlation = {correlations[idx]:.3f}")

# ===== STEP 5: SPATIAL STRUCTURE ANALYSIS =====
print("\n" + "=" * 80)
print("STEP 5: SPATIAL STRUCTURE ANALYSIS")
print("=" * 80)

# Spatial gradients
spatial_gradients = np.gradient(u_real, axis=1)
mean_gradient = np.mean(np.abs(spatial_gradients))
max_gradient = np.max(np.abs(spatial_gradients))

print(f"\n∇ Spatial Gradients:")
print(f"   Mean |∇u|: {mean_gradient:.4f}")
print(f"   Max |∇u|: {max_gradient:.4f}")

if mean_gradient > 0.1:
    print(f"   → ✅ Strong spatial structure (good for derivatives)")
elif mean_gradient > 0.05:
    print(f"   → ⚠️  Moderate spatial structure")
else:
    print(f"   → ❌ Weak spatial structure (derivatives may be noisy)")

# Dominant wavelength estimation (FFT)
print(f"\n🌊 Wavelength Analysis (using FFT):")
# Average over time to get mean spatial profile
mean_spatial_profile = np.mean(u_real, axis=0)
# Remove mean and apply window
windowed = (mean_spatial_profile - mean_spatial_profile.mean()) * np.hanning(len(mean_spatial_profile))
# FFT
fft_vals = np.abs(fft(windowed))
freqs = fftfreq(len(windowed), d=1.0)  # d=1 pixel
# Only positive frequencies
pos_freqs = freqs[:len(freqs)//2]
pos_fft = fft_vals[:len(fft_vals)//2]
# Find dominant frequency
dominant_idx = np.argmax(pos_fft[1:]) + 1  # Skip DC component
dominant_freq = pos_freqs[dominant_idx]
dominant_wavelength = 1 / dominant_freq if dominant_freq > 0 else np.inf

print(f"   Dominant spatial frequency: {dominant_freq:.6f} cycles/pixel")
print(f"   Dominant wavelength: {dominant_wavelength:.2f} pixels")

# Find all significant peaks
peak_threshold = 0.3 * pos_fft[1:].max()
peaks, _ = signal.find_peaks(pos_fft[1:], height=peak_threshold)
if len(peaks) > 0:
    print(f"   Number of significant peaks: {len(peaks)}")
    # Compute wavelengths separately to avoid f-string formatting issues
    peak_wavelengths = [1/pos_freqs[p+1] for p in peaks[:5]]
    peak_wavelengths_str = ', '.join([f'{w:.1f}' for w in peak_wavelengths])
    print(f"   Peak wavelengths: [{peak_wavelengths_str}] pixels (top 5)")
    
# ===== STEP 6: DISTRIBUTION ANALYSIS =====
print("\n" + "=" * 80)
print("STEP 6: INTENSITY DISTRIBUTION")
print("=" * 80)

# Check for normality
_, p_value = stats.normaltest(u_real.flatten())
is_normal = p_value > 0.05

print(f"\n📈 Distribution Properties:")
print(f"   Skewness: {stats.skew(u_real.flatten()):.4f}")
print(f"   Kurtosis: {stats.kurtosis(u_real.flatten()):.4f}")
print(f"   Normality test p-value: {p_value:.4e}")
if is_normal:
    print(f"   → ✅ Distribution is approximately normal")
else:
    print(f"   → ⚠️  Distribution is non-normal (may indicate artifacts)")

# Outlier detection
q1, q3 = np.percentile(u_real.flatten(), [25, 75])
iqr = q3 - q1
outlier_threshold_low = q1 - 3 * iqr
outlier_threshold_high = q3 + 3 * iqr
outliers = (u_real < outlier_threshold_low) | (u_real > outlier_threshold_high)
outlier_count = np.sum(outliers)
outlier_percentage = 100 * outlier_count / u_real.size

print(f"\n🎯 Outlier Detection (3×IQR method):")
print(f"   IQR: {iqr:.4f}")
print(f"   Outlier thresholds: [{outlier_threshold_low:.2f}, {outlier_threshold_high:.2f}]")
print(f"   Outliers found: {outlier_count} ({outlier_percentage:.3f}%)")

if outlier_percentage < 0.1:
    print(f"   → ✅ Very few outliers")
elif outlier_percentage < 1.0:
    print(f"   → ⚠️  Some outliers present")
else:
    print(f"   → ❌ Many outliers - investigate data quality")

# ===== STEP 7: STATIONARITY CHECK =====
print("\n" + "=" * 80)
print("STEP 7: STATIONARITY ANALYSIS")
print("=" * 80)

# Split data into chunks and compare statistics
n_chunks = 4
chunk_size = Nt // n_chunks
chunk_means = []
chunk_stds = []

for i in range(n_chunks):
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < n_chunks - 1 else Nt
    chunk = u_real[start:end, :]
    chunk_means.append(chunk.mean())
    chunk_stds.append(chunk.std())

mean_drift = max(chunk_means) - min(chunk_means)
std_drift = max(chunk_stds) - min(chunk_stds)

print(f"\n📊 Temporal Stationarity (split into {n_chunks} chunks):")
print(f"   Chunk means: {[f'{m:.3f}' for m in chunk_means]}")
print(f"   Chunk stds: {[f'{s:.3f}' for s in chunk_stds]}")
print(f"   Mean drift: {mean_drift:.4f}")
print(f"   Std drift: {std_drift:.4f}")

if mean_drift < 0.1 * u_real.std():
    print(f"   → ✅ Mean is stationary (drift < 10% of std)")
else:
    print(f"   → ⚠️  Significant mean drift detected")

if std_drift < 0.2 * u_real.std():
    print(f"   → ✅ Variance is stationary")
else:
    print(f"   → ⚠️  Variance changes over time")

# ===== STEP 8: DERIVATIVE READINESS =====
print("\n" + "=" * 80)
print("STEP 8: DERIVATIVE COMPUTATION READINESS")
print("=" * 80)

# Estimate derivative accuracy by computing test derivatives
print(f"\n🧮 Testing Derivative Computation:")

# Temporal derivative (simple finite difference)
dt = 1.0  # Frame spacing
u_t_test = np.diff(u_real, axis=0) / dt
u_t_noise_estimate = estimate_noise_mad(u_t_test).mean()
print(f"   Temporal derivative u_t:")
print(f"      Mean |u_t|: {np.mean(np.abs(u_t_test)):.4f}")
print(f"      Estimated noise in u_t: {u_t_noise_estimate:.4f}")

# Spatial first derivative (central difference)
dx = 1.0  # Pixel spacing
u_x_test = np.gradient(u_real, axis=1) / dx
u_x_noise_estimate = estimate_noise_mad(u_x_test).mean()
print(f"   Spatial derivative u_x:")
print(f"      Mean |u_x|: {np.mean(np.abs(u_x_test)):.4f}")
print(f"      Estimated noise in u_x: {u_x_noise_estimate:.4f}")

# Spatial second derivative
u_xx_test = np.gradient(u_x_test, axis=1) / dx
u_xx_noise_estimate = estimate_noise_mad(u_xx_test).mean()
print(f"   Spatial derivative u_xx:")
print(f"      Mean |u_xx|: {np.mean(np.abs(u_xx_test)):.4f}")
print(f"      Estimated noise in u_xx: {u_xx_noise_estimate:.4f}")

# SNR for derivatives
u_t_snr = 10 * np.log10(np.var(u_t_test) / (u_t_noise_estimate**2))
u_x_snr = 10 * np.log10(np.var(u_x_test) / (u_x_noise_estimate**2))
u_xx_snr = 10 * np.log10(np.var(u_xx_test) / (u_xx_noise_estimate**2))

print(f"\n📡 Derivative SNR Estimates:")
print(f"   u_t SNR: {u_t_snr:.2f} dB")
print(f"   u_x SNR: {u_x_snr:.2f} dB")
print(f"   u_xx SNR: {u_xx_snr:.2f} dB")

if u_t_snr > 10 and u_x_snr > 10 and u_xx_snr > 10:
    print(f"   → ✅ All derivatives have acceptable SNR (> 10 dB)")
elif u_t_snr > 5 and u_x_snr > 5 and u_xx_snr > 5:
    print(f"   → ⚠️  Moderate derivative SNR (5-10 dB) - Denoising recommended")
else:
    print(f"   → ❌ Low derivative SNR (< 5 dB) - Denoising REQUIRED before SINDy")

# ===== COMPREHENSIVE QUALITY SCORE =====
print("\n" + "=" * 80)
print("OVERALL DATA QUALITY ASSESSMENT")
print("=" * 80)

quality_score = 0
max_score = 8

# Criterion 1: SNR
if snr_db > 20:
    quality_score += 2
    snr_status = "✅ Excellent"
elif snr_db > 10:
    quality_score += 1
    snr_status = "⚠️  Moderate"
else:
    snr_status = "❌ Poor"

# Criterion 2: Temporal correlation
if mean_correlation > 0.8:
    quality_score += 1
    corr_status = "✅ High"
elif mean_correlation > 0.6:
    quality_score += 0.5
    corr_status = "⚠️  Moderate"
else:
    corr_status = "❌ Low"

# Criterion 3: Spatial gradients
if mean_gradient > 0.1:
    quality_score += 1
    grad_status = "✅ Strong"
elif mean_gradient > 0.05:
    quality_score += 0.5
    grad_status = "⚠️  Moderate"
else:
    grad_status = "❌ Weak"

# Criterion 4: Outliers
if outlier_percentage < 0.1:
    quality_score += 1
    outlier_status = "✅ Minimal"
elif outlier_percentage < 1.0:
    quality_score += 0.5
    outlier_status = "⚠️  Some"
else:
    outlier_status = "❌ Many"

# Criterion 5: Stationarity
if mean_drift < 0.1 * u_real.std():
    quality_score += 1
    station_status = "✅ Stationary"
else:
    station_status = "⚠️  Non-stationary"

# Criterion 6: Derivative SNR
if u_t_snr > 10 and u_x_snr > 10:
    quality_score += 2
    deriv_status = "✅ Good"
elif u_t_snr > 5 and u_x_snr > 5:
    quality_score += 1
    deriv_status = "⚠️  Marginal"
else:
    deriv_status = "❌ Poor"

print(f"\n📊 Quality Criteria Summary:")
print(f"   1. SNR: {snr_status} ({snr_db:.2f} dB)")
print(f"   2. Temporal correlation: {corr_status} ({mean_correlation:.3f})")
print(f"   3. Spatial gradients: {grad_status} ({mean_gradient:.4f})")
print(f"   4. Outliers: {outlier_status} ({outlier_percentage:.2f}%)")
print(f"   5. Stationarity: {station_status}")
print(f"   6. Derivative SNR: {deriv_status}")

print(f"\n🎯 OVERALL QUALITY SCORE: {quality_score}/{max_score}")

if quality_score >= 7:
    overall_assessment = "✅ EXCELLENT - Ready for SINDy!"
    recommendation = "Proceed directly to Phase 4.3 (Baseline SINDy)"
elif quality_score >= 5:
    overall_assessment = "✅ GOOD - SINDy should work with minor preprocessing"
    recommendation = "Proceed to Phase 4.3, but keep Phase 4.5 (denoising) ready"
elif quality_score >= 3:
    overall_assessment = "⚠️  FAIR - Preprocessing recommended"
    recommendation = "Apply denoising (Phase 4.5) before Phase 4.3"
else:
    overall_assessment = "❌ POOR - Significant preprocessing needed"
    recommendation = "Apply aggressive denoising (Phase 4.5) and consider robust regression (Phase 4.7)"

print(f"\n📋 Assessment: {overall_assessment}")
print(f"💡 Recommendation: {recommendation}")

# ===== STEP 9: COMPREHENSIVE VISUALIZATIONS =====
if SAVE_PLOTS:
    print("\n" + "=" * 80)
    print("STEP 9: GENERATING COMPREHENSIVE VISUALIZATIONS")
    print("=" * 80)
    
    # === FIGURE 1: Basic Properties ===
    print("\n  Creating Figure 1: Basic properties...")
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)
    
    # Main spatiotemporal plot
    ax1 = fig.add_subplot(gs[0:2, 0:2])
    im1 = ax1.imshow(u_real, aspect='auto', cmap='seismic', 
                     extent=[x[0], x[-1], t[-1], t[0]], vmin=-30, vmax=30)
    ax1.set_xlabel('X Position (pixels)', fontsize=12)
    ax1.set_ylabel('Frame Number', fontsize=12)
    ax1.set_title('Extracted 1D Spatiotemporal Data', fontsize=14, fontweight='bold')
    plt.colorbar(im1, ax=ax1, label='Intensity')
    
    # Temporal mean evolution
    ax2 = fig.add_subplot(gs[0, 2])
    ax2.plot(t, temporal_means, 'b-', linewidth=2)
    ax2.fill_between(t, temporal_means - temporal_stds, temporal_means + temporal_stds, 
                      alpha=0.3, color='blue')
    ax2.set_xlabel('Frame Number', fontsize=10)
    ax2.set_ylabel('Mean Intensity', fontsize=10)
    ax2.set_title('Temporal Mean ± Std', fontsize=11)
    ax2.grid(True, alpha=0.3)
    
    # Temporal std evolution
    ax3 = fig.add_subplot(gs[0, 3])
    ax3.plot(t, temporal_stds, 'r-', linewidth=2)
    ax3.set_xlabel('Frame Number', fontsize=10)
    ax3.set_ylabel('Std Intensity', fontsize=10)
    ax3.set_title('Temporal Variability', fontsize=11)
    ax3.grid(True, alpha=0.3)
    
    # Spatial mean profile
    ax4 = fig.add_subplot(gs[1, 2])
    ax4.plot(x, spatial_means, 'g-', linewidth=2)
    ax4.fill_between(x, spatial_means - spatial_stds, spatial_means + spatial_stds,
                      alpha=0.3, color='green')
    ax4.set_xlabel('X Position (pixels)', fontsize=10)
    ax4.set_ylabel('Mean Intensity', fontsize=10)
    ax4.set_title('Spatial Mean ± Std', fontsize=11)
    ax4.grid(True, alpha=0.3)
    
    # Spatial std profile
    ax5 = fig.add_subplot(gs[1, 3])
    ax5.plot(x, spatial_stds, 'orange', linewidth=2)
    ax5.set_xlabel('X Position (pixels)', fontsize=10)
    ax5.set_ylabel('Std Intensity', fontsize=10)
    ax5.set_title('Spatial Variability', fontsize=11)
    ax5.grid(True, alpha=0.3)
    
    # Intensity distribution
    ax6 = fig.add_subplot(gs[2, 0])
    ax6.hist(u_real.flatten(), bins=100, color='steelblue', alpha=0.7, edgecolor='black')
    ax6.axvline(u_real.mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    ax6.axvline(np.median(u_real), color='green', linestyle='--', linewidth=2, label='Median')
    ax6.set_xlabel('Intensity', fontsize=10)
    ax6.set_ylabel('Frequency', fontsize=10)
    ax6.set_title('Intensity Distribution', fontsize=11)
    ax6.legend()
    ax6.grid(True, alpha=0.3, axis='y')
    
    # Correlation evolution
    ax7 = fig.add_subplot(gs[2, 1])
    ax7.plot(t[:-1], correlations, 'purple', linewidth=2)
    ax7.axhline(mean_correlation, color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {mean_correlation:.3f}')
    ax7.set_xlabel('Frame Number', fontsize=10)
    ax7.set_ylabel('Correlation', fontsize=10)
    ax7.set_title('Frame-to-Frame Correlation', fontsize=11)
    ax7.legend()
    ax7.grid(True, alpha=0.3)
    
    # Noise level evolution
    ax8 = fig.add_subplot(gs[2, 2])
    ax8.plot(t, noise_per_frame, 'brown', linewidth=2)
    ax8.axhline(mean_noise, color='black', linestyle='--', linewidth=2,
                label=f'Mean: {mean_noise:.3f}')
    ax8.set_xlabel('Frame Number', fontsize=10)
    ax8.set_ylabel('Noise Level (σ)', fontsize=10)
    ax8.set_title('Estimated Noise Evolution', fontsize=11)
    ax8.legend()
    ax8.grid(True, alpha=0.3)
    
    # FFT power spectrum
    ax9 = fig.add_subplot(gs[2, 3])
    ax9.semilogy(pos_freqs[1:100], pos_fft[1:100], 'navy', linewidth=2)
    if len(peaks) > 0:
        for peak in peaks[:3]:
            ax9.axvline(pos_freqs[peak+1], color='red', linestyle='--', alpha=0.5)
    ax9.set_xlabel('Spatial Frequency (cycles/pixel)', fontsize=10)
    ax9.set_ylabel('Power', fontsize=10)
    ax9.set_title('Spatial Power Spectrum', fontsize=11)
    ax9.grid(True, alpha=0.3)
    
    plt.suptitle('Comprehensive Quality Metrics - Extracted 1D Data', 
                 fontsize=16, fontweight='bold', y=0.995)
    
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_02b_comprehensive_metrics.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"  ✓ Saved: {plot_file}")
    plt.close()
    
    # === FIGURE 2: Sample Frames ===
    print("  Creating Figure 2: Sample spatial profiles...")
    fig, axes = plt.subplots(3, 3, figsize=(18, 12))
    fig.suptitle('Sample Spatial Profiles Over Time', fontsize=16, fontweight='bold')
    
    sample_indices = np.linspace(0, Nt-1, 9, dtype=int)
    
    for idx, (ax, frame_idx) in enumerate(zip(axes.flat, sample_indices)):
        ax.plot(x, u_real[frame_idx, :], linewidth=1.5, color='navy')
        ax.axhline(0, color='red', linestyle='--', alpha=0.5, linewidth=1)
        ax.set_xlabel('X Position (pixels)', fontsize=9)
        ax.set_ylabel('Intensity', fontsize=9)
        ax.set_title(f'Frame {frame_idx} (t={t[frame_idx]})', fontsize=10)
        ax.grid(True, alpha=0.3)
        ax.set_ylim([u_real.min()-5, u_real.max()+5])
    
    plt.tight_layout()
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_02b_sample_profiles.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"  ✓ Saved: {plot_file}")
    plt.close()
    
    # === FIGURE 3: Derivative Analysis ===
    print("  Creating Figure 3: Derivative quality check...")
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle('Derivative Quality Assessment', fontsize=16, fontweight='bold')
    
    # u_t
    im0 = axes[0, 0].imshow(u_t_test, aspect='auto', cmap='RdBu_r', 
                            extent=[x[0], x[-1], t[-2], t[0]])
    axes[0, 0].set_title('Temporal Derivative (u_t)')
    axes[0, 0].set_xlabel('X Position')
    axes[0, 0].set_ylabel('Frame Number')
    plt.colorbar(im0, ax=axes[0, 0])
    
    axes[1, 0].hist(u_t_test.flatten(), bins=100, color='red', alpha=0.7)
    axes[1, 0].set_title(f'u_t Distribution (SNR: {u_t_snr:.1f} dB)')
    axes[1, 0].set_xlabel('u_t')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].grid(True, alpha=0.3, axis='y')
    
    # u_x
    im1 = axes[0, 1].imshow(u_x_test, aspect='auto', cmap='RdBu_r',
                            extent=[x[0], x[-1], t[-1], t[0]])
    axes[0, 1].set_title('Spatial Derivative (u_x)')
    axes[0, 1].set_xlabel('X Position')
    axes[0, 1].set_ylabel('Frame Number')
    plt.colorbar(im1, ax=axes[0, 1])
    
    axes[1, 1].hist(u_x_test.flatten(), bins=100, color='green', alpha=0.7)
    axes[1, 1].set_title(f'u_x Distribution (SNR: {u_x_snr:.1f} dB)')
    axes[1, 1].set_xlabel('u_x')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    # u_xx
    im2 = axes[0, 2].imshow(u_xx_test, aspect='auto', cmap='RdBu_r',
                            extent=[x[0], x[-1], t[-1], t[0]])
    axes[0, 2].set_title('Second Derivative (u_xx)')
    axes[0, 2].set_xlabel('X Position')
    axes[0, 2].set_ylabel('Frame Number')
    plt.colorbar(im2, ax=axes[0, 2])
    
    axes[1, 2].hist(u_xx_test.flatten(), bins=100, color='blue', alpha=0.7)
    axes[1, 2].set_title(f'u_xx Distribution (SNR: {u_xx_snr:.1f} dB)')
    axes[1, 2].set_xlabel('u_xx')
    axes[1, 2].set_ylabel('Frequency')
    axes[1, 2].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_02b_derivative_quality.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"  ✓ Saved: {plot_file}")
    plt.close()
    
    # === FIGURE 4: Quality Summary Dashboard ===
    print("  Creating Figure 4: Quality dashboard...")
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)
    
    # Quality scores radar/bar chart
    ax1 = fig.add_subplot(gs[0, 0])
    criteria = ['SNR', 'Correlation', 'Gradients', 'Outliers', 'Stationarity', 'Deriv SNR']
    scores_norm = [
        min(snr_db/20, 1),
        mean_correlation,
        min(mean_gradient/0.1, 1),
        1 - min(outlier_percentage/1.0, 1),
        1 - min(mean_drift/(0.1*u_real.std()), 1),
        min(min(u_t_snr, u_x_snr)/10, 1)
    ]
    colors_bar = ['green' if s > 0.8 else 'orange' if s > 0.5 else 'red' for s in scores_norm]
    ax1.barh(criteria, scores_norm, color=colors_bar, alpha=0.7)
    ax1.set_xlim([0, 1])
    ax1.set_xlabel('Normalized Score', fontsize=10)
    ax1.set_title('Quality Criteria Scores', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='x')
    
    # Overall assessment text
    ax2 = fig.add_subplot(gs[0, 1:])
    ax2.axis('off')
    summary_text = f"""
DATA QUALITY SUMMARY
{'='*60}

Overall Score: {quality_score}/{max_score}
Assessment: {overall_assessment}

Key Metrics:
  • Signal-to-Noise Ratio: {snr_db:.2f} dB
  • Temporal Correlation: {mean_correlation:.4f}
  • Spatial Gradient: {mean_gradient:.4f}
  • Noise Level: {mean_noise:.4f}
  • Outlier Percentage: {outlier_percentage:.3f}%
  
Data Properties:
  • Shape: {u_real.shape} (Nt × Nx)
  • Intensity Range: [{u_real.min():.2f}, {u_real.max():.2f}]
  • Mean: {u_real.mean():.4f}, Std: {u_real.std():.4f}
  • Dominant Wavelength: {dominant_wavelength:.1f} pixels

Derivative Readiness:
  • u_t SNR: {u_t_snr:.2f} dB
  • u_x SNR: {u_x_snr:.2f} dB
  • u_xx SNR: {u_xx_snr:.2f} dB

RECOMMENDATION:
{recommendation}
    """
    ax2.text(0.05, 0.95, summary_text, fontsize=10, family='monospace',
             verticalalignment='top', transform=ax2.transAxes,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    # Temporal stability
    ax3 = fig.add_subplot(gs[1, 0])
    chunk_x = np.arange(len(chunk_means))
    ax3.plot(chunk_x, chunk_means, 'o-', linewidth=2, markersize=8, label='Mean', color='blue')
    ax3_twin = ax3.twinx()
    ax3_twin.plot(chunk_x, chunk_stds, 's-', linewidth=2, markersize=8, label='Std', color='red')
    ax3.set_xlabel('Temporal Chunk', fontsize=10)
    ax3.set_ylabel('Mean', fontsize=10, color='blue')
    ax3_twin.set_ylabel('Std', fontsize=10, color='red')
    ax3.set_title('Temporal Stationarity Check', fontsize=11)
    ax3.grid(True, alpha=0.3)
    ax3.legend(loc='upper left')
    ax3_twin.legend(loc='upper right')
    
    # Correlation heatmap (small subset)
    ax4 = fig.add_subplot(gs[1, 1])
    # Compute correlation matrix for subset of frames
    subset_size = min(20, Nt)
    subset_indices = np.linspace(0, Nt-1, subset_size, dtype=int)
    corr_matrix = np.corrcoef(u_real[subset_indices, :])
    im = ax4.imshow(corr_matrix, cmap='RdYlGn', vmin=0, vmax=1, aspect='auto')
    ax4.set_xlabel('Frame Index', fontsize=10)
    ax4.set_ylabel('Frame Index', fontsize=10)
    ax4.set_title('Frame Correlation Matrix', fontsize=11)
    plt.colorbar(im, ax=ax4, label='Correlation')
    
    # Noise distribution
    ax5 = fig.add_subplot(gs[1, 2])
    ax5.plot(t, noise_per_frame, 'o-', linewidth=1.5, markersize=4, color='brown')
    ax5.axhline(mean_noise, color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {mean_noise:.3f}')
    ax5.fill_between(t, mean_noise - std_noise, mean_noise + std_noise,
                      alpha=0.3, color='red', label=f'±1σ: {std_noise:.3f}')
    ax5.set_xlabel('Frame Number', fontsize=10)
    ax5.set_ylabel('Noise Level', fontsize=10)
    ax5.set_title('Noise Consistency', fontsize=11)
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    plt.suptitle('Quality Dashboard', fontsize=16, fontweight='bold', y=0.98)
    
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_02b_quality_dashboard.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"  ✓ Saved: {plot_file}")
    plt.close()

# ===== FINAL SUMMARY =====
print("\n" + "=" * 80)
print("PHASE 4.2B COMPLETE - QUALITY ASSESSMENT SUMMARY")
print("=" * 80)

print(f"\n✅ Analysis Complete:")
print(f"   • Input: {INPUT_FILE}")
print(f"   • Data shape: {u_real.shape} (Nt={Nt}, Nx={Nx})")
print(f"   • Quality score: {quality_score}/{max_score}")
print(f"   • Assessment: {overall_assessment}")

if SAVE_PLOTS:
    print(f"\n📁 Generated Visualizations:")
    print(f"   ✓ phase4_02b_comprehensive_metrics.png")
    print(f"   ✓ phase4_02b_sample_profiles.png")
    print(f"   ✓ phase4_02b_derivative_quality.png")
    print(f"   ✓ phase4_02b_quality_dashboard.png")

print(f"\n🎯 Next Steps:")
print(f"   {recommendation}")

if quality_score >= 5:
    print(f"\n✅ DATA IS READY FOR PHASE 4.3 (BASELINE SINDY)")
else:
    print(f"\n⚠️  PREPROCESSING RECOMMENDED BEFORE PHASE 4.3")
    print(f"   Consider applying Phase 4.5 (Denoising) first")

print("\n" + "=" * 80)
print("🚀 READY TO PROCEED!")
print("=" * 80)

In [ ]:
"""
PHASE 4.1: BASELINE 2D SINDY (NO PREPROCESSING)

Input: Raw 2D images (51, 2052, 2560)
Output: Baseline coefficient recovery metrics
"""


# Load raw images
IMAGE_DIRECTORY = "Real-Images" 
images_raw = load_tiff_images(IMAGE_DIRECTORY)  # (51, 2052, 2560)

# Spatially subsample (reduce computational cost)
subsample_factor = 8
images_subsampled = images_raw[:, ::subsample_factor, ::subsample_factor]
# New shape: (51, 257, 320)

# Apply 2D SINDy
coeffs_baseline = apply_2d_sindy_fft(images_subsampled)

# Save results
results = {
    'method': 'baseline_no_preprocessing',
    'coefficients': coeffs_baseline,
    'rmse': compute_rmse(coeffs_baseline),
    'r2': compute_r2(coeffs_baseline)
}

In [ ]:
"""
PHASE 4.1: BASELINE 2D SINDY (NO PREPROCESSING)

Goal: Establish baseline coefficient recovery from raw 2D images
Input: ZIP file containing TIFF images
Output: Baseline metrics, visualizations, CSV results

Author: SINDy Real Image Processing Project
Date: Phase 4.1
"""

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas as pd
import zipfile
import os
from sklearn.linear_model import Lasso
from tifffile import imread
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (18, 12)

print("=" * 80)
print("PHASE 4.1: BASELINE 2D SINDY - NO PREPROCESSING")
print("=" * 80)

# ===== CONFIGURATION =====
ZIP_FILE = "path/to/your/Real_Images.zip"  # UPDATE THIS PATH
OUTPUT_DIR = "phase4_results"
SUBSAMPLE_FACTOR = 8  # Reduce from (2052, 2560) to (257, 320)
SINDY_ALPHA = 1e-4  # Lasso regularization parameter
SAVE_PLOTS = True

# True coefficients for 2D Kuramoto-Sivashinsky equation
# u_t = a*u_xx + b*u_yy + c*u_xxxx + d*u_yyyy + e*u*u_x + f*u*u_y
TRUE_COEFFS = np.array([-1.0, -1.0, -1.0, -1.0, -1.0, -1.0])

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ===== STEP 1: LOAD IMAGES FROM ZIP =====
print("\n" + "=" * 80)
print("STEP 1: LOADING IMAGES FROM ZIP FILE")
print("=" * 80)

def load_images_from_zip(zip_path):
    """
    Load TIFF images from a ZIP file
    
    Parameters:
    -----------
    zip_path : str
        Path to ZIP file containing TIFF images
        
    Returns:
    --------
    images : ndarray
        Array of shape (Nt, H, W) containing all images
    filenames : list
        List of image filenames
    """
    print(f"\n📦 Opening ZIP file: {zip_path}")
    
    images = []
    filenames = []
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Get all TIFF files in ZIP
        tiff_files = [f for f in zip_ref.namelist() 
                      if f.lower().endswith(('.tif', '.tiff')) 
                      and not f.startswith('__MACOSX')]
        
        tiff_files = sorted(tiff_files)
        
        print(f"   Found {len(tiff_files)} TIFF files")
        
        # Extract to temporary directory and load
        temp_dir = "temp_extracted_images"
        os.makedirs(temp_dir, exist_ok=True)
        
        for idx, tiff_file in enumerate(tiff_files):
            # Extract file
            zip_ref.extract(tiff_file, temp_dir)
            
            # Load image
            img_path = os.path.join(temp_dir, tiff_file)
            img = imread(img_path)
            
            images.append(img)
            filenames.append(os.path.basename(tiff_file))
            
            if (idx + 1) % 10 == 0:
                print(f"   Loaded {idx + 1}/{len(tiff_files)} images...")
        
        print(f"   ✓ All images loaded successfully")
        
        # Clean up temporary directory
        import shutil
        shutil.rmtree(temp_dir)
    
    images = np.array(images)
    return images, filenames

# Load images
try:
    images_raw, filenames = load_images_from_zip(ZIP_FILE)
    Nt, H, W = images_raw.shape
    
    print(f"\n✅ Loaded raw images:")
    print(f"   Number of frames (Nt): {Nt}")
    print(f"   Image height (H): {H}")
    print(f"   Image width (W): {W}")
    print(f"   Total shape: {images_raw.shape}")
    print(f"   Data type: {images_raw.dtype}")
    print(f"   Intensity range: [{images_raw.min()}, {images_raw.max()}]")
    
except FileNotFoundError:
    print(f"\n❌ ERROR: ZIP file not found at: {ZIP_FILE}")
    print(f"   Please update the ZIP_FILE path in the configuration section")
    exit(1)
except Exception as e:
    print(f"\n❌ ERROR loading images: {str(e)}")
    exit(1)

# ===== STEP 2: SPATIAL SUBSAMPLING =====
print("\n" + "=" * 80)
print("STEP 2: SPATIAL SUBSAMPLING")
print("=" * 80)

print(f"\n🔽 Subsampling by factor: {SUBSAMPLE_FACTOR}")
print(f"   Original shape: ({Nt}, {H}, {W})")

images_subsampled = images_raw[:, ::SUBSAMPLE_FACTOR, ::SUBSAMPLE_FACTOR]
Nt_sub, H_sub, W_sub = images_subsampled.shape

print(f"   Subsampled shape: ({Nt_sub}, {H_sub}, {W_sub})")
print(f"   Reduction: {H*W} → {H_sub*W_sub} pixels per frame")
print(f"   Total points for SINDy: {Nt_sub * H_sub * W_sub:,}")

# Normalize to zero mean (important for SINDy)
images_normalized = images_subsampled - images_subsampled.mean()
print(f"\n   ✓ Normalized to zero mean")
print(f"   New intensity range: [{images_normalized.min():.2f}, {images_normalized.max():.2f}]")

# ===== STEP 3: COMPUTE 2D SPATIAL DERIVATIVES (FFT) =====
print("\n" + "=" * 80)
print("STEP 3: COMPUTING 2D SPATIAL DERIVATIVES (FFT)")
print("=" * 80)

def compute_2d_fft_derivatives(u):
    """
    Compute 2D spatial derivatives using FFT
    
    Parameters:
    -----------
    u : ndarray
        3D array of shape (Nt, Ny, Nx)
        
    Returns:
    --------
    derivatives : dict
        Dictionary containing u_x, u_y, u_xx, u_yy, u_xxxx, u_yyyy
    """
    Nt, Ny, Nx = u.shape
    
    print(f"\n🧮 Computing spatial derivatives for shape: ({Nt}, {Ny}, {Nx})")
    
    # Wavenumbers (assuming dx = dy = 1.0 pixel)
    ky = 2 * np.pi * np.fft.fftfreq(Ny, d=1.0)
    kx = 2 * np.pi * np.fft.fftfreq(Nx, d=1.0)
    
    # Create 2D meshgrid
    KY, KX = np.meshgrid(ky, kx, indexing='ij')
    
    print(f"   Wavenumber ranges:")
    print(f"      kx: [{kx.min():.4f}, {kx.max():.4f}]")
    print(f"      ky: [{ky.min():.4f}, {ky.max():.4f}]")
    
    # FFT of all frames
    print(f"   Computing 2D FFT for all {Nt} frames...")
    u_fft = np.fft.fft2(u, axes=(1, 2))
    
    # First derivatives
    print(f"   Computing first derivatives (u_x, u_y)...")
    u_x_fft = 1j * KX[np.newaxis, :, :] * u_fft
    u_y_fft = 1j * KY[np.newaxis, :, :] * u_fft
    
    u_x = np.fft.ifft2(u_x_fft, axes=(1, 2)).real
    u_y = np.fft.ifft2(u_y_fft, axes=(1, 2)).real
    
    # Second derivatives
    print(f"   Computing second derivatives (u_xx, u_yy)...")
    u_xx_fft = -(KX[np.newaxis, :, :]**2) * u_fft
    u_yy_fft = -(KY[np.newaxis, :, :]**2) * u_fft
    
    u_xx = np.fft.ifft2(u_xx_fft, axes=(1, 2)).real
    u_yy = np.fft.ifft2(u_yy_fft, axes=(1, 2)).real
    
    # Fourth derivatives
    print(f"   Computing fourth derivatives (u_xxxx, u_yyyy)...")
    u_xxxx_fft = (KX[np.newaxis, :, :]**4) * u_fft
    u_yyyy_fft = (KY[np.newaxis, :, :]**4) * u_fft
    
    u_xxxx = np.fft.ifft2(u_xxxx_fft, axes=(1, 2)).real
    u_yyyy = np.fft.ifft2(u_yyyy_fft, axes=(1, 2)).real
    
    print(f"   ✓ All spatial derivatives computed")
    
    derivatives = {
        'u_x': u_x,
        'u_y': u_y,
        'u_xx': u_xx,
        'u_yy': u_yy,
        'u_xxxx': u_xxxx,
        'u_yyyy': u_yyyy
    }
    
    # Print statistics
    print(f"\n   Derivative statistics:")
    for name, deriv in derivatives.items():
        print(f"      {name:8s}: mean={deriv.mean():8.4f}, std={deriv.std():8.4f}, "
              f"range=[{deriv.min():8.2f}, {deriv.max():8.2f}]")
    
    return derivatives

derivatives = compute_2d_fft_derivatives(images_normalized)

# ===== STEP 4: COMPUTE TEMPORAL DERIVATIVE =====
print("\n" + "=" * 80)
print("STEP 4: COMPUTING TEMPORAL DERIVATIVE")
print("=" * 80)

print(f"\n⏱️  Computing u_t using np.gradient...")
u_t = np.gradient(images_normalized, axis=0)

print(f"   u_t shape: {u_t.shape}")
print(f"   u_t statistics:")
print(f"      mean: {u_t.mean():.4f}")
print(f"      std: {u_t.std():.4f}")
print(f"      range: [{u_t.min():.2f}, {u_t.max():.2f}]")

# ===== STEP 5: BUILD LIBRARY MATRIX =====
print("\n" + "=" * 80)
print("STEP 5: BUILDING LIBRARY MATRIX (THETA)")
print("=" * 80)

print(f"\n📚 Building library for 2D Kuramoto-Sivashinsky equation:")
print(f"   u_t = a*u_xx + b*u_yy + c*u_xxxx + d*u_yyyy + e*u*u_x + f*u*u_y")

# Extract derivatives
u = images_normalized
u_x = derivatives['u_x']
u_y = derivatives['u_y']
u_xx = derivatives['u_xx']
u_yy = derivatives['u_yy']
u_xxxx = derivatives['u_xxxx']
u_yyyy = derivatives['u_yyyy']

# Build library matrix
Theta = np.column_stack([
    u_xx.flatten(),      # Column 0: u_xx
    u_yy.flatten(),      # Column 1: u_yy
    u_xxxx.flatten(),    # Column 2: u_xxxx
    u_yyyy.flatten(),    # Column 3: u_yyyy
    (u * u_x).flatten(), # Column 4: u*u_x
    (u * u_y).flatten()  # Column 5: u*u_y
])

print(f"\n   Library matrix (Theta) shape: {Theta.shape}")
print(f"   Number of data points: {Theta.shape[0]:,}")
print(f"   Number of features: {Theta.shape[1]}")
print(f"   Feature names: ['u_xx', 'u_yy', 'u_xxxx', 'u_yyyy', 'u*u_x', 'u*u_y']")

# Target vector
y = u_t.flatten()
print(f"\n   Target vector (u_t) shape: {y.shape}")

# Check for NaN/Inf
if np.any(np.isnan(Theta)) or np.any(np.isinf(Theta)):
    print(f"\n   ⚠️  WARNING: NaN or Inf detected in library matrix!")
    print(f"      NaN count: {np.isnan(Theta).sum()}")
    print(f"      Inf count: {np.isinf(Theta).sum()}")
else:
    print(f"\n   ✓ No NaN or Inf values detected")

# ===== STEP 6: APPLY SINDY (LASSO REGRESSION) =====
print("\n" + "=" * 80)
print("STEP 6: APPLYING SINDY (LASSO REGRESSION)")
print("=" * 80)

print(f"\n🔍 Fitting Lasso model with alpha={SINDY_ALPHA}")

lasso = Lasso(alpha=SINDY_ALPHA, max_iter=10000, tol=1e-4)
lasso.fit(Theta, y)

coeffs_baseline = lasso.coef_
r2_baseline = lasso.score(Theta, y)

print(f"\n   ✓ Model fitted successfully")
print(f"   R² score: {r2_baseline:.6f}")
print(f"   Number of iterations: {lasso.n_iter_}")

# ===== STEP 7: EVALUATE RESULTS =====
print("\n" + "=" * 80)
print("STEP 7: EVALUATING COEFFICIENT RECOVERY")
print("=" * 80)

# Compute errors
coeff_errors = np.abs(coeffs_baseline - TRUE_COEFFS)
rmse = np.sqrt(np.mean((coeffs_baseline - TRUE_COEFFS)**2))
mae = np.mean(coeff_errors)
max_error = np.max(coeff_errors)

print(f"\n📊 Coefficient Recovery Results:")
print(f"\n   {'Feature':<12} {'True':<10} {'Recovered':<12} {'Error':<10} {'Rel Error (%)':<15}")
print(f"   {'-'*60}")

feature_names = ['u_xx', 'u_yy', 'u_xxxx', 'u_yyyy', 'u*u_x', 'u*u_y']
for i, name in enumerate(feature_names):
    true_val = TRUE_COEFFS[i]
    rec_val = coeffs_baseline[i]
    error = coeff_errors[i]
    rel_error = (error / np.abs(true_val)) * 100 if true_val != 0 else np.inf
    
    print(f"   {name:<12} {true_val:<10.4f} {rec_val:<12.4f} {error:<10.4f} {rel_error:<15.2f}")

print(f"\n   Overall Metrics:")
print(f"      RMSE: {rmse:.6f}")
print(f"      MAE:  {mae:.6f}")
print(f"      Max Error: {max_error:.6f}")
print(f"      R² Score: {r2_baseline:.6f}")

# Prediction
y_pred = lasso.predict(Theta)
prediction_rmse = np.sqrt(np.mean((y - y_pred)**2))
print(f"      Prediction RMSE: {prediction_rmse:.6f}")

# ===== STEP 8: SAVE RESULTS =====
print("\n" + "=" * 80)
print("STEP 8: SAVING RESULTS")
print("=" * 80)

results = {
    'method': 'baseline_no_preprocessing',
    'subsample_factor': SUBSAMPLE_FACTOR,
    'alpha': SINDY_ALPHA,
    'r2_score': r2_baseline,
    'rmse': rmse,
    'mae': mae,
    'max_error': max_error,
    'prediction_rmse': prediction_rmse,
    'coeff_u_xx': coeffs_baseline[0],
    'coeff_u_yy': coeffs_baseline[1],
    'coeff_u_xxxx': coeffs_baseline[2],
    'coeff_u_yyyy': coeffs_baseline[3],
    'coeff_u_ux': coeffs_baseline[4],
    'coeff_u_uy': coeffs_baseline[5],
    'error_u_xx': coeff_errors[0],
    'error_u_yy': coeff_errors[1],
    'error_u_xxxx': coeff_errors[2],
    'error_u_yyyy': coeff_errors[3],
    'error_u_ux': coeff_errors[4],
    'error_u_uy': coeff_errors[5]
}

results_df = pd.DataFrame([results])
csv_file = os.path.join(OUTPUT_DIR, 'phase4_01_baseline_results.csv')
results_df.to_csv(csv_file, index=False)
print(f"\n   ✓ Results saved to: {csv_file}")

# Save detailed arrays
npz_file = os.path.join(OUTPUT_DIR, 'phase4_01_baseline_data.npz')
np.savez(npz_file,
         images_raw=images_raw,
         images_subsampled=images_normalized,
         coefficients=coeffs_baseline,
         true_coefficients=TRUE_COEFFS,
         u_t=u_t,
         **derivatives)
print(f"   ✓ Data arrays saved to: {npz_file}")

# ===== STEP 9: VISUALIZATIONS =====
if SAVE_PLOTS:
    print("\n" + "=" * 80)
    print("STEP 9: GENERATING VISUALIZATIONS")
    print("=" * 80)
    
    # === FIGURE 1: Coefficient Recovery ===
    print("\n   Creating Figure 1: Coefficient recovery comparison...")
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle('Phase 4.1: Baseline Coefficient Recovery (No Preprocessing)', 
                 fontsize=16, fontweight='bold')
    
    for i, (ax, name) in enumerate(zip(axes.flat, feature_names)):
        x = ['True', 'Recovered']
        y = [TRUE_COEFFS[i], coeffs_baseline[i]]
        colors = ['green', 'red' if coeff_errors[i] > 0.1 else 'orange']
        
        bars = ax.bar(x, y, color=colors, alpha=0.7, edgecolor='black')
        ax.axhline(TRUE_COEFFS[i], color='green', linestyle='--', 
                   linewidth=2, label='True Value')
        ax.set_ylabel('Coefficient Value', fontsize=10)
        ax.set_title(f'{name}\nError: {coeff_errors[i]:.4f} '
                     f'({(coeff_errors[i]/np.abs(TRUE_COEFFS[i])*100):.1f}%)',
                     fontsize=11, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add value labels on bars
        for bar, val in zip(bars, y):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{val:.4f}',
                   ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_01_coefficient_recovery.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"   ✓ Saved: {plot_file}")
    plt.close()
    
    # === FIGURE 2: Sample Images and Derivatives ===
    print("   Creating Figure 2: Sample images and derivatives...")
    fig, axes = plt.subplots(3, 4, figsize=(20, 12))
    fig.suptitle('Phase 4.1: Sample Frames and Computed Derivatives', 
                 fontsize=16, fontweight='bold')
    
    # Show 3 sample frames
    sample_indices = [0, Nt//2, Nt-1]
    
    for row, idx in enumerate(sample_indices):
        # Original image
        im0 = axes[row, 0].imshow(images_normalized[idx], cmap='gray', aspect='auto')
        axes[row, 0].set_title(f'Frame {idx}\nOriginal', fontsize=10)
        axes[row, 0].axis('off')
        plt.colorbar(im0, ax=axes[row, 0])
        
        # u_xx
        im1 = axes[row, 1].imshow(u_xx[idx], cmap='RdBu_r', aspect='auto')
        axes[row, 1].set_title(f'u_xx', fontsize=10)
        axes[row, 1].axis('off')
        plt.colorbar(im1, ax=axes[row, 1])
        
        # u_xxxx
        im2 = axes[row, 2].imshow(u_xxxx[idx], cmap='RdBu_r', aspect='auto')
        axes[row, 2].set_title(f'u_xxxx', fontsize=10)
        axes[row, 2].axis('off')
        plt.colorbar(im2, ax=axes[row, 2])
        
        # u*u_x (nonlinear term)
        im3 = axes[row, 3].imshow((u*u_x)[idx], cmap='RdBu_r', aspect='auto')
        axes[row, 3].set_title(f'u*u_x', fontsize=10)
        axes[row, 3].axis('off')
        plt.colorbar(im3, ax=axes[row, 3])
    
    plt.tight_layout()
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_01_sample_derivatives.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"   ✓ Saved: {plot_file}")
    plt.close()
    
    # === FIGURE 3: Prediction Quality ===
    print("   Creating Figure 3: Prediction quality assessment...")
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle('Phase 4.1: SINDy Prediction Quality', 
                 fontsize=16, fontweight='bold')
    
    # Scatter plot: True vs Predicted
    axes[0].scatter(y, y_pred, alpha=0.1, s=1)
    axes[0].plot([y.min(), y.max()], [y.min(), y.max()], 
                 'r--', linewidth=2, label='Perfect Prediction')
    axes[0].set_xlabel('True u_t', fontsize=11)
    axes[0].set_ylabel('Predicted u_t', fontsize=11)
    axes[0].set_title(f'True vs Predicted\nR² = {r2_baseline:.4f}', fontsize=12)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Residuals
    residuals = y - y_pred
    axes[1].hist(residuals, bins=100, color='steelblue', alpha=0.7, edgecolor='black')
    axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
    axes[1].set_xlabel('Residual (True - Predicted)', fontsize=11)
    axes[1].set_ylabel('Frequency', fontsize=11)
    axes[1].set_title(f'Residual Distribution\nRMSE = {prediction_rmse:.6f}', fontsize=12)
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # Sample temporal predictions
    sample_idx = H_sub // 2
    sample_row = slice(sample_idx, sample_idx + 1)
    u_t_sample = u_t[:, sample_row, :].flatten()
    y_pred_sample = y_pred.reshape(u_t.shape)[:, sample_row, :].flatten()
    
    axes[2].plot(u_t_sample[:500], 'b-', linewidth=1.5, label='True u_t', alpha=0.7)
    axes[2].plot(y_pred_sample[:500], 'r--', linewidth=1.5, label='Predicted u_t', alpha=0.7)
    axes[2].set_xlabel('Spatial Point', fontsize=11)
    axes[2].set_ylabel('u_t', fontsize=11)
    axes[2].set_title('Sample Temporal Derivative', fontsize=12)
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_01_prediction_quality.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"   ✓ Saved: {plot_file}")
    plt.close()
    
    # === FIGURE 4: Summary Dashboard ===
    print("   Creating Figure 4: Summary dashboard...")
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)
    
    # Coefficient comparison
    ax1 = fig.add_subplot(gs[0, 0])
    x_pos = np.arange(len(feature_names))
    width = 0.35
    ax1.bar(x_pos - width/2, TRUE_COEFFS, width, label='True', 
            color='green', alpha=0.7, edgecolor='black')
    ax1.bar(x_pos + width/2, coeffs_baseline, width, label='Recovered',
            color='red', alpha=0.7, edgecolor='black')
    ax1.set_xlabel('Coefficient', fontsize=11)
    ax1.set_ylabel('Value', fontsize=11)
    ax1.set_title('Coefficient Comparison', fontsize=12, fontweight='bold')
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(feature_names, rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    ax1.axhline(0, color='black', linewidth=0.5)
    
    # Error bars
    ax2 = fig.add_subplot(gs[0, 1])
    colors_err = ['green' if e < 0.05 else 'orange' if e < 0.1 else 'red' 
                  for e in coeff_errors]
    bars = ax2.barh(feature_names, coeff_errors, color=colors_err, 
                     alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Absolute Error', fontsize=11)
    ax2.set_title('Coefficient Errors', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='x')
    
    # Add error values on bars
    for bar, err in zip(bars, coeff_errors):
        width = bar.get_width()
        ax2.text(width, bar.get_y() + bar.get_height()/2,
                f' {err:.4f}',
                ha='left', va='center', fontsize=9)
    
    # Summary text
    ax3 = fig.add_subplot(gs[1, :])
    ax3.axis('off')
    
    summary_text = f"""
PHASE 4.1: BASELINE 2D SINDY RESULTS (NO PREPROCESSING)
{'='*80}

INPUT DATA:
  • Original images: {Nt} frames of {H} × {W} pixels
  • Subsampled: {Nt_sub} frames of {H_sub} × {W_sub} pixels
  • Subsample factor: {SUBSAMPLE_FACTOR}
  • Total data points: {Nt_sub * H_sub * W_sub:,}

SINDY CONFIGURATION:
  • Regularization (α): {SINDY_ALPHA}
  • Method: Lasso regression
  • Library features: 6 (u_xx, u_yy, u_xxxx, u_yyyy, u*u_x, u*u_y)

COEFFICIENT RECOVERY:
  • RMSE: {rmse:.6f}
  • MAE:  {mae:.6f}
  • Max Error: {max_error:.6f}
  • R² Score: {r2_baseline:.6f}

PREDICTION QUALITY:
  • Prediction RMSE: {prediction_rmse:.6f}
  • R² Score: {r2_baseline:.6f}

ASSESSMENT:
  {'✅ GOOD' if rmse < 0.1 else '⚠️  MODERATE' if rmse < 0.3 else '❌ POOR'} - Baseline coefficient recovery
  {'✅ GOOD' if r2_baseline > 0.8 else '⚠️  MODERATE' if r2_baseline > 0.5 else '❌ POOR'} - Model fit quality

NEXT STEPS:
  → Phase 4.2: Apply registration methods
  → Phase 4.3: Apply denoising methods
  → Compare improvements over baseline
    """
    
    ax3.text(0.05, 0.95, summary_text, fontsize=10, family='monospace',
             verticalalignment='top', transform=ax3.transAxes,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.suptitle('Phase 4.1: Baseline Summary Dashboard', 
                 fontsize=16, fontweight='bold', y=0.98)
    
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_01_summary_dashboard.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"   ✓ Saved: {plot_file}")
    plt.close()

# ===== FINAL SUMMARY =====
print("\n" + "=" * 80)
print("PHASE 4.1 COMPLETE - BASELINE 2D SINDY")
print("=" * 80)

print(f"\n✅ Successfully completed baseline analysis:")
print(f"   • Loaded {Nt} images from ZIP file")
print(f"   • Subsampled to ({Nt_sub}, {H_sub}, {W_sub})")
print(f"   • Computed 2D spatial derivatives (FFT)")
print(f"   • Applied SINDy with Lasso regression")
print(f"   • Coefficient RMSE: {rmse:.6f}")
print(f"   • Model R² score: {r2_baseline:.6f}")

print(f"\n📁 Output files saved to: {OUTPUT_DIR}/")
print(f"   ✓ phase4_01_baseline_results.csv")
print(f"   ✓ phase4_01_baseline_data.npz")

if SAVE_PLOTS:
    print(f"   ✓ phase4_01_coefficient_recovery.png")
    print(f"   ✓ phase4_01_sample_derivatives.png")
    print(f"   ✓ phase4_01_prediction_quality.png")
    print(f"   ✓ phase4_01_summary_dashboard.png")

assessment = "GOOD" if rmse < 0.1 else "MODERATE" if rmse < 0.3 else "POOR"
print(f"\n🎯 Overall Assessment: {assessment}")

if rmse < 0.1:
    print(f"   ✅ Baseline performance is excellent!")
    print(f"   → Registration and denoising may provide marginal improvements")
elif rmse < 0.3:
    print(f"   ⚠️  Baseline performance is moderate")
    print(f"   → Registration and denoising should improve results significantly")
else:
    print(f"   ❌ Baseline performance is poor")
    print(f"   → Preprocessing (registration + denoising) is ESSENTIAL")

print(f"\n🚀 Ready for Phase 4.2: Registration Methods")
print("=" * 80)

In [ ]:
"""
PHASE 4.1: BASELINE 2D SINDY (NO PREPROCESSING)

Goal: Establish baseline coefficient recovery from raw 2D images
Input: ZIP file containing TIFF images
Output: Baseline metrics, visualizations, CSV results

Author: SINDy Real Image Processing Project
Date: Phase 4.1
"""

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas as pd
import zipfile
import os
from sklearn.linear_model import Lasso
from tifffile import imread
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (18, 12)

print("=" * 80)
print("PHASE 4.1: BASELINE 2D SINDY - NO PREPROCESSING")
print("=" * 80)

# ===== CONFIGURATION =====
ZIP_FILE = "path/to/your/Real_Images.zip"  # UPDATE THIS PATH
OUTPUT_DIR = "phase4_results"
SUBSAMPLE_FACTOR = 8  # Reduce from (2052, 2560) to (257, 320)
SINDY_ALPHA = 1e-4  # Lasso regularization parameter
SAVE_PLOTS = True

# True coefficients for 2D Kuramoto-Sivashinsky equation
# u_t = a*u_xx + b*u_yy + c*u_xxxx + d*u_yyyy + e*u*u_x + f*u*u_y
TRUE_COEFFS = np.array([-1.0, -1.0, -1.0, -1.0, -1.0, -1.0])

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ===== STEP 1: LOAD IMAGES FROM ZIP =====
print("\n" + "=" * 80)
print("STEP 1: LOADING IMAGES FROM ZIP FILE")
print("=" * 80)

def load_images_from_zip(zip_path):
    """
    Load TIFF images from a ZIP file
    
    Parameters:
    -----------
    zip_path : str
        Path to ZIP file containing TIFF images
        
    Returns:
    --------
    images : ndarray
        Array of shape (Nt, H, W) containing all images
    filenames : list
        List of image filenames
    """
    print(f"\n📦 Opening ZIP file: {zip_path}")
    
    images = []
    filenames = []
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Get all TIFF files in ZIP
        tiff_files = [f for f in zip_ref.namelist() 
                      if f.lower().endswith(('.tif', '.tiff')) 
                      and not f.startswith('__MACOSX')]
        
        tiff_files = sorted(tiff_files)
        
        print(f"   Found {len(tiff_files)} TIFF files")
        
        # Extract to temporary directory and load
        temp_dir = "temp_extracted_images"
        os.makedirs(temp_dir, exist_ok=True)
        
        for idx, tiff_file in enumerate(tiff_files):
            # Extract file
            zip_ref.extract(tiff_file, temp_dir)
            
            # Load image
            img_path = os.path.join(temp_dir, tiff_file)
            img = imread(img_path)
            
            images.append(img)
            filenames.append(os.path.basename(tiff_file))
            
            if (idx + 1) % 10 == 0:
                print(f"   Loaded {idx + 1}/{len(tiff_files)} images...")
        
        print(f"   ✓ All images loaded successfully")
        
        # Clean up temporary directory
        import shutil
        shutil.rmtree(temp_dir)
    
    images = np.array(images)
    return images, filenames

# Load images
try:
    images_raw, filenames = load_images_from_zip(ZIP_FILE)
    Nt, H, W = images_raw.shape
    
    print(f"\n✅ Loaded raw images:")
    print(f"   Number of frames (Nt): {Nt}")
    print(f"   Image height (H): {H}")
    print(f"   Image width (W): {W}")
    print(f"   Total shape: {images_raw.shape}")
    print(f"   Data type: {images_raw.dtype}")
    print(f"   Intensity range: [{images_raw.min()}, {images_raw.max()}]")
    
except FileNotFoundError:
    print(f"\n❌ ERROR: ZIP file not found at: {ZIP_FILE}")
    print(f"   Please update the ZIP_FILE path in the configuration section")
    exit(1)
except Exception as e:
    print(f"\n❌ ERROR loading images: {str(e)}")
    exit(1)

# ===== STEP 2: SPATIAL SUBSAMPLING =====
print("\n" + "=" * 80)
print("STEP 2: SPATIAL SUBSAMPLING")
print("=" * 80)

print(f"\n🔽 Subsampling by factor: {SUBSAMPLE_FACTOR}")
print(f"   Original shape: ({Nt}, {H}, {W})")

images_subsampled = images_raw[:, ::SUBSAMPLE_FACTOR, ::SUBSAMPLE_FACTOR]
Nt_sub, H_sub, W_sub = images_subsampled.shape

print(f"   Subsampled shape: ({Nt_sub}, {H_sub}, {W_sub})")
print(f"   Reduction: {H*W} → {H_sub*W_sub} pixels per frame")
print(f"   Total points for SINDy: {Nt_sub * H_sub * W_sub:,}")

# Normalize to zero mean (important for SINDy)
images_normalized = images_subsampled - images_subsampled.mean()
print(f"\n   ✓ Normalized to zero mean")
print(f"   New intensity range: [{images_normalized.min():.2f}, {images_normalized.max():.2f}]")

# ===== STEP 3: COMPUTE 2D SPATIAL DERIVATIVES (FFT) =====
print("\n" + "=" * 80)
print("STEP 3: COMPUTING 2D SPATIAL DERIVATIVES (FFT)")
print("=" * 80)

def compute_2d_fft_derivatives(u):
    """
    Compute 2D spatial derivatives using FFT
    
    Parameters:
    -----------
    u : ndarray
        3D array of shape (Nt, Ny, Nx)
        
    Returns:
    --------
    derivatives : dict
        Dictionary containing u_x, u_y, u_xx, u_yy, u_xxxx, u_yyyy
    """
    Nt, Ny, Nx = u.shape
    
    print(f"\n🧮 Computing spatial derivatives for shape: ({Nt}, {Ny}, {Nx})")
    
    # Wavenumbers (assuming dx = dy = 1.0 pixel)
    ky = 2 * np.pi * np.fft.fftfreq(Ny, d=1.0)
    kx = 2 * np.pi * np.fft.fftfreq(Nx, d=1.0)
    
    # Create 2D meshgrid
    KY, KX = np.meshgrid(ky, kx, indexing='ij')
    
    print(f"   Wavenumber ranges:")
    print(f"      kx: [{kx.min():.4f}, {kx.max():.4f}]")
    print(f"      ky: [{ky.min():.4f}, {ky.max():.4f}]")
    
    # FFT of all frames
    print(f"   Computing 2D FFT for all {Nt} frames...")
    u_fft = np.fft.fft2(u, axes=(1, 2))
    
    # First derivatives
    print(f"   Computing first derivatives (u_x, u_y)...")
    u_x_fft = 1j * KX[np.newaxis, :, :] * u_fft
    u_y_fft = 1j * KY[np.newaxis, :, :] * u_fft
    
    u_x = np.fft.ifft2(u_x_fft, axes=(1, 2)).real
    u_y = np.fft.ifft2(u_y_fft, axes=(1, 2)).real
    
    # Second derivatives
    print(f"   Computing second derivatives (u_xx, u_yy)...")
    u_xx_fft = -(KX[np.newaxis, :, :]**2) * u_fft
    u_yy_fft = -(KY[np.newaxis, :, :]**2) * u_fft
    
    u_xx = np.fft.ifft2(u_xx_fft, axes=(1, 2)).real
    u_yy = np.fft.ifft2(u_yy_fft, axes=(1, 2)).real
    
    # Fourth derivatives
    print(f"   Computing fourth derivatives (u_xxxx, u_yyyy)...")
    u_xxxx_fft = (KX[np.newaxis, :, :]**4) * u_fft
    u_yyyy_fft = (KY[np.newaxis, :, :]**4) * u_fft
    
    u_xxxx = np.fft.ifft2(u_xxxx_fft, axes=(1, 2)).real
    u_yyyy = np.fft.ifft2(u_yyyy_fft, axes=(1, 2)).real
    
    print(f"   ✓ All spatial derivatives computed")
    
    derivatives = {
        'u_x': u_x,
        'u_y': u_y,
        'u_xx': u_xx,
        'u_yy': u_yy,
        'u_xxxx': u_xxxx,
        'u_yyyy': u_yyyy
    }
    
    # Print statistics
    print(f"\n   Derivative statistics:")
    for name, deriv in derivatives.items():
        print(f"      {name:8s}: mean={deriv.mean():8.4f}, std={deriv.std():8.4f}, "
              f"range=[{deriv.min():8.2f}, {deriv.max():8.2f}]")
    
    return derivatives

derivatives = compute_2d_fft_derivatives(images_normalized)

# ===== STEP 4: COMPUTE TEMPORAL DERIVATIVE =====
print("\n" + "=" * 80)
print("STEP 4: COMPUTING TEMPORAL DERIVATIVE")
print("=" * 80)

print(f"\n⏱️  Computing u_t using np.gradient...")
u_t = np.gradient(images_normalized, axis=0)

print(f"   u_t shape: {u_t.shape}")
print(f"   u_t statistics:")
print(f"      mean: {u_t.mean():.4f}")
print(f"      std: {u_t.std():.4f}")
print(f"      range: [{u_t.min():.2f}, {u_t.max():.2f}]")

# ===== STEP 5: BUILD LIBRARY MATRIX =====
print("\n" + "=" * 80)
print("STEP 5: BUILDING LIBRARY MATRIX (THETA)")
print("=" * 80)

print(f"\n📚 Building library for 2D Kuramoto-Sivashinsky equation:")
print(f"   u_t = a*u_xx + b*u_yy + c*u_xxxx + d*u_yyyy + e*u*u_x + f*u*u_y")

# Extract derivatives
u = images_normalized
u_x = derivatives['u_x']
u_y = derivatives['u_y']
u_xx = derivatives['u_xx']
u_yy = derivatives['u_yy']
u_xxxx = derivatives['u_xxxx']
u_yyyy = derivatives['u_yyyy']

# Build library matrix
Theta = np.column_stack([
    u_xx.flatten(),      # Column 0: u_xx
    u_yy.flatten(),      # Column 1: u_yy
    u_xxxx.flatten(),    # Column 2: u_xxxx
    u_yyyy.flatten(),    # Column 3: u_yyyy
    (u * u_x).flatten(), # Column 4: u*u_x
    (u * u_y).flatten()  # Column 5: u*u_y
])

print(f"\n   Library matrix (Theta) shape: {Theta.shape}")
print(f"   Number of data points: {Theta.shape[0]:,}")
print(f"   Number of features: {Theta.shape[1]}")
print(f"   Feature names: ['u_xx', 'u_yy', 'u_xxxx', 'u_yyyy', 'u*u_x', 'u*u_y']")

# Target vector
y = u_t.flatten()
print(f"\n   Target vector (u_t) shape: {y.shape}")

# Check for NaN/Inf
if np.any(np.isnan(Theta)) or np.any(np.isinf(Theta)):
    print(f"\n   ⚠️  WARNING: NaN or Inf detected in library matrix!")
    print(f"      NaN count: {np.isnan(Theta).sum()}")
    print(f"      Inf count: {np.isinf(Theta).sum()}")
else:
    print(f"\n   ✓ No NaN or Inf values detected")

# ===== STEP 6: APPLY SINDY (LASSO REGRESSION) =====
print("\n" + "=" * 80)
print("STEP 6: APPLYING SINDY (LASSO REGRESSION)")
print("=" * 80)

print(f"\n🔍 Fitting Lasso model with alpha={SINDY_ALPHA}")

lasso = Lasso(alpha=SINDY_ALPHA, max_iter=10000, tol=1e-4)
lasso.fit(Theta, y)

coeffs_baseline = lasso.coef_
r2_baseline = lasso.score(Theta, y)

print(f"\n   ✓ Model fitted successfully")
print(f"   R² score: {r2_baseline:.6f}")
print(f"   Number of iterations: {lasso.n_iter_}")

# ===== STEP 7: EVALUATE RESULTS =====
print("\n" + "=" * 80)
print("STEP 7: EVALUATING COEFFICIENT RECOVERY")
print("=" * 80)

# Compute errors
coeff_errors = np.abs(coeffs_baseline - TRUE_COEFFS)
rmse = np.sqrt(np.mean((coeffs_baseline - TRUE_COEFFS)**2))
mae = np.mean(coeff_errors)
max_error = np.max(coeff_errors)

print(f"\n📊 Coefficient Recovery Results:")
print(f"\n   {'Feature':<12} {'True':<10} {'Recovered':<12} {'Error':<10} {'Rel Error (%)':<15}")
print(f"   {'-'*60}")

feature_names = ['u_xx', 'u_yy', 'u_xxxx', 'u_yyyy', 'u*u_x', 'u*u_y']
for i, name in enumerate(feature_names):
    true_val = TRUE_COEFFS[i]
    rec_val = coeffs_baseline[i]
    error = coeff_errors[i]
    rel_error = (error / np.abs(true_val)) * 100 if true_val != 0 else np.inf
    
    print(f"   {name:<12} {true_val:<10.4f} {rec_val:<12.4f} {error:<10.4f} {rel_error:<15.2f}")

print(f"\n   Overall Metrics:")
print(f"      RMSE: {rmse:.6f}")
print(f"      MAE:  {mae:.6f}")
print(f"      Max Error: {max_error:.6f}")
print(f"      R² Score: {r2_baseline:.6f}")

# Prediction
y_pred = lasso.predict(Theta)
prediction_rmse = np.sqrt(np.mean((y - y_pred)**2))
print(f"      Prediction RMSE: {prediction_rmse:.6f}")

# ===== STEP 8: SAVE RESULTS =====
print("\n" + "=" * 80)
print("STEP 8: SAVING RESULTS")
print("=" * 80)

results = {
    'method': 'baseline_no_preprocessing',
    'subsample_factor': SUBSAMPLE_FACTOR,
    'alpha': SINDY_ALPHA,
    'r2_score': r2_baseline,
    'rmse': rmse,
    'mae': mae,
    'max_error': max_error,
    'prediction_rmse': prediction_rmse,
    'coeff_u_xx': coeffs_baseline[0],
    'coeff_u_yy': coeffs_baseline[1],
    'coeff_u_xxxx': coeffs_baseline[2],
    'coeff_u_yyyy': coeffs_baseline[3],
    'coeff_u_ux': coeffs_baseline[4],
    'coeff_u_uy': coeffs_baseline[5],
    'error_u_xx': coeff_errors[0],
    'error_u_yy': coeff_errors[1],
    'error_u_xxxx': coeff_errors[2],
    'error_u_yyyy': coeff_errors[3],
    'error_u_ux': coeff_errors[4],
    'error_u_uy': coeff_errors[5]
}

results_df = pd.DataFrame([results])
csv_file = os.path.join(OUTPUT_DIR, 'phase4_01_baseline_results.csv')
results_df.to_csv(csv_file, index=False)
print(f"\n   ✓ Results saved to: {csv_file}")

# Save detailed arrays
npz_file = os.path.join(OUTPUT_DIR, 'phase4_01_baseline_data.npz')
np.savez(npz_file,
         images_raw=images_raw,
         images_subsampled=images_normalized,
         coefficients=coeffs_baseline,
         true_coefficients=TRUE_COEFFS,
         u_t=u_t,
         **derivatives)
print(f"   ✓ Data arrays saved to: {npz_file}")

# ===== STEP 9: VISUALIZATIONS =====
if SAVE_PLOTS:
    print("\n" + "=" * 80)
    print("STEP 9: GENERATING VISUALIZATIONS")
    print("=" * 80)
    
    # === FIGURE 1: Coefficient Recovery ===
    print("\n   Creating Figure 1: Coefficient recovery comparison...")
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle('Phase 4.1: Baseline Coefficient Recovery (No Preprocessing)', 
                 fontsize=16, fontweight='bold')
    
    for i, (ax, name) in enumerate(zip(axes.flat, feature_names)):
        x_labels = ['True', 'Recovered']
        y_vals = [TRUE_COEFFS[i], coeffs_baseline[i]]
        colors = ['green', 'red' if coeff_errors[i] > 0.1 else 'orange']
        
        bars = ax.bar(x_labels, y_vals, color=colors, alpha=0.7, edgecolor='black')
        ax.axhline(TRUE_COEFFS[i], color='green', linestyle='--', 
                   linewidth=2, label='True Value')
        ax.set_ylabel('Coefficient Value', fontsize=10)
        ax.set_title(f'{name}\nError: {coeff_errors[i]:.4f} '
                     f'({(coeff_errors[i]/np.abs(TRUE_COEFFS[i])*100):.1f}%)',
                     fontsize=11, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add value labels on bars
        for bar, val in zip(bars, y_vals):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{val:.4f}',
                   ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_01_coefficient_recovery.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"   ✓ Saved: {plot_file}")
    plt.close()
    
    # === FIGURE 2: Sample Images and Derivatives ===
    print("   Creating Figure 2: Sample images and derivatives...")
    fig, axes = plt.subplots(3, 4, figsize=(20, 12))
    fig.suptitle('Phase 4.1: Sample Frames and Computed Derivatives', 
                 fontsize=16, fontweight='bold')
    
    # Show 3 sample frames
    sample_indices = [0, Nt_sub//2, Nt_sub-1]
    
    for row, idx in enumerate(sample_indices):
        # Original image
        im0 = axes[row, 0].imshow(images_normalized[idx], cmap='gray', aspect='auto')
        axes[row, 0].set_title(f'Frame {idx}\nOriginal', fontsize=10)
        axes[row, 0].axis('off')
        plt.colorbar(im0, ax=axes[row, 0])
        
        # u_xx
        im1 = axes[row, 1].imshow(u_xx[idx], cmap='RdBu_r', aspect='auto')
        axes[row, 1].set_title(f'u_xx', fontsize=10)
        axes[row, 1].axis('off')
        plt.colorbar(im1, ax=axes[row, 1])
        
        # u_xxxx
        im2 = axes[row, 2].imshow(u_xxxx[idx], cmap='RdBu_r', aspect='auto')
        axes[row, 2].set_title(f'u_xxxx', fontsize=10)
        axes[row, 2].axis('off')
        plt.colorbar(im2, ax=axes[row, 2])
        
        # u*u_x (nonlinear term)
        im3 = axes[row, 3].imshow((u*u_x)[idx], cmap='RdBu_r', aspect='auto')
        axes[row, 3].set_title(f'u*u_x', fontsize=10)
        axes[row, 3].axis('off')
        plt.colorbar(im3, ax=axes[row, 3])
    
    plt.tight_layout()
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_01_sample_derivatives.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"   ✓ Saved: {plot_file}")
    plt.close()
    
    # === FIGURE 3: Prediction Quality ===
    print("   Creating Figure 3: Prediction quality assessment...")
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle('Phase 4.1: SINDy Prediction Quality', 
                 fontsize=16, fontweight='bold')
    
    # Scatter plot: True vs Predicted
    axes[0].scatter(y, y_pred, alpha=0.1, s=1)
    axes[0].plot([y.min(), y.max()], [y.min(), y.max()], 
                 'r--', linewidth=2, label='Perfect Prediction')
    axes[0].set_xlabel('True u_t', fontsize=11)
    axes[0].set_ylabel('Predicted u_t', fontsize=11)
    axes[0].set_title(f'True vs Predicted\nR² = {r2_baseline:.4f}', fontsize=12)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Residuals
    residuals = y - y_pred
    axes[1].hist(residuals, bins=100, color='steelblue', alpha=0.7, edgecolor='black')
    axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
    axes[1].set_xlabel('Residual (True - Predicted)', fontsize=11)
    axes[1].set_ylabel('Frequency', fontsize=11)
    axes[1].set_title(f'Residual Distribution\nRMSE = {prediction_rmse:.6f}', fontsize=12)
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # Sample temporal predictions
    sample_idx = H_sub // 2
    sample_row = slice(sample_idx, sample_idx + 1)
    u_t_sample = u_t[:, sample_row, :].flatten()
    y_pred_sample = y_pred.reshape(u_t.shape)[:, sample_row, :].flatten()
    
    axes[2].plot(u_t_sample[:500], 'b-', linewidth=1.5, label='True u_t', alpha=0.7)
    axes[2].plot(y_pred_sample[:500], 'r--', linewidth=1.5, label='Predicted u_t', alpha=0.7)
    axes[2].set_xlabel('Spatial Point', fontsize=11)
    axes[2].set_ylabel('u_t', fontsize=11)
    axes[2].set_title('Sample Temporal Derivative', fontsize=12)
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_01_prediction_quality.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"   ✓ Saved: {plot_file}")
    plt.close()
    
    # === FIGURE 4: Summary Dashboard ===
    print("   Creating Figure 4: Summary dashboard...")
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)
    
    # Coefficient comparison
    ax1 = fig.add_subplot(gs[0, 0])
    x_pos = np.arange(len(feature_names))
    bar_width = 0.35
    ax1.bar(x_pos - bar_width/2, TRUE_COEFFS, bar_width, label='True', 
            color='green', alpha=0.7, edgecolor='black')
    ax1.bar(x_pos + bar_width/2, coeffs_baseline, bar_width, label='Recovered',
            color='red', alpha=0.7, edgecolor='black')
    ax1.set_xlabel('Coefficient', fontsize=11)
    ax1.set_ylabel('Value', fontsize=11)
    ax1.set_title('Coefficient Comparison', fontsize=12, fontweight='bold')
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(feature_names, rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    ax1.axhline(0, color='black', linewidth=0.5)
    
    # Error bars
    ax2 = fig.add_subplot(gs[0, 1])
    colors_err = ['green' if e < 0.05 else 'orange' if e < 0.1 else 'red' 
                  for e in coeff_errors]
    bars = ax2.barh(feature_names, coeff_errors, color=colors_err, 
                     alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Absolute Error', fontsize=11)
    ax2.set_title('Coefficient Errors', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='x')
    
    # Add error values on bars
    for bar, err in zip(bars, coeff_errors):
        bar_w = bar.get_width()
        ax2.text(bar_w, bar.get_y() + bar.get_height()/2,
                f' {err:.4f}',
                ha='left', va='center', fontsize=9)
    
    # Summary text
    ax3 = fig.add_subplot(gs[1, :])
    ax3.axis('off')
    
    summary_text = f"""
PHASE 4.1: BASELINE 2D SINDY RESULTS (NO PREPROCESSING)
{'='*80}

INPUT DATA:
  • Original images: {Nt} frames of {H} × {W} pixels
  • Subsampled: {Nt_sub} frames of {H_sub} × {W_sub} pixels
  • Subsample factor: {SUBSAMPLE_FACTOR}
  • Total data points: {Nt_sub * H_sub * W_sub:,}

SINDY CONFIGURATION:
  • Regularization (α): {SINDY_ALPHA}
  • Method: Lasso regression
  • Library features: 6 (u_xx, u_yy, u_xxxx, u_yyyy, u*u_x, u*u_y)

COEFFICIENT RECOVERY:
  • RMSE: {rmse:.6f}
  • MAE:  {mae:.6f}
  • Max Error: {max_error:.6f}
  • R² Score: {r2_baseline:.6f}

PREDICTION QUALITY:
  • Prediction RMSE: {prediction_rmse:.6f}
  • R² Score: {r2_baseline:.6f}

ASSESSMENT:
  {'✅ GOOD' if rmse < 0.1 else '⚠️  MODERATE' if rmse < 0.3 else '❌ POOR'} - Baseline coefficient recovery
  {'✅ GOOD' if r2_baseline > 0.8 else '⚠️  MODERATE' if r2_baseline > 0.5 else '❌ POOR'} - Model fit quality

NEXT STEPS:
  → Phase 4.2: Apply registration methods
  → Phase 4.3: Apply denoising methods
  → Compare improvements over baseline
    """
    
    ax3.text(0.05, 0.95, summary_text, fontsize=10, family='monospace',
             verticalalignment='top', transform=ax3.transAxes,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.suptitle('Phase 4.1: Baseline Summary Dashboard', 
                 fontsize=16, fontweight='bold', y=0.98)
    
    plot_file = os.path.join(OUTPUT_DIR, 'phase4_01_summary_dashboard.png')
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"   ✓ Saved: {plot_file}")
    plt.close()

# ===== FINAL SUMMARY =====
print("\n" + "=" * 80)
print("PHASE 4.1 COMPLETE - BASELINE 2D SINDY")
print("=" * 80)

print(f"\n✅ Successfully completed baseline analysis:")
print(f"   • Loaded {Nt} images from ZIP file")
print(f"   • Subsampled to ({Nt_sub}, {H_sub}, {W_sub})")
print(f"   • Computed 2D spatial derivatives (FFT)")
print(f"   • Applied SINDy with Lasso regression")
print(f"   • Coefficient RMSE: {rmse:.6f}")
print(f"   • Model R² score: {r2_baseline:.6f}")

print(f"\n📁 Output files saved to: {OUTPUT_DIR}/")
print(f"   ✓ phase4_01_baseline_results.csv")
print(f"   ✓ phase4_01_baseline_data.npz")

if SAVE_PLOTS:
    print(f"   ✓ phase4_01_coefficient_recovery.png")
    print(f"   ✓ phase4_01_sample_derivatives.png")
    print(f"   ✓ phase4_01_prediction_quality.png")
    print(f"   ✓ phase4_01_summary_dashboard.png")

assessment = "GOOD" if rmse < 0.1 else "MODERATE" if rmse < 0.3 else "POOR"
print(f"\n🎯 Overall Assessment: {assessment}")

if rmse < 0.1:
    print(f"   ✅ Baseline performance is excellent!")
    print(f"   → Registration and denoising may provide marginal improvements")
elif rmse < 0.3:
    print(f"   ⚠️  Baseline performance is moderate")
    print(f"   → Registration and denoising should improve results significantly")
else:
    print(f"   ❌ Baseline performance is poor")
    print(f"   → Preprocessing (registration + denoising) is ESSENTIAL")

print(f"\n🚀 Ready for Phase 4.2: Registration Methods")
print("=" * 80)